# Homework 4 Solutions

BEE 4850/5850

## Overview

### Instructions

The goal of this homework assignment is to practice simulation-based
uncertainty quantification, including the bootstrap and Markov chain
Monte Carlo.

-   Problem 1 asks you to use the parametric and non-parametric
    bootstrap to estimate the median of water level data.
-   Problem 2 asks you to use the parametric bootstrap to estimate
    parameter uncertainty in a semi-empirical sea-level rise model.
-   Problem 3 (only required for students in BEE 5850) asks you to use
    the bootstrap to quantify uncertainties in the
    gender-impact-on-hurricane-damages dataset from Homework 2.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

The following packages are included in the environment (to help you find
other similar packages in other languages). The code below loads these
packages for use in the subsequent notebook (the desired functionality
for each package is commented next to the package).

In [1]:
using Random # random number generation and seed-setting
using DataFrames # tabular data structure
using DataFramesMeta # API which can simplify chains of DataFrames transformations
using CSV# reads/writes .csv files
using Distributions # interface to work with probability distributions
using Plots # plotting library
using StatsBase # statistical quantities like mean, median, etc
using StatsPlots # some additional statistical plotting tools
using Optim
using LaTeXStrings
using Dates

Random.seed!(1)

## Problems

### Scoring

-   Problem 1 is worth 10 points;
-   Problem 2 is worth 10 points;
-   Problem 3 is worth 5 points.

### Problem 1

Let’s load the data from ‘data/salamanders.csv’.

In [1]:
dat = CSV.read(joinpath("data", "salamanders.csv"), DataFrame, delim=";")

Since we’re using a Poisson model, the model specification is

where $x$ is the percent groundcover of the plot and $y$ is the
salamander count.

Let’s find the maximum likelihood.

In [1]:
function salamander_pcover(p, counts, pctcover) 
    a, b = p
    λ = exp.(a * pctcover .+ b)
    ll = sum(logpdf.(Poisson.(λ), counts))
    return ll
end

lb = [-10.0, -50.0]
ub = [10.0, 50.0]
p0 = [0.0, 0.0]

# function to make standardizing the predictor more convenient
stdz(x) = (x .- mean(x)) / std(x)

result = optimize(p -> -salamander_pcover(p, dat.SALAMAN, stdz(dat.PCTCOVER)), lb, ub, p0)
θ_mle = result.minimizer

2-element Vector{Float64}:
 1.1595061318756
 0.42951129115546005

Each bootstrap replicate consists of a new dataset formed by resampling
plot datum, to which we repeat the above analysis to refit the model.

In [1]:
nboot = 1_000
θ_boot = zeros(nboot, 2) # storage for bootstrap replicates
for i = 1:nboot
    idx = sample(1:nrow(dat), nrow(dat), replace=true)
    boot_dat = dat[idx, :]
    result = optimize(p -> -salamander_pcover(p, boot_dat.SALAMAN, stdz(boot_dat.PCTCOVER)), lb, ub, p0)
    θ_boot[i, :] = result.minimizer
end
# show the bootstrap mean
θ̂ = mean(θ_boot; dims=1)
@show θ̂;

θ̂ = [1.1794723089593893 0.38099303881850943]

We can now visualize the bootstrapped sampling distributions and compare
to the MLE.

``` julia
p1 = histogram(θ_boot[:, 1], xlabel="a", ylabel="Count", fillcolor=:white, label=false)
vline!(p1, [θ_mle[1]], color=:red, lw=2, label="MLE")
vline!(p1, [θ̂[1]], color=:purple, lw=2, label="Bootstrap Mean")
plot!(p1, size=(400, 400))

p2 = histogram(θ_boot[:, 2], xlabel="b", ylabel="Count", fillcolor=:white, label=false)
vline!(p2, [θ_mle[2]], color=:red, lw=2, label="MLE")
vline!(p2, [θ̂[2]], color=:purple, lw=2, label="Bootstrap Mean")
plot!(p2, size=(400, 400))

display(p1)
display(p2)
```

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<tbody>
<tr>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="400" height="400" viewBox="0 0 1600 1600">
<defs>
  <clipPath id="clip030">
    <rect x="0" y="0" width="1600" height="1600"/>
  </clipPath>
</defs>
<path clip-path="url(#clip030)" d="M0 1600 L1600 1600 L1600 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip031">
    <rect x="320" y="159" width="1121" height="1121"/>
  </clipPath>
</defs>
<path clip-path="url(#clip030)" d="M248.149 1382.14 L1552.76 1382.14 L1552.76 47.2441 L248.149 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip032">
    <rect x="248" y="47" width="1306" height="1336"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="372.682,1382.14 372.682,47.2441 "/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="636.567,1382.14 636.567,47.2441 "/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="900.453,1382.14 900.453,47.2441 "/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1164.34,1382.14 1164.34,47.2441 "/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1428.22,1382.14 1428.22,47.2441 "/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.149,1382.14 1552.76,1382.14 "/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.149,1005.05 1552.76,1005.05 "/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.149,627.961 1552.76,627.961 "/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.149,250.872 1552.76,250.872 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,1382.14 1552.76,1382.14 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="372.682,1382.14 372.682,1363.24 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="636.567,1382.14 636.567,1363.24 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="900.453,1382.14 900.453,1363.24 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1164.34,1382.14 1164.34,1363.24 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1428.22,1382.14 1428.22,1363.24 "/>
<path clip-path="url(#clip030)" d="M350.564 1406.66 Q346.953 1406.66 345.124 1410.22 Q343.319 1413.76 343.319 1420.89 Q343.319 1428 345.124 1431.57 Q346.953 1435.11 350.564 1435.11 Q354.198 1435.11 356.004 1431.57 Q357.833 1428 357.833 1420.89 Q357.833 1413.76 356.004 1410.22 Q354.198 1406.66 350.564 1406.66 M350.564 1402.95 Q356.374 1402.95 359.43 1407.56 Q362.509 1412.14 362.509 1420.89 Q362.509 1429.62 359.43 1434.23 Q356.374 1438.81 350.564 1438.81 Q344.754 1438.81 341.675 1434.23 Q338.62 1429.62 338.62 1420.89 Q338.62 1412.14 341.675 1407.56 Q344.754 1402.95 350.564 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M370.726 1432.26 L375.61 1432.26 L375.61 1438.14 L370.726 1438.14 L370.726 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M385.842 1403.58 L404.198 1403.58 L404.198 1407.52 L390.124 1407.52 L390.124 1415.99 Q391.143 1415.64 392.161 1415.48 Q393.18 1415.29 394.198 1415.29 Q399.985 1415.29 403.365 1418.46 Q406.744 1421.64 406.744 1427.05 Q406.744 1432.63 403.272 1435.73 Q399.8 1438.81 393.481 1438.81 Q391.305 1438.81 389.036 1438.44 Q386.791 1438.07 384.383 1437.33 L384.383 1432.63 Q386.467 1433.76 388.689 1434.32 Q390.911 1434.88 393.388 1434.88 Q397.393 1434.88 399.731 1432.77 Q402.069 1430.66 402.069 1427.05 Q402.069 1423.44 399.731 1421.33 Q397.393 1419.23 393.388 1419.23 Q391.513 1419.23 389.638 1419.64 Q387.786 1420.06 385.842 1420.94 L385.842 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M603.72 1434.2 L611.359 1434.2 L611.359 1407.84 L603.049 1409.51 L603.049 1405.25 L611.313 1403.58 L615.989 1403.58 L615.989 1434.2 L623.628 1434.2 L623.628 1438.14 L603.72 1438.14 L603.72 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M633.072 1432.26 L637.956 1432.26 L637.956 1438.14 L633.072 1438.14 L633.072 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M658.141 1406.66 Q654.53 1406.66 652.702 1410.22 Q650.896 1413.76 650.896 1420.89 Q650.896 1428 652.702 1431.57 Q654.53 1435.11 658.141 1435.11 Q661.776 1435.11 663.581 1431.57 Q665.41 1428 665.41 1420.89 Q665.41 1413.76 663.581 1410.22 Q661.776 1406.66 658.141 1406.66 M658.141 1402.95 Q663.951 1402.95 667.007 1407.56 Q670.086 1412.14 670.086 1420.89 Q670.086 1429.62 667.007 1434.23 Q663.951 1438.81 658.141 1438.81 Q652.331 1438.81 649.252 1434.23 Q646.197 1429.62 646.197 1420.89 Q646.197 1412.14 649.252 1407.56 Q652.331 1402.95 658.141 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M868.103 1434.2 L875.742 1434.2 L875.742 1407.84 L867.432 1409.51 L867.432 1405.25 L875.696 1403.58 L880.372 1403.58 L880.372 1434.2 L888.011 1434.2 L888.011 1438.14 L868.103 1438.14 L868.103 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M897.455 1432.26 L902.339 1432.26 L902.339 1438.14 L897.455 1438.14 L897.455 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M912.571 1403.58 L930.927 1403.58 L930.927 1407.52 L916.853 1407.52 L916.853 1415.99 Q917.872 1415.64 918.89 1415.48 Q919.909 1415.29 920.927 1415.29 Q926.714 1415.29 930.094 1418.46 Q933.473 1421.64 933.473 1427.05 Q933.473 1432.63 930.001 1435.73 Q926.529 1438.81 920.209 1438.81 Q918.034 1438.81 915.765 1438.44 Q913.52 1438.07 911.112 1437.33 L911.112 1432.63 Q913.196 1433.76 915.418 1434.32 Q917.64 1434.88 920.117 1434.88 Q924.121 1434.88 926.459 1432.77 Q928.797 1430.66 928.797 1427.05 Q928.797 1423.44 926.459 1421.33 Q924.121 1419.23 920.117 1419.23 Q918.242 1419.23 916.367 1419.64 Q914.515 1420.06 912.571 1420.94 L912.571 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1135.58 1434.2 L1151.9 1434.2 L1151.9 1438.14 L1129.95 1438.14 L1129.95 1434.2 Q1132.61 1431.45 1137.2 1426.82 Q1141.8 1422.17 1142.98 1420.83 Q1145.23 1418.3 1146.11 1416.57 Q1147.01 1414.81 1147.01 1413.12 Q1147.01 1410.36 1145.07 1408.63 Q1143.15 1406.89 1140.04 1406.89 Q1137.84 1406.89 1135.39 1407.65 Q1132.96 1408.42 1130.18 1409.97 L1130.18 1405.25 Q1133.01 1404.11 1135.46 1403.53 Q1137.91 1402.95 1139.95 1402.95 Q1145.32 1402.95 1148.52 1405.64 Q1151.71 1408.33 1151.71 1412.82 Q1151.71 1414.95 1150.9 1416.87 Q1150.11 1418.76 1148.01 1421.36 Q1147.43 1422.03 1144.33 1425.25 Q1141.22 1428.44 1135.58 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1161.71 1432.26 L1166.59 1432.26 L1166.59 1438.14 L1161.71 1438.14 L1161.71 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1186.78 1406.66 Q1183.17 1406.66 1181.34 1410.22 Q1179.53 1413.76 1179.53 1420.89 Q1179.53 1428 1181.34 1431.57 Q1183.17 1435.11 1186.78 1435.11 Q1190.41 1435.11 1192.22 1431.57 Q1194.05 1428 1194.05 1420.89 Q1194.05 1413.76 1192.22 1410.22 Q1190.41 1406.66 1186.78 1406.66 M1186.78 1402.95 Q1192.59 1402.95 1195.65 1407.56 Q1198.72 1412.14 1198.72 1420.89 Q1198.72 1429.62 1195.65 1434.23 Q1192.59 1438.81 1186.78 1438.81 Q1180.97 1438.81 1177.89 1434.23 Q1174.84 1429.62 1174.84 1420.89 Q1174.84 1412.14 1177.89 1407.56 Q1180.97 1402.95 1186.78 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1399.96 1434.2 L1416.28 1434.2 L1416.28 1438.14 L1394.33 1438.14 L1394.33 1434.2 Q1397 1431.45 1401.58 1426.82 Q1406.19 1422.17 1407.37 1420.83 Q1409.61 1418.3 1410.49 1416.57 Q1411.39 1414.81 1411.39 1413.12 Q1411.39 1410.36 1409.45 1408.63 Q1407.53 1406.89 1404.43 1406.89 Q1402.23 1406.89 1399.77 1407.65 Q1397.34 1408.42 1394.57 1409.97 L1394.57 1405.25 Q1397.39 1404.11 1399.84 1403.53 Q1402.3 1402.95 1404.33 1402.95 Q1409.7 1402.95 1412.9 1405.64 Q1416.09 1408.33 1416.09 1412.82 Q1416.09 1414.95 1415.28 1416.87 Q1414.5 1418.76 1412.39 1421.36 Q1411.81 1422.03 1408.71 1425.25 Q1405.61 1428.44 1399.96 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1426.09 1432.26 L1430.98 1432.26 L1430.98 1438.14 L1426.09 1438.14 L1426.09 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1441.21 1403.58 L1459.57 1403.58 L1459.57 1407.52 L1445.49 1407.52 L1445.49 1415.99 Q1446.51 1415.64 1447.53 1415.48 Q1448.55 1415.29 1449.57 1415.29 Q1455.35 1415.29 1458.73 1418.46 Q1462.11 1421.64 1462.11 1427.05 Q1462.11 1432.63 1458.64 1435.73 Q1455.17 1438.81 1448.85 1438.81 Q1446.67 1438.81 1444.4 1438.44 Q1442.16 1438.07 1439.75 1437.33 L1439.75 1432.63 Q1441.83 1433.76 1444.06 1434.32 Q1446.28 1434.88 1448.76 1434.88 Q1452.76 1434.88 1455.1 1432.77 Q1457.44 1430.66 1457.44 1427.05 Q1457.44 1423.44 1455.1 1421.33 Q1452.76 1419.23 1448.76 1419.23 Q1446.88 1419.23 1445.01 1419.64 Q1443.15 1420.06 1441.21 1420.94 L1441.21 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M903.826 1497.08 Q896.729 1497.08 893.991 1498.71 Q891.254 1500.33 891.254 1504.25 Q891.254 1507.36 893.291 1509.21 Q895.36 1511.03 898.893 1511.03 Q903.763 1511.03 906.691 1507.59 Q909.651 1504.12 909.651 1498.39 L909.651 1497.08 L903.826 1497.08 M915.508 1494.67 L915.508 1515 L909.651 1515 L909.651 1509.59 Q907.646 1512.84 904.654 1514.4 Q901.662 1515.93 897.333 1515.93 Q891.859 1515.93 888.612 1512.87 Q885.398 1509.78 885.398 1504.63 Q885.398 1498.61 889.408 1495.56 Q893.45 1492.5 901.439 1492.5 L909.651 1492.5 L909.651 1491.93 Q909.651 1487.89 906.977 1485.69 Q904.336 1483.46 899.53 1483.46 Q896.474 1483.46 893.578 1484.19 Q890.681 1484.93 888.008 1486.39 L888.008 1480.98 Q891.222 1479.74 894.246 1479.13 Q897.27 1478.5 900.134 1478.5 Q907.869 1478.5 911.688 1482.51 Q915.508 1486.52 915.508 1494.67 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,1382.14 248.149,47.2441 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,1382.14 267.047,1382.14 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,1005.05 267.047,1005.05 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,627.961 267.047,627.961 "/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,250.872 267.047,250.872 "/>
<path clip-path="url(#clip030)" d="M212.205 1367.94 Q208.594 1367.94 206.765 1371.5 Q204.96 1375.04 204.96 1382.17 Q204.96 1389.28 206.765 1392.85 Q208.594 1396.39 212.205 1396.39 Q215.839 1396.39 217.645 1392.85 Q219.473 1389.28 219.473 1382.17 Q219.473 1375.04 217.645 1371.5 Q215.839 1367.94 212.205 1367.94 M212.205 1364.23 Q218.015 1364.23 221.071 1368.84 Q224.149 1373.42 224.149 1382.17 Q224.149 1390.9 221.071 1395.51 Q218.015 1400.09 212.205 1400.09 Q206.395 1400.09 203.316 1395.51 Q200.261 1390.9 200.261 1382.17 Q200.261 1373.42 203.316 1368.84 Q206.395 1364.23 212.205 1364.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M172.089 987.771 L190.446 987.771 L190.446 991.706 L176.372 991.706 L176.372 1000.18 Q177.39 999.831 178.409 999.669 Q179.427 999.484 180.446 999.484 Q186.233 999.484 189.612 1002.65 Q192.992 1005.83 192.992 1011.24 Q192.992 1016.82 189.52 1019.92 Q186.048 1023 179.728 1023 Q177.552 1023 175.284 1022.63 Q173.039 1022.26 170.631 1021.52 L170.631 1016.82 Q172.714 1017.96 174.937 1018.51 Q177.159 1019.07 179.636 1019.07 Q183.64 1019.07 185.978 1016.96 Q188.316 1014.85 188.316 1011.24 Q188.316 1007.63 185.978 1005.53 Q183.64 1003.42 179.636 1003.42 Q177.761 1003.42 175.886 1003.84 Q174.034 1004.25 172.089 1005.13 L172.089 987.771 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M212.205 990.849 Q208.594 990.849 206.765 994.414 Q204.96 997.956 204.96 1005.09 Q204.96 1012.19 206.765 1015.76 Q208.594 1019.3 212.205 1019.3 Q215.839 1019.3 217.645 1015.76 Q219.473 1012.19 219.473 1005.09 Q219.473 997.956 217.645 994.414 Q215.839 990.849 212.205 990.849 M212.205 987.146 Q218.015 987.146 221.071 991.752 Q224.149 996.335 224.149 1005.09 Q224.149 1013.81 221.071 1018.42 Q218.015 1023 212.205 1023 Q206.395 1023 203.316 1018.42 Q200.261 1013.81 200.261 1005.09 Q200.261 996.335 203.316 991.752 Q206.395 987.146 212.205 987.146 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M142.691 641.306 L150.33 641.306 L150.33 614.941 L142.02 616.607 L142.02 612.348 L150.284 610.681 L154.96 610.681 L154.96 641.306 L162.599 641.306 L162.599 645.241 L142.691 645.241 L142.691 641.306 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M182.043 613.76 Q178.432 613.76 176.603 617.325 Q174.798 620.867 174.798 627.996 Q174.798 635.103 176.603 638.667 Q178.432 642.209 182.043 642.209 Q185.677 642.209 187.483 638.667 Q189.312 635.103 189.312 627.996 Q189.312 620.867 187.483 617.325 Q185.677 613.76 182.043 613.76 M182.043 610.056 Q187.853 610.056 190.909 614.663 Q193.987 619.246 193.987 627.996 Q193.987 636.723 190.909 641.329 Q187.853 645.913 182.043 645.913 Q176.233 645.913 173.154 641.329 Q170.099 636.723 170.099 627.996 Q170.099 619.246 173.154 614.663 Q176.233 610.056 182.043 610.056 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M212.205 613.76 Q208.594 613.76 206.765 617.325 Q204.96 620.867 204.96 627.996 Q204.96 635.103 206.765 638.667 Q208.594 642.209 212.205 642.209 Q215.839 642.209 217.645 638.667 Q219.473 635.103 219.473 627.996 Q219.473 620.867 217.645 617.325 Q215.839 613.76 212.205 613.76 M212.205 610.056 Q218.015 610.056 221.071 614.663 Q224.149 619.246 224.149 627.996 Q224.149 636.723 221.071 641.329 Q218.015 645.913 212.205 645.913 Q206.395 645.913 203.316 641.329 Q200.261 636.723 200.261 627.996 Q200.261 619.246 203.316 614.663 Q206.395 610.056 212.205 610.056 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M142.691 264.217 L150.33 264.217 L150.33 237.851 L142.02 239.518 L142.02 235.259 L150.284 233.592 L154.96 233.592 L154.96 264.217 L162.599 264.217 L162.599 268.152 L142.691 268.152 L142.691 264.217 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M172.089 233.592 L190.446 233.592 L190.446 237.527 L176.372 237.527 L176.372 246 Q177.39 245.652 178.409 245.49 Q179.427 245.305 180.446 245.305 Q186.233 245.305 189.612 248.476 Q192.992 251.648 192.992 257.064 Q192.992 262.643 189.52 265.745 Q186.048 268.824 179.728 268.824 Q177.552 268.824 175.284 268.453 Q173.039 268.083 170.631 267.342 L170.631 262.643 Q172.714 263.777 174.937 264.333 Q177.159 264.888 179.636 264.888 Q183.64 264.888 185.978 262.782 Q188.316 260.675 188.316 257.064 Q188.316 253.453 185.978 251.347 Q183.64 249.24 179.636 249.24 Q177.761 249.24 175.886 249.657 Q174.034 250.074 172.089 250.953 L172.089 233.592 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M212.205 236.671 Q208.594 236.671 206.765 240.236 Q204.96 243.777 204.96 250.907 Q204.96 258.013 206.765 261.578 Q208.594 265.12 212.205 265.12 Q215.839 265.12 217.645 261.578 Q219.473 258.013 219.473 250.907 Q219.473 243.777 217.645 240.236 Q215.839 236.671 212.205 236.671 M212.205 232.967 Q218.015 232.967 221.071 237.574 Q224.149 242.157 224.149 250.907 Q224.149 259.634 221.071 264.24 Q218.015 268.824 212.205 268.824 Q206.395 268.824 203.316 264.24 Q200.261 259.634 200.261 250.907 Q200.261 242.157 203.316 237.574 Q206.395 232.967 212.205 232.967 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M59.9044 770.551 L66.6839 770.551 Q63.6602 773.798 62.1643 777.49 Q60.6683 781.15 60.6683 785.288 Q60.6683 793.436 65.6654 797.764 Q70.6307 802.093 80.0519 802.093 Q89.4413 802.093 94.4384 797.764 Q99.4036 793.436 99.4036 785.288 Q99.4036 781.15 97.9077 777.49 Q96.4118 773.798 93.3881 770.551 L100.104 770.551 Q102.396 773.925 103.541 777.712 Q104.687 781.468 104.687 785.67 Q104.687 796.459 98.0987 802.666 Q91.4783 808.873 80.0519 808.873 Q68.5936 808.873 62.0051 802.666 Q55.3848 796.459 55.3848 785.67 Q55.3848 781.405 56.5306 777.649 Q57.6446 773.861 59.9044 770.551 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M72.2221 747.062 Q72.2221 751.772 75.9142 754.509 Q79.5745 757.247 85.972 757.247 Q92.3695 757.247 96.0617 754.541 Q99.7219 751.804 99.7219 747.062 Q99.7219 742.383 96.0298 739.646 Q92.3377 736.908 85.972 736.908 Q79.6381 736.908 75.946 739.646 Q72.2221 742.383 72.2221 747.062 M67.2568 747.062 Q67.2568 739.423 72.2221 735.062 Q77.1873 730.702 85.972 730.702 Q94.7249 730.702 99.7219 735.062 Q104.687 739.423 104.687 747.062 Q104.687 754.732 99.7219 759.093 Q94.7249 763.421 85.972 763.421 Q77.1873 763.421 72.2221 759.093 Q67.2568 754.732 67.2568 747.062 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M89.6959 721.599 L68.1162 721.599 L68.1162 715.742 L89.4731 715.742 Q94.5339 715.742 97.0802 713.769 Q99.5946 711.796 99.5946 707.849 Q99.5946 703.106 96.5709 700.369 Q93.5472 697.6 88.3273 697.6 L68.1162 697.6 L68.1162 691.744 L103.764 691.744 L103.764 697.6 L98.2896 697.6 Q101.536 699.733 103.128 702.565 Q104.687 705.366 104.687 709.09 Q104.687 715.233 100.868 718.416 Q97.0483 721.599 89.6959 721.599 M67.2568 706.862 L67.2568 706.862 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M82.2481 650.048 L103.764 650.048 L103.764 655.905 L82.439 655.905 Q77.3783 655.905 74.8638 657.878 Q72.3494 659.851 72.3494 663.798 Q72.3494 668.541 75.3731 671.278 Q78.3968 674.015 83.6167 674.015 L103.764 674.015 L103.764 679.903 L68.1162 679.903 L68.1162 674.015 L73.6544 674.015 Q70.4397 671.914 68.8483 669.082 Q67.2568 666.217 67.2568 662.493 Q67.2568 656.35 71.0763 653.199 Q74.8638 650.048 82.2481 650.048 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M57.9947 632.574 L68.1162 632.574 L68.1162 620.511 L72.6677 620.511 L72.6677 632.574 L92.0194 632.574 Q96.3799 632.574 97.6213 631.397 Q98.8626 630.187 98.8626 626.527 L98.8626 620.511 L103.764 620.511 L103.764 626.527 Q103.764 633.306 101.25 635.885 Q98.7034 638.463 92.0194 638.463 L72.6677 638.463 L72.6677 642.759 L68.1162 642.759 L68.1162 638.463 L57.9947 638.463 L57.9947 632.574 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip032)" d="M319.905 1374.6 L319.905 1382.14 L372.682 1382.14 L372.682 1374.6 L319.905 1374.6 L319.905 1374.6  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="319.905,1374.6 319.905,1382.14 372.682,1382.14 372.682,1374.6 319.905,1374.6 "/>
<path clip-path="url(#clip032)" d="M372.682 1367.06 L372.682 1382.14 L425.459 1382.14 L425.459 1367.06 L372.682 1367.06 L372.682 1367.06  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="372.682,1367.06 372.682,1382.14 425.459,1382.14 425.459,1367.06 372.682,1367.06 "/>
<path clip-path="url(#clip032)" d="M425.459 1344.43 L425.459 1382.14 L478.236 1382.14 L478.236 1344.43 L425.459 1344.43 L425.459 1344.43  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="425.459,1344.43 425.459,1382.14 478.236,1382.14 478.236,1344.43 425.459,1344.43 "/>
<path clip-path="url(#clip032)" d="M478.236 1155.89 L478.236 1382.14 L531.013 1382.14 L531.013 1155.89 L478.236 1155.89 L478.236 1155.89  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="478.236,1155.89 478.236,1382.14 531.013,1382.14 531.013,1155.89 478.236,1155.89 "/>
<path clip-path="url(#clip032)" d="M531.013 907.007 L531.013 1382.14 L583.79 1382.14 L583.79 907.007 L531.013 907.007 L531.013 907.007  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="531.013,907.007 531.013,1382.14 583.79,1382.14 583.79,907.007 531.013,907.007 "/>
<path clip-path="url(#clip032)" d="M583.79 469.584 L583.79 1382.14 L636.567 1382.14 L636.567 469.584 L583.79 469.584 L583.79 469.584  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="583.79,469.584 583.79,1382.14 636.567,1382.14 636.567,469.584 583.79,469.584 "/>
<path clip-path="url(#clip032)" d="M636.567 47.2441 L636.567 1382.14 L689.344 1382.14 L689.344 47.2441 L636.567 47.2441 L636.567 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="636.567,47.2441 636.567,1382.14 689.344,1382.14 689.344,47.2441 636.567,47.2441 "/>
<path clip-path="url(#clip032)" d="M689.344 62.3277 L689.344 1382.14 L742.121 1382.14 L742.121 62.3277 L689.344 62.3277 L689.344 62.3277  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="689.344,62.3277 689.344,1382.14 742.121,1382.14 742.121,62.3277 689.344,62.3277 "/>
<path clip-path="url(#clip032)" d="M742.121 258.414 L742.121 1382.14 L794.899 1382.14 L794.899 258.414 L742.121 258.414 L742.121 258.414  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="742.121,258.414 742.121,1382.14 794.899,1382.14 794.899,258.414 742.121,258.414 "/>
<path clip-path="url(#clip032)" d="M794.899 597.794 L794.899 1382.14 L847.676 1382.14 L847.676 597.794 L794.899 597.794 L794.899 597.794  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="794.899,597.794 794.899,1382.14 847.676,1382.14 847.676,597.794 794.899,597.794 "/>
<path clip-path="url(#clip032)" d="M847.676 763.714 L847.676 1382.14 L900.453 1382.14 L900.453 763.714 L847.676 763.714 L847.676 763.714  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="847.676,763.714 847.676,1382.14 900.453,1382.14 900.453,763.714 847.676,763.714 "/>
<path clip-path="url(#clip032)" d="M900.453 1072.93 L900.453 1382.14 L953.23 1382.14 L953.23 1072.93 L900.453 1072.93 L900.453 1072.93  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="900.453,1072.93 900.453,1382.14 953.23,1382.14 953.23,1072.93 900.453,1072.93 "/>
<path clip-path="url(#clip032)" d="M953.23 1208.68 L953.23 1382.14 L1006.01 1382.14 L1006.01 1208.68 L953.23 1208.68 L953.23 1208.68  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="953.23,1208.68 953.23,1382.14 1006.01,1382.14 1006.01,1208.68 953.23,1208.68 "/>
<path clip-path="url(#clip032)" d="M1006.01 1276.55 L1006.01 1382.14 L1058.78 1382.14 L1058.78 1276.55 L1006.01 1276.55 L1006.01 1276.55  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1006.01,1276.55 1006.01,1382.14 1058.78,1382.14 1058.78,1276.55 1006.01,1276.55 "/>
<path clip-path="url(#clip032)" d="M1058.78 1329.35 L1058.78 1382.14 L1111.56 1382.14 L1111.56 1329.35 L1058.78 1329.35 L1058.78 1329.35  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1058.78,1329.35 1058.78,1382.14 1111.56,1382.14 1111.56,1329.35 1058.78,1329.35 "/>
<path clip-path="url(#clip032)" d="M1111.56 1351.97 L1111.56 1382.14 L1164.34 1382.14 L1164.34 1351.97 L1111.56 1351.97 L1111.56 1351.97  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1111.56,1351.97 1111.56,1382.14 1164.34,1382.14 1164.34,1351.97 1111.56,1351.97 "/>
<path clip-path="url(#clip032)" d="M1164.34 1374.6 L1164.34 1382.14 L1217.11 1382.14 L1217.11 1374.6 L1164.34 1374.6 L1164.34 1374.6  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1164.34,1374.6 1164.34,1382.14 1217.11,1382.14 1217.11,1374.6 1164.34,1374.6 "/>
<path clip-path="url(#clip032)" d="M1217.11 1382.14 L1217.11 1382.14 L1269.89 1382.14 L1269.89 1382.14 L1217.11 1382.14 L1217.11 1382.14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1217.11,1382.14 1217.11,1382.14 1269.89,1382.14 1217.11,1382.14 "/>
<path clip-path="url(#clip032)" d="M1269.89 1382.14 L1269.89 1382.14 L1322.67 1382.14 L1322.67 1382.14 L1269.89 1382.14 L1269.89 1382.14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1269.89,1382.14 1269.89,1382.14 1322.67,1382.14 1269.89,1382.14 "/>
<path clip-path="url(#clip032)" d="M1322.67 1382.14 L1322.67 1382.14 L1375.45 1382.14 L1375.45 1382.14 L1322.67 1382.14 L1322.67 1382.14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1322.67,1382.14 1322.67,1382.14 1375.45,1382.14 1322.67,1382.14 "/>
<path clip-path="url(#clip032)" d="M1375.45 1382.14 L1375.45 1382.14 L1428.22 1382.14 L1428.22 1382.14 L1375.45 1382.14 L1375.45 1382.14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1375.45,1382.14 1375.45,1382.14 1428.22,1382.14 1375.45,1382.14 "/>
<path clip-path="url(#clip032)" d="M1428.22 1374.6 L1428.22 1382.14 L1481 1382.14 L1481 1374.6 L1428.22 1374.6 L1428.22 1374.6  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip032)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1428.22,1374.6 1428.22,1382.14 1481,1382.14 1481,1374.6 1428.22,1374.6 "/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="346.294" cy="1374.6" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="399.071" cy="1367.06" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="451.848" cy="1344.43" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="504.625" cy="1155.89" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="557.402" cy="907.007" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="610.179" cy="469.584" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="662.956" cy="47.2441" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="715.733" cy="62.3277" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="768.51" cy="258.414" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="821.287" cy="597.794" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="874.064" cy="763.714" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="926.841" cy="1072.93" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="979.618" cy="1208.68" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1032.4" cy="1276.55" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1085.17" cy="1329.35" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1137.95" cy="1351.97" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1190.73" cy="1374.6" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1243.5" cy="1382.14" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1296.28" cy="1382.14" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1349.06" cy="1382.14" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1401.83" cy="1382.14" r="2"/>
<circle clip-path="url(#clip032)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1454.61" cy="1374.6" r="2"/>
<polyline clip-path="url(#clip032)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:8; stroke-opacity:1; fill:none" points="720.75,2717.04 720.75,-1287.65 "/>
<polyline clip-path="url(#clip032)" style="stroke:#800080; stroke-linecap:round; stroke-linejoin:round; stroke-width:8; stroke-opacity:1; fill:none" points="731.288,2717.04 731.288,-1287.65 "/>
<path clip-path="url(#clip030)" d="M1012.21 247.261 L1509.27 247.261 L1509.27 91.7406 L1012.21 91.7406  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip030)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1012.21,247.261 1509.27,247.261 1509.27,91.7406 1012.21,91.7406 1012.21,247.261 "/>
<polyline clip-path="url(#clip030)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:8; stroke-opacity:1; fill:none" points="1026.71,143.581 1113.68,143.581 "/>
<path clip-path="url(#clip030)" d="M1128.18 126.301 L1135.15 126.301 L1143.97 149.819 L1152.83 126.301 L1159.8 126.301 L1159.8 160.861 L1155.24 160.861 L1155.24 130.514 L1146.33 154.217 L1141.63 154.217 L1132.72 130.514 L1132.72 160.861 L1128.18 160.861 L1128.18 126.301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1169.08 126.301 L1173.76 126.301 L1173.76 156.925 L1190.59 156.925 L1190.59 160.861 L1169.08 160.861 L1169.08 126.301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1195.49 126.301 L1217.34 126.301 L1217.34 130.236 L1200.17 130.236 L1200.17 140.467 L1216.63 140.467 L1216.63 144.402 L1200.17 144.402 L1200.17 156.925 L1217.76 156.925 L1217.76 160.861 L1195.49 160.861 L1195.49 126.301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip030)" style="stroke:#800080; stroke-linecap:round; stroke-linejoin:round; stroke-width:8; stroke-opacity:1; fill:none" points="1026.71,195.421 1113.68,195.421 "/>
<path clip-path="url(#clip030)" d="M1132.85 196.196 L1132.85 208.858 L1140.35 208.858 Q1144.13 208.858 1145.93 207.307 Q1147.76 205.733 1147.76 202.515 Q1147.76 199.275 1145.93 197.747 Q1144.13 196.196 1140.35 196.196 L1132.85 196.196 M1132.85 181.983 L1132.85 192.4 L1139.78 192.4 Q1143.2 192.4 1144.87 191.127 Q1146.56 189.83 1146.56 187.191 Q1146.56 184.576 1144.87 183.279 Q1143.2 181.983 1139.78 181.983 L1132.85 181.983 M1128.18 178.141 L1140.12 178.141 Q1145.47 178.141 1148.36 180.363 Q1151.26 182.585 1151.26 186.682 Q1151.26 189.854 1149.78 191.729 Q1148.29 193.603 1145.42 194.066 Q1148.87 194.807 1150.77 197.168 Q1152.69 199.506 1152.69 203.025 Q1152.69 207.654 1149.54 210.177 Q1146.4 212.701 1140.59 212.701 L1128.18 212.701 L1128.18 178.141 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1170.56 189.761 Q1167.14 189.761 1165.15 192.446 Q1163.15 195.108 1163.15 199.761 Q1163.15 204.414 1165.12 207.099 Q1167.11 209.761 1170.56 209.761 Q1173.96 209.761 1175.96 207.076 Q1177.95 204.39 1177.95 199.761 Q1177.95 195.154 1175.96 192.469 Q1173.96 189.761 1170.56 189.761 M1170.56 186.15 Q1176.12 186.15 1179.29 189.761 Q1182.46 193.372 1182.46 199.761 Q1182.46 206.127 1179.29 209.761 Q1176.12 213.372 1170.56 213.372 Q1164.98 213.372 1161.81 209.761 Q1158.66 206.127 1158.66 199.761 Q1158.66 193.372 1161.81 189.761 Q1164.98 186.15 1170.56 186.15 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1199.57 189.761 Q1196.14 189.761 1194.15 192.446 Q1192.16 195.108 1192.16 199.761 Q1192.16 204.414 1194.13 207.099 Q1196.12 209.761 1199.57 209.761 Q1202.97 209.761 1204.96 207.076 Q1206.95 204.39 1206.95 199.761 Q1206.95 195.154 1204.96 192.469 Q1202.97 189.761 1199.57 189.761 M1199.57 186.15 Q1205.12 186.15 1208.29 189.761 Q1211.46 193.372 1211.46 199.761 Q1211.46 206.127 1208.29 209.761 Q1205.12 213.372 1199.57 213.372 Q1193.99 213.372 1190.82 209.761 Q1187.67 206.127 1187.67 199.761 Q1187.67 193.372 1190.82 189.761 Q1193.99 186.15 1199.57 186.15 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1222.74 179.414 L1222.74 186.775 L1231.51 186.775 L1231.51 190.085 L1222.74 190.085 L1222.74 204.159 Q1222.74 207.33 1223.59 208.233 Q1224.47 209.136 1227.14 209.136 L1231.51 209.136 L1231.51 212.701 L1227.14 212.701 Q1222.21 212.701 1220.33 210.872 Q1218.46 209.02 1218.46 204.159 L1218.46 190.085 L1215.33 190.085 L1215.33 186.775 L1218.46 186.775 L1218.46 179.414 L1222.74 179.414 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1253.64 187.539 L1253.64 191.566 Q1251.83 190.641 1249.89 190.178 Q1247.95 189.715 1245.86 189.715 Q1242.69 189.715 1241.09 190.687 Q1239.52 191.659 1239.52 193.603 Q1239.52 195.085 1240.65 195.941 Q1241.79 196.775 1245.21 197.539 L1246.67 197.863 Q1251.21 198.835 1253.11 200.617 Q1255.03 202.377 1255.03 205.548 Q1255.03 209.159 1252.16 211.265 Q1249.31 213.372 1244.31 213.372 Q1242.23 213.372 1239.96 212.955 Q1237.71 212.562 1235.21 211.752 L1235.21 207.353 Q1237.58 208.58 1239.87 209.205 Q1242.16 209.807 1244.4 209.807 Q1247.41 209.807 1249.03 208.789 Q1250.65 207.747 1250.65 205.872 Q1250.65 204.136 1249.47 203.21 Q1248.32 202.284 1244.36 201.428 L1242.88 201.08 Q1238.92 200.247 1237.16 198.534 Q1235.4 196.798 1235.4 193.789 Q1235.4 190.131 1237.99 188.141 Q1240.58 186.15 1245.35 186.15 Q1247.71 186.15 1249.8 186.497 Q1251.88 186.844 1253.64 187.539 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1266.02 179.414 L1266.02 186.775 L1274.8 186.775 L1274.8 190.085 L1266.02 190.085 L1266.02 204.159 Q1266.02 207.33 1266.88 208.233 Q1267.76 209.136 1270.42 209.136 L1274.8 209.136 L1274.8 212.701 L1270.42 212.701 Q1265.49 212.701 1263.62 210.872 Q1261.74 209.02 1261.74 204.159 L1261.74 190.085 L1258.62 190.085 L1258.62 186.775 L1261.74 186.775 L1261.74 179.414 L1266.02 179.414 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1295.42 190.756 Q1294.71 190.34 1293.85 190.154 Q1293.02 189.946 1292 189.946 Q1288.39 189.946 1286.44 192.307 Q1284.52 194.645 1284.52 199.043 L1284.52 212.701 L1280.24 212.701 L1280.24 186.775 L1284.52 186.775 L1284.52 190.803 Q1285.86 188.441 1288.02 187.307 Q1290.17 186.15 1293.25 186.15 Q1293.69 186.15 1294.22 186.219 Q1294.75 186.266 1295.4 186.381 L1295.42 190.756 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1311.67 199.668 Q1306.51 199.668 1304.52 200.849 Q1302.53 202.029 1302.53 204.877 Q1302.53 207.145 1304.01 208.488 Q1305.52 209.807 1308.08 209.807 Q1311.63 209.807 1313.76 207.307 Q1315.91 204.784 1315.91 200.617 L1315.91 199.668 L1311.67 199.668 M1320.17 197.909 L1320.17 212.701 L1315.91 212.701 L1315.91 208.765 Q1314.45 211.127 1312.27 212.261 Q1310.1 213.372 1306.95 213.372 Q1302.97 213.372 1300.61 211.15 Q1298.27 208.904 1298.27 205.154 Q1298.27 200.779 1301.19 198.557 Q1304.13 196.335 1309.94 196.335 L1315.91 196.335 L1315.91 195.918 Q1315.91 192.979 1313.96 191.381 Q1312.04 189.761 1308.55 189.761 Q1306.33 189.761 1304.22 190.293 Q1302.11 190.826 1300.17 191.891 L1300.17 187.955 Q1302.51 187.053 1304.7 186.613 Q1306.9 186.15 1308.99 186.15 Q1314.61 186.15 1317.39 189.066 Q1320.17 191.983 1320.17 197.909 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1333.06 208.812 L1333.06 222.562 L1328.78 222.562 L1328.78 186.775 L1333.06 186.775 L1333.06 190.71 Q1334.4 188.395 1336.44 187.284 Q1338.5 186.15 1341.35 186.15 Q1346.07 186.15 1349.01 189.9 Q1351.97 193.65 1351.97 199.761 Q1351.97 205.872 1349.01 209.622 Q1346.07 213.372 1341.35 213.372 Q1338.5 213.372 1336.44 212.261 Q1334.4 211.127 1333.06 208.812 M1347.55 199.761 Q1347.55 195.062 1345.61 192.4 Q1343.69 189.715 1340.31 189.715 Q1336.93 189.715 1334.98 192.4 Q1333.06 195.062 1333.06 199.761 Q1333.06 204.46 1334.98 207.145 Q1336.93 209.807 1340.31 209.807 Q1343.69 209.807 1345.61 207.145 Q1347.55 204.46 1347.55 199.761 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1374.29 178.141 L1381.26 178.141 L1390.07 201.659 L1398.94 178.141 L1405.91 178.141 L1405.91 212.701 L1401.35 212.701 L1401.35 182.354 L1392.44 206.057 L1387.74 206.057 L1378.82 182.354 L1378.82 212.701 L1374.29 212.701 L1374.29 178.141 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1437.18 198.673 L1437.18 200.756 L1417.6 200.756 Q1417.88 205.154 1420.24 207.469 Q1422.62 209.761 1426.86 209.761 Q1429.31 209.761 1431.6 209.159 Q1433.92 208.557 1436.19 207.353 L1436.19 211.381 Q1433.89 212.353 1431.49 212.863 Q1429.08 213.372 1426.6 213.372 Q1420.4 213.372 1416.76 209.761 Q1413.15 206.15 1413.15 199.992 Q1413.15 193.627 1416.58 189.9 Q1420.03 186.15 1425.86 186.15 Q1431.09 186.15 1434.13 189.529 Q1437.18 192.886 1437.18 198.673 M1432.92 197.423 Q1432.88 193.928 1430.95 191.844 Q1429.06 189.761 1425.91 189.761 Q1422.34 189.761 1420.19 191.775 Q1418.06 193.789 1417.74 197.446 L1432.92 197.423 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1455.95 199.668 Q1450.79 199.668 1448.8 200.849 Q1446.81 202.029 1446.81 204.877 Q1446.81 207.145 1448.29 208.488 Q1449.8 209.807 1452.37 209.807 Q1455.91 209.807 1458.04 207.307 Q1460.19 204.784 1460.19 200.617 L1460.19 199.668 L1455.95 199.668 M1464.45 197.909 L1464.45 212.701 L1460.19 212.701 L1460.19 208.765 Q1458.73 211.127 1456.56 212.261 Q1454.38 213.372 1451.23 213.372 Q1447.25 213.372 1444.89 211.15 Q1442.55 208.904 1442.55 205.154 Q1442.55 200.779 1445.47 198.557 Q1448.41 196.335 1454.22 196.335 L1460.19 196.335 L1460.19 195.918 Q1460.19 192.979 1458.25 191.381 Q1456.32 189.761 1452.83 189.761 Q1450.61 189.761 1448.5 190.293 Q1446.39 190.826 1444.45 191.891 L1444.45 187.955 Q1446.79 187.053 1448.99 186.613 Q1451.19 186.15 1453.27 186.15 Q1458.89 186.15 1461.67 189.066 Q1464.45 191.983 1464.45 197.909 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip030)" d="M1494.77 197.053 L1494.77 212.701 L1490.51 212.701 L1490.51 197.191 Q1490.51 193.511 1489.08 191.682 Q1487.64 189.854 1484.77 189.854 Q1481.32 189.854 1479.33 192.053 Q1477.34 194.252 1477.34 198.048 L1477.34 212.701 L1473.06 212.701 L1473.06 186.775 L1477.34 186.775 L1477.34 190.803 Q1478.87 188.465 1480.93 187.307 Q1483.01 186.15 1485.72 186.15 Q1490.19 186.15 1492.48 188.928 Q1494.77 191.682 1494.77 197.053 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /></svg>
<p>(a) Bootstrapped estimates of parameter values for Problem 1. The red
line is the MLE for the given parameter and the purple line is the
bootstrap mean.</p>
</div>
</div></td>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="400" height="400" viewBox="0 0 1600 1600">
<defs>
  <clipPath id="clip090">
    <rect x="0" y="0" width="1600" height="1600"/>
  </clipPath>
</defs>
<path clip-path="url(#clip090)" d="M0 1600 L1600 1600 L1600 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip091">
    <rect x="320" y="159" width="1121" height="1121"/>
  </clipPath>
</defs>
<path clip-path="url(#clip090)" d="M248.149 1382.14 L1552.76 1382.14 L1552.76 47.2441 L248.149 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip092">
    <rect x="248" y="47" width="1306" height="1336"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="264.615,1382.14 264.615,47.2441 "/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="541.066,1382.14 541.066,47.2441 "/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="817.517,1382.14 817.517,47.2441 "/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1093.97,1382.14 1093.97,47.2441 "/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1370.42,1382.14 1370.42,47.2441 "/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.149,1382.14 1552.76,1382.14 "/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.149,977.626 1552.76,977.626 "/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.149,573.112 1552.76,573.112 "/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.149,168.598 1552.76,168.598 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,1382.14 1552.76,1382.14 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="264.615,1382.14 264.615,1363.24 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="541.066,1382.14 541.066,1363.24 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="817.517,1382.14 817.517,1363.24 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1093.97,1382.14 1093.97,1363.24 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1370.42,1382.14 1370.42,1363.24 "/>
<path clip-path="url(#clip090)" d="M211.143 1421.31 L240.819 1421.31 L240.819 1425.25 L211.143 1425.25 L211.143 1421.31 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M251.721 1434.2 L259.36 1434.2 L259.36 1407.84 L251.05 1409.51 L251.05 1405.25 L259.314 1403.58 L263.99 1403.58 L263.99 1434.2 L271.629 1434.2 L271.629 1438.14 L251.721 1438.14 L251.721 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M281.073 1432.26 L285.957 1432.26 L285.957 1438.14 L281.073 1438.14 L281.073 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M306.142 1406.66 Q302.531 1406.66 300.703 1410.22 Q298.897 1413.76 298.897 1420.89 Q298.897 1428 300.703 1431.57 Q302.531 1435.11 306.142 1435.11 Q309.777 1435.11 311.582 1431.57 Q313.411 1428 313.411 1420.89 Q313.411 1413.76 311.582 1410.22 Q309.777 1406.66 306.142 1406.66 M306.142 1402.95 Q311.952 1402.95 315.008 1407.56 Q318.087 1412.14 318.087 1420.89 Q318.087 1429.62 315.008 1434.23 Q311.952 1438.81 306.142 1438.81 Q300.332 1438.81 297.253 1434.23 Q294.198 1429.62 294.198 1420.89 Q294.198 1412.14 297.253 1407.56 Q300.332 1402.95 306.142 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M488.092 1421.31 L517.768 1421.31 L517.768 1425.25 L488.092 1425.25 L488.092 1421.31 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M537.86 1406.66 Q534.249 1406.66 532.42 1410.22 Q530.615 1413.76 530.615 1420.89 Q530.615 1428 532.42 1431.57 Q534.249 1435.11 537.86 1435.11 Q541.494 1435.11 543.3 1431.57 Q545.128 1428 545.128 1420.89 Q545.128 1413.76 543.3 1410.22 Q541.494 1406.66 537.86 1406.66 M537.86 1402.95 Q543.67 1402.95 546.726 1407.56 Q549.804 1412.14 549.804 1420.89 Q549.804 1429.62 546.726 1434.23 Q543.67 1438.81 537.86 1438.81 Q532.05 1438.81 528.971 1434.23 Q525.916 1429.62 525.916 1420.89 Q525.916 1412.14 528.971 1407.56 Q532.05 1402.95 537.86 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M558.022 1432.26 L562.906 1432.26 L562.906 1438.14 L558.022 1438.14 L558.022 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M573.138 1403.58 L591.494 1403.58 L591.494 1407.52 L577.42 1407.52 L577.42 1415.99 Q578.439 1415.64 579.457 1415.48 Q580.476 1415.29 581.494 1415.29 Q587.281 1415.29 590.661 1418.46 Q594.04 1421.64 594.04 1427.05 Q594.04 1432.63 590.568 1435.73 Q587.096 1438.81 580.776 1438.81 Q578.601 1438.81 576.332 1438.44 Q574.087 1438.07 571.679 1437.33 L571.679 1432.63 Q573.763 1433.76 575.985 1434.32 Q578.207 1434.88 580.684 1434.88 Q584.688 1434.88 587.026 1432.77 Q589.364 1430.66 589.364 1427.05 Q589.364 1423.44 587.026 1421.33 Q584.688 1419.23 580.684 1419.23 Q578.809 1419.23 576.934 1419.64 Q575.082 1420.06 573.138 1420.94 L573.138 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M794.902 1406.66 Q791.291 1406.66 789.462 1410.22 Q787.656 1413.76 787.656 1420.89 Q787.656 1428 789.462 1431.57 Q791.291 1435.11 794.902 1435.11 Q798.536 1435.11 800.341 1431.57 Q802.17 1428 802.17 1420.89 Q802.17 1413.76 800.341 1410.22 Q798.536 1406.66 794.902 1406.66 M794.902 1402.95 Q800.712 1402.95 803.767 1407.56 Q806.846 1412.14 806.846 1420.89 Q806.846 1429.62 803.767 1434.23 Q800.712 1438.81 794.902 1438.81 Q789.091 1438.81 786.013 1434.23 Q782.957 1429.62 782.957 1420.89 Q782.957 1412.14 786.013 1407.56 Q789.091 1402.95 794.902 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M815.064 1432.26 L819.948 1432.26 L819.948 1438.14 L815.064 1438.14 L815.064 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M840.133 1406.66 Q836.522 1406.66 834.693 1410.22 Q832.888 1413.76 832.888 1420.89 Q832.888 1428 834.693 1431.57 Q836.522 1435.11 840.133 1435.11 Q843.767 1435.11 845.573 1431.57 Q847.401 1428 847.401 1420.89 Q847.401 1413.76 845.573 1410.22 Q843.767 1406.66 840.133 1406.66 M840.133 1402.95 Q845.943 1402.95 848.999 1407.56 Q852.077 1412.14 852.077 1420.89 Q852.077 1429.62 848.999 1434.23 Q845.943 1438.81 840.133 1438.81 Q834.323 1438.81 831.244 1434.23 Q828.188 1429.62 828.188 1420.89 Q828.188 1412.14 831.244 1407.56 Q834.323 1402.95 840.133 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1071.85 1406.66 Q1068.24 1406.66 1066.41 1410.22 Q1064.61 1413.76 1064.61 1420.89 Q1064.61 1428 1066.41 1431.57 Q1068.24 1435.11 1071.85 1435.11 Q1075.48 1435.11 1077.29 1431.57 Q1079.12 1428 1079.12 1420.89 Q1079.12 1413.76 1077.29 1410.22 Q1075.48 1406.66 1071.85 1406.66 M1071.85 1402.95 Q1077.66 1402.95 1080.72 1407.56 Q1083.79 1412.14 1083.79 1420.89 Q1083.79 1429.62 1080.72 1434.23 Q1077.66 1438.81 1071.85 1438.81 Q1066.04 1438.81 1062.96 1434.23 Q1059.91 1429.62 1059.91 1420.89 Q1059.91 1412.14 1062.96 1407.56 Q1066.04 1402.95 1071.85 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1092.01 1432.26 L1096.9 1432.26 L1096.9 1438.14 L1092.01 1438.14 L1092.01 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1107.13 1403.58 L1125.48 1403.58 L1125.48 1407.52 L1111.41 1407.52 L1111.41 1415.99 Q1112.43 1415.64 1113.45 1415.48 Q1114.47 1415.29 1115.48 1415.29 Q1121.27 1415.29 1124.65 1418.46 Q1128.03 1421.64 1128.03 1427.05 Q1128.03 1432.63 1124.56 1435.73 Q1121.09 1438.81 1114.77 1438.81 Q1112.59 1438.81 1110.32 1438.44 Q1108.08 1438.07 1105.67 1437.33 L1105.67 1432.63 Q1107.75 1433.76 1109.98 1434.32 Q1112.2 1434.88 1114.67 1434.88 Q1118.68 1434.88 1121.02 1432.77 Q1123.35 1430.66 1123.35 1427.05 Q1123.35 1423.44 1121.02 1421.33 Q1118.68 1419.23 1114.67 1419.23 Q1112.8 1419.23 1110.92 1419.64 Q1109.07 1420.06 1107.13 1420.94 L1107.13 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1337.57 1434.2 L1345.21 1434.2 L1345.21 1407.84 L1336.9 1409.51 L1336.9 1405.25 L1345.17 1403.58 L1349.84 1403.58 L1349.84 1434.2 L1357.48 1434.2 L1357.48 1438.14 L1337.57 1438.14 L1337.57 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1366.92 1432.26 L1371.81 1432.26 L1371.81 1438.14 L1366.92 1438.14 L1366.92 1432.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1391.99 1406.66 Q1388.38 1406.66 1386.55 1410.22 Q1384.75 1413.76 1384.75 1420.89 Q1384.75 1428 1386.55 1431.57 Q1388.38 1435.11 1391.99 1435.11 Q1395.63 1435.11 1397.43 1431.57 Q1399.26 1428 1399.26 1420.89 Q1399.26 1413.76 1397.43 1410.22 Q1395.63 1406.66 1391.99 1406.66 M1391.99 1402.95 Q1397.8 1402.95 1400.86 1407.56 Q1403.94 1412.14 1403.94 1420.89 Q1403.94 1429.62 1400.86 1434.23 Q1397.8 1438.81 1391.99 1438.81 Q1386.18 1438.81 1383.1 1434.23 Q1380.05 1429.62 1380.05 1420.89 Q1380.05 1412.14 1383.1 1407.56 Q1386.18 1402.95 1391.99 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M910.319 1497.21 Q910.319 1490.75 907.646 1487.09 Q905.004 1483.4 900.357 1483.4 Q895.71 1483.4 893.037 1487.09 Q890.395 1490.75 890.395 1497.21 Q890.395 1503.67 893.037 1507.36 Q895.71 1511.03 900.357 1511.03 Q905.004 1511.03 907.646 1507.36 Q910.319 1503.67 910.319 1497.21 M890.395 1484.77 Q892.241 1481.58 895.042 1480.06 Q897.875 1478.5 901.789 1478.5 Q908.282 1478.5 912.325 1483.65 Q916.399 1488.81 916.399 1497.21 Q916.399 1505.61 912.325 1510.77 Q908.282 1515.93 901.789 1515.93 Q897.875 1515.93 895.042 1514.4 Q892.241 1512.84 890.395 1509.66 L890.395 1515 L884.507 1515 L884.507 1465.48 L890.395 1465.48 L890.395 1484.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,1382.14 248.149,47.2441 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,1382.14 267.047,1382.14 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,977.626 267.047,977.626 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,573.112 267.047,573.112 "/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.149,168.598 267.047,168.598 "/>
<path clip-path="url(#clip090)" d="M212.205 1367.94 Q208.594 1367.94 206.765 1371.5 Q204.96 1375.04 204.96 1382.17 Q204.96 1389.28 206.765 1392.85 Q208.594 1396.39 212.205 1396.39 Q215.839 1396.39 217.645 1392.85 Q219.473 1389.28 219.473 1382.17 Q219.473 1375.04 217.645 1371.5 Q215.839 1367.94 212.205 1367.94 M212.205 1364.23 Q218.015 1364.23 221.071 1368.84 Q224.149 1373.42 224.149 1382.17 Q224.149 1390.9 221.071 1395.51 Q218.015 1400.09 212.205 1400.09 Q206.395 1400.09 203.316 1395.51 Q200.261 1390.9 200.261 1382.17 Q200.261 1373.42 203.316 1368.84 Q206.395 1364.23 212.205 1364.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M172.089 960.346 L190.446 960.346 L190.446 964.281 L176.372 964.281 L176.372 972.753 Q177.39 972.406 178.409 972.244 Q179.427 972.059 180.446 972.059 Q186.233 972.059 189.612 975.23 Q192.992 978.401 192.992 983.818 Q192.992 989.397 189.52 992.499 Q186.048 995.577 179.728 995.577 Q177.552 995.577 175.284 995.207 Q173.039 994.837 170.631 994.096 L170.631 989.397 Q172.714 990.531 174.937 991.087 Q177.159 991.642 179.636 991.642 Q183.64 991.642 185.978 989.536 Q188.316 987.429 188.316 983.818 Q188.316 980.207 185.978 978.101 Q183.64 975.994 179.636 975.994 Q177.761 975.994 175.886 976.411 Q174.034 976.827 172.089 977.707 L172.089 960.346 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M212.205 963.425 Q208.594 963.425 206.765 966.989 Q204.96 970.531 204.96 977.661 Q204.96 984.767 206.765 988.332 Q208.594 991.874 212.205 991.874 Q215.839 991.874 217.645 988.332 Q219.473 984.767 219.473 977.661 Q219.473 970.531 217.645 966.989 Q215.839 963.425 212.205 963.425 M212.205 959.721 Q218.015 959.721 221.071 964.327 Q224.149 968.911 224.149 977.661 Q224.149 986.388 221.071 990.994 Q218.015 995.577 212.205 995.577 Q206.395 995.577 203.316 990.994 Q200.261 986.388 200.261 977.661 Q200.261 968.911 203.316 964.327 Q206.395 959.721 212.205 959.721 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M142.691 586.457 L150.33 586.457 L150.33 560.091 L142.02 561.758 L142.02 557.499 L150.284 555.832 L154.96 555.832 L154.96 586.457 L162.599 586.457 L162.599 590.392 L142.691 590.392 L142.691 586.457 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M182.043 558.911 Q178.432 558.911 176.603 562.476 Q174.798 566.017 174.798 573.147 Q174.798 580.253 176.603 583.818 Q178.432 587.36 182.043 587.36 Q185.677 587.36 187.483 583.818 Q189.312 580.253 189.312 573.147 Q189.312 566.017 187.483 562.476 Q185.677 558.911 182.043 558.911 M182.043 555.207 Q187.853 555.207 190.909 559.814 Q193.987 564.397 193.987 573.147 Q193.987 581.874 190.909 586.48 Q187.853 591.063 182.043 591.063 Q176.233 591.063 173.154 586.48 Q170.099 581.874 170.099 573.147 Q170.099 564.397 173.154 559.814 Q176.233 555.207 182.043 555.207 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M212.205 558.911 Q208.594 558.911 206.765 562.476 Q204.96 566.017 204.96 573.147 Q204.96 580.253 206.765 583.818 Q208.594 587.36 212.205 587.36 Q215.839 587.36 217.645 583.818 Q219.473 580.253 219.473 573.147 Q219.473 566.017 217.645 562.476 Q215.839 558.911 212.205 558.911 M212.205 555.207 Q218.015 555.207 221.071 559.814 Q224.149 564.397 224.149 573.147 Q224.149 581.874 221.071 586.48 Q218.015 591.063 212.205 591.063 Q206.395 591.063 203.316 586.48 Q200.261 581.874 200.261 573.147 Q200.261 564.397 203.316 559.814 Q206.395 555.207 212.205 555.207 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M142.691 181.943 L150.33 181.943 L150.33 155.577 L142.02 157.244 L142.02 152.985 L150.284 151.318 L154.96 151.318 L154.96 181.943 L162.599 181.943 L162.599 185.878 L142.691 185.878 L142.691 181.943 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M172.089 151.318 L190.446 151.318 L190.446 155.253 L176.372 155.253 L176.372 163.726 Q177.39 163.378 178.409 163.216 Q179.427 163.031 180.446 163.031 Q186.233 163.031 189.612 166.202 Q192.992 169.374 192.992 174.79 Q192.992 180.369 189.52 183.471 Q186.048 186.55 179.728 186.55 Q177.552 186.55 175.284 186.179 Q173.039 185.809 170.631 185.068 L170.631 180.369 Q172.714 181.503 174.937 182.059 Q177.159 182.614 179.636 182.614 Q183.64 182.614 185.978 180.508 Q188.316 178.401 188.316 174.79 Q188.316 171.179 185.978 169.073 Q183.64 166.966 179.636 166.966 Q177.761 166.966 175.886 167.383 Q174.034 167.8 172.089 168.679 L172.089 151.318 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M212.205 154.397 Q208.594 154.397 206.765 157.962 Q204.96 161.503 204.96 168.633 Q204.96 175.739 206.765 179.304 Q208.594 182.846 212.205 182.846 Q215.839 182.846 217.645 179.304 Q219.473 175.739 219.473 168.633 Q219.473 161.503 217.645 157.962 Q215.839 154.397 212.205 154.397 M212.205 150.693 Q218.015 150.693 221.071 155.3 Q224.149 159.883 224.149 168.633 Q224.149 177.36 221.071 181.966 Q218.015 186.55 212.205 186.55 Q206.395 186.55 203.316 181.966 Q200.261 177.36 200.261 168.633 Q200.261 159.883 203.316 155.3 Q206.395 150.693 212.205 150.693 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M59.9044 770.551 L66.6839 770.551 Q63.6602 773.798 62.1643 777.49 Q60.6683 781.15 60.6683 785.288 Q60.6683 793.436 65.6654 797.764 Q70.6307 802.093 80.0519 802.093 Q89.4413 802.093 94.4384 797.764 Q99.4036 793.436 99.4036 785.288 Q99.4036 781.15 97.9077 777.49 Q96.4118 773.798 93.3881 770.551 L100.104 770.551 Q102.396 773.925 103.541 777.712 Q104.687 781.468 104.687 785.67 Q104.687 796.459 98.0987 802.666 Q91.4783 808.873 80.0519 808.873 Q68.5936 808.873 62.0051 802.666 Q55.3848 796.459 55.3848 785.67 Q55.3848 781.405 56.5306 777.649 Q57.6446 773.861 59.9044 770.551 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M72.2221 747.062 Q72.2221 751.772 75.9142 754.509 Q79.5745 757.247 85.972 757.247 Q92.3695 757.247 96.0617 754.541 Q99.7219 751.804 99.7219 747.062 Q99.7219 742.383 96.0298 739.646 Q92.3377 736.908 85.972 736.908 Q79.6381 736.908 75.946 739.646 Q72.2221 742.383 72.2221 747.062 M67.2568 747.062 Q67.2568 739.423 72.2221 735.062 Q77.1873 730.702 85.972 730.702 Q94.7249 730.702 99.7219 735.062 Q104.687 739.423 104.687 747.062 Q104.687 754.732 99.7219 759.093 Q94.7249 763.421 85.972 763.421 Q77.1873 763.421 72.2221 759.093 Q67.2568 754.732 67.2568 747.062 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M89.6959 721.599 L68.1162 721.599 L68.1162 715.742 L89.4731 715.742 Q94.5339 715.742 97.0802 713.769 Q99.5946 711.796 99.5946 707.849 Q99.5946 703.106 96.5709 700.369 Q93.5472 697.6 88.3273 697.6 L68.1162 697.6 L68.1162 691.744 L103.764 691.744 L103.764 697.6 L98.2896 697.6 Q101.536 699.733 103.128 702.565 Q104.687 705.366 104.687 709.09 Q104.687 715.233 100.868 718.416 Q97.0483 721.599 89.6959 721.599 M67.2568 706.862 L67.2568 706.862 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M82.2481 650.048 L103.764 650.048 L103.764 655.905 L82.439 655.905 Q77.3783 655.905 74.8638 657.878 Q72.3494 659.851 72.3494 663.798 Q72.3494 668.541 75.3731 671.278 Q78.3968 674.015 83.6167 674.015 L103.764 674.015 L103.764 679.903 L68.1162 679.903 L68.1162 674.015 L73.6544 674.015 Q70.4397 671.914 68.8483 669.082 Q67.2568 666.217 67.2568 662.493 Q67.2568 656.35 71.0763 653.199 Q74.8638 650.048 82.2481 650.048 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M57.9947 632.574 L68.1162 632.574 L68.1162 620.511 L72.6677 620.511 L72.6677 632.574 L92.0194 632.574 Q96.3799 632.574 97.6213 631.397 Q98.8626 630.187 98.8626 626.527 L98.8626 620.511 L103.764 620.511 L103.764 626.527 Q103.764 633.306 101.25 635.885 Q98.7034 638.463 92.0194 638.463 L72.6677 638.463 L72.6677 642.759 L68.1162 642.759 L68.1162 638.463 L57.9947 638.463 L57.9947 632.574 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip092)" d="M319.905 1374.05 L319.905 1382.14 L375.195 1382.14 L375.195 1374.05 L319.905 1374.05 L319.905 1374.05  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="319.905,1374.05 319.905,1382.14 375.195,1382.14 375.195,1374.05 319.905,1374.05 "/>
<path clip-path="url(#clip092)" d="M375.195 1382.14 L375.195 1382.14 L430.486 1382.14 L430.486 1382.14 L375.195 1382.14 L375.195 1382.14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="375.195,1382.14 375.195,1382.14 430.486,1382.14 375.195,1382.14 "/>
<path clip-path="url(#clip092)" d="M430.486 1382.14 L430.486 1382.14 L485.776 1382.14 L485.776 1382.14 L430.486 1382.14 L430.486 1382.14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="430.486,1382.14 430.486,1382.14 485.776,1382.14 430.486,1382.14 "/>
<path clip-path="url(#clip092)" d="M485.776 1357.87 L485.776 1382.14 L541.066 1382.14 L541.066 1357.87 L485.776 1357.87 L485.776 1357.87  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="485.776,1357.87 485.776,1382.14 541.066,1382.14 541.066,1357.87 485.776,1357.87 "/>
<path clip-path="url(#clip092)" d="M541.066 1374.05 L541.066 1382.14 L596.356 1382.14 L596.356 1374.05 L541.066 1374.05 L541.066 1374.05  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="541.066,1374.05 541.066,1382.14 596.356,1382.14 596.356,1374.05 541.066,1374.05 "/>
<path clip-path="url(#clip092)" d="M596.356 1341.69 L596.356 1382.14 L651.647 1382.14 L651.647 1341.69 L596.356 1341.69 L596.356 1341.69  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="596.356,1341.69 596.356,1382.14 651.647,1382.14 651.647,1341.69 596.356,1341.69 "/>
<path clip-path="url(#clip092)" d="M651.647 1309.33 L651.647 1382.14 L706.937 1382.14 L706.937 1309.33 L651.647 1309.33 L651.647 1309.33  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="651.647,1309.33 651.647,1382.14 706.937,1382.14 706.937,1309.33 651.647,1309.33 "/>
<path clip-path="url(#clip092)" d="M706.937 1228.42 L706.937 1382.14 L762.227 1382.14 L762.227 1228.42 L706.937 1228.42 L706.937 1228.42  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="706.937,1228.42 706.937,1382.14 762.227,1382.14 762.227,1228.42 706.937,1228.42 "/>
<path clip-path="url(#clip092)" d="M762.227 1074.71 L762.227 1382.14 L817.517 1382.14 L817.517 1074.71 L762.227 1074.71 L762.227 1074.71  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="762.227,1074.71 762.227,1382.14 817.517,1382.14 817.517,1074.71 762.227,1074.71 "/>
<path clip-path="url(#clip092)" d="M817.517 937.175 L817.517 1382.14 L872.808 1382.14 L872.808 937.175 L817.517 937.175 L817.517 937.175  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="817.517,937.175 817.517,1382.14 872.808,1382.14 872.808,937.175 817.517,937.175 "/>
<path clip-path="url(#clip092)" d="M872.808 605.473 L872.808 1382.14 L928.098 1382.14 L928.098 605.473 L872.808 605.473 L872.808 605.473  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="872.808,605.473 872.808,1382.14 928.098,1382.14 928.098,605.473 872.808,605.473 "/>
<path clip-path="url(#clip092)" d="M928.098 378.945 L928.098 1382.14 L983.388 1382.14 L983.388 378.945 L928.098 378.945 L928.098 378.945  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="928.098,378.945 928.098,1382.14 983.388,1382.14 983.388,378.945 928.098,378.945 "/>
<path clip-path="url(#clip092)" d="M983.388 111.966 L983.388 1382.14 L1038.68 1382.14 L1038.68 111.966 L983.388 111.966 L983.388 111.966  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="983.388,111.966 983.388,1382.14 1038.68,1382.14 1038.68,111.966 983.388,111.966 "/>
<path clip-path="url(#clip092)" d="M1038.68 47.2441 L1038.68 1382.14 L1093.97 1382.14 L1093.97 47.2441 L1038.68 47.2441 L1038.68 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1038.68,47.2441 1038.68,1382.14 1093.97,1382.14 1093.97,47.2441 1038.68,47.2441 "/>
<path clip-path="url(#clip092)" d="M1093.97 411.307 L1093.97 1382.14 L1149.26 1382.14 L1149.26 411.307 L1093.97 411.307 L1093.97 411.307  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1093.97,411.307 1093.97,1382.14 1149.26,1382.14 1149.26,411.307 1093.97,411.307 "/>
<path clip-path="url(#clip092)" d="M1149.26 508.39 L1149.26 1382.14 L1204.55 1382.14 L1204.55 508.39 L1149.26 508.39 L1149.26 508.39  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1149.26,508.39 1149.26,1382.14 1204.55,1382.14 1204.55,508.39 1149.26,508.39 "/>
<path clip-path="url(#clip092)" d="M1204.55 872.452 L1204.55 1382.14 L1259.84 1382.14 L1259.84 872.452 L1204.55 872.452 L1204.55 872.452  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1204.55,872.452 1204.55,1382.14 1259.84,1382.14 1259.84,872.452 1204.55,872.452 "/>
<path clip-path="url(#clip092)" d="M1259.84 1179.88 L1259.84 1382.14 L1315.13 1382.14 L1315.13 1179.88 L1259.84 1179.88 L1259.84 1179.88  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1259.84,1179.88 1259.84,1382.14 1315.13,1382.14 1315.13,1179.88 1259.84,1179.88 "/>
<path clip-path="url(#clip092)" d="M1315.13 1325.51 L1315.13 1382.14 L1370.42 1382.14 L1370.42 1325.51 L1315.13 1325.51 L1315.13 1325.51  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1315.13,1325.51 1315.13,1382.14 1370.42,1382.14 1370.42,1325.51 1315.13,1325.51 "/>
<path clip-path="url(#clip092)" d="M1370.42 1357.87 L1370.42 1382.14 L1425.71 1382.14 L1425.71 1357.87 L1370.42 1357.87 L1370.42 1357.87  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1370.42,1357.87 1370.42,1382.14 1425.71,1382.14 1425.71,1357.87 1370.42,1357.87 "/>
<path clip-path="url(#clip092)" d="M1425.71 1374.05 L1425.71 1382.14 L1481 1382.14 L1481 1374.05 L1425.71 1374.05 L1425.71 1374.05  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip092)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1425.71,1374.05 1425.71,1382.14 1481,1382.14 1481,1374.05 1425.71,1374.05 "/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="347.55" cy="1374.05" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="402.84" cy="1382.14" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="458.131" cy="1382.14" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="513.421" cy="1357.87" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="568.711" cy="1374.05" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="624.001" cy="1341.69" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="679.292" cy="1309.33" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="734.582" cy="1228.42" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="789.872" cy="1074.71" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="845.162" cy="937.175" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="900.453" cy="605.473" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="955.743" cy="378.945" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1011.03" cy="111.966" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1066.32" cy="47.2441" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1121.61" cy="411.307" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1176.9" cy="508.39" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1232.19" cy="872.452" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1287.48" cy="1179.88" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1342.77" cy="1325.51" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1398.06" cy="1357.87" r="2"/>
<circle clip-path="url(#clip092)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1453.36" cy="1374.05" r="2"/>
<polyline clip-path="url(#clip092)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:8; stroke-opacity:1; fill:none" points="1055,2717.04 1055,-1287.65 "/>
<polyline clip-path="url(#clip092)" style="stroke:#800080; stroke-linecap:round; stroke-linejoin:round; stroke-width:8; stroke-opacity:1; fill:none" points="1028.17,2717.04 1028.17,-1287.65 "/>
<path clip-path="url(#clip090)" d="M1012.21 247.261 L1509.27 247.261 L1509.27 91.7406 L1012.21 91.7406  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip090)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1012.21,247.261 1509.27,247.261 1509.27,91.7406 1012.21,91.7406 1012.21,247.261 "/>
<polyline clip-path="url(#clip090)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:8; stroke-opacity:1; fill:none" points="1026.71,143.581 1113.68,143.581 "/>
<path clip-path="url(#clip090)" d="M1128.18 126.301 L1135.15 126.301 L1143.97 149.819 L1152.83 126.301 L1159.8 126.301 L1159.8 160.861 L1155.24 160.861 L1155.24 130.514 L1146.33 154.217 L1141.63 154.217 L1132.72 130.514 L1132.72 160.861 L1128.18 160.861 L1128.18 126.301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1169.08 126.301 L1173.76 126.301 L1173.76 156.925 L1190.59 156.925 L1190.59 160.861 L1169.08 160.861 L1169.08 126.301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1195.49 126.301 L1217.34 126.301 L1217.34 130.236 L1200.17 130.236 L1200.17 140.467 L1216.63 140.467 L1216.63 144.402 L1200.17 144.402 L1200.17 156.925 L1217.76 156.925 L1217.76 160.861 L1195.49 160.861 L1195.49 126.301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip090)" style="stroke:#800080; stroke-linecap:round; stroke-linejoin:round; stroke-width:8; stroke-opacity:1; fill:none" points="1026.71,195.421 1113.68,195.421 "/>
<path clip-path="url(#clip090)" d="M1132.85 196.196 L1132.85 208.858 L1140.35 208.858 Q1144.13 208.858 1145.93 207.307 Q1147.76 205.733 1147.76 202.515 Q1147.76 199.275 1145.93 197.747 Q1144.13 196.196 1140.35 196.196 L1132.85 196.196 M1132.85 181.983 L1132.85 192.4 L1139.78 192.4 Q1143.2 192.4 1144.87 191.127 Q1146.56 189.83 1146.56 187.191 Q1146.56 184.576 1144.87 183.279 Q1143.2 181.983 1139.78 181.983 L1132.85 181.983 M1128.18 178.141 L1140.12 178.141 Q1145.47 178.141 1148.36 180.363 Q1151.26 182.585 1151.26 186.682 Q1151.26 189.854 1149.78 191.729 Q1148.29 193.603 1145.42 194.066 Q1148.87 194.807 1150.77 197.168 Q1152.69 199.506 1152.69 203.025 Q1152.69 207.654 1149.54 210.177 Q1146.4 212.701 1140.59 212.701 L1128.18 212.701 L1128.18 178.141 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1170.56 189.761 Q1167.14 189.761 1165.15 192.446 Q1163.15 195.108 1163.15 199.761 Q1163.15 204.414 1165.12 207.099 Q1167.11 209.761 1170.56 209.761 Q1173.96 209.761 1175.96 207.076 Q1177.95 204.39 1177.95 199.761 Q1177.95 195.154 1175.96 192.469 Q1173.96 189.761 1170.56 189.761 M1170.56 186.15 Q1176.12 186.15 1179.29 189.761 Q1182.46 193.372 1182.46 199.761 Q1182.46 206.127 1179.29 209.761 Q1176.12 213.372 1170.56 213.372 Q1164.98 213.372 1161.81 209.761 Q1158.66 206.127 1158.66 199.761 Q1158.66 193.372 1161.81 189.761 Q1164.98 186.15 1170.56 186.15 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1199.57 189.761 Q1196.14 189.761 1194.15 192.446 Q1192.16 195.108 1192.16 199.761 Q1192.16 204.414 1194.13 207.099 Q1196.12 209.761 1199.57 209.761 Q1202.97 209.761 1204.96 207.076 Q1206.95 204.39 1206.95 199.761 Q1206.95 195.154 1204.96 192.469 Q1202.97 189.761 1199.57 189.761 M1199.57 186.15 Q1205.12 186.15 1208.29 189.761 Q1211.46 193.372 1211.46 199.761 Q1211.46 206.127 1208.29 209.761 Q1205.12 213.372 1199.57 213.372 Q1193.99 213.372 1190.82 209.761 Q1187.67 206.127 1187.67 199.761 Q1187.67 193.372 1190.82 189.761 Q1193.99 186.15 1199.57 186.15 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1222.74 179.414 L1222.74 186.775 L1231.51 186.775 L1231.51 190.085 L1222.74 190.085 L1222.74 204.159 Q1222.74 207.33 1223.59 208.233 Q1224.47 209.136 1227.14 209.136 L1231.51 209.136 L1231.51 212.701 L1227.14 212.701 Q1222.21 212.701 1220.33 210.872 Q1218.46 209.02 1218.46 204.159 L1218.46 190.085 L1215.33 190.085 L1215.33 186.775 L1218.46 186.775 L1218.46 179.414 L1222.74 179.414 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1253.64 187.539 L1253.64 191.566 Q1251.83 190.641 1249.89 190.178 Q1247.95 189.715 1245.86 189.715 Q1242.69 189.715 1241.09 190.687 Q1239.52 191.659 1239.52 193.603 Q1239.52 195.085 1240.65 195.941 Q1241.79 196.775 1245.21 197.539 L1246.67 197.863 Q1251.21 198.835 1253.11 200.617 Q1255.03 202.377 1255.03 205.548 Q1255.03 209.159 1252.16 211.265 Q1249.31 213.372 1244.31 213.372 Q1242.23 213.372 1239.96 212.955 Q1237.71 212.562 1235.21 211.752 L1235.21 207.353 Q1237.58 208.58 1239.87 209.205 Q1242.16 209.807 1244.4 209.807 Q1247.41 209.807 1249.03 208.789 Q1250.65 207.747 1250.65 205.872 Q1250.65 204.136 1249.47 203.21 Q1248.32 202.284 1244.36 201.428 L1242.88 201.08 Q1238.92 200.247 1237.16 198.534 Q1235.4 196.798 1235.4 193.789 Q1235.4 190.131 1237.99 188.141 Q1240.58 186.15 1245.35 186.15 Q1247.71 186.15 1249.8 186.497 Q1251.88 186.844 1253.64 187.539 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1266.02 179.414 L1266.02 186.775 L1274.8 186.775 L1274.8 190.085 L1266.02 190.085 L1266.02 204.159 Q1266.02 207.33 1266.88 208.233 Q1267.76 209.136 1270.42 209.136 L1274.8 209.136 L1274.8 212.701 L1270.42 212.701 Q1265.49 212.701 1263.62 210.872 Q1261.74 209.02 1261.74 204.159 L1261.74 190.085 L1258.62 190.085 L1258.62 186.775 L1261.74 186.775 L1261.74 179.414 L1266.02 179.414 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1295.42 190.756 Q1294.71 190.34 1293.85 190.154 Q1293.02 189.946 1292 189.946 Q1288.39 189.946 1286.44 192.307 Q1284.52 194.645 1284.52 199.043 L1284.52 212.701 L1280.24 212.701 L1280.24 186.775 L1284.52 186.775 L1284.52 190.803 Q1285.86 188.441 1288.02 187.307 Q1290.17 186.15 1293.25 186.15 Q1293.69 186.15 1294.22 186.219 Q1294.75 186.266 1295.4 186.381 L1295.42 190.756 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1311.67 199.668 Q1306.51 199.668 1304.52 200.849 Q1302.53 202.029 1302.53 204.877 Q1302.53 207.145 1304.01 208.488 Q1305.52 209.807 1308.08 209.807 Q1311.63 209.807 1313.76 207.307 Q1315.91 204.784 1315.91 200.617 L1315.91 199.668 L1311.67 199.668 M1320.17 197.909 L1320.17 212.701 L1315.91 212.701 L1315.91 208.765 Q1314.45 211.127 1312.27 212.261 Q1310.1 213.372 1306.95 213.372 Q1302.97 213.372 1300.61 211.15 Q1298.27 208.904 1298.27 205.154 Q1298.27 200.779 1301.19 198.557 Q1304.13 196.335 1309.94 196.335 L1315.91 196.335 L1315.91 195.918 Q1315.91 192.979 1313.96 191.381 Q1312.04 189.761 1308.55 189.761 Q1306.33 189.761 1304.22 190.293 Q1302.11 190.826 1300.17 191.891 L1300.17 187.955 Q1302.51 187.053 1304.7 186.613 Q1306.9 186.15 1308.99 186.15 Q1314.61 186.15 1317.39 189.066 Q1320.17 191.983 1320.17 197.909 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1333.06 208.812 L1333.06 222.562 L1328.78 222.562 L1328.78 186.775 L1333.06 186.775 L1333.06 190.71 Q1334.4 188.395 1336.44 187.284 Q1338.5 186.15 1341.35 186.15 Q1346.07 186.15 1349.01 189.9 Q1351.97 193.65 1351.97 199.761 Q1351.97 205.872 1349.01 209.622 Q1346.07 213.372 1341.35 213.372 Q1338.5 213.372 1336.44 212.261 Q1334.4 211.127 1333.06 208.812 M1347.55 199.761 Q1347.55 195.062 1345.61 192.4 Q1343.69 189.715 1340.31 189.715 Q1336.93 189.715 1334.98 192.4 Q1333.06 195.062 1333.06 199.761 Q1333.06 204.46 1334.98 207.145 Q1336.93 209.807 1340.31 209.807 Q1343.69 209.807 1345.61 207.145 Q1347.55 204.46 1347.55 199.761 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1374.29 178.141 L1381.26 178.141 L1390.07 201.659 L1398.94 178.141 L1405.91 178.141 L1405.91 212.701 L1401.35 212.701 L1401.35 182.354 L1392.44 206.057 L1387.74 206.057 L1378.82 182.354 L1378.82 212.701 L1374.29 212.701 L1374.29 178.141 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1437.18 198.673 L1437.18 200.756 L1417.6 200.756 Q1417.88 205.154 1420.24 207.469 Q1422.62 209.761 1426.86 209.761 Q1429.31 209.761 1431.6 209.159 Q1433.92 208.557 1436.19 207.353 L1436.19 211.381 Q1433.89 212.353 1431.49 212.863 Q1429.08 213.372 1426.6 213.372 Q1420.4 213.372 1416.76 209.761 Q1413.15 206.15 1413.15 199.992 Q1413.15 193.627 1416.58 189.9 Q1420.03 186.15 1425.86 186.15 Q1431.09 186.15 1434.13 189.529 Q1437.18 192.886 1437.18 198.673 M1432.92 197.423 Q1432.88 193.928 1430.95 191.844 Q1429.06 189.761 1425.91 189.761 Q1422.34 189.761 1420.19 191.775 Q1418.06 193.789 1417.74 197.446 L1432.92 197.423 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1455.95 199.668 Q1450.79 199.668 1448.8 200.849 Q1446.81 202.029 1446.81 204.877 Q1446.81 207.145 1448.29 208.488 Q1449.8 209.807 1452.37 209.807 Q1455.91 209.807 1458.04 207.307 Q1460.19 204.784 1460.19 200.617 L1460.19 199.668 L1455.95 199.668 M1464.45 197.909 L1464.45 212.701 L1460.19 212.701 L1460.19 208.765 Q1458.73 211.127 1456.56 212.261 Q1454.38 213.372 1451.23 213.372 Q1447.25 213.372 1444.89 211.15 Q1442.55 208.904 1442.55 205.154 Q1442.55 200.779 1445.47 198.557 Q1448.41 196.335 1454.22 196.335 L1460.19 196.335 L1460.19 195.918 Q1460.19 192.979 1458.25 191.381 Q1456.32 189.761 1452.83 189.761 Q1450.61 189.761 1448.5 190.293 Q1446.39 190.826 1444.45 191.891 L1444.45 187.955 Q1446.79 187.053 1448.99 186.613 Q1451.19 186.15 1453.27 186.15 Q1458.89 186.15 1461.67 189.066 Q1464.45 191.983 1464.45 197.909 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip090)" d="M1494.77 197.053 L1494.77 212.701 L1490.51 212.701 L1490.51 197.191 Q1490.51 193.511 1489.08 191.682 Q1487.64 189.854 1484.77 189.854 Q1481.32 189.854 1479.33 192.053 Q1477.34 194.252 1477.34 198.048 L1477.34 212.701 L1473.06 212.701 L1473.06 186.775 L1477.34 186.775 L1477.34 190.803 Q1478.87 188.465 1480.93 187.307 Q1483.01 186.15 1485.72 186.15 Q1490.19 186.15 1492.48 188.928 Q1494.77 191.682 1494.77 197.053 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /></svg>
<p>(b)</p>
</div>
</div></td>
</tr>
</tbody>
</table>

Figure 1

We can see from <a href="#fig-p1-boot" class="quarto-xref">Figure 1</a>
that both of the bootstrapped distributions are skewed, but that the
estimates only have a small bias (recall that the bootstrap estimate of
bias of a statistic $t$ is $\mathbb{E}[\tilde{t}] - \hat{t}$, where
$\hat{t}$ is the MLE and $\tilde{t}$ is a bootstrap estimate); $a$ has a
bias of 0.02 and $b$ has a bias of -0.05.

To find the 90% confidence intervals, we use the basic bootstrap formula
$$\left(\hat{t} - (Q_{\tilde{t}}(1-\alpha/2) - \hat{t}), \hat{t} - (Q_{\tilde{t}}(\alpha/2) - \hat{t})\right),$$
where $\alpha = 0.1$.

In [1]:
a_q = quantile(θ_boot[:, 1], [0.95, 0.05])
b_q = quantile(θ_boot[:, 2], [0.95, 0.05])

a_ci = (2 * θ_mle[1] - a_q[1], 2 * θ_mle[1] - a_q[2]) 
b_ci = (2 * θ_mle[2] - b_q[1], 2 * θ_mle[2] - b_q[2]) 
@show round.(a_ci, digits=2);
@show round.(b_ci, digits=2);

round.(a_ci, digits = 2) = (0.72, 1.49)
round.(b_ci, digits = 2) = (0.1, 0.91)

One important note is that your answers might differ a bit from these:
the confidence interval estimates are a bit sensitive to the random
bootstrap replicates with this number of replicates, but the answer
should be roughly similar to this.

### Problem 2

First, load the data

In [1]:

norm_yrs = 1980:1999

sl_dat = DataFrame(CSV.File(joinpath("data",  "CSIRO_Recons_gmsl_yr_2015.csv")))

rename!(sl_dat, [:Year, :GMSLR, :SD]) # rename to make columns easier to work with
sl_dat[!, :Year] .-= 0.5 # shift year to line up with years instead of being half-year 
sl_dat[!, :GMSLR] .-= mean(filter(row -> row.Year ∈ norm_yrs, sl_dat)[!, :GMSLR]) # rescale to be relative to 1880-1900 mean for consistency with temperature anomaly

# load temperature data
temp_dat = DataFrame(CSV.File(joinpath("data", "HadCRUT.5.0.2.0.analysis.summary_series.global.annual.csv")))
rename!(temp_dat, [:Year, :Temp, :Lower, :Upper]) # rename to make columns easier to work with
filter!(row -> row.Year ∈ sl_dat[!, :Year], temp_dat) # reduce to the same years that we have SL data for
temp_normalize = mean(filter(row -> row.Year ∈ norm_yrs, temp_dat)[!, :Temp]) # get renormalization to rescale temperature to 1880-1900 mean
temp_dat[!, :Temp] .-= temp_normalize
temp_dat[!, :Lower] .-= temp_normalize
temp_dat[!, :Upper] .-=  temp_normalize

sl_plot = scatter(sl_dat[!, :Year], sl_dat[!, :GMSLR], yerr=sl_dat[!, :SD], color=:black, label="Observations", ylabel="(mm)", xlabel="Year", title="Sea Level Anomaly")

temp_plot = scatter(temp_dat[!, :Year], temp_dat[!, :Temp], yerr=(temp_dat[!, :Temp] - temp_dat[!, :Lower], temp_dat[!, :Upper] - temp_dat[!, :Temp]), color=:black, label="Observations", ylabel="(°C)", xlabel="Year", title="Temperature")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path clip-path="url(#clip150)" d="M642.161 1798.1 L642.161 1793.84 Q643.92 1794.68 645.726 1795.12 Q647.531 1795.56 649.267 1795.56 Q653.897 1795.56 656.327 1792.46 Q658.781 1789.33 659.128 1782.99 Q657.786 1784.98 655.726 1786.04 Q653.665 1787.11 651.165 1787.11 Q645.98 1787.11 642.948 1783.98 Q639.939 1780.83 639.939 1775.4 Q639.939 1770.07 643.087 1766.85 Q646.235 1763.64 651.466 1763.64 Q657.462 1763.64 660.61 1768.24 Q663.781 1772.83 663.781 1781.58 Q663.781 1789.75 659.892 1794.63 Q656.027 1799.49 649.476 1799.49 Q647.716 1799.49 645.911 1799.15 Q644.105 1798.8 642.161 1798.1 M651.466 1783.45 Q654.615 1783.45 656.443 1781.3 Q658.295 1779.15 658.295 1775.4 Q658.295 1771.67 656.443 1769.52 Q654.615 1767.34 651.466 1767.34 Q648.318 1767.34 646.466 1769.52 Q644.638 1771.67 644.638 1775.4 Q644.638 1779.15 646.466 1781.3 Q648.318 1783.45 651.466 1783.45 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip150)" d="M682.184 1767.34 Q678.573 1767.34 676.744 1770.9 Q674.938 1774.45 674.938 1781.58 Q674.938 1788.68 676.744 1792.25 Q678.573 1795.79 682.184 1795.79 Q685.818 1795.79 687.624 1792.25 Q689.452 1788.68 689.452 1781.58 Q689.452 1774.45 687.624 1770.9 Q685.818 1767.34 682.184 1767.34 M682.184 1763.64 Q687.994 1763.64 691.05 1768.24 Q694.128 1772.83 694.128 1781.58 Q694.128 1790.3 691.05 1794.91 Q687.994 1799.49 682.184 1799.49 Q676.374 1799.49 673.295 1794.91 Q670.239 1790.3 670.239 1781.58 Q670.239 1772.83 673.295 1768.24 Q676.374 1763.64 682.184 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip150)" d="M712.346 1767.34 Q708.735 1767.34 706.906 1770.9 Q705.1 1774.45 705.1 1781.58 Q705.1 1788.68 706.906 1792.25 Q708.735 1795.79 712.346 1795.79 Q715.98 1795.79 717.785 1792.25 Q719.614 1788.68 719.614 1781.58 Q719.614 1774.45 717.785 1770.9 Q715.98 1767.34 712.346 1767.34 M712.346 1763.64 Q718.156 1763.64 721.211 1768.24 Q724.29 1772.83 724.29 1781.58 Q724.29 1790.3 721.211 1794.91 Q718.156 1799.49 712.346 1799.49 Q706.536 1799.49 703.457 1794.91 Q700.401 1790.3 700.401 1781.58 Q700.401 1772.83 703.457 1768.24 Q706.536 1763.64 712.346 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip150)" d="M1064.19 1798.1 L1064.19 1793.84 Q1065.95 1794.68 1067.76 1795.12 Q1069.56 1795.56 1071.3 1795.56 Q1075.93 1795.56 1078.36 1792.46 Q1080.81 1789.33 1081.16 1782.99 Q1079.82 1784.98 1077.76 1786.04 Q1075.7 1787.11 1073.2 1787.11 Q1068.01 1787.11 1064.98 1783.98 Q1061.97 1780.83 1061.97 1775.4 Q1061.97 1770.07 1065.12 1766.85 Q1068.27 1763.64 1073.5 1763.64 Q1079.49 1763.64 1082.64 1768.24 Q1085.81 1772.83 1085.81 1781.58 Q1085.81 1789.75 1081.92 1794.63 Q1078.06 1799.49 1071.51 1799.49 Q1069.75 1799.49 1067.94 1799.15 Q1066.14 1798.8 1064.19 1798.1 M1073.5 1783.45 Q1076.65 1783.45 1078.48 1781.3 Q1080.33 1779.15 1080.33 1775.4 Q1080.33 1771.67 1078.48 1769.52 Q1076.65 1767.34 1073.5 1767.34 Q1070.35 1767.34 1068.5 1769.52 Q1066.67 1771.67 1066.67 1775.4 Q1066.67 1779.15 1068.5 1781.3 Q1070.35 1783.45 1073.5 1783.45 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip150)" d="M1098.24 1794.89 L1114.56 1794.89 L1114.56 1798.82 L1092.62 1798.82 L1092.62 1794.89 Q1095.28 1792.13 1099.86 1787.5 Q1104.47 1782.85 1105.65 1781.51 Q1107.9 1778.98 1108.78 1777.25 Q1109.68 1775.49 1109.68 1773.8 Q1109.68 1771.04 1107.73 1769.31 Q1105.81 1767.57 1102.71 1767.57 Q1100.51 1767.57 1098.06 1768.34 Q1095.63 1769.1 1092.85 1770.65 L1092.85 1765.93 Q1095.67 1764.79 1098.13 1764.21 Q1100.58 1763.64 1102.62 1763.64 Q1107.99 1763.64 1111.18 1766.32 Q1114.38 1769.01 1114.38 1773.5 Q1114.38 1775.63 1113.57 1777.55 Q1112.78 1779.45 1110.67 1782.04 Q1110.1 1782.71 1106.99 1785.93 Q1103.89 1789.12 1098.24 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip150)" d="M1124.42 17

The Grinsted model in code (skipping past the discretization since we
did this in HW2):

In [1]:
function grinsted_slr(params, temps; Δt=1)
    a, b, τ, S₀ = params
    S = zeros(length(temps)) # initialize storage
    Seq = a * temps .+ b
    S[1] = S₀
    for i = 2:length(S)
        S[i] = S[i-1] +  Δt * (Seq[i] - S[i-1]) / τ
    end
    return S[1:end]
end

grinsted_slr (generic function with 1 method)

The log likelihood function with AR(1) residuals:

In [1]:
function ar1_loglik(params, temp_dat, slr_obs, Δt=1.0)
    a, b, τ, S₀, ρ, σ = params 
    slr_sim = grinsted_slr((a, b, τ, S₀), temp_dat; Δt = Δt)
    # whiten residuals
    resids = slr_obs .- slr_sim
    ll = 0
    for t = 1:length(temp_dat) - 1
        if t == 1
            ll += logpdf(Normal(0, sqrt(σ^2 / (1 - ρ^2))), resids[t])
        else
            ll += sum(logpdf(Normal(ρ * resids[t], σ), resids[t+1]))
        end
    end 
    return ll
end

ar1_loglik (generic function with 2 methods)

Now, let’s find the MLE by optimizing the `ar1_loglik` function.

In [1]:
low_bds = [-2000.0, -2000.0, 0.1, sl_dat.GMSLR[1] - 1.96 * sl_dat.SD[1], -1.0, 0.1]
up_bds = [20_000.0, 30_000.0, 10_000.0, sl_dat.GMSLR[1] + 1.96 * sl_dat.SD[1], 0.99, 100.0]
p₀ = [10.0, 0.0, 1500.0, sl_dat.GMSLR[1], 0.0, 10.0]

mle_optim = optimize(p -> -ar1_loglik(p, temp_dat.Temp, sl_dat.GMSLR), low_bds, up_bds, p₀)
p_mle = mle_optim.minimizer
@show p_mle;

p_mle = [546.9549427127922, 410.3873816251446, 179.69518456677608, -158.18775262248883, 0.5215568507538185, 4.960943964640367]

We can plot the fitted model and the residuals:

``` julia
slr_fit = grinsted_slr(p_mle[1:end-2], temp_dat.Temp)
resids = sl_dat.GMSLR - slr_fit

pfit = plot(sl_dat.Year, slr_fit, color=:blue, xlabel="Year", ylabel="Global Mean Sea Level (mm)", label="Model Fit")
scatter!(pfit, sl_dat.Year, sl_dat.GMSLR, color=:black, label="Observations")
plot!(pfit, size=(400, 400))

presids = scatter(sl_dat.Year, resids, color=:black, xlabel="Year", ylabel="Model Residuals (mm)", legend=false)
plot!(presids, size=(400, 400))

display(pfit)
display(presids)
```

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<tbody>
<tr>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="400" height="400" viewBox="0 0 1600 1600">
<defs>
  <clipPath id="clip210">
    <rect x="0" y="0" width="1600" height="1600"/>
  </clipPath>
</defs>
<path clip-path="url(#clip210)" d="M0 1600 L1600 1600 L1600 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip211">
    <rect x="320" y="159" width="1121" height="1121"/>
  </clipPath>
</defs>
<path clip-path="url(#clip210)" d="M288.057 1382.14 L1552.76 1382.14 L1552.76 47.2441 L288.057 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip212">
    <rect x="288" y="47" width="1266" height="1336"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="503.265,1382.14 503.265,47.2441 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="727.535,1382.14 727.535,47.2441 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="951.804,1382.14 951.804,47.2441 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1176.07,1382.14 1176.07,47.2441 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1400.34,1382.14 1400.34,47.2441 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="288.057,1240.57 1552.76,1240.57 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="288.057,984.604 1552.76,984.604 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="288.057,728.642 1552.76,728.642 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="288.057,472.679 1552.76,472.679 "/>
<polyline clip-path="url(#clip212)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="288.057,216.717 1552.76,216.717 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="288.057,1382.14 1552.76,1382.14 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="503.265,1382.14 503.265,1363.24 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="727.535,1382.14 727.535,1363.24 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="951.804,1382.14 951.804,1363.24 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1176.07,1382.14 1176.07,1363.24 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1400.34,1382.14 1400.34,1363.24 "/>
<path clip-path="url(#clip210)" d="M447.791 1434.2 L455.43 1434.2 L455.43 1407.84 L447.12 1409.51 L447.12 1405.25 L455.384 1403.58 L460.06 1403.58 L460.06 1434.2 L467.698 1434.2 L467.698 1438.14 L447.791 1438.14 L447.791 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M477.282 1437.42 L477.282 1433.16 Q479.041 1434 480.846 1434.44 Q482.652 1434.88 484.388 1434.88 Q489.018 1434.88 491.448 1431.77 Q493.902 1428.65 494.249 1422.31 Q492.907 1424.3 490.846 1425.36 Q488.786 1426.43 486.286 1426.43 Q481.101 1426.43 478.069 1423.3 Q475.059 1420.15 475.059 1414.71 Q475.059 1409.39 478.208 1406.17 Q481.356 1402.95 486.587 1402.95 Q492.583 1402.95 495.731 1407.56 Q498.902 1412.14 498.902 1420.89 Q498.902 1429.07 495.013 1433.95 Q491.147 1438.81 484.596 1438.81 Q482.837 1438.81 481.032 1438.46 Q479.226 1438.12 477.282 1437.42 M486.587 1422.77 Q489.735 1422.77 491.564 1420.62 Q493.416 1418.46 493.416 1414.71 Q493.416 1410.99 491.564 1408.83 Q489.735 1406.66 486.587 1406.66 Q483.439 1406.66 481.587 1408.83 Q479.759 1410.99 479.759 1414.71 Q479.759 1418.46 481.587 1420.62 Q483.439 1422.77 486.587 1422.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M517.305 1406.66 Q513.694 1406.66 511.865 1410.22 Q510.059 1413.76 510.059 1420.89 Q510.059 1428 511.865 1431.57 Q513.694 1435.11 517.305 1435.11 Q520.939 1435.11 522.744 1431.57 Q524.573 1428 524.573 1420.89 Q524.573 1413.76 522.744 1410.22 Q520.939 1406.66 517.305 1406.66 M517.305 1402.95 Q523.115 1402.95 526.17 1407.56 Q529.249 1412.14 529.249 1420.89 Q529.249 1429.62 526.17 1434.23 Q523.115 1438.81 517.305 1438.81 Q511.494 1438.81 508.416 1434.23 Q505.36 1429.62 505.36 1420.89 Q505.36 1412.14 508.416 1407.56 Q511.494 1402.95 517.305 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M547.467 1406.66 Q543.855 1406.66 542.027 1410.22 Q540.221 1413.76 540.221 1420.89 Q540.221 1428 542.027 1431.57 Q543.855 1435.11 547.467 1435.11 Q551.101 1435.11 552.906 1431.57 Q554.735 1428 554.735 1420.89 Q554.735 1413.76 552.906 1410.22 Q551.101 1406.66 547.467 1406.66 M547.467 1402.95 Q553.277 1402.95 556.332 1407.56 Q559.411 1412.14 559.411 1420.89 Q559.411 1429.62 556.332 1434.23 Q553.277 1438.81 547.467 1438.81 Q541.656 1438.81 538.578 1434.23 Q535.522 1429.62 535.522 1420.89 Q535.522 1412.14 538.578 1407.56 Q541.656 1402.95 547.467 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M672.558 1434.2 L680.197 1434.2 L680.197 1407.84 L671.887 1409.51 L671.887 1405.25 L680.151 1403.58 L684.827 1403.58 L684.827 1434.2 L692.465 1434.2 L692.465 1438.14 L672.558 1438.14 L672.558 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M702.049 1437.42 L702.049 1433.16 Q703.808 1434 705.613 1434.44 Q707.419 1434.88 709.155 1434.88 Q713.785 1434.88 716.215 1431.77 Q718.669 1428.65 719.016 1422.31 Q717.674 1424.3 715.613 1425.36 Q713.553 1426.43 711.053 1426.43 Q705.868 1426.43 702.836 1423.3 Q699.826 1420.15 699.826 1414.71 Q699.826 1409.39 702.975 1406.17 Q706.123 1402.95 711.354 1402.95 Q717.35 1402.95 720.498 1407.56 Q723.669 1412.14 723.669 1420.89 Q723.669 1429.07 719.78 1433.95 Q715.914 1438.81 709.363 1438.81 Q707.604 1438.81 705.799 1438.46 Q703.993 1438.12 702.049 1437.42 M711.354 1422.77 Q714.502 1422.77 716.331 1420.62 Q718.183 1418.46 718.183 1414.71 Q718.183 1410.99 716.331 1408.83 Q714.502 1406.66 711.354 1406.66 Q708.206 1406.66 706.354 1408.83 Q704.526 1410.99 704.526 1414.71 Q704.526 1418.46 706.354 1420.62 Q708.206 1422.77 711.354 1422.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M736.099 1434.2 L752.419 1434.2 L752.419 1438.14 L730.474 1438.14 L730.474 1434.2 Q733.136 1431.45 737.72 1426.82 Q742.326 1422.17 743.507 1420.83 Q745.752 1418.3 746.632 1416.57 Q747.535 1414.81 747.535 1413.12 Q747.535 1410.36 745.59 1408.63 Q743.669 1406.89 740.567 1406.89 Q738.368 1406.89 735.914 1407.65 Q733.484 1408.42 730.706 1409.97 L730.706 1405.25 Q733.53 1404.11 735.984 1403.53 Q738.437 1402.95 740.474 1402.95 Q745.845 1402.95 749.039 1405.64 Q752.234 1408.33 752.234 1412.82 Q752.234 1414.95 751.423 1416.87 Q750.636 1418.76 748.53 1421.36 Q747.951 1422.03 744.849 1425.25 Q741.748 1428.44 736.099 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M762.28 1403.58 L780.636 1403.58 L780.636 1407.52 L766.562 1407.52 L766.562 1415.99 Q767.581 1415.64 768.599 1415.48 Q769.618 1415.29 770.636 1415.29 Q776.423 1415.29 779.803 1418.46 Q783.182 1421.64 783.182 1427.05 Q783.182 1432.63 779.71 1435.73 Q776.238 1438.81 769.919 1438.81 Q767.743 1438.81 765.474 1438.44 Q763.229 1438.07 760.821 1437.33 L760.821 1432.63 Q762.905 1433.76 765.127 1434.32 Q767.349 1434.88 769.826 1434.88 Q773.831 1434.88 776.169 1432.77 Q778.507 1430.66 778.507 1427.05 Q778.507 1423.44 776.169 1421.33 Q773.831 1419.23 769.826 1419.23 Q767.951 1419.23 766.076 1419.64 Q764.224 1420.06 762.28 1420.94 L762.28 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M896.33 1434.2 L903.969 1434.2 L903.969 1407.84 L895.658 1409.51 L895.658 1405.25 L903.922 1403.58 L908.598 1403.58 L908.598 1434.2 L916.237 1434.2 L916.237 1438.14 L896.33 1438.14 L896.33 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M925.82 1437.42 L925.82 1433.16 Q927.58 1434 929.385 1434.44 Q931.191 1434.88 932.927 1434.88 Q937.556 1434.88 939.987 1431.77 Q942.441 1428.65 942.788 1422.31 Q941.445 1424.3 939.385 1425.36 Q937.325 1426.43 934.825 1426.43 Q929.64 1426.43 926.607 1423.3 Q923.598 1420.15 923.598 1414.71 Q923.598 1409.39 926.746 1406.17 Q929.894 1402.95 935.126 1402.95 Q941.121 1402.95 944.269 1407.56 Q947.441 1412.14 947.441 1420.89 Q947.441 1429.07 943.552 1433.95 Q939.686 1438.81 933.135 1438.81 Q931.376 1438.81 929.57 1438.46 Q927.765 1438.12 925.82 1437.42 M935.126 1422.77 Q938.274 1422.77 940.103 1420.62 Q941.954 1418.46 941.954 1414.71 Q941.954 1410.99 940.103 1408.83 Q938.274 1406.66 935.126 1406.66 Q931.978 1406.66 930.126 1408.83 Q928.297 1410.99 928.297 1414.71 Q928.297 1418.46 930.126 1420.62 Q931.978 1422.77 935.126 1422.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M955.89 1403.58 L974.246 1403.58 L974.246 1407.52 L960.172 1407.52 L960.172 1415.99 Q961.19 1415.64 962.209 1415.48 Q963.227 1415.29 964.246 1415.29 Q970.033 1415.29 973.413 1418.46 Q976.792 1421.64 976.792 1427.05 Q976.792 1432.63 973.32 1435.73 Q969.848 1438.81 963.528 1438.81 Q961.352 1438.81 959.084 1438.44 Q956.839 1438.07 954.431 1437.33 L954.431 1432.63 Q956.515 1433.76 958.737 1434.32 Q960.959 1434.88 963.436 1434.88 Q967.44 1434.88 969.778 1432.77 Q972.116 1430.66 972.116 1427.05 Q972.116 1423.44 969.778 1421.33 Q967.44 1419.23 963.436 1419.23 Q961.561 1419.23 959.686 1419.64 Q957.834 1420.06 955.89 1420.94 L955.89 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M996.005 1406.66 Q992.394 1406.66 990.565 1410.22 Q988.76 1413.76 988.76 1420.89 Q988.76 1428 990.565 1431.57 Q992.394 1435.11 996.005 1435.11 Q999.639 1435.11 1001.44 1431.57 Q1003.27 1428 1003.27 1420.89 Q1003.27 1413.76 1001.44 1410.22 Q999.639 1406.66 996.005 1406.66 M996.005 1402.95 Q1001.82 1402.95 1004.87 1407.56 Q1007.95 1412.14 1007.95 1420.89 Q1007.95 1429.62 1004.87 1434.23 Q1001.82 1438.81 996.005 1438.81 Q990.195 1438.81 987.116 1434.23 Q984.061 1429.62 984.061 1420.89 Q984.061 1412.14 987.116 1407.56 Q990.195 1402.95 996.005 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M1121.1 1434.2 L1128.74 1434.2 L1128.74 1407.84 L1120.43 1409.51 L1120.43 1405.25 L1128.69 1403.58 L1133.37 1403.58 L1133.37 1434.2 L1141 1434.2 L1141 1438.14 L1121.1 1438.14 L1121.1 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M1150.59 1437.42 L1150.59 1433.16 Q1152.35 1434 1154.15 1434.44 Q1155.96 1434.88 1157.69 1434.88 Q1162.32 1434.88 1164.75 1431.77 Q1167.21 1428.65 1167.55 1422.31 Q1166.21 1424.3 1164.15 1425.36 Q1162.09 1426.43 1159.59 1426.43 Q1154.41 1426.43 1151.37 1423.3 Q1148.37 1420.15 1148.37 1414.71 Q1148.37 1409.39 1151.51 1406.17 Q1154.66 1402.95 1159.89 1402.95 Q1165.89 1402.95 1169.04 1407.56 Q1172.21 1412.14 1172.21 1420.89 Q1172.21 1429.07 1168.32 1433.95 Q1164.45 1438.81 1157.9 1438.81 Q1156.14 1438.81 1154.34 1438.46 Q1152.53 1438.12 1150.59 1437.42 M1159.89 1422.77 Q1163.04 1422.77 1164.87 1420.62 Q1166.72 1418.46 1166.72 1414.71 Q1166.72 1410.99 1164.87 1408.83 Q1163.04 1406.66 1159.89 1406.66 Q1156.74 1406.66 1154.89 1408.83 Q1153.06 1410.99 1153.06 1414.71 Q1153.06 1418.46 1154.89 1420.62 Q1156.74 1422.77 1159.89 1422.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M1179.43 1403.58 L1201.65 1403.58 L1201.65 1405.57 L1189.11 1438.14 L1184.22 1438.14 L1196.03 1407.52 L1179.43 1407.52 L1179.43 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M1210.82 1403.58 L1229.17 1403.58 L1229.17 1407.52 L1215.1 1407.52 L1215.1 1415.99 Q1216.12 1415.64 1217.14 1415.48 Q1218.16 1415.29 1219.17 1415.29 Q1224.96 1415.29 1228.34 1418.46 Q1231.72 1421.64 1231.72 1427.05 Q1231.72 1432.63 1228.25 1435.73 Q1224.78 1438.81 1218.46 1438.81 Q1216.28 1438.81 1214.01 1438.44 Q1211.77 1438.07 1209.36 1437.33 L1209.36 1432.63 Q1211.44 1433.76 1213.67 1434.32 Q1215.89 1434.88 1218.36 1434.88 Q1222.37 1434.88 1224.71 1432.77 Q1227.05 1430.66 1227.05 1427.05 Q1227.05 1423.44 1224.71 1421.33 Q1222.37 1419.23 1218.36 1419.23 Q1216.49 1419.23 1214.61 1419.64 Q1212.76 1420.06 1210.82 1420.94 L1210.82 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M1348.95 1434.2 L1365.27 1434.2 L1365.27 1438.14 L1343.33 1438.14 L1343.33 1434.2 Q1345.99 1431.45 1350.57 1426.82 Q1355.18 1422.17 1356.36 1420.83 Q1358.61 1418.3 1359.49 1416.57 Q1360.39 1414.81 1360.39 1413.12 Q1360.39 1410.36 1358.44 1408.63 Q1356.52 1406.89 1353.42 1406.89 Q1351.22 1406.89 1348.77 1407.65 Q1346.34 1408.42 1343.56 1409.97 L1343.56 1405.25 Q1346.38 1404.11 1348.84 1403.53 Q1351.29 1402.95 1353.33 1402.95 Q1358.7 1402.95 1361.89 1405.64 Q1365.09 1408.33 1365.09 1412.82 Q1365.09 1414.95 1364.28 1416.87 Q1363.49 1418.76 1361.38 1421.36 Q1360.81 1422.03 1357.7 1425.25 Q1354.6 1428.44 1348.95 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M1385.09 1406.66 Q1381.48 1406.66 1379.65 1410.22 Q1377.84 1413.76 1377.84 1420.89 Q1377.84 1428 1379.65 1431.57 Q1381.48 1435.11 1385.09 1435.11 Q1388.72 1435.11 1390.53 1431.57 Q1392.36 1428 1392.36 1420.89 Q1392.36 1413.76 1390.53 1410.22 Q1388.72 1406.66 1385.09 1406.66 M1385.09 1402.95 Q1390.9 1402.95 1393.95 1407.56 Q1397.03 1412.14 1397.03 1420.89 Q1397.03 1429.62 1393.95 1434.23 Q1390.9 1438.81 1385.09 1438.81 Q1379.28 1438.81 1376.2 1434.23 Q1373.14 1429.62 1373.14 1420.89 Q1373.14 1412.14 1376.2 1407.56 Q1379.28 1402.95 1385.09 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M1415.25 1406.66 Q1411.64 1406.66 1409.81 1410.22 Q1408 1413.76 1408 1420.89 Q1408 1428 1409.81 1431.57 Q1411.64 1435.11 1415.25 1435.11 Q1418.88 1435.11 1420.69 1431.57 Q1422.52 1428 1422.52 1420.89 Q1422.52 1413.76 1420.69 1410.22 Q1418.88 1406.66 1415.25 1406.66 M1415.25 1402.95 Q1421.06 1402.95 1424.12 1407.56 Q1427.19 1412.14 1427.19 1420.89 Q1427.19 1429.62 1424.12 1434.23 Q1421.06 1438.81 1415.25 1438.81 Q1409.44 1438.81 1406.36 1434.23 Q1403.31 1429.62 1403.31 1420.89 Q1403.31 1412.14 1406.36 1407.56 Q1409.44 1402.95 1415.25 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M1445.41 1406.66 Q1441.8 1406.66 1439.97 1410.22 Q1438.17 1413.76 1438.17 1420.89 Q1438.17 1428 1439.97 1431.57 Q1441.8 1435.11 1445.41 1435.11 Q1449.05 1435.11 1450.85 1431.57 Q1452.68 1428 1452.68 1420.89 Q1452.68 1413.76 1450.85 1410.22 Q1449.05 1406.66 1445.41 1406.66 M1445.41 1402.95 Q1451.22 1402.95 1454.28 1407.56 Q1457.36 1412.14 1457.36 1420.89 Q1457.36 1429.62 1454.28 1434.23 Q1451.22 1438.81 1445.41 1438.81 Q1439.6 1438.81 1436.52 1434.23 Q1433.47 1429.62 1433.47 1420.89 Q1433.47 1412.14 1436.52 1407.56 Q1439.6 1402.95 1445.41 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M851.338 1467.48 L858.245 1467.48 L871.422 1487.03 L884.504 1467.48 L891.41 1467.48 L874.605 1492.37 L874.605 1515 L868.144 1515 L868.144 1492.37 L851.338 1467.48 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M919.26 1495.72 L919.26 1498.58 L892.333 1498.58 Q892.715 1504.63 895.962 1507.81 Q899.24 1510.96 905.065 1510.96 Q908.439 1510.96 911.59 1510.13 Q914.773 1509.31 917.892 1507.65 L917.892 1513.19 Q914.741 1514.53 911.431 1515.23 Q908.12 1515.93 904.715 1515.93 Q896.185 1515.93 891.188 1510.96 Q886.222 1506 886.222 1497.53 Q886.222 1488.78 890.933 1483.65 Q895.675 1478.5 903.696 1478.5 Q910.889 1478.5 915.059 1483.14 Q919.26 1487.76 919.26 1495.72 M913.404 1494 Q913.34 1489.19 910.699 1486.33 Q908.089 1483.46 903.76 1483.46 Q898.858 1483.46 895.898 1486.23 Q892.97 1489 892.524 1494.03 L913.404 1494 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M945.073 1497.08 Q937.976 1497.08 935.238 1498.71 Q932.501 1500.33 932.501 1504.25 Q932.501 1507.36 934.538 1509.21 Q936.607 1511.03 940.14 1511.03 Q945.01 1511.03 947.938 1507.59 Q950.898 1504.12 950.898 1498.39 L950.898 1497.08 L945.073 1497.08 M956.754 1494.67 L956.754 1515 L950.898 1515 L950.898 1509.59 Q948.893 1512.84 945.901 1514.4 Q942.909 1515.93 938.58 1515.93 Q933.106 1515.93 929.859 1512.87 Q926.645 1509.78 926.645 1504.63 Q926.645 1498.61 930.655 1495.56 Q934.697 1492.5 942.686 1492.5 L950.898 1492.5 L950.898 1491.93 Q950.898 1487.89 948.224 1485.69 Q945.583 1483.46 940.776 1483.46 Q937.721 1483.46 934.825 1484.19 Q931.928 1484.93 929.255 1486.39 L929.255 1480.98 Q932.469 1479.74 935.493 1479.13 Q938.517 1478.5 941.381 1478.5 Q949.116 1478.5 952.935 1482.51 Q956.754 1486.52 956.754 1494.67 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M989.474 1484.83 Q988.487 1484.26 987.31 1484 Q986.164 1483.72 984.764 1483.72 Q979.798 1483.72 977.125 1486.96 Q974.483 1490.18 974.483 1496.23 L974.483 1515 L968.595 1515 L968.595 1479.36 L974.483 1479.36 L974.483 1484.89 Q976.329 1481.65 979.289 1480.09 Q982.249 1478.5 986.482 1478.5 Q987.087 1478.5 987.819 1478.59 Q988.551 1478.66 989.442 1478.81 L989.474 1484.83 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="288.057,1382.14 288.057,47.2441 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="288.057,1240.57 306.954,1240.57 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="288.057,984.604 306.954,984.604 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="288.057,728.642 306.954,728.642 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="288.057,472.679 306.954,472.679 "/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="288.057,216.717 306.954,216.717 "/>
<path clip-path="url(#clip210)" d="M142.02 1241.02 L171.696 1241.02 L171.696 1244.95 L142.02 1244.95 L142.02 1241.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M182.599 1253.91 L190.237 1253.91 L190.237 1227.55 L181.927 1229.21 L181.927 1224.95 L190.191 1223.29 L194.867 1223.29 L194.867 1253.91 L202.506 1253.91 L202.506 1257.85 L182.599 1257.85 L182.599 1253.91 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M211.997 1223.29 L230.353 1223.29 L230.353 1227.22 L216.279 1227.22 L216.279 1235.69 Q217.298 1235.35 218.316 1235.19 Q219.335 1235 220.353 1235 Q226.14 1235 229.52 1238.17 Q232.899 1241.34 232.899 1246.76 Q232.899 1252.34 229.427 1255.44 Q225.955 1258.52 219.635 1258.52 Q217.46 1258.52 215.191 1258.15 Q212.946 1257.78 210.538 1257.04 L210.538 1252.34 Q212.622 1253.47 214.844 1254.03 Q217.066 1254.58 219.543 1254.58 Q223.547 1254.58 225.885 1252.48 Q228.223 1250.37 228.223 1246.76 Q228.223 1243.15 225.885 1241.04 Q223.547 1238.94 219.543 1238.94 Q217.668 1238.94 215.793 1239.35 Q213.941 1239.77 211.997 1240.65 L211.997 1223.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M252.112 1226.37 Q248.501 1226.37 246.672 1229.93 Q244.867 1233.47 244.867 1240.6 Q244.867 1247.71 246.672 1251.27 Q248.501 1254.81 252.112 1254.81 Q255.746 1254.81 257.552 1251.27 Q259.381 1247.71 259.381 1240.6 Q259.381 1233.47 257.552 1229.93 Q255.746 1226.37 252.112 1226.37 M252.112 1222.66 Q257.922 1222.66 260.978 1227.27 Q264.057 1231.85 264.057 1240.6 Q264.057 1249.33 260.978 1253.93 Q257.922 1258.52 252.112 1258.52 Q246.302 1258.52 243.223 1253.93 Q240.168 1249.33 240.168 1240.6 Q240.168 1231.85 243.223 1227.27 Q246.302 1222.66 252.112 1222.66 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M142.02 985.056 L171.696 985.056 L171.696 988.991 L142.02 988.991 L142.02 985.056 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M182.599 997.949 L190.237 997.949 L190.237 971.584 L181.927 973.25 L181.927 968.991 L190.191 967.324 L194.867 967.324 L194.867 997.949 L202.506 997.949 L202.506 1001.88 L182.599 1001.88 L182.599 997.949 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M221.95 970.403 Q218.339 970.403 216.51 973.968 Q214.705 977.51 214.705 984.639 Q214.705 991.746 216.51 995.31 Q218.339 998.852 221.95 998.852 Q225.585 998.852 227.39 995.31 Q229.219 991.746 229.219 984.639 Q229.219 977.51 227.39 973.968 Q225.585 970.403 221.95 970.403 M221.95 966.699 Q227.76 966.699 230.816 971.306 Q233.895 975.889 233.895 984.639 Q233.895 993.366 230.816 997.972 Q227.76 1002.56 221.95 1002.56 Q216.14 1002.56 213.061 997.972 Q210.006 993.366 210.006 984.639 Q210.006 975.889 213.061 971.306 Q216.14 966.699 221.95 966.699 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M252.112 970.403 Q248.501 970.403 246.672 973.968 Q244.867 977.51 244.867 984.639 Q244.867 991.746 246.672 995.31 Q248.501 998.852 252.112 998.852 Q255.746 998.852 257.552 995.31 Q259.381 991.746 259.381 984.639 Q259.381 977.51 257.552 973.968 Q255.746 970.403 252.112 970.403 M252.112 966.699 Q257.922 966.699 260.978 971.306 Q264.057 975.889 264.057 984.639 Q264.057 993.366 260.978 997.972 Q257.922 1002.56 252.112 1002.56 Q246.302 1002.56 243.223 997.972 Q240.168 993.366 240.168 984.639 Q240.168 975.889 243.223 971.306 Q246.302 966.699 252.112 966.699 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M172.182 729.093 L201.858 729.093 L201.858 733.028 L172.182 733.028 L172.182 729.093 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M211.997 711.362 L230.353 711.362 L230.353 715.297 L216.279 715.297 L216.279 723.769 Q217.298 723.422 218.316 723.26 Q219.335 723.075 220.353 723.075 Q226.14 723.075 229.52 726.246 Q232.899 729.417 232.899 734.834 Q232.899 740.413 229.427 743.515 Q225.955 746.593 219.635 746.593 Q217.46 746.593 215.191 746.223 Q212.946 745.852 210.538 745.112 L210.538 740.413 Q212.622 741.547 214.844 742.103 Q217.066 742.658 219.543 742.658 Q223.547 742.658 225.885 740.552 Q228.223 738.445 228.223 734.834 Q228.223 731.223 225.885 729.116 Q223.547 727.01 219.543 727.01 Q217.668 727.01 215.793 727.427 Q213.941 727.843 211.997 728.723 L211.997 711.362 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M252.112 714.441 Q248.501 714.441 246.672 718.005 Q244.867 721.547 244.867 728.677 Q244.867 735.783 246.672 739.348 Q248.501 742.89 252.112 742.89 Q255.746 742.89 257.552 739.348 Q259.381 735.783 259.381 728.677 Q259.381 721.547 257.552 718.005 Q255.746 714.441 252.112 714.441 M252.112 710.737 Q257.922 710.737 260.978 715.343 Q264.057 719.927 264.057 728.677 Q264.057 737.403 260.978 742.01 Q257.922 746.593 252.112 746.593 Q246.302 746.593 243.223 742.01 Q240.168 737.403 240.168 728.677 Q240.168 719.927 243.223 715.343 Q246.302 710.737 252.112 710.737 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M252.112 458.478 Q248.501 458.478 246.672 462.043 Q244.867 465.585 244.867 472.714 Q244.867 479.821 246.672 483.385 Q248.501 486.927 252.112 486.927 Q255.746 486.927 257.552 483.385 Q259.381 479.821 259.381 472.714 Q259.381 465.585 257.552 462.043 Q255.746 458.478 252.112 458.478 M252.112 454.774 Q257.922 454.774 260.978 459.381 Q264.057 463.964 264.057 472.714 Q264.057 481.441 260.978 486.047 Q257.922 490.631 252.112 490.631 Q246.302 490.631 243.223 486.047 Q240.168 481.441 240.168 472.714 Q240.168 463.964 243.223 459.381 Q246.302 454.774 252.112 454.774 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M211.997 199.437 L230.353 199.437 L230.353 203.372 L216.279 203.372 L216.279 211.844 Q217.298 211.497 218.316 211.335 Q219.335 211.15 220.353 211.15 Q226.14 211.15 229.52 214.321 Q232.899 217.492 232.899 222.909 Q232.899 228.488 229.427 231.589 Q225.955 234.668 219.635 234.668 Q217.46 234.668 215.191 234.298 Q212.946 233.927 210.538 233.187 L210.538 228.488 Q212.622 229.622 214.844 230.177 Q217.066 230.733 219.543 230.733 Q223.547 230.733 225.885 228.627 Q228.223 226.52 228.223 222.909 Q228.223 219.298 225.885 217.191 Q223.547 215.085 219.543 215.085 Q217.668 215.085 215.793 215.502 Q213.941 215.918 211.997 216.798 L211.997 199.437 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M252.112 202.516 Q248.501 202.516 246.672 206.08 Q244.867 209.622 244.867 216.752 Q244.867 223.858 246.672 227.423 Q248.501 230.964 252.112 230.964 Q255.746 230.964 257.552 227.423 Q259.381 223.858 259.381 216.752 Q259.381 209.622 257.552 206.08 Q255.746 202.516 252.112 202.516 M252.112 198.812 Q257.922 198.812 260.978 203.418 Q264.057 208.002 264.057 216.752 Q264.057 225.478 260.978 230.085 Q257.922 234.668 252.112 234.668 Q246.302 234.668 243.223 230.085 Q240.168 225.478 240.168 216.752 Q240.168 208.002 243.223 203.418 Q246.302 198.812 252.112 198.812 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M96.9847 1144.81 L84.2214 1144.81 L84.2214 1155.31 L78.9379 1155.31 L78.9379 1138.44 L99.34 1138.44 Q101.982 1142.17 103.35 1146.65 Q104.687 1151.14 104.687 1156.23 Q104.687 1167.37 98.1942 1173.68 Q91.6693 1179.95 80.0519 1179.95 Q68.4027 1179.95 61.9096 1173.68 Q55.3848 1167.37 55.3848 1156.23 Q55.3848 1151.59 56.5306 1147.42 Q57.6764 1143.22 59.9044 1139.68 L66.7476 1139.68 Q63.7239 1143.25 62.1961 1147.26 Q60.6683 1151.27 60.6683 1155.69 Q60.6683 1164.41 65.5381 1168.81 Q70.4079 1173.17 80.0519 1173.17 Q89.6641 1173.17 94.5339 1168.81 Q99.4036 1164.41 99.4036 1155.69 Q99.4036 1152.29 98.8307 1149.61 Q98.226 1146.94 96.9847 1144.81 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M54.239 1126.95 L54.239 1121.09 L103.764 1121.09 L103.764 1126.95 L54.239 1126.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M72.2221 1095.03 Q72.2221 1099.74 75.9142 1102.47 Q79.5745 1105.21 85.972 1105.21 Q92.3695 1105.21 96.0617 1102.51 Q99.7219 1099.77 99.7219 1095.03 Q99.7219 1090.35 96.0298 1087.61 Q92.3377 1084.87 85.972 1084.87 Q79.6381 1084.87 75.946 1087.61 Q72.2221 1090.35 72.2221 1095.03 M67.2568 1095.03 Q67.2568 1087.39 72.2221 1083.03 Q77.1873 1078.67 85.972 1078.67 Q94.7249 1078.67 99.7219 1083.03 Q104.687 1087.39 104.687 1095.03 Q104.687 1102.7 99.7219 1107.06 Q94.7249 1111.39 85.972 1111.39 Q77.1873 1111.39 72.2221 1107.06 Q67.2568 1102.7 67.2568 1095.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M85.972 1043.37 Q79.5108 1043.37 75.8505 1046.04 Q72.1584 1048.68 72.1584 1053.33 Q72.1584 1057.98 75.8505 1060.65 Q79.5108 1063.29 85.972 1063.29 Q92.4332 1063.29 96.1253 1060.65 Q99.7856 1057.98 99.7856 1053.33 Q99.7856 1048.68 96.1253 1046.04 Q92.4332 1043.37 85.972 1043.37 M73.5271 1063.29 Q70.3442 1061.45 68.8164 1058.65 Q67.2568 1055.81 67.2568 1051.9 Q67.2568 1045.41 72.4131 1041.36 Q77.5693 1037.29 85.972 1037.29 Q94.3747 1037.29 99.531 1041.36 Q104.687 1045.41 104.687 1051.9 Q104.687 1055.81 103.159 1058.65 Q101.6 1061.45 98.417 1063.29 L103.764 1063.29 L103.764 1069.18 L54.239 1069.18 L54.239 1063.29 L73.5271 1063.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M85.8447 1011.38 Q85.8447 1018.48 87.4679 1021.22 Q89.0912 1023.95 93.0061 1023.95 Q96.1253 1023.95 97.9714 1021.92 Q99.7856 1019.85 99.7856 1016.32 Q99.7856 1011.45 96.3481 1008.52 Q92.8788 1005.56 87.1497 1005.56 L85.8447 1005.56 L85.8447 1011.38 M83.4257 999.701 L103.764 999.701 L103.764 1005.56 L98.3533 1005.56 Q101.6 1007.56 103.159 1010.55 Q104.687 1013.55 104.687 1017.87 Q104.687 1023.35 101.632 1026.6 Q98.5443 1029.81 93.3881 1029.81 Q87.3725 1029.81 84.3169 1025.8 Q81.2614 1021.76 81.2614 1013.77 L81.2614 1005.56 L80.6885 1005.56 Q76.6462 1005.56 74.4501 1008.23 Q72.2221 1010.87 72.2221 1015.68 Q72.2221 1018.73 72.9541 1021.63 Q73.6862 1024.53 75.1503 1027.2 L69.7395 1027.2 Q68.4981 1023.99 67.8934 1020.96 Q67.2568 1017.94 67.2568 1015.07 Q67.2568 1007.34 71.2672 1003.52 Q75.2776 999.701 83.4257 999.701 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M54.239 987.638 L54.239 981.781 L103.764 981.781 L103.764 987.638 L54.239 987.638 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M56.2442 948.552 L56.2442 938.972 L88.5819 926.845 L56.2442 914.655 L56.2442 905.074 L103.764 905.074 L103.764 911.345 L62.037 911.345 L94.6294 923.599 L94.6294 930.06 L62.037 942.314 L103.764 942.314 L103.764 948.552 L56.2442 948.552 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M84.4761 862.074 L87.3406 862.074 L87.3406 889.001 Q93.3881 888.619 96.5709 885.372 Q99.7219 882.094 99.7219 876.27 Q99.7219 872.896 98.8944 869.745 Q98.0669 866.562 96.4118 863.443 L101.95 863.443 Q103.287 866.594 103.987 869.904 Q104.687 873.214 104.687 876.62 Q104.687 885.15 99.7219 890.147 Q94.7567 895.112 86.2903 895.112 Q77.5374 895.112 72.4131 890.401 Q67.2568 885.659 67.2568 877.638 Q67.2568 870.445 71.9038 866.275 Q76.5189 862.074 84.4761 862.074 M82.7573 867.93 Q77.9512 867.994 75.0866 870.636 Q72.2221 873.246 72.2221 877.574 Q72.2221 882.476 74.9912 885.436 Q77.7602 888.364 82.7892 888.81 L82.7573 867.93 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M85.8447 836.261 Q85.8447 843.359 87.4679 846.096 Q89.0912 848.833 93.0061 848.833 Q96.1253 848.833 97.9714 846.796 Q99.7856 844.727 99.7856 841.194 Q99.7856 836.325 96.3481 833.396 Q92.8788 830.436 87.1497 830.436 L85.8447 830.436 L85.8447 836.261 M83.4257 824.58 L103.764 824.58 L103.764 830.436 L98.3533 830.436 Q101.6 832.442 103.159 835.434 Q104.687 838.425 104.687 842.754 Q104.687 848.229 101.632 851.475 Q98.5443 854.69 93.3881 854.69 Q87.3725 854.69 84.3169 850.679 Q81.2614 846.637 81.2614 838.648 L81.2614 830.436 L80.6885 830.436 Q76.6462 830.436 74.4501 833.11 Q72.2221 835.752 72.2221 840.558 Q72.2221 843.613 72.9541 846.51 Q73.6862 849.406 75.1503 852.08 L69.7395 852.08 Q68.4981 848.865 67.8934 845.841 Q67.2568 842.818 67.2568 839.953 Q67.2568 832.219 71.2672 828.399 Q75.2776 824.58 83.4257 824.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M82.2481 782.885 L103.764 782.885 L103.764 788.741 L82.439 788.741 Q77.3783 788.741 74.8638 790.714 Q72.3494 792.688 72.3494 796.635 Q72.3494 801.377 75.3731 804.114 Q78.3968 806.851 83.6167 806.851 L103.764 806.851 L103.764 812.74 L68.1162 812.74 L68.1162 806.851 L73.6544 806.851 Q70.4397 804.751 68.8483 801.918 Q67.2568 799.053 67.2568 795.33 Q67.2568 789.187 71.0763 786.036 Q74.8638 782.885 82.2481 782.885 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M57.8038 721.742 L64.074 721.742 Q62.3234 725.402 61.464 728.649 Q60.6047 731.895 60.6047 734.919 Q60.6047 740.171 62.6417 743.035 Q64.6787 745.868 68.4345 745.868 Q71.5855 745.868 73.2088 743.99 Q74.8002 742.08 75.7869 736.797 L76.5826 732.914 Q77.9512 725.721 81.4205 722.315 Q84.858 718.877 90.6508 718.877 Q97.5576 718.877 101.122 723.524 Q104.687 728.14 104.687 737.083 Q104.687 740.457 103.923 744.277 Q103.159 748.064 101.663 752.138 L95.0431 752.138 Q97.2393 748.223 98.3533 744.468 Q99.4673 740.712 99.4673 737.083 Q99.4673 731.577 97.303 728.585 Q95.1386 725.593 91.1282 725.593 Q87.6271 725.593 85.6537 727.758 Q83.6804 729.89 82.6937 734.792 L81.9298 738.707 Q80.4975 745.9 77.442 749.115 Q74.3864 752.329 68.9437 752.329 Q62.6417 752.329 59.0132 747.905 Q55.3848 743.449 55.3848 735.651 Q55.3848 732.309 55.9895 728.84 Q56.5943 725.37 57.8038 721.742 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M84.4761 678.614 L87.3406 678.614 L87.3406 705.541 Q93.3881 705.159 96.5709 701.913 Q99.7219 698.634 99.7219 692.81 Q99.7219 689.436 98.8944 686.285 Q98.0669 683.102 96.4118 679.983 L101.95 679.983 Q103.287 683.134 103.987 686.444 Q104.687 689.754 104.687 693.16 Q104.687 701.69 99.7219 706.687 Q94.7567 711.652 86.2903 711.652 Q77.5374 711.652 72.4131 706.942 Q67.2568 702.199 67.2568 694.178 Q67.2568 686.985 71.9038 682.816 Q76.5189 678.614 84.4761 678.614 M82.7573 684.471 Q77.9512 684.534 75.0866 687.176 Q72.2221 689.786 72.2221 694.115 Q72.2221 699.016 74.9912 701.976 Q77.7602 704.905 82.7892 705.35 L82.7573 684.471 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M85.8447 652.801 Q85.8447 659.899 87.4679 662.636 Q89.0912 665.374 93.0061 665.374 Q96.1253 665.374 97.9714 663.337 Q99.7856 661.268 99.7856 657.735 Q99.7856 652.865 96.3481 649.937 Q92.8788 646.977 87.1497 646.977 L85.8447 646.977 L85.8447 652.801 M83.4257 641.12 L103.764 641.12 L103.764 646.977 L98.3533 646.977 Q101.6 648.982 103.159 651.974 Q104.687 654.966 104.687 659.294 Q104.687 664.769 101.632 668.015 Q98.5443 671.23 93.3881 671.23 Q87.3725 671.23 84.3169 667.22 Q81.2614 663.177 81.2614 655.189 L81.2614 646.977 L80.6885 646.977 Q76.6462 646.977 74.4501 649.65 Q72.2221 652.292 72.2221 657.098 Q72.2221 660.154 72.9541 663.05 Q73.6862 665.947 75.1503 668.62 L69.7395 668.62 Q68.4981 665.405 67.8934 662.382 Q67.2568 659.358 67.2568 656.493 Q67.2568 648.759 71.2672 644.94 Q75.2776 641.12 83.4257 641.12 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M56.2442 608.082 L56.2442 601.653 L98.3533 601.653 L98.3533 578.514 L103.764 578.514 L103.764 608.082 L56.2442 608.082 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M84.4761 542.675 L87.3406 542.675 L87.3406 569.602 Q93.3881 569.22 96.5709 565.973 Q99.7219 562.695 99.7219 556.87 Q99.7219 553.496 98.8944 550.345 Q98.0669 547.162 96.4118 544.043 L101.95 544.043 Q103.287 547.194 103.987 550.504 Q104.687 553.815 104.687 557.22 Q104.687 565.75 99.7219 570.747 Q94.7567 575.713 86.2903 575.713 Q77.5374 575.713 72.4131 571.002 Q67.2568 566.26 67.2568 558.239 Q67.2568 551.046 71.9038 546.876 Q76.5189 542.675 84.4761 542.675 M82.7573 548.531 Q77.9512 548.595 75.0866 551.237 Q72.2221 553.846 72.2221 558.175 Q72.2221 563.077 74.9912 566.037 Q77.7602 568.965 82.7892 569.411 L82.7573 548.531 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M68.1162 537.264 L68.1162 531.057 L98.035 519.917 L68.1162 508.777 L68.1162 502.571 L103.764 515.939 L103.764 523.896 L68.1162 537.264 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M84.4761 463.995 L87.3406 463.995 L87.3406 490.921 Q93.3881 490.54 96.5709 487.293 Q99.7219 484.015 99.7219 478.19 Q99.7219 474.816 98.8944 471.665 Q98.0669 468.482 96.4118 465.363 L101.95 465.363 Q103.287 468.514 103.987 471.824 Q104.687 475.135 104.687 478.54 Q104.687 487.07 99.7219 492.067 Q94.7567 497.033 86.2903 497.033 Q77.5374 497.033 72.4131 492.322 Q67.2568 487.579 67.2568 479.559 Q67.2568 472.365 71.9038 468.196 Q76.5189 463.995 84.4761 463.995 M82.7573 469.851 Q77.9512 469.915 75.0866 472.556 Q72.2221 475.166 72.2221 479.495 Q72.2221 484.397 74.9912 487.357 Q77.7602 490.285 82.7892 490.73 L82.7573 469.851 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M54.239 454.382 L54.239 448.526 L103.764 448.526 L103.764 454.382 L54.239 454.382 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M54.3026 401.483 Q61.6232 405.748 68.7846 407.817 Q75.946 409.886 83.2984 409.886 Q90.6508 409.886 97.8759 407.817 Q105.069 405.716 112.358 401.483 L112.358 406.576 Q104.878 411.35 97.6531 413.737 Q90.428 416.093 83.2984 416.093 Q76.2006 416.093 69.0074 413.737 Q61.8142 411.382 54.3026 406.576 L54.3026 401.483 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M74.9593 362.366 Q71.0126 360.17 69.1347 357.114 Q67.2568 354.059 67.2568 349.921 Q67.2568 344.351 71.1717 341.327 Q75.0548 338.304 82.2481 338.304 L103.764 338.304 L103.764 344.192 L82.439 344.192 Q77.3146 344.192 74.832 346.006 Q72.3494 347.82 72.3494 351.544 Q72.3494 356.096 75.3731 358.738 Q78.3968 361.379 83.6167 361.379 L103.764 361.379 L103.764 367.268 L82.439 367.268 Q77.2828 367.268 74.832 369.082 Q72.3494 370.896 72.3494 374.684 Q72.3494 379.171 75.4049 381.813 Q78.4286 384.455 83.6167 384.455 L103.764 384.455 L103.764 390.343 L68.1162 390.343 L68.1162 384.455 L73.6544 384.455 Q70.376 382.45 68.8164 379.649 Q67.2568 376.848 67.2568 372.997 Q67.2568 369.114 69.2302 366.408 Q71.2036 363.671 74.9593 362.366 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M74.9593 298.868 Q71.0126 296.672 69.1347 293.616 Q67.2568 290.561 67.2568 286.423 Q67.2568 280.853 71.1717 277.829 Q75.0548 274.806 82.2481 274.806 L103.764 274.806 L103.764 280.694 L82.439 280.694 Q77.3146 280.694 74.832 282.508 Q72.3494 284.322 72.3494 288.046 Q72.3494 292.598 75.3731 295.24 Q78.3968 297.881 83.6167 297.881 L103.764 297.881 L103.764 303.77 L82.439 303.77 Q77.2828 303.77 74.832 305.584 Q72.3494 307.398 72.3494 311.186 Q72.3494 315.674 75.4049 318.315 Q78.4286 320.957 83.6167 320.957 L103.764 320.957 L103.764 326.845 L68.1162 326.845 L68.1162 320.957 L73.6544 320.957 Q70.376 318.952 68.8164 316.151 Q67.2568 313.35 67.2568 309.499 Q67.2568 305.616 69.2302 302.91 Q71.2036 300.173 74.9593 298.868 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M54.3026 264.048 L54.3026 258.955 Q61.8142 254.181 69.0074 251.826 Q76.2006 249.438 83.2984 249.438 Q90.428 249.438 97.6531 251.826 Q104.878 254.181 112.358 258.955 L112.358 264.048 Q105.069 259.814 97.8759 257.746 Q90.6508 255.645 83.2984 255.645 Q75.946 255.645 68.7846 257.746 Q61.6232 259.814 54.3026 264.048 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip212)" style="stroke:#0000ff; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="323.85,1282.48 332.821,1273.65 341.791,1265.84 350.762,1258.88 359.733,1254.23 368.704,1249.27 377.675,1243.56 386.645,1239.09 395.616,1232.79 404.587,1224.5 413.558,1220.27 422.528,1214.41 431.499,1210.24 440.47,1205.9 449.441,1201.4 458.411,1196.39 467.382,1188.84 476.353,1180.96 485.324,1176.63 494.295,1170.31 503.265,1162.13 512.236,1154.91 521.207,1150.01 530.178,1146.6 539.148,1144.21 548.119,1138.88 557.09,1132.2 566.061,1128.43 575.032,1124.84 584.002,1121.61 592.973,1118.32 601.944,1115.18 610.915,1111.06 619.885,1106.84 628.856,1099.45 637.827,1091 646.798,1086.15 655.768,1083.25 664.739,1078.51 673.71,1072.26 682.681,1065.63 691.652,1058.12 700.622,1052.2 709.593,1045.97 718.564,1039.69 727.535,1032.99 736.505,1023.83 745.476,1016.38 754.447,1008.62 763.418,1003.81 772.388,995.657 781.359,986.409 790.33,977.868 799.301,972.13 808.272,964.118 817.242,956.645 826.213,948.644 835.184,938.345 844.155,927.995 853.125,918.148 862.096,906.537 871.067,895.579 880.038,885.254 889.009,874.909 897.979,862.476 906.95,851.687 915.921,843.48 924.892,834.888 933.862,826.866 942.833,819.187 951.804,812.841 960.775,803.951 969.745,793.92 978.716,782.973 987.687,775.117 996.658,768.559 1005.63,763.064 1014.6,754.049 1023.57,744.809 1032.54,736.094 1041.51,728.478 1050.48,719.417 1059.45,711.093 1068.42,702.39 1077.39,697.929 1086.37,691.912 1095.34,685.063 1104.31,677.763 1113.28,671.301 1122.25,662.735 1131.22,655.054 1140.19,649.298 1149.16,641.828 1158.13,632.159 1167.1,626.011 1176.07,618.934 1185.04,613.534 1194.01,603.195 1202.99,594.438 1211.96,584.395 1220.93,572.769 1229.9,560.368 1238.87,551.397 1247.84,539.522 1256.81,530.453 1265.78,521.408 1274.75,511.696 1283.72,499.743 1292.69,487.247 1301.66,476.424 1310.63,462.836 1319.61,449.661 1328.58,439.894 1337.55,429.545 1346.52,418.197 1355.49,404.678 1364.46,392.796 1373.43,378.712 1382.4,362.289 1391.37,349.898 1400.34,337.474 1409.31,322.653 1418.28,307.071 1427.25,291.564 1436.23,277.34 1445.2,261.022 1454.17,245.329 1463.14,229.426 1472.11,215.575 1481.08,199.757 1490.05,182.726 1499.02,168.012 1507.99,152.758 1516.96,136.873 "/>
<circle clip-path="url(#clip212)" cx="323.85" cy="1287.02" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="332.821" cy="1258.36" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="341.791" cy="1344.36" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="350.762" cy="1317.23" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="359.733" cy="1210.24" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="368.704" cy="1217.91" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="377.675" cy="1230.2" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="386.645" cy="1258.87" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="395.616" cy="1248.12" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="404.587" cy="1239.93" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="413.558" cy="1229.69" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="422.528" cy="1238.39" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="431.499" cy="1222.01" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="440.47" cy="1197.95" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="449.441" cy="1247.61" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="458.411" cy="1187.2" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="467.382" cy="1226.11" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="476.353" cy="1199.48" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="485.324" cy="1151.36" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="494.295" cy="1112.97" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="503.265" cy="1140.61" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="512.236" cy="1142.66" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="521.207" cy="1119.11" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="530.178" cy="1078.16" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="539.148" cy="1130.89" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="548.119" cy="1159.04" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="557.09" cy="1124.23" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="566.061" cy="1131.4" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="575.032" cy="1144.2" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="584.002" cy="1121.16" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="592.973" cy="1121.67" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="601.944" cy="1079.18" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="610.915" cy="1095.05" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="619.885" cy="1085.84" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="628.856" cy="1053.59" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="637.827" cy="1013.14" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="646.798" cy="1022.87" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="655.768" cy="1045.91" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="664.739" cy="1054.1" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="673.71" cy="1045.91" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="682.681" cy="1039.25" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="691.652" cy="1028.5" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="700.622" cy="1033.11" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="709.593" cy="1026.97" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="718.564" cy="1064.34" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="727.535" cy="1054.1" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="736.505" cy="1020.82" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="745.476" cy="1026.45" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="754.447" cy="1046.42" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="763.418" cy="1039.25" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="772.388" cy="1018.78" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="781.359" cy="1020.82" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="790.33" cy="991.643" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="799.301" cy="969.631" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="808.272" cy="997.275" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="817.242" cy="968.607" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="826.213" cy="988.572" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="835.184" cy="959.392" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="844.155" cy="946.082" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="853.125" cy="919.462" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="862.096" cy="946.594" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="871.067" cy="884.139" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="880.038" cy="884.139" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="889.009" cy="884.139" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="897.979" cy="916.902" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="906.95" cy="902.568" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="915.921" cy="864.174" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="924.892" cy="848.304" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="933.862" cy="823.732" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="942.833" cy="830.387" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="951.804" cy="819.125" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="960.775" cy="770.492" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="969.745" cy="783.802" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="978.716" cy="761.277" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="987.687" cy="776.123" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="996.658" cy="771.516" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1005.63" cy="797.624" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1014.6" cy="729.026" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1023.57" cy="721.859" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1032.54" cy="720.323" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1041.51" cy="701.382" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1050.48" cy="669.643" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1059.45" cy="696.263" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1068.42" cy="704.453" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1077.39" cy="744.896" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1086.37" cy="687.56" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1095.34" cy="715.204" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1104.31" cy="708.037" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1113.28" cy="703.942" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1122.25" cy="669.131" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1131.22" cy="678.857" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1140.19" cy="652.237" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1149.16" cy="605.652" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1158.13" cy="636.367" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1167.1" cy="575.448" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1176.07" cy="583.639" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1185.04" cy="588.758" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1194.01" cy="597.461" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1202.99" cy="564.698" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1211.96" cy="589.782" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1220.93" cy="559.067" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1229.9" cy="495.588" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1238.87" cy="525.28" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1247.84" cy="482.278" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1256.81" cy="486.885" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1265.78" cy="539.614" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1274.75" cy="536.542" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1283.72" cy="533.471" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1292.69" cy="509.41" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1301.66" cy="486.373" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1310.63" cy="474.599" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1319.61" cy="461.801" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1328.58" cy="458.218" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1337.55" cy="467.432" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1346.52" cy="440.812" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1355.49" cy="425.966" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1364.46" cy="404.977" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1373.43" cy="369.143" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1382.4" cy="419.823" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1391.37" cy="376.31" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1400.34" cy="369.654" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1409.31" cy="341.499" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1418.28" cy="329.212" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1427.25" cy="282.115" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1436.23" cy="284.163" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1445.2" cy="283.651" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1454.17" cy="261.638" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1463.14" cy="251.912" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1472.11" cy="207.374" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1481.08" cy="177.683" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1490.05" cy="138.264" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1499.02" cy="130.074" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1507.99" cy="85.0242" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip212)" cx="1516.96" cy="128.026" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<path clip-path="url(#clip210)" d="M330.213 247.261 L765.084 247.261 L765.084 91.7406 L330.213 91.7406  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip210)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="330.213,247.261 765.084,247.261 765.084,91.7406 330.213,91.7406 330.213,247.261 "/>
<polyline clip-path="url(#clip210)" style="stroke:#0000ff; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="344.265,143.581 428.579,143.581 "/>
<path clip-path="url(#clip210)" d="M442.631 126.301 L449.598 126.301 L458.418 149.819 L467.284 126.301 L474.251 126.301 L474.251 160.861 L469.691 160.861 L469.691 130.514 L460.779 154.217 L456.08 154.217 L447.168 130.514 L447.168 160.861 L442.631 160.861 L442.631 126.301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M493.395 137.921 Q489.969 137.921 487.978 140.606 Q485.987 143.268 485.987 147.921 Q485.987 152.574 487.955 155.259 Q489.945 157.921 493.395 157.921 Q496.797 157.921 498.788 155.236 Q500.779 152.55 500.779 147.921 Q500.779 143.314 498.788 140.629 Q496.797 137.921 493.395 137.921 M493.395 134.31 Q498.95 134.31 502.121 137.921 Q505.293 141.532 505.293 147.921 Q505.293 154.287 502.121 157.921 Q498.95 161.532 493.395 161.532 Q487.816 161.532 484.645 157.921 Q481.496 154.287 481.496 147.921 Q481.496 141.532 484.645 137.921 Q487.816 134.31 493.395 134.31 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M529.413 138.87 L529.413 124.842 L533.672 124.842 L533.672 160.861 L529.413 160.861 L529.413 156.972 Q528.07 159.287 526.01 160.421 Q523.973 161.532 521.103 161.532 Q516.404 161.532 513.441 157.782 Q510.501 154.032 510.501 147.921 Q510.501 141.81 513.441 138.06 Q516.404 134.31 521.103 134.31 Q523.973 134.31 526.01 135.444 Q528.07 136.555 529.413 138.87 M514.899 147.921 Q514.899 152.62 516.82 155.305 Q518.765 157.967 522.144 157.967 Q525.524 157.967 527.468 155.305 Q529.413 152.62 529.413 147.921 Q529.413 143.222 527.468 140.56 Q525.524 137.875 522.144 137.875 Q518.765 137.875 516.82 140.56 Q514.899 143.222 514.899 147.921 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M564.621 146.833 L564.621 148.916 L545.038 148.916 Q545.316 153.314 547.677 155.629 Q550.061 157.921 554.297 157.921 Q556.751 157.921 559.042 157.319 Q561.357 156.717 563.626 155.513 L563.626 159.541 Q561.334 160.513 558.927 161.023 Q556.519 161.532 554.042 161.532 Q547.839 161.532 544.204 157.921 Q540.593 154.31 540.593 148.152 Q540.593 141.787 544.019 138.06 Q547.468 134.31 553.302 134.31 Q558.533 134.31 561.565 137.689 Q564.621 141.046 564.621 146.833 M560.362 145.583 Q560.315 142.088 558.394 140.004 Q556.496 137.921 553.348 137.921 Q549.783 137.921 547.63 139.935 Q545.501 141.949 545.177 145.606 L560.362 145.583 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M571.612 124.842 L575.871 124.842 L575.871 160.861 L571.612 160.861 L571.612 124.842 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M600.037 126.301 L619.898 126.301 L619.898 130.236 L604.713 130.236 L604.713 140.421 L618.417 140.421 L618.417 144.356 L604.713 144.356 L604.713 160.861 L600.037 160.861 L600.037 126.301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M623.672 134.935 L627.931 134.935 L627.931 160.861 L623.672 160.861 L623.672 134.935 M623.672 124.842 L627.931 124.842 L627.931 130.236 L623.672 130.236 L623.672 124.842 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M641.056 127.574 L641.056 134.935 L649.829 134.935 L649.829 138.245 L641.056 138.245 L641.056 152.319 Q641.056 155.49 641.912 156.393 Q642.792 157.296 645.454 157.296 L649.829 157.296 L649.829 160.861 L645.454 160.861 Q640.523 160.861 638.648 159.032 Q636.773 157.18 636.773 152.319 L636.773 138.245 L633.648 138.245 L633.648 134.935 L636.773 134.935 L636.773 127.574 L641.056 127.574 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><circle clip-path="url(#clip210)" cx="386.422" cy="195.421" r="20.48" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<path clip-path="url(#clip210)" d="M458.649 181.312 Q453.557 181.312 450.548 185.108 Q447.561 188.904 447.561 195.455 Q447.561 201.983 450.548 205.779 Q453.557 209.576 458.649 209.576 Q463.742 209.576 466.705 205.779 Q469.691 201.983 469.691 195.455 Q469.691 188.904 466.705 185.108 Q463.742 181.312 458.649 181.312 M458.649 177.516 Q465.918 177.516 470.27 182.4 Q474.621 187.261 474.621 195.455 Q474.621 203.627 470.27 208.511 Q465.918 213.372 458.649 213.372 Q451.358 213.372 446.983 208.511 Q442.631 203.65 442.631 195.455 Q442.631 187.261 446.983 182.4 Q451.358 177.516 458.649 177.516 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M500.362 199.761 Q500.362 195.062 498.418 192.4 Q496.496 189.715 493.117 189.715 Q489.737 189.715 487.793 192.4 Q485.871 195.062 485.871 199.761 Q485.871 204.46 487.793 207.145 Q489.737 209.807 493.117 209.807 Q496.496 209.807 498.418 207.145 Q500.362 204.46 500.362 199.761 M485.871 190.71 Q487.214 188.395 489.251 187.284 Q491.311 186.15 494.158 186.15 Q498.881 186.15 501.82 189.9 Q504.783 193.65 504.783 199.761 Q504.783 205.872 501.82 209.622 Q498.881 213.372 494.158 213.372 Q491.311 213.372 489.251 212.261 Q487.214 211.127 485.871 208.812 L485.871 212.701 L481.589 212.701 L481.589 176.682 L485.871 176.682 L485.871 190.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M528.371 187.539 L528.371 191.566 Q526.566 190.641 524.621 190.178 Q522.677 189.715 520.593 189.715 Q517.422 189.715 515.825 190.687 Q514.251 191.659 514.251 193.603 Q514.251 195.085 515.385 195.941 Q516.519 196.775 519.945 197.539 L521.404 197.863 Q525.941 198.835 527.839 200.617 Q529.76 202.377 529.76 205.548 Q529.76 209.159 526.89 211.265 Q524.043 213.372 519.043 213.372 Q516.959 213.372 514.691 212.955 Q512.445 212.562 509.945 211.752 L509.945 207.353 Q512.306 208.58 514.598 209.205 Q516.89 209.807 519.135 209.807 Q522.144 209.807 523.765 208.789 Q525.385 207.747 525.385 205.872 Q525.385 204.136 524.205 203.21 Q523.047 202.284 519.089 201.428 L517.607 201.08 Q513.649 200.247 511.89 198.534 Q510.131 196.798 510.131 193.789 Q510.131 190.131 512.723 188.141 Q515.316 186.15 520.084 186.15 Q522.445 186.15 524.529 186.497 Q526.612 186.844 528.371 187.539 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M558.718 198.673 L558.718 200.756 L539.135 200.756 Q539.413 205.154 541.774 207.469 Q544.158 209.761 548.394 209.761 Q550.848 209.761 553.14 209.159 Q555.454 208.557 557.723 207.353 L557.723 211.381 Q555.431 212.353 553.024 212.863 Q550.616 213.372 548.14 213.372 Q541.936 213.372 538.302 209.761 Q534.691 206.15 534.691 199.992 Q534.691 193.627 538.117 189.9 Q541.566 186.15 547.399 186.15 Q552.63 186.15 555.663 189.529 Q558.718 192.886 558.718 198.673 M554.459 197.423 Q554.413 193.928 552.491 191.844 Q550.593 189.761 547.445 189.761 Q543.88 189.761 541.728 191.775 Q539.598 193.789 539.274 197.446 L554.459 197.423 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M580.732 190.756 Q580.014 190.34 579.158 190.154 Q578.325 189.946 577.306 189.946 Q573.695 189.946 571.751 192.307 Q569.829 194.645 569.829 199.043 L569.829 212.701 L565.547 212.701 L565.547 186.775 L569.829 186.775 L569.829 190.803 Q571.172 188.441 573.325 187.307 Q575.477 186.15 578.556 186.15 Q578.996 186.15 579.528 186.219 Q580.061 186.266 580.709 186.381 L580.732 190.756 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M582.144 186.775 L586.658 186.775 L594.76 208.534 L602.862 186.775 L607.375 186.775 L597.653 212.701 L591.866 212.701 L582.144 186.775 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M625.037 199.668 Q619.875 199.668 617.885 200.849 Q615.894 202.029 615.894 204.877 Q615.894 207.145 617.375 208.488 Q618.88 209.807 621.449 209.807 Q624.991 209.807 627.121 207.307 Q629.273 204.784 629.273 200.617 L629.273 199.668 L625.037 199.668 M633.533 197.909 L633.533 212.701 L629.273 212.701 L629.273 208.765 Q627.815 211.127 625.639 212.261 Q623.463 213.372 620.315 213.372 Q616.334 213.372 613.973 211.15 Q611.635 208.904 611.635 205.154 Q611.635 200.779 614.551 198.557 Q617.491 196.335 623.301 196.335 L629.273 196.335 L629.273 195.918 Q629.273 192.979 627.329 191.381 Q625.408 189.761 621.912 189.761 Q619.69 189.761 617.584 190.293 Q615.477 190.826 613.533 191.891 L613.533 187.955 Q615.871 187.053 618.07 186.613 Q620.269 186.15 622.352 186.15 Q627.977 186.15 630.755 189.066 Q633.533 191.983 633.533 197.909 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M646.519 179.414 L646.519 186.775 L655.292 186.775 L655.292 190.085 L646.519 190.085 L646.519 204.159 Q646.519 207.33 647.375 208.233 Q648.255 209.136 650.917 209.136 L655.292 209.136 L655.292 212.701 L650.917 212.701 Q645.986 212.701 644.111 210.872 Q642.236 209.02 642.236 204.159 L642.236 190.085 L639.111 190.085 L639.111 186.775 L642.236 186.775 L642.236 179.414 L646.519 179.414 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M660.894 186.775 L665.153 186.775 L665.153 212.701 L660.894 212.701 L660.894 186.775 M660.894 176.682 L665.153 176.682 L665.153 182.076 L660.894 182.076 L660.894 176.682 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M684.111 189.761 Q680.685 189.761 678.694 192.446 Q676.704 195.108 676.704 199.761 Q676.704 204.414 678.671 207.099 Q680.662 209.761 684.111 209.761 Q687.514 209.761 689.505 207.076 Q691.495 204.39 691.495 199.761 Q691.495 195.154 689.505 192.469 Q687.514 189.761 684.111 189.761 M684.111 186.15 Q689.667 186.15 692.838 189.761 Q696.009 193.372 696.009 199.761 Q696.009 206.127 692.838 209.761 Q689.667 213.372 684.111 213.372 Q678.532 213.372 675.361 209.761 Q672.213 206.127 672.213 199.761 Q672.213 193.372 675.361 189.761 Q678.532 186.15 684.111 186.15 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M724.62 197.053 L724.62 212.701 L720.361 212.701 L720.361 197.191 Q720.361 193.511 718.926 191.682 Q717.491 189.854 714.62 189.854 Q711.171 189.854 709.18 192.053 Q707.19 194.252 707.19 198.048 L707.19 212.701 L702.907 212.701 L702.907 186.775 L707.19 186.775 L707.19 190.803 Q708.717 188.465 710.778 187.307 Q712.861 186.15 715.569 186.15 Q720.037 186.15 722.328 188.928 Q724.62 191.682 724.62 197.053 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip210)" d="M749.643 187.539 L749.643 191.566 Q747.838 190.641 745.893 190.178 Q743.949 189.715 741.865 189.715 Q738.694 189.715 737.097 190.687 Q735.523 191.659 735.523 193.603 Q735.523 195.085 736.657 195.941 Q737.791 196.775 741.217 197.539 L742.676 197.863 Q747.213 198.835 749.111 200.617 Q751.032 202.377 751.032 205.548 Q751.032 209.159 748.162 211.265 Q745.314 213.372 740.314 213.372 Q738.231 213.372 735.963 212.955 Q733.717 212.562 731.217 211.752 L731.217 207.353 Q733.578 208.58 735.87 209.205 Q738.162 209.807 740.407 209.807 Q743.416 209.807 745.037 208.789 Q746.657 207.747 746.657 205.872 Q746.657 204.136 745.477 203.21 Q744.319 202.284 740.361 201.428 L738.879 201.08 Q734.921 200.247 733.162 198.534 Q731.403 196.798 731.403 193.789 Q731.403 190.131 733.995 188.141 Q736.588 186.15 741.356 186.15 Q743.717 186.15 745.801 186.497 Q747.884 186.844 749.643 187.539 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /></svg>
<p>(a) Sea-level hindcast using the fitted Grinsted model and the model
residuals.</p>
</div>
</div></td>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="400" height="400" viewBox="0 0 1600 1600">
<defs>
  <clipPath id="clip270">
    <rect x="0" y="0" width="1600" height="1600"/>
  </clipPath>
</defs>
<path clip-path="url(#clip270)" d="M0 1600 L1600 1600 L1600 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip271">
    <rect x="320" y="159" width="1121" height="1121"/>
  </clipPath>
</defs>
<path clip-path="url(#clip270)" d="M257.895 1382.14 L1552.76 1382.14 L1552.76 47.2441 L257.895 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip272">
    <rect x="257" y="47" width="1296" height="1336"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="478.236,1382.14 478.236,47.2441 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="707.854,1382.14 707.854,47.2441 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="937.472,1382.14 937.472,47.2441 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1167.09,1382.14 1167.09,47.2441 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1396.71,1382.14 1396.71,47.2441 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="257.895,1330.45 1552.76,1330.45 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="257.895,1124.31 1552.76,1124.31 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="257.895,918.168 1552.76,918.168 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="257.895,712.027 1552.76,712.027 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="257.895,505.886 1552.76,505.886 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="257.895,299.745 1552.76,299.745 "/>
<polyline clip-path="url(#clip272)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="257.895,93.6035 1552.76,93.6035 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,1382.14 1552.76,1382.14 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="478.236,1382.14 478.236,1363.24 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="707.854,1382.14 707.854,1363.24 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="937.472,1382.14 937.472,1363.24 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1167.09,1382.14 1167.09,1363.24 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1396.71,1382.14 1396.71,1363.24 "/>
<path clip-path="url(#clip270)" d="M422.762 1434.2 L430.401 1434.2 L430.401 1407.84 L422.09 1409.51 L422.09 1405.25 L430.354 1403.58 L435.03 1403.58 L435.03 1434.2 L442.669 1434.2 L442.669 1438.14 L422.762 1438.14 L422.762 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M452.252 1437.42 L452.252 1433.16 Q454.012 1434 455.817 1434.44 Q457.623 1434.88 459.359 1434.88 Q463.988 1434.88 466.419 1431.77 Q468.873 1428.65 469.22 1422.31 Q467.877 1424.3 465.817 1425.36 Q463.757 1426.43 461.257 1426.43 Q456.072 1426.43 453.039 1423.3 Q450.03 1420.15 450.03 1414.71 Q450.03 1409.39 453.178 1406.17 Q456.326 1402.95 461.558 1402.95 Q467.553 1402.95 470.701 1407.56 Q473.873 1412.14 473.873 1420.89 Q473.873 1429.07 469.984 1433.95 Q466.118 1438.81 459.567 1438.81 Q457.808 1438.81 456.002 1438.46 Q454.197 1438.12 452.252 1437.42 M461.558 1422.77 Q464.706 1422.77 466.535 1420.62 Q468.387 1418.46 468.387 1414.71 Q468.387 1410.99 466.535 1408.83 Q464.706 1406.66 461.558 1406.66 Q458.41 1406.66 456.558 1408.83 Q454.729 1410.99 454.729 1414.71 Q454.729 1418.46 456.558 1420.62 Q458.41 1422.77 461.558 1422.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M492.275 1406.66 Q488.664 1406.66 486.835 1410.22 Q485.03 1413.76 485.03 1420.89 Q485.03 1428 486.835 1431.57 Q488.664 1435.11 492.275 1435.11 Q495.91 1435.11 497.715 1431.57 Q499.544 1428 499.544 1420.89 Q499.544 1413.76 497.715 1410.22 Q495.91 1406.66 492.275 1406.66 M492.275 1402.95 Q498.085 1402.95 501.141 1407.56 Q504.22 1412.14 504.22 1420.89 Q504.22 1429.62 501.141 1434.23 Q498.085 1438.81 492.275 1438.81 Q486.465 1438.81 483.386 1434.23 Q480.331 1429.62 480.331 1420.89 Q480.331 1412.14 483.386 1407.56 Q486.465 1402.95 492.275 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M522.437 1406.66 Q518.826 1406.66 516.997 1410.22 Q515.192 1413.76 515.192 1420.89 Q515.192 1428 516.997 1431.57 Q518.826 1435.11 522.437 1435.11 Q526.071 1435.11 527.877 1431.57 Q529.706 1428 529.706 1420.89 Q529.706 1413.76 527.877 1410.22 Q526.071 1406.66 522.437 1406.66 M522.437 1402.95 Q528.247 1402.95 531.303 1407.56 Q534.382 1412.14 534.382 1420.89 Q534.382 1429.62 531.303 1434.23 Q528.247 1438.81 522.437 1438.81 Q516.627 1438.81 513.548 1434.23 Q510.493 1429.62 510.493 1420.89 Q510.493 1412.14 513.548 1407.56 Q516.627 1402.95 522.437 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M652.877 1434.2 L660.516 1434.2 L660.516 1407.84 L652.206 1409.51 L652.206 1405.25 L660.47 1403.58 L665.146 1403.58 L665.146 1434.2 L672.785 1434.2 L672.785 1438.14 L652.877 1438.14 L652.877 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M682.368 1437.42 L682.368 1433.16 Q684.127 1434 685.933 1434.44 Q687.738 1434.88 689.474 1434.88 Q694.104 1434.88 696.535 1431.77 Q698.988 1428.65 699.335 1422.31 Q697.993 1424.3 695.933 1425.36 Q693.872 1426.43 691.373 1426.43 Q686.187 1426.43 683.155 1423.3 Q680.146 1420.15 680.146 1414.71 Q680.146 1409.39 683.294 1406.17 Q686.442 1402.95 691.673 1402.95 Q697.669 1402.95 700.817 1407.56 Q703.988 1412.14 703.988 1420.89 Q703.988 1429.07 700.099 1433.95 Q696.234 1438.81 689.683 1438.81 Q687.923 1438.81 686.118 1438.46 Q684.312 1438.12 682.368 1437.42 M691.673 1422.77 Q694.822 1422.77 696.65 1420.62 Q698.502 1418.46 698.502 1414.71 Q698.502 1410.99 696.65 1408.83 Q694.822 1406.66 691.673 1406.66 Q688.525 1406.66 686.673 1408.83 Q684.845 1410.99 684.845 1414.71 Q684.845 1418.46 686.673 1420.62 Q688.525 1422.77 691.673 1422.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M716.419 1434.2 L732.738 1434.2 L732.738 1438.14 L710.794 1438.14 L710.794 1434.2 Q713.456 1431.45 718.039 1426.82 Q722.645 1422.17 723.826 1420.83 Q726.071 1418.3 726.951 1416.57 Q727.854 1414.81 727.854 1413.12 Q727.854 1410.36 725.909 1408.63 Q723.988 1406.89 720.886 1406.89 Q718.687 1406.89 716.233 1407.65 Q713.803 1408.42 711.025 1409.97 L711.025 1405.25 Q713.849 1404.11 716.303 1403.53 Q718.757 1402.95 720.794 1402.95 Q726.164 1402.95 729.358 1405.64 Q732.553 1408.33 732.553 1412.82 Q732.553 1414.95 731.743 1416.87 Q730.956 1418.76 728.849 1421.36 Q728.27 1422.03 725.169 1425.25 Q722.067 1428.44 716.419 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M742.599 1403.58 L760.955 1403.58 L760.955 1407.52 L746.881 1407.52 L746.881 1415.99 Q747.9 1415.64 748.918 1415.48 Q749.937 1415.29 750.956 1415.29 Q756.743 1415.29 760.122 1418.46 Q763.502 1421.64 763.502 1427.05 Q763.502 1432.63 760.03 1435.73 Q756.557 1438.81 750.238 1438.81 Q748.062 1438.81 745.794 1438.44 Q743.548 1438.07 741.141 1437.33 L741.141 1432.63 Q743.224 1433.76 745.446 1434.32 Q747.668 1434.88 750.145 1434.88 Q754.15 1434.88 756.488 1432.77 Q758.826 1430.66 758.826 1427.05 Q758.826 1423.44 756.488 1421.33 Q754.15 1419.23 750.145 1419.23 Q748.27 1419.23 746.395 1419.64 Q744.544 1420.06 742.599 1420.94 L742.599 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M881.998 1434.2 L889.636 1434.2 L889.636 1407.84 L881.326 1409.51 L881.326 1405.25 L889.59 1403.58 L894.266 1403.58 L894.266 1434.2 L901.905 1434.2 L901.905 1438.14 L881.998 1438.14 L881.998 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M911.488 1437.42 L911.488 1433.16 Q913.247 1434 915.053 1434.44 Q916.858 1434.88 918.595 1434.88 Q923.224 1434.88 925.655 1431.77 Q928.108 1428.65 928.456 1422.31 Q927.113 1424.3 925.053 1425.36 Q922.993 1426.43 920.493 1426.43 Q915.308 1426.43 912.275 1423.3 Q909.266 1420.15 909.266 1414.71 Q909.266 1409.39 912.414 1406.17 Q915.562 1402.95 920.794 1402.95 Q926.789 1402.95 929.937 1407.56 Q933.108 1412.14 933.108 1420.89 Q933.108 1429.07 929.22 1433.95 Q925.354 1438.81 918.803 1438.81 Q917.044 1438.81 915.238 1438.46 Q913.433 1438.12 911.488 1437.42 M920.794 1422.77 Q923.942 1422.77 925.77 1420.62 Q927.622 1418.46 927.622 1414.71 Q927.622 1410.99 925.77 1408.83 Q923.942 1406.66 920.794 1406.66 Q917.646 1406.66 915.794 1408.83 Q913.965 1410.99 913.965 1414.71 Q913.965 1418.46 915.794 1420.62 Q917.646 1422.77 920.794 1422.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M941.557 1403.58 L959.914 1403.58 L959.914 1407.52 L945.84 1407.52 L945.84 1415.99 Q946.858 1415.64 947.877 1415.48 Q948.895 1415.29 949.914 1415.29 Q955.701 1415.29 959.08 1418.46 Q962.46 1421.64 962.46 1427.05 Q962.46 1432.63 958.988 1435.73 Q955.516 1438.81 949.196 1438.81 Q947.02 1438.81 944.752 1438.44 Q942.506 1438.07 940.099 1437.33 L940.099 1432.63 Q942.182 1433.76 944.405 1434.32 Q946.627 1434.88 949.104 1434.88 Q953.108 1434.88 955.446 1432.77 Q957.784 1430.66 957.784 1427.05 Q957.784 1423.44 955.446 1421.33 Q953.108 1419.23 949.104 1419.23 Q947.229 1419.23 945.354 1419.64 Q943.502 1420.06 941.557 1420.94 L941.557 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M981.673 1406.66 Q978.062 1406.66 976.233 1410.22 Q974.428 1413.76 974.428 1420.89 Q974.428 1428 976.233 1431.57 Q978.062 1435.11 981.673 1435.11 Q985.307 1435.11 987.113 1431.57 Q988.941 1428 988.941 1420.89 Q988.941 1413.76 987.113 1410.22 Q985.307 1406.66 981.673 1406.66 M981.673 1402.95 Q987.483 1402.95 990.539 1407.56 Q993.617 1412.14 993.617 1420.89 Q993.617 1429.62 990.539 1434.23 Q987.483 1438.81 981.673 1438.81 Q975.863 1438.81 972.784 1434.23 Q969.729 1429.62 969.729 1420.89 Q969.729 1412.14 972.784 1407.56 Q975.863 1402.95 981.673 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M1112.11 1434.2 L1119.75 1434.2 L1119.75 1407.84 L1111.44 1409.51 L1111.44 1405.25 L1119.71 1403.58 L1124.38 1403.58 L1124.38 1434.2 L1132.02 1434.2 L1132.02 1438.14 L1112.11 1438.14 L1112.11 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M1141.6 1437.42 L1141.6 1433.16 Q1143.36 1434 1145.17 1434.44 Q1146.97 1434.88 1148.71 1434.88 Q1153.34 1434.88 1155.77 1431.77 Q1158.22 1428.65 1158.57 1422.31 Q1157.23 1424.3 1155.17 1425.36 Q1153.11 1426.43 1150.61 1426.43 Q1145.42 1426.43 1142.39 1423.3 Q1139.38 1420.15 1139.38 1414.71 Q1139.38 1409.39 1142.53 1406.17 Q1145.68 1402.95 1150.91 1402.95 Q1156.9 1402.95 1160.05 1407.56 Q1163.22 1412.14 1163.22 1420.89 Q1163.22 1429.07 1159.34 1433.95 Q1155.47 1438.81 1148.92 1438.81 Q1147.16 1438.81 1145.35 1438.46 Q1143.55 1438.12 1141.6 1437.42 M1150.91 1422.77 Q1154.06 1422.77 1155.89 1420.62 Q1157.74 1418.46 1157.74 1414.71 Q1157.74 1410.99 1155.89 1408.83 Q1154.06 1406.66 1150.91 1406.66 Q1147.76 1406.66 1145.91 1408.83 Q1144.08 1410.99 1144.08 1414.71 Q1144.08 1418.46 1145.91 1420.62 Q1147.76 1422.77 1150.91 1422.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M1170.45 1403.58 L1192.67 1403.58 L1192.67 1405.57 L1180.12 1438.14 L1175.24 1438.14 L1187.04 1407.52 L1170.45 1407.52 L1170.45 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M1201.83 1403.58 L1220.19 1403.58 L1220.19 1407.52 L1206.12 1407.52 L1206.12 1415.99 Q1207.14 1415.64 1208.15 1415.48 Q1209.17 1415.29 1210.19 1415.29 Q1215.98 1415.29 1219.36 1418.46 Q1222.74 1421.64 1222.74 1427.05 Q1222.74 1432.63 1219.27 1435.73 Q1215.79 1438.81 1209.47 1438.81 Q1207.3 1438.81 1205.03 1438.44 Q1202.78 1438.07 1200.38 1437.33 L1200.38 1432.63 Q1202.46 1433.76 1204.68 1434.32 Q1206.9 1434.88 1209.38 1434.88 Q1213.39 1434.88 1215.72 1432.77 Q1218.06 1430.66 1218.06 1427.05 Q1218.06 1423.44 1215.72 1421.33 Q1213.39 1419.23 1209.38 1419.23 Q1207.51 1419.23 1205.63 1419.64 Q1203.78 1420.06 1201.83 1420.94 L1201.83 1403.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M1345.32 1434.2 L1361.64 1434.2 L1361.64 1438.14 L1339.69 1438.14 L1339.69 1434.2 Q1342.36 1431.45 1346.94 1426.82 Q1351.55 1422.17 1352.73 1420.83 Q1354.97 1418.3 1355.85 1416.57 Q1356.75 1414.81 1356.75 1413.12 Q1356.75 1410.36 1354.81 1408.63 Q1352.89 1406.89 1349.79 1406.89 Q1347.59 1406.89 1345.13 1407.65 Q1342.7 1408.42 1339.93 1409.97 L1339.93 1405.25 Q1342.75 1404.11 1345.2 1403.53 Q1347.66 1402.95 1349.69 1402.95 Q1355.06 1402.95 1358.26 1405.64 Q1361.45 1408.33 1361.45 1412.82 Q1361.45 1414.95 1360.64 1416.87 Q1359.86 1418.76 1357.75 1421.36 Q1357.17 1422.03 1354.07 1425.25 Q1350.97 1428.44 1345.32 1434.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M1381.45 1406.66 Q1377.84 1406.66 1376.01 1410.22 Q1374.21 1413.76 1374.21 1420.89 Q1374.21 1428 1376.01 1431.57 Q1377.84 1435.11 1381.45 1435.11 Q1385.09 1435.11 1386.89 1431.57 Q1388.72 1428 1388.72 1420.89 Q1388.72 1413.76 1386.89 1410.22 Q1385.09 1406.66 1381.45 1406.66 M1381.45 1402.95 Q1387.26 1402.95 1390.32 1407.56 Q1393.4 1412.14 1393.4 1420.89 Q1393.4 1429.62 1390.32 1434.23 Q1387.26 1438.81 1381.45 1438.81 Q1375.64 1438.81 1372.56 1434.23 Q1369.51 1429.62 1369.51 1420.89 Q1369.51 1412.14 1372.56 1407.56 Q1375.64 1402.95 1381.45 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M1411.61 1406.66 Q1408 1406.66 1406.18 1410.22 Q1404.37 1413.76 1404.37 1420.89 Q1404.37 1428 1406.18 1431.57 Q1408 1435.11 1411.61 1435.11 Q1415.25 1435.11 1417.05 1431.57 Q1418.88 1428 1418.88 1420.89 Q1418.88 1413.76 1417.05 1410.22 Q1415.25 1406.66 1411.61 1406.66 M1411.61 1402.95 Q1417.43 1402.95 1420.48 1407.56 Q1423.56 1412.14 1423.56 1420.89 Q1423.56 1429.62 1420.48 1434.23 Q1417.43 1438.81 1411.61 1438.81 Q1405.8 1438.81 1402.73 1434.23 Q1399.67 1429.62 1399.67 1420.89 Q1399.67 1412.14 1402.73 1407.56 Q1405.8 1402.95 1411.61 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M1441.78 1406.66 Q1438.17 1406.66 1436.34 1410.22 Q1434.53 1413.76 1434.53 1420.89 Q1434.53 1428 1436.34 1431.57 Q1438.17 1435.11 1441.78 1435.11 Q1445.41 1435.11 1447.22 1431.57 Q1449.05 1428 1449.05 1420.89 Q1449.05 1413.76 1447.22 1410.22 Q1445.41 1406.66 1441.78 1406.66 M1441.78 1402.95 Q1447.59 1402.95 1450.64 1407.56 Q1453.72 1412.14 1453.72 1420.89 Q1453.72 1429.62 1450.64 1434.23 Q1447.59 1438.81 1441.78 1438.81 Q1435.97 1438.81 1432.89 1434.23 Q1429.83 1429.62 1429.83 1420.89 Q1429.83 1412.14 1432.89 1407.56 Q1435.97 1402.95 1441.78 1402.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M836.257 1467.48 L843.164 1467.48 L856.341 1487.03 L869.423 1467.48 L876.329 1467.48 L859.524 1492.37 L859.524 1515 L853.063 1515 L853.063 1492.37 L836.257 1467.48 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M904.179 1495.72 L904.179 1498.58 L877.253 1498.58 Q877.634 1504.63 880.881 1507.81 Q884.159 1510.96 889.984 1510.96 Q893.358 1510.96 896.509 1510.13 Q899.692 1509.31 902.811 1507.65 L902.811 1513.19 Q899.66 1514.53 896.35 1515.23 Q893.039 1515.93 889.634 1515.93 Q881.104 1515.93 876.107 1510.96 Q871.141 1506 871.141 1497.53 Q871.141 1488.78 875.852 1483.65 Q880.595 1478.5 888.615 1478.5 Q895.809 1478.5 899.978 1483.14 Q904.179 1487.76 904.179 1495.72 M898.323 1494 Q898.259 1489.19 895.618 1486.33 Q893.008 1483.46 888.679 1483.46 Q883.777 1483.46 880.817 1486.23 Q877.889 1489 877.443 1494.03 L898.323 1494 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M929.992 1497.08 Q922.895 1497.08 920.157 1498.71 Q917.42 1500.33 917.42 1504.25 Q917.42 1507.36 919.457 1509.21 Q921.526 1511.03 925.059 1511.03 Q929.929 1511.03 932.857 1507.59 Q935.817 1504.12 935.817 1498.39 L935.817 1497.08 L929.992 1497.08 M941.673 1494.67 L941.673 1515 L935.817 1515 L935.817 1509.59 Q933.812 1512.84 930.82 1514.4 Q927.828 1515.93 923.499 1515.93 Q918.025 1515.93 914.778 1512.87 Q911.564 1509.78 911.564 1504.63 Q911.564 1498.61 915.574 1495.56 Q919.616 1492.5 927.605 1492.5 L935.817 1492.5 L935.817 1491.93 Q935.817 1487.89 933.143 1485.69 Q930.502 1483.46 925.696 1483.46 Q922.64 1483.46 919.744 1484.19 Q916.847 1484.93 914.174 1486.39 L914.174 1480.98 Q917.388 1479.74 920.412 1479.13 Q923.436 1478.5 926.3 1478.5 Q934.035 1478.5 937.854 1482.51 Q941.673 1486.52 941.673 1494.67 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M974.393 1484.83 Q973.407 1484.26 972.229 1484 Q971.083 1483.72 969.683 1483.72 Q964.717 1483.72 962.044 1486.96 Q959.402 1490.18 959.402 1496.23 L959.402 1515 L953.514 1515 L953.514 1479.36 L959.402 1479.36 L959.402 1484.89 Q961.248 1481.65 964.208 1480.09 Q967.168 1478.5 971.401 1478.5 Q972.006 1478.5 972.738 1478.59 Q973.47 1478.66 974.361 1478.81 L974.393 1484.83 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,1382.14 257.895,47.2441 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,1330.45 276.792,1330.45 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,1124.31 276.792,1124.31 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,918.168 276.792,918.168 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,712.027 276.792,712.027 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,505.886 276.792,505.886 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,299.745 276.792,299.745 "/>
<polyline clip-path="url(#clip270)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="257.895,93.6035 276.792,93.6035 "/>
<path clip-path="url(#clip270)" d="M143.016 1330.9 L172.691 1330.9 L172.691 1334.84 L143.016 1334.84 L143.016 1330.9 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M183.594 1343.79 L191.233 1343.79 L191.233 1317.43 L182.923 1319.1 L182.923 1314.84 L191.187 1313.17 L195.862 1313.17 L195.862 1343.79 L203.501 1343.79 L203.501 1347.73 L183.594 1347.73 L183.594 1343.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M212.992 1313.17 L231.348 1313.17 L231.348 1317.11 L217.274 1317.11 L217.274 1325.58 Q218.293 1325.23 219.311 1325.07 Q220.33 1324.88 221.348 1324.88 Q227.135 1324.88 230.515 1328.05 Q233.895 1331.23 233.895 1336.64 Q233.895 1342.22 230.422 1345.32 Q226.95 1348.4 220.631 1348.4 Q218.455 1348.4 216.186 1348.03 Q213.941 1347.66 211.534 1346.92 L211.534 1342.22 Q213.617 1343.35 215.839 1343.91 Q218.061 1344.47 220.538 1344.47 Q224.543 1344.47 226.881 1342.36 Q229.219 1340.25 229.219 1336.64 Q229.219 1333.03 226.881 1330.92 Q224.543 1328.82 220.538 1328.82 Q218.663 1328.82 216.788 1329.23 Q214.936 1329.65 212.992 1330.53 L212.992 1313.17 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M142.02 1124.76 L171.696 1124.76 L171.696 1128.7 L142.02 1128.7 L142.02 1124.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M182.599 1137.65 L190.237 1137.65 L190.237 1111.29 L181.927 1112.95 L181.927 1108.7 L190.191 1107.03 L194.867 1107.03 L194.867 1137.65 L202.506 1137.65 L202.506 1141.59 L182.599 1141.59 L182.599 1137.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M221.95 1110.11 Q218.339 1110.11 216.51 1113.67 Q214.705 1117.21 214.705 1124.34 Q214.705 1131.45 216.51 1135.01 Q218.339 1138.56 221.95 1138.56 Q225.585 1138.56 227.39 1135.01 Q229.219 1131.45 229.219 1124.34 Q229.219 1117.21 227.39 1113.67 Q225.585 1110.11 221.95 1110.11 M221.95 1106.4 Q227.76 1106.4 230.816 1111.01 Q233.895 1115.59 233.895 1124.34 Q233.895 1133.07 230.816 1137.68 Q227.76 1142.26 221.95 1142.26 Q216.14 1142.26 213.061 1137.68 Q210.006 1133.07 210.006 1124.34 Q210.006 1115.59 213.061 1111.01 Q216.14 1106.4 221.95 1106.4 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M173.177 918.619 L202.853 918.619 L202.853 922.554 L173.177 922.554 L173.177 918.619 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M212.992 900.888 L231.348 900.888 L231.348 904.823 L217.274 904.823 L217.274 913.295 Q218.293 912.948 219.311 912.786 Q220.33 912.601 221.348 912.601 Q227.135 912.601 230.515 915.772 Q233.895 918.943 233.895 924.36 Q233.895 929.939 230.422 933.04 Q226.95 936.119 220.631 936.119 Q218.455 936.119 216.186 935.749 Q213.941 935.378 211.534 934.638 L211.534 929.939 Q213.617 931.073 215.839 931.628 Q218.061 932.184 220.538 932.184 Q224.543 932.184 226.881 930.077 Q229.219 927.971 229.219 924.36 Q229.219 920.749 226.881 918.642 Q224.543 916.536 220.538 916.536 Q218.663 916.536 216.788 916.953 Q214.936 917.369 212.992 918.249 L212.992 900.888 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M221.95 697.825 Q218.339 697.825 216.51 701.39 Q214.705 704.932 214.705 712.061 Q214.705 719.168 216.51 722.733 Q218.339 726.274 221.95 726.274 Q225.585 726.274 227.39 722.733 Q229.219 719.168 229.219 712.061 Q229.219 704.932 227.39 701.39 Q225.585 697.825 221.95 697.825 M221.95 694.122 Q227.76 694.122 230.816 698.728 Q233.895 703.311 233.895 712.061 Q233.895 720.788 230.816 725.395 Q227.76 729.978 221.95 729.978 Q216.14 729.978 213.061 725.395 Q210.006 720.788 210.006 712.061 Q210.006 703.311 213.061 698.728 Q216.14 694.122 221.95 694.122 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M212.992 488.606 L231.348 488.606 L231.348 492.541 L217.274 492.541 L217.274 501.013 Q218.293 500.666 219.311 500.504 Q220.33 500.319 221.348 500.319 Q227.135 500.319 230.515 503.49 Q233.895 506.661 233.895 512.078 Q233.895 517.656 230.422 520.758 Q226.95 523.837 220.631 523.837 Q218.455 523.837 216.186 523.467 Q213.941 523.096 211.534 522.355 L211.534 517.656 Q213.617 518.791 215.839 519.346 Q218.061 519.902 220.538 519.902 Q224.543 519.902 226.881 517.795 Q229.219 515.689 229.219 512.078 Q229.219 508.467 226.881 506.36 Q224.543 504.254 220.538 504.254 Q218.663 504.254 216.788 504.67 Q214.936 505.087 212.992 505.967 L212.992 488.606 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M182.599 313.089 L190.237 313.089 L190.237 286.724 L181.927 288.39 L181.927 284.131 L190.191 282.465 L194.867 282.465 L194.867 313.089 L202.506 313.089 L202.506 317.025 L182.599 317.025 L182.599 313.089 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M221.95 285.543 Q218.339 285.543 216.51 289.108 Q214.705 292.65 214.705 299.779 Q214.705 306.886 216.51 310.451 Q218.339 313.992 221.95 313.992 Q225.585 313.992 227.39 310.451 Q229.219 306.886 229.219 299.779 Q229.219 292.65 227.39 289.108 Q225.585 285.543 221.95 285.543 M221.95 281.84 Q227.76 281.84 230.816 286.446 Q233.895 291.029 233.895 299.779 Q233.895 308.506 230.816 313.113 Q227.76 317.696 221.95 317.696 Q216.14 317.696 213.061 313.113 Q210.006 308.506 210.006 299.779 Q210.006 291.029 213.061 286.446 Q216.14 281.84 221.95 281.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M183.594 106.948 L191.233 106.948 L191.233 80.5827 L182.923 82.2494 L182.923 77.9901 L191.187 76.3235 L195.862 76.3235 L195.862 106.948 L203.501 106.948 L203.501 110.883 L183.594 110.883 L183.594 106.948 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M212.992 76.3235 L231.348 76.3235 L231.348 80.2587 L217.274 80.2587 L217.274 88.7308 Q218.293 88.3836 219.311 88.2216 Q220.33 88.0364 221.348 88.0364 Q227.135 88.0364 230.515 91.2077 Q233.895 94.3789 233.895 99.7956 Q233.895 105.374 230.422 108.476 Q226.95 111.555 220.631 111.555 Q218.455 111.555 216.186 111.184 Q213.941 110.814 211.534 110.073 L211.534 105.374 Q213.617 106.509 215.839 107.064 Q218.061 107.62 220.538 107.62 Q224.543 107.62 226.881 105.513 Q229.219 103.407 229.219 99.7956 Q229.219 96.1845 226.881 94.078 Q224.543 91.9716 220.538 91.9716 Q218.663 91.9716 216.788 92.3882 Q214.936 92.8049 212.992 93.6845 L212.992 76.3235 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M56.2442 1070.82 L56.2442 1061.24 L88.5819 1049.11 L56.2442 1036.92 L56.2442 1027.34 L103.764 1027.34 L103.764 1033.61 L62.037 1033.61 L94.6294 1045.87 L94.6294 1052.33 L62.037 1064.58 L103.764 1064.58 L103.764 1070.82 L56.2442 1070.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M72.2221 1001.02 Q72.2221 1005.73 75.9142 1008.47 Q79.5745 1011.21 85.972 1011.21 Q92.3695 1011.21 96.0617 1008.5 Q99.7219 1005.76 99.7219 1001.02 Q99.7219 996.343 96.0298 993.605 Q92.3377 990.868 85.972 990.868 Q79.6381 990.868 75.946 993.605 Q72.2221 996.343 72.2221 1001.02 M67.2568 1001.02 Q67.2568 993.383 72.2221 989.022 Q77.1873 984.662 85.972 984.662 Q94.7249 984.662 99.7219 989.022 Q104.687 993.383 104.687 1001.02 Q104.687 1008.69 99.7219 1013.05 Q94.7249 1017.38 85.972 1017.38 Q77.1873 1017.38 72.2221 1013.05 Q67.2568 1008.69 67.2568 1001.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M73.5271 951.496 L54.239 951.496 L54.239 945.64 L103.764 945.64 L103.764 951.496 L98.417 951.496 Q101.6 953.342 103.159 956.175 Q104.687 958.976 104.687 962.923 Q104.687 969.384 99.531 973.458 Q94.3747 977.5 85.972 977.5 Q77.5693 977.5 72.4131 973.458 Q67.2568 969.384 67.2568 962.923 Q67.2568 958.976 68.8164 956.175 Q70.3442 953.342 73.5271 951.496 M85.972 971.453 Q92.4332 971.453 96.1253 968.811 Q99.7856 966.137 99.7856 961.49 Q99.7856 956.843 96.1253 954.17 Q92.4332 951.496 85.972 951.496 Q79.5108 951.496 75.8505 954.17 Q72.1584 956.843 72.1584 961.49 Q72.1584 966.137 75.8505 968.811 Q79.5108 971.453 85.972 971.453 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M84.4761 903.085 L87.3406 903.085 L87.3406 930.012 Q93.3881 929.63 96.5709 926.384 Q99.7219 923.105 99.7219 917.281 Q99.7219 913.907 98.8944 910.756 Q98.0669 907.573 96.4118 904.454 L101.95 904.454 Q103.287 907.605 103.987 910.915 Q104.687 914.225 104.687 917.631 Q104.687 926.161 99.7219 931.158 Q94.7567 936.123 86.2903 936.123 Q77.5374 936.123 72.4131 931.412 Q67.2568 926.67 67.2568 918.649 Q67.2568 911.456 71.9038 907.286 Q76.5189 903.085 84.4761 903.085 M82.7573 908.942 Q77.9512 909.005 75.0866 911.647 Q72.2221 914.257 72.2221 918.586 Q72.2221 923.487 74.9912 926.447 Q77.7602 929.375 82.7892 929.821 L82.7573 908.942 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M54.239 893.473 L54.239 887.616 L103.764 887.616 L103.764 893.473 L54.239 893.473 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M81.4842 831.853 Q82.1844 829.784 84.4761 827.842 Q86.7677 825.869 90.7781 823.896 L103.764 817.371 L103.764 824.278 L91.5738 830.357 Q86.7995 832.712 85.24 834.94 Q83.6804 837.136 83.6804 840.956 L83.6804 847.958 L103.764 847.958 L103.764 854.387 L56.2442 854.387 L56.2442 839.874 Q56.2442 831.725 59.6498 827.715 Q63.0555 823.705 69.9304 823.705 Q74.4182 823.705 77.3783 825.805 Q80.3384 827.874 81.4842 831.853 M61.5277 847.958 L78.3968 847.958 L78.3968 839.874 Q78.3968 835.227 76.2643 832.871 Q74.1 830.484 69.9304 830.484 Q65.7609 830.484 63.6602 832.871 Q61.5277 835.227 61.5277 839.874 L61.5277 847.958 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M84.4761 781.787 L87.3406 781.787 L87.3406 808.713 Q93.3881 808.332 96.5709 805.085 Q99.7219 801.807 99.7219 795.982 Q99.7219 792.608 98.8944 789.457 Q98.0669 786.274 96.4118 783.155 L101.95 783.155 Q103.287 786.306 103.987 789.616 Q104.687 792.927 104.687 796.332 Q104.687 804.862 99.7219 809.859 Q94.7567 814.825 86.2903 814.825 Q77.5374 814.825 72.4131 810.114 Q67.2568 805.371 67.2568 797.351 Q67.2568 790.157 71.9038 785.988 Q76.5189 781.787 84.4761 781.787 M82.7573 787.643 Q77.9512 787.707 75.0866 790.348 Q72.2221 792.958 72.2221 797.287 Q72.2221 802.189 74.9912 805.149 Q77.7602 808.077 82.7892 808.522 L82.7573 787.643 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M69.1665 749.449 L74.7047 749.449 Q73.4316 751.931 72.795 754.605 Q72.1584 757.279 72.1584 760.143 Q72.1584 764.504 73.4952 766.7 Q74.832 768.864 77.5056 768.864 Q79.5426 768.864 80.7203 767.305 Q81.8661 765.745 82.9165 761.034 L83.3621 759.029 Q84.6989 752.791 87.1497 750.181 Q89.5686 747.539 93.9291 747.539 Q98.8944 747.539 101.791 751.486 Q104.687 755.401 104.687 762.276 Q104.687 765.14 104.114 768.259 Q103.573 771.347 102.459 774.784 L96.4118 774.784 Q98.0987 771.538 98.958 768.387 Q99.7856 765.236 99.7856 762.148 Q99.7856 758.011 98.3851 755.783 Q96.9529 753.555 94.3747 753.555 Q91.9876 753.555 90.7145 755.178 Q89.4413 756.769 88.2637 762.212 L87.7862 764.249 Q86.6404 769.692 84.2851 772.111 Q81.898 774.53 77.7602 774.53 Q72.7313 774.53 69.9941 770.965 Q67.2568 767.4 67.2568 760.843 Q67.2568 757.597 67.7343 754.732 Q68.2117 751.868 69.1665 749.449 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M68.1162 738.213 L68.1162 732.357 L103.764 732.357 L103.764 738.213 L68.1162 738.213 M54.239 738.213 L54.239 732.357 L61.655 732.357 L61.655 738.213 L54.239 738.213 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M73.5271 696.645 L54.239 696.645 L54.239 690.789 L103.764 690.789 L103.764 696.645 L98.417 696.645 Q101.6 698.491 103.159 701.324 Q104.687 704.125 104.687 708.072 Q104.687 714.533 99.531 718.607 Q94.3747 722.649 85.972 722.649 Q77.5693 722.649 72.4131 718.607 Q67.2568 714.533 67.2568 708.072 Q67.2568 704.125 68.8164 701.324 Q70.3442 698.491 73.5271 696.645 M85.972 716.602 Q92.4332 716.602 96.1253 713.96 Q99.7856 711.286 99.7856 706.639 Q99.7856 701.992 96.1253 699.319 Q92.4332 696.645 85.972 696.645 Q79.5108 696.645 75.8505 699.319 Q72.1584 701.992 72.1584 706.639 Q72.1584 711.286 75.8505 713.96 Q79.5108 716.602 85.972 716.602 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M89.6959 679.33 L68.1162 679.33 L68.1162 673.474 L89.4731 673.474 Q94.5339 673.474 97.0802 671.501 Q99.5946 669.527 99.5946 665.581 Q99.5946 660.838 96.5709 658.101 Q93.5472 655.332 88.3273 655.332 L68.1162 655.332 L68.1162 649.475 L103.764 649.475 L103.764 655.332 L98.2896 655.332 Q101.536 657.464 103.128 660.297 Q104.687 663.098 104.687 666.822 Q104.687 672.965 100.868 676.148 Q97.0483 679.33 89.6959 679.33 M67.2568 664.594 L67.2568 664.594 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M85.8447 621.212 Q85.8447 628.309 87.4679 631.047 Q89.0912 633.784 93.0061 633.784 Q96.1253 633.784 97.9714 631.747 Q99.7856 629.678 99.7856 626.145 Q99.7856 621.275 96.3481 618.347 Q92.8788 615.387 87.1497 615.387 L85.8447 615.387 L85.8447 621.212 M83.4257 609.53 L103.764 609.53 L103.764 615.387 L98.3533 615.387 Q101.6 617.392 103.159 620.384 Q104.687 623.376 104.687 627.705 Q104.687 633.179 101.632 636.426 Q98.5443 639.64 93.3881 639.64 Q87.3725 639.64 84.3169 635.63 Q81.2614 631.588 81.2614 623.599 L81.2614 615.387 L80.6885 615.387 Q76.6462 615.387 74.4501 618.061 Q72.2221 620.702 72.2221 625.508 Q72.2221 628.564 72.9541 631.46 Q73.6862 634.357 75.1503 637.03 L69.7395 637.03 Q68.4981 633.816 67.8934 630.792 Q67.2568 627.768 67.2568 624.904 Q67.2568 617.169 71.2672 613.35 Q75.2776 609.53 83.4257 609.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M54.239 597.467 L54.239 591.611 L103.764 591.611 L103.764 597.467 L54.239 597.467 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M69.1665 556.631 L74.7047 556.631 Q73.4316 559.114 72.795 561.788 Q72.1584 564.461 72.1584 567.326 Q72.1584 571.686 73.4952 573.883 Q74.832 576.047 77.5056 576.047 Q79.5426 576.047 80.7203 574.487 Q81.8661 572.928 82.9165 568.217 L83.3621 566.212 Q84.6989 559.973 87.1497 557.364 Q89.5686 554.722 93.9291 554.722 Q98.8944 554.722 101.791 558.668 Q104.687 562.583 104.687 569.458 Q104.687 572.323 104.114 575.442 Q103.573 578.53 102.459 581.967 L96.4118 581.967 Q98.0987 578.72 98.958 575.569 Q99.7856 572.418 99.7856 569.331 Q99.7856 565.193 98.3851 562.965 Q96.9529 560.737 94.3747 560.737 Q91.9876 560.737 90.7145 562.361 Q89.4413 563.952 88.2637 569.395 L87.7862 571.432 Q86.6404 576.874 84.2851 579.293 Q81.898 581.712 77.7602 581.712 Q72.7313 581.712 69.9941 578.148 Q67.2568 574.583 67.2568 568.026 Q67.2568 564.78 67.7343 561.915 Q68.2117 559.05 69.1665 556.631 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M54.3026 510.607 Q61.6232 514.872 68.7846 516.941 Q75.946 519.01 83.2984 519.01 Q90.6508 519.01 97.8759 516.941 Q105.069 514.841 112.358 510.607 L112.358 515.7 Q104.878 520.474 97.6531 522.861 Q90.428 525.217 83.2984 525.217 Q76.2006 525.217 69.0074 522.861 Q61.8142 520.506 54.3026 515.7 L54.3026 510.607 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M74.9593 471.49 Q71.0126 469.294 69.1347 466.238 Q67.2568 463.183 67.2568 459.045 Q67.2568 453.475 71.1717 450.451 Q75.0548 447.428 82.2481 447.428 L103.764 447.428 L103.764 453.316 L82.439 453.316 Q77.3146 453.316 74.832 455.13 Q72.3494 456.945 72.3494 460.668 Q72.3494 465.22 75.3731 467.862 Q78.3968 470.503 83.6167 470.503 L103.764 470.503 L103.764 476.392 L82.439 476.392 Q77.2828 476.392 74.832 478.206 Q72.3494 480.02 72.3494 483.808 Q72.3494 488.296 75.4049 490.937 Q78.4286 493.579 83.6167 493.579 L103.764 493.579 L103.764 499.467 L68.1162 499.467 L68.1162 493.579 L73.6544 493.579 Q70.376 491.574 68.8164 488.773 Q67.2568 485.972 67.2568 482.121 Q67.2568 478.238 69.2302 475.532 Q71.2036 472.795 74.9593 471.49 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M74.9593 407.992 Q71.0126 405.796 69.1347 402.741 Q67.2568 399.685 67.2568 395.547 Q67.2568 389.977 71.1717 386.954 Q75.0548 383.93 82.2481 383.93 L103.764 383.93 L103.764 389.818 L82.439 389.818 Q77.3146 389.818 74.832 391.632 Q72.3494 393.447 72.3494 397.171 Q72.3494 401.722 75.3731 404.364 Q78.3968 407.006 83.6167 407.006 L103.764 407.006 L103.764 412.894 L82.439 412.894 Q77.2828 412.894 74.832 414.708 Q72.3494 416.522 72.3494 420.31 Q72.3494 424.798 75.4049 427.439 Q78.4286 430.081 83.6167 430.081 L103.764 430.081 L103.764 435.97 L68.1162 435.97 L68.1162 430.081 L73.6544 430.081 Q70.376 428.076 68.8164 425.275 Q67.2568 422.474 67.2568 418.623 Q67.2568 414.74 69.2302 412.034 Q71.2036 409.297 74.9593 407.992 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip270)" d="M54.3026 373.172 L54.3026 368.079 Q61.8142 363.305 69.0074 360.95 Q76.2006 358.563 83.2984 358.563 Q90.428 358.563 97.6531 360.95 Q104.878 363.305 112.358 368.079 L112.358 373.172 Q105.069 368.939 97.8759 366.87 Q90.6508 364.769 83.2984 364.769 Q75.946 364.769 68.7846 366.87 Q61.6232 368.939 54.3026 373.172 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><circle clip-path="url(#clip272)" cx="294.542" cy="748.606" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="303.726" cy="588.896" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="312.911" cy="1344.36" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="322.096" cy="1181.94" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="331.281" cy="357.741" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="340.465" cy="459.507" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="349.65" cy="604.459" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="358.835" cy="871.312" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="368.019" cy="835.492" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="377.204" cy="836.252" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="386.389" cy="787.904" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="395.574" cy="905.154" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="404.758" cy="806.792" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="413.943" cy="648.024" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="423.128" cy="1084.11" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="432.312" cy="637.981" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="441.497" cy="1012.13" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="450.682" cy="861.255" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="459.867" cy="508.51" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="469.051" cy="250.263" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="478.236" cy="538.753" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="487.421" cy="613.342" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="496.605" cy="463.203" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="505.79" cy="160.83" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="514.975" cy="604.72" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="524.16" cy="874.417" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="533.344" cy="647.896" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="542.529" cy="735.935" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="551.714" cy="867.934" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="560.898" cy="708.431" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="570.083" cy="739.032" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="579.268" cy="422.143" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="588.453" cy="583.089" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="597.637" cy="542.902" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="606.822" cy="342.671" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="616.007" cy="85.0242" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="625.191" cy="202.379" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="634.376" cy="411.301" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="643.561" cy="515.395" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="652.746" cy="499.78" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="661.93" cy="499.633" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="671.115" cy="473.473" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="680.3" cy="558.293" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="689.484" cy="558.944" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="698.669" cy="910.494" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="707.854" cy="882.063" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="717.039" cy="687.83" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="726.223" cy="793.149" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="735.408" cy="1016.42" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="744.593" cy="997.487" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="753.777" cy="898.209" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="762.962" cy="989.182" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="772.147" cy="822.97" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="781.332" cy="691.898" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="790.516" cy="979.055" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="799.701" cy="808.362" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="808.886" cy="1033.59" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="818.07" cy="881.528" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="827.255" cy="857.692" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="836.44" cy="722.61" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="845.625" cy="1034.63" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="854.809" cy="619.896" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="863.994" cy="703.044" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="873.179" cy="786.359" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="882.363" cy="1150.35" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="891.548" cy="1121.81" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="900.733" cy="878.689" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="909.918" cy="820.073" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="919.102" cy="686.785" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="928.287" cy="802.229" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="937.472" cy="762.636" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="946.657" cy="442.557" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="955.841" cy="630.543" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="965.026" cy="537.296" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="974.211" cy="720.132" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="983.395" cy="735.839" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="992.58" cy="990.358" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1001.76" cy="510.498" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1010.95" cy="527.195" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1020.13" cy="585.017" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1029.32" cy="493.808" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1038.5" cy="311.166" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1047.69" cy="592.592" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1056.87" cy="728.642" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1066.06" cy="1090.28" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1075.24" cy="676.98" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1084.43" cy="954.771" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1093.61" cy="955.839" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1102.8" cy="974.902" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1111.98" cy="763.532" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1121.17" cy="903.727" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1130.35" cy="735.696" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1139.54" cy="420.683" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1148.72" cy="745.92" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1157.9" cy="304.82" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1167.09" cy="427.777" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1176.27" cy="512.493" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1185.46" cy="665.848" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1194.64" cy="472.515" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1203.83" cy="755.411" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1213.01" cy="601.672" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1222.2" cy="190.32" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1231.38" cy="501.693" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1240.57" cy="251.008" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1249.75" cy="361.15" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1258.94" cy="858.649" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1268.12" cy="912.122" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1277.31" cy="983.65" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1286.49" cy="890.516" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1295.68" cy="792.154" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1304.86" cy="806.765" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1314.05" cy="809.8" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1323.23" cy="859.6" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1332.41" cy="1017.15" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1341.6" cy="894.161" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1350.78" cy="883.469" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1359.97" cy="810.126" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1369.15" cy="634.962" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1378.34" cy="1175.38" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1387.52" cy="924.73" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1396.71" cy="971.197" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1405.89" cy="863.8" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1415.08" cy="890.347" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1424.26" cy="635.931" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1433.45" cy="766.974" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1442.63" cy="894.27" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1451.82" cy="843.371" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1461" cy="893.119" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1470.19" cy="645.984" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1479.37" cy="534.246" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1488.55" cy="353.956" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1497.74" cy="406.489" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1506.92" cy="166.529" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
<circle clip-path="url(#clip272)" cx="1516.11" cy="640.779" r="14.4" fill="#000000" fill-rule="evenodd" fill-opacity="1" stroke="none"/>
</svg>
<p>(b)</p>
</div>
</div></td>
</tr>
</tbody>
</table>

Figure 2

Now we want to generate 1,000 replicates of the residuals using the
fitted AR(1) process and add them back to the model hindcast.

In [1]:
function ar1_sim(ρ, σ, n_sim, T)

    sim_out = zeros(n_sim, T)
    for t = 1:T
        if t == 1
            sim_out[:, t] = rand(Normal(0, sqrt(σ^2 / (1 - ρ^2))), n_sim)
        else
            sim_out[:, t] = rand.(Normal.(ρ * sim_out[:, t-1], σ))
        end
    end
    return sim_out
end

n_sim = 1_000
resids_boot = ar1_sim(p_mle[end-1], p_mle[end], n_sim, nrow(temp_dat))
slr_boot = mapslices(row -> row .+ slr_fit, resids_boot; dims=2)

1000×134 Matrix{Float64}:
 -155.578  -160.371  -150.263  …  65.6277  66.8024  60.9281  66.3092
 -166.817  -150.491  -144.177     50.7711  53.8146  58.418   54.321
 -159.91   -162.683  -163.252     50.7926  68.1767  61.0413  63.3705
 -162.234  -151.419  -150.336     56.3515  66.5042  65.2169  70.7606
 -154.785  -153.74   -152.474     53.0952  59.1195  62.5341  61.0469
 -158.985  -157.702  -154.061  …  53.8543  58.2929  60.5728  58.7454
 -161.992  -149.346  -157.096     57.0526  65.5743  61.8247  55.8163
 -157.933  -162.833  -164.287     52.874   55.6301  63.7023  70.2268
 -157.496  -149.485  -152.446     48.1722  47.95    52.4044  63.0504
 -145.989  -149.469  -149.711     64.1377  57.1585  53.9137  51.6984
    ⋮                          ⋱   ⋮                         
 -152.459  -161.553  -158.747     61.4694  70.3682  68.5933  74.4939
 -163.098  -154.776  -146.78      59.051   64.7032  62.6031  66.4458
 -156.917  -158.073  -143.377     64.4128  63.236   70.6126  71.2534
 -158.449  -159.

Now we refit the model to each replicate to get the estimates of the
bootstrap parameters.

In [1]:
a_boot = zeros(n_sim)
b_boot = zeros(n_sim)
τ_boot = zeros(n_sim)
S₀_boot = zeros(n_sim)
ρ_boot = zeros(n_sim)
σ_boot = zeros(n_sim)

for i = 1:n_sim
    mle_optim = optimize(p -> -ar1_loglik(p, temp_dat.Temp, slr_boot[i, :]), low_bds, up_bds, p₀)
    a_boot[i], b_boot[i], τ_boot[i], S₀_boot[i], ρ_boot[i], σ_boot[i] = mle_optim.minimizer
end

Plotting the histograms:

``` julia
hist1 = histogram(a_boot, xlabel=L"$a$", ylabel="Count", legend=false, title=L"$a$")
vline!(hist1, [p_mle[1]], color=:red)
plot!(hist1, size=(300, 300))

hist2 = histogram(b_boot, xlabel=L"$b$", ylabel="Count", legend=false, title=L"$b$")
vline!(hist2, [p_mle[2]], color=:red)
plot!(hist2, size=(300, 300))

hist3 = histogram(τ_boot, xlabel=L"$\tau$", ylabel="Count", legend=false, title=L"$\tau$")
vline!(hist3, [p_mle[3]], color=:red)
plot!(hist3, size=(300, 300))

hist4 = histogram(S₀_boot, xlabel=L"$S_0$", ylabel="Count", legend=false, title=L"$S_0$")
vline!(hist4, [p_mle[4]], color=:red)
plot!(hist4, size=(300, 300))

hist5 = histogram(ρ_boot, xlabel=L"$\rho$", ylabel="Count", legend=false, title=L"$\rho$")
vline!(hist5, [p_mle[5]], color=:red)
plot!(hist5, size=(300, 300))

hist6 = histogram(σ_boot, xlabel=L"$\sigma$", ylabel="Count", legend=false, title=L"$\sigma$")
vline!(hist6, [p_mle[6]], color=:red)
plot!(hist6, size=(300, 300))

display(hist1)
display(hist2)
display(hist3)
display(hist4)
display(hist5)
display(hist6)
```

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<tbody>
<tr>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="300" height="300" viewBox="0 0 1200 1200">
<defs>
  <clipPath id="clip330">
    <rect x="0" y="0" width="1200" height="1200"/>
  </clipPath>
</defs>
<path clip-path="url(#clip330)" d="M0 1200 L1200 1200 L1200 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip331">
    <rect x="240" y="119" width="841" height="841"/>
  </clipPath>
</defs>
<path clip-path="url(#clip330)" d="M245.885 967.715 L1152.76 967.715 L1152.76 153.712 L245.885 153.712  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip332">
    <rect x="245" y="153" width="908" height="815"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="287.612,967.715 287.612,153.712 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="491.428,967.715 491.428,153.712 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="695.244,967.715 695.244,153.712 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="899.06,967.715 899.06,153.712 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1102.88,967.715 1102.88,153.712 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,967.715 1152.76,967.715 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,785.203 1152.76,785.203 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,602.691 1152.76,602.691 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,420.179 1152.76,420.179 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,237.668 1152.76,237.668 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,967.715 1152.76,967.715 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="287.612,967.715 287.612,948.817 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="491.428,967.715 491.428,948.817 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="695.244,967.715 695.244,948.817 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="899.06,967.715 899.06,948.817 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1102.88,967.715 1102.88,948.817 "/>
<path clip-path="url(#clip330)" d="M287.612 989.034 Q284.001 989.034 282.173 992.599 Q280.367 996.14 280.367 1003.27 Q280.367 1010.38 282.173 1013.94 Q284.001 1017.48 287.612 1017.48 Q291.247 1017.48 293.052 1013.94 Q294.881 1010.38 294.881 1003.27 Q294.881 996.14 293.052 992.599 Q291.247 989.034 287.612 989.034 M287.612 985.33 Q293.422 985.33 296.478 989.937 Q299.557 994.52 299.557 1003.27 Q299.557 1012 296.478 1016.6 Q293.422 1021.19 287.612 1021.19 Q281.802 1021.19 278.723 1016.6 Q275.668 1012 275.668 1003.27 Q275.668 994.52 278.723 989.937 Q281.802 985.33 287.612 985.33 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M384.245 1001.62 L402.601 1001.62 L402.601 1005.55 L388.527 1005.55 L388.527 1014.02 Q389.546 1013.68 390.564 1013.51 Q391.583 1013.33 392.601 1013.33 Q398.388 1013.33 401.768 1016.5 Q405.148 1019.67 405.148 1025.09 Q405.148 1030.67 401.675 1033.77 Q398.203 1036.85 391.884 1036.85 Q389.708 1036.85 387.439 1036.48 Q385.194 1036.11 382.787 1035.36 L382.787 1030.67 Q384.87 1031.8 387.092 1032.36 Q389.314 1032.91 391.791 1032.91 Q395.796 1032.91 398.134 1030.8 Q400.472 1028.7 400.472 1025.09 Q400.472 1021.48 398.134 1019.37 Q395.796 1017.26 391.791 1017.26 Q389.916 1017.26 388.041 1017.68 Q386.189 1018.1 384.245 1018.98 L384.245 1001.62 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M414.361 1030.3 L419.245 1030.3 L419.245 1036.18 L414.361 1036.18 L414.361 1030.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M439.43 1004.69 Q435.819 1004.69 433.99 1008.26 Q432.185 1011.8 432.185 1018.93 Q432.185 1026.04 433.99 1029.6 Q435.819 1033.14 439.43 1033.14 Q443.064 1033.14 444.87 1029.6 Q446.698 1026.04 446.698 1018.93 Q446.698 1011.8 444.87 1008.26 Q443.064 1004.69 439.43 1004.69 M439.43 1000.99 Q445.24 1000.99 448.296 1005.6 Q451.374 1010.18 451.374 1018.93 Q451.374 1027.66 448.296 1032.26 Q445.24 1036.85 439.43 1036.85 Q433.62 1036.85 430.541 1032.26 Q427.486 1027.66 427.486 1018.93 Q427.486 1010.18 430.541 1005.6 Q433.62 1000.99 439.43 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M469.592 1004.69 Q465.981 1004.69 464.152 1008.26 Q462.346 1011.8 462.346 1018.93 Q462.346 1026.04 464.152 1029.6 Q465.981 1033.14 469.592 1033.14 Q473.226 1033.14 475.032 1029.6 Q476.86 1026.04 476.86 1018.93 Q476.86 1011.8 475.032 1008.26 Q473.226 1004.69 469.592 1004.69 M469.592 1000.99 Q475.402 1000.99 478.457 1005.6 Q481.536 1010.18 481.536 1018.93 Q481.536 1027.66 478.457 1032.26 Q475.402 1036.85 469.592 1036.85 Q463.782 1036.85 460.703 1032.26 Q457.647 1027.66 457.647 1018.93 Q457.647 1010.18 460.703 1005.6 Q463.782 1000.99 469.592 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M517.925 1010.71 L507.346 1021.34 L517.925 1031.92 L515.17 1034.72 L504.545 1024.09 L493.92 1034.72 L491.189 1031.92 L501.744 1021.34 L491.189 1010.71 L493.92 1007.91 L504.545 1018.54 L515.17 1007.91 L517.925 1010.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M530.286 1032.24 L537.925 1032.24 L537.925 1005.87 L529.615 1007.54 L529.615 1003.28 L537.878 1001.62 L542.554 1001.62 L542.554 1032.24 L550.193 1032.24 L550.193 1036.18 L530.286 1036.18 L530.286 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M569.638 1004.69 Q566.026 1004.69 564.198 1008.26 Q562.392 1011.8 562.392 1018.93 Q562.392 1026.04 564.198 1029.6 Q566.026 1033.14 569.638 1033.14 Q573.272 1033.14 575.077 1029.6 Q576.906 1026.04 576.906 1018.93 Q576.906 1011.8 575.077 1008.26 Q573.272 1004.69 569.638 1004.69 M569.638 1000.99 Q575.448 1000.99 578.503 1005.6 Q581.582 1010.18 581.582 1018.93 Q581.582 1027.66 578.503 1032.26 Q575.448 1036.85 569.638 1036.85 Q563.827 1036.85 560.749 1032.26 Q557.693 1027.66 557.693 1018.93 Q557.693 1010.18 560.749 1005.6 Q563.827 1000.99 569.638 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M594.277 993.624 Q597.004 994.207 598.528 996.051 Q600.07 997.894 600.07 1000.6 Q600.07 1004.76 597.211 1007.03 Q594.352 1009.31 589.086 1009.31 Q587.318 1009.31 585.438 1008.95 Q583.576 1008.61 581.582 1007.92 L581.582 1004.25 Q583.162 1005.17 585.043 1005.64 Q586.923 1006.11 588.973 1006.11 Q592.547 1006.11 594.409 1004.7 Q596.29 1003.29 596.29 1000.6 Q596.29 998.119 594.54 996.728 Q592.81 995.317 589.707 995.317 L586.434 995.317 L586.434 992.195 L589.857 992.195 Q592.66 992.195 594.146 991.085 Q595.631 989.957 595.631 987.85 Q595.631 985.688 594.089 984.54 Q592.566 983.374 589.707 983.374 Q588.146 983.374 586.359 983.713 Q584.572 984.051 582.428 984.766 L582.428 981.381 Q584.591 980.779 586.472 980.478 Q588.372 980.177 590.045 980.177 Q594.371 980.177 596.891 982.152 Q599.412 984.108 599.412 987.455 Q599.412 989.788 598.076 991.405 Q596.741 993.004 594.277 993.624 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M587.062 1032.24 L594.701 1032.24 L594.701 1005.87 L586.391 1007.54 L586.391 1003.28 L594.655 1001.62 L599.331 1001.62 L599.331 1032.24 L606.969 1032.24 L606.969 1036.18 L587.062 1036.18 L587.062 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M616.414 1030.3 L621.298 1030.3 L621.298 1036.18 L616.414 1036.18 L616.414 1030.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M641.483 1004.69 Q637.872 1004.69 636.043 1008.26 Q634.238 1011.8 634.238 1018.93 Q634.238 1026.04 636.043 1029.6 Q637.872 1033.14 641.483 1033.14 Q645.117 1033.14 646.923 1029.6 Q648.752 1026.04 648.752 1018.93 Q648.752 1011.8 646.923 1008.26 Q645.117 1004.69 641.483 1004.69 M641.483 1000.99 Q647.293 1000.99 650.349 1005.6 Q653.427 1010.18 653.427 1018.93 Q653.427 1027.66 650.349 1032.26 Q647.293 1036.85 641.483 1036.85 Q635.673 1036.85 632.594 1032.26 Q629.539 1027.66 629.539 1018.93 Q629.539 1010.18 632.594 1005.6 Q635.673 1000.99 641.483 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M671.645 1004.69 Q668.034 1004.69 666.205 1008.26 Q664.4 1011.8 664.4 1018.93 Q664.4 1026.04 666.205 1029.6 Q668.034 1033.14 671.645 1033.14 Q675.279 1033.14 677.085 1029.6 Q678.913 1026.04 678.913 1018.93 Q678.913 1011.8 677.085 1008.26 Q675.279 1004.69 671.645 1004.69 M671.645 1000.99 Q677.455 1000.99 680.511 1005.6 Q683.589 1010.18 683.589 1018.93 Q683.589 1027.66 680.511 1032.26 Q677.455 1036.85 671.645 1036.85 Q665.835 1036.85 662.756 1032.26 Q659.701 1027.66 659.701 1018.93 Q659.701 1010.18 662.756 1005.6 Q665.835 1000.99 671.645 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M719.978 1010.71 L709.399 1021.34 L719.978 1031.92 L717.223 1034.72 L706.598 1024.09 L695.974 1034.72 L693.242 1031.92 L703.798 1021.34 L693.242 1010.71 L695.974 1007.91 L706.598 1018.54 L717.223 1007.91 L719.978 1010.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M732.339 1032.24 L739.978 1032.24 L739.978 1005.87 L731.668 1007.54 L731.668 1003.28 L739.932 1001.62 L744.608 1001.62 L744.608 1032.24 L752.246 1032.24 L752.246 1036.18 L732.339 1036.18 L732.339 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M771.691 1004.69 Q768.08 1004.69 766.251 1008.26 Q764.445 1011.8 764.445 1018.93 Q764.445 1026.04 766.251 1029.6 Q768.08 1033.14 771.691 1033.14 Q775.325 1033.14 777.13 1029.6 Q778.959 1026.04 778.959 1018.93 Q778.959 1011.8 777.13 1008.26 Q775.325 1004.69 771.691 1004.69 M771.691 1000.99 Q777.501 1000.99 780.556 1005.6 Q783.635 1010.18 783.635 1018.93 Q783.635 1027.66 780.556 1032.26 Q777.501 1036.85 771.691 1036.85 Q765.881 1036.85 762.802 1032.26 Q759.746 1027.66 759.746 1018.93 Q759.746 1010.18 762.802 1005.6 Q765.881 1000.99 771.691 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M796.312 983.995 L786.72 998.985 L796.312 998.985 L796.312 983.995 M795.315 980.685 L800.092 980.685 L800.092 998.985 L804.098 998.985 L804.098 1002.14 L800.092 1002.14 L800.092 1008.76 L796.312 1008.76 L796.312 1002.14 L783.635 1002.14 L783.635 998.477 L795.315 980.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M790.878 1032.24 L798.517 1032.24 L798.517 1005.87 L790.207 1007.54 L790.207 1003.28 L798.471 1001.62 L803.147 1001.62 L803.147 1032.24 L810.785 1032.24 L810.785 1036.18 L790.878 1036.18 L790.878 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M820.23 1030.3 L825.114 1030.3 L825.114 1036.18 L820.23 1036.18 L820.23 1030.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M835.345 1001.62 L853.702 1001.62 L853.702 1005.55 L839.628 1005.55 L839.628 1014.02 Q840.646 1013.68 841.665 1013.51 Q842.683 1013.33 843.702 1013.33 Q849.489 1013.33 852.869 1016.5 Q856.248 1019.67 856.248 1025.09 Q856.248 1030.67 852.776 1033.77 Q849.304 1036.85 842.984 1036.85 Q840.808 1036.85 838.54 1036.48 Q836.295 1036.11 833.887 1035.36 L833.887 1030.67 Q835.97 1031.8 838.193 1032.36 Q840.415 1032.91 842.892 1032.91 Q846.896 1032.91 849.234 1030.8 Q851.572 1028.7 851.572 1025.09 Q851.572 1021.48 849.234 1019.37 Q846.896 1017.26 842.892 1017.26 Q841.017 1017.26 839.142 1017.68 Q837.29 1018.1 835.345 1018.98 L835.345 1001.62 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M875.461 1004.69 Q871.85 1004.69 870.021 1008.26 Q868.216 1011.8 868.216 1018.93 Q868.216 1026.04 870.021 1029.6 Q871.85 1033.14 875.461 1033.14 Q879.095 1033.14 880.901 1029.6 Q882.729 1026.04 882.729 1018.93 Q882.729 1011.8 880.901 1008.26 Q879.095 1004.69 875.461 1004.69 M875.461 1000.99 Q881.271 1000.99 884.327 1005.6 Q887.405 1010.18 887.405 1018.93 Q887.405 1027.66 884.327 1032.26 Q881.271 1036.85 875.461 1036.85 Q869.651 1036.85 866.572 1032.26 Q863.517 1027.66 863.517 1018.93 Q863.517 1010.18 866.572 1005.6 Q869.651 1000.99 875.461 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M923.794 1010.71 L913.215 1021.34 L923.794 1031.92 L921.039 1034.72 L910.414 1024.09 L899.79 1034.72 L897.058 1031.92 L907.614 1021.34 L897.058 1010.71 L899.79 1007.91 L910.414 1018.54 L921.039 1007.91 L923.794 1010.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M936.155 1032.24 L943.794 1032.24 L943.794 1005.87 L935.484 1007.54 L935.484 1003.28 L943.748 1001.62 L948.424 1001.62 L948.424 1032.24 L956.062 1032.24 L956.062 1036.18 L936.155 1036.18 L936.155 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M975.507 1004.69 Q971.896 1004.69 970.067 1008.26 Q968.261 1011.8 968.261 1018.93 Q968.261 1026.04 970.067 1029.6 Q971.896 1033.14 975.507 1033.14 Q979.141 1033.14 980.947 1029.6 Q982.775 1026.04 982.775 1018.93 Q982.775 1011.8 980.947 1008.26 Q979.141 1004.69 975.507 1004.69 M975.507 1000.99 Q981.317 1000.99 984.372 1005.6 Q987.451 1010.18 987.451 1018.93 Q987.451 1027.66 984.372 1032.26 Q981.317 1036.85 975.507 1036.85 Q969.697 1036.85 966.618 1032.26 Q963.562 1027.66 963.562 1018.93 Q963.562 1010.18 966.618 1005.6 Q969.697 1000.99 975.507 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1000.13 983.995 L990.536 998.985 L1000.13 998.985 L1000.13 983.995 M999.131 980.685 L1003.91 980.685 L1003.91 998.985 L1007.91 998.985 L1007.91 1002.14 L1003.91 1002.14 L1003.91 1008.76 L1000.13 1008.76 L1000.13 1002.14 L987.451 1002.14 L987.451 998.477 L999.131 980.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M998.78 1032.24 L1015.1 1032.24 L1015.1 1036.18 L993.155 1036.18 L993.155 1032.24 Q995.817 1029.49 1000.4 1024.86 Q1005.01 1020.2 1006.19 1018.86 Q1008.43 1016.34 1009.31 1014.6 Q1010.21 1012.84 1010.21 1011.15 Q1010.21 1008.4 1008.27 1006.66 Q1006.35 1004.93 1003.25 1004.93 Q1001.05 1004.93 998.595 1005.69 Q996.164 1006.45 993.386 1008 L993.386 1003.28 Q996.21 1002.15 998.664 1001.57 Q1001.12 1000.99 1003.15 1000.99 Q1008.53 1000.99 1011.72 1003.68 Q1014.91 1006.36 1014.91 1010.85 Q1014.91 1012.98 1014.1 1014.9 Q1013.32 1016.8 1011.21 1019.39 Q1010.63 1020.06 1007.53 1023.28 Q1004.43 1026.48 998.78 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1024.91 1030.3 L1029.8 1030.3 L1029.8 1036.18 L1024.91 1036.18 L1024.91 1030.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1049.98 1004.69 Q1046.37 1004.69 1044.54 1008.26 Q1042.74 1011.8 1042.74 1018.93 Q1042.74 1026.04 1044.54 1029.6 Q1046.37 1033.14 1049.98 1033.14 Q1053.62 1033.14 1055.42 1029.6 Q1057.25 1026.04 1057.25 1018.93 Q1057.25 1011.8 1055.42 1008.26 Q1053.62 1004.69 1049.98 1004.69 M1049.98 1000.99 Q1055.79 1000.99 1058.85 1005.6 Q1061.93 1010.18 1061.93 1018.93 Q1061.93 1027.66 1058.85 1032.26 Q1055.79 1036.85 1049.98 1036.85 Q1044.17 1036.85 1041.09 1032.26 Q1038.04 1027.66 1038.04 1018.93 Q1038.04 1010.18 1041.09 1005.6 Q1044.17 1000.99 1049.98 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1080.15 1004.69 Q1076.53 1004.69 1074.71 1008.26 Q1072.9 1011.8 1072.9 1018.93 Q1072.9 1026.04 1074.71 1029.6 Q1076.53 1033.14 1080.15 1033.14 Q1083.78 1033.14 1085.58 1029.6 Q1087.41 1026.04 1087.41 1018.93 Q1087.41 1011.8 1085.58 1008.26 Q1083.78 1004.69 1080.15 1004.69 M1080.15 1000.99 Q1085.96 1000.99 1089.01 1005.6 Q1092.09 1010.18 1092.09 1018.93 Q1092.09 1027.66 1089.01 1032.26 Q1085.96 1036.85 1080.15 1036.85 Q1074.33 1036.85 1071.26 1032.26 Q1068.2 1027.66 1068.2 1018.93 Q1068.2 1010.18 1071.26 1005.6 Q1074.33 1000.99 1080.15 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1128.48 1010.71 L1117.9 1021.34 L1128.48 1031.92 L1125.72 1034.72 L1115.1 1024.09 L1104.47 1034.72 L1101.74 1031.92 L1112.3 1021.34 L1101.74 1010.71 L1104.47 1007.91 L1115.1 1018.54 L1125.72 1007.91 L1128.48 1010.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1140.84 1032.24 L1148.48 1032.24 L1148.48 1005.87 L1140.17 1007.54 L1140.17 1003.28 L1148.43 1001.62 L1153.11 1001.62 L1153.11 1032.24 L1160.75 1032.24 L1160.75 1036.18 L1140.84 1036.18 L1140.84 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1180.19 1004.69 Q1176.58 1004.69 1174.75 1008.26 Q1172.95 1011.8 1172.95 1018.93 Q1172.95 1026.04 1174.75 1029.6 Q1176.58 1033.14 1180.19 1033.14 Q1183.83 1033.14 1185.63 1029.6 Q1187.46 1026.04 1187.46 1018.93 Q1187.46 1011.8 1185.63 1008.26 Q1183.83 1004.69 1180.19 1004.69 M1180.19 1000.99 Q1186 1000.99 1189.06 1005.6 Q1192.14 1010.18 1192.14 1018.93 Q1192.14 1027.66 1189.06 1032.26 Q1186 1036.85 1180.19 1036.85 Q1174.38 1036.85 1171.3 1032.26 Q1168.25 1027.66 1168.25 1018.93 Q1168.25 1010.18 1171.3 1005.6 Q1174.38 1000.99 1180.19 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1204.81 983.995 L1195.22 998.985 L1204.81 998.985 L1204.81 983.995 M1203.81 980.685 L1208.59 980.685 L1208.59 998.985 L1212.6 998.985 L1212.6 1002.14 L1208.59 1002.14 L1208.59 1008.76 L1204.81 1008.76 L1204.81 1002.14 L1192.14 1002.14 L1192.14 998.477 L1203.81 980.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M712.257 1089.12 Q711.387 1093.28 710.26 1095.66 Q708.456 1099.3 705.397 1099.3 Q703.078 1099.3 701.436 1097.88 Q699.793 1096.46 699.407 1094.34 Q695.188 1099.3 690.808 1099.3 Q686.943 1099.3 684.495 1096.37 Q682.048 1093.41 682.048 1088.74 Q682.048 1084.03 684.366 1079.52 Q686.717 1075.02 690.421 1072.21 Q694.125 1069.41 697.925 1069.41 Q701.951 1069.41 704.012 1073.63 Q704.173 1072.47 704.914 1071.6 Q705.655 1070.73 706.782 1070.73 Q707.651 1070.73 708.167 1071.22 Q708.682 1071.67 708.682 1072.5 Q708.682 1072.92 708.231 1074.76 L705.848 1083.97 L704.399 1089.93 Q703.851 1092.12 703.69 1092.92 Q703.561 1093.73 703.561 1094.79 Q703.561 1097.82 705.526 1097.82 Q706.492 1097.82 707.265 1097.21 Q708.038 1096.59 708.65 1095.31 Q709.262 1094.02 709.648 1092.83 Q710.035 1091.6 710.55 1089.67 Q710.711 1088.9 710.84 1088.7 Q711.001 1088.48 711.452 1088.48 Q712.257 1088.48 712.257 1089.12 M703.207 1076.66 Q703.207 1076.37 702.981 1075.56 Q702.756 1074.76 702.241 1073.63 Q701.725 1072.5 700.598 1071.7 Q699.503 1070.86 698.022 1070.86 Q696.025 1070.86 693.932 1072.57 Q691.87 1074.28 690.357 1077.17 Q689.197 1079.49 687.941 1084.36 Q686.717 1089.22 686.717 1091.63 Q686.717 1093.12 687.04 1094.44 Q687.394 1095.72 688.392 1096.79 Q689.391 1097.82 690.936 1097.82 Q694.705 1097.82 698.408 1093.15 Q699.117 1092.34 699.278 1092.02 Q699.471 1091.67 699.729 1090.7 L703.014 1077.79 Q703.207 1076.82 703.207 1076.66 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,967.715 245.885,153.712 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,967.715 264.783,967.715 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,785.203 264.783,785.203 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,602.691 264.783,602.691 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,420.179 264.783,420.179 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,237.668 264.783,237.668 "/>
<path clip-path="url(#clip330)" d="M215.941 953.514 Q212.33 953.514 210.501 957.079 Q208.696 960.62 208.696 967.75 Q208.696 974.856 210.501 978.421 Q212.33 981.963 215.941 981.963 Q219.575 981.963 221.381 978.421 Q223.21 974.856 223.21 967.75 Q223.21 960.62 221.381 957.079 Q219.575 953.514 215.941 953.514 M215.941 949.81 Q221.751 949.81 224.807 954.417 Q227.885 959 227.885 967.75 Q227.885 976.477 224.807 981.083 Q221.751 985.666 215.941 985.666 Q210.131 985.666 207.052 981.083 Q203.997 976.477 203.997 967.75 Q203.997 959 207.052 954.417 Q210.131 949.81 215.941 949.81 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M175.826 767.923 L194.182 767.923 L194.182 771.858 L180.108 771.858 L180.108 780.331 Q181.126 779.983 182.145 779.821 Q183.163 779.636 184.182 779.636 Q189.969 779.636 193.349 782.807 Q196.728 785.979 196.728 791.395 Q196.728 796.974 193.256 800.076 Q189.784 803.154 183.464 803.154 Q181.288 803.154 179.02 802.784 Q176.775 802.414 174.367 801.673 L174.367 796.974 Q176.451 798.108 178.673 798.664 Q180.895 799.219 183.372 799.219 Q187.376 799.219 189.714 797.113 Q192.052 795.006 192.052 791.395 Q192.052 787.784 189.714 785.678 Q187.376 783.571 183.372 783.571 Q181.497 783.571 179.622 783.988 Q177.77 784.405 175.826 785.284 L175.826 767.923 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M215.941 771.002 Q212.33 771.002 210.501 774.567 Q208.696 778.108 208.696 785.238 Q208.696 792.344 210.501 795.909 Q212.33 799.451 215.941 799.451 Q219.575 799.451 221.381 795.909 Q223.21 792.344 223.21 785.238 Q223.21 778.108 221.381 774.567 Q219.575 771.002 215.941 771.002 M215.941 767.298 Q221.751 767.298 224.807 771.905 Q227.885 776.488 227.885 785.238 Q227.885 793.965 224.807 798.571 Q221.751 803.154 215.941 803.154 Q210.131 803.154 207.052 798.571 Q203.997 793.965 203.997 785.238 Q203.997 776.488 207.052 771.905 Q210.131 767.298 215.941 767.298 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M146.428 616.036 L154.066 616.036 L154.066 589.671 L145.756 591.337 L145.756 587.078 L154.02 585.411 L158.696 585.411 L158.696 616.036 L166.335 616.036 L166.335 619.971 L146.428 619.971 L146.428 616.036 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M185.779 588.49 Q182.168 588.49 180.339 592.055 Q178.534 595.596 178.534 602.726 Q178.534 609.832 180.339 613.397 Q182.168 616.939 185.779 616.939 Q189.413 616.939 191.219 613.397 Q193.048 609.832 193.048 602.726 Q193.048 595.596 191.219 592.055 Q189.413 588.49 185.779 588.49 M185.779 584.786 Q191.589 584.786 194.645 589.393 Q197.724 593.976 197.724 602.726 Q197.724 611.453 194.645 616.059 Q191.589 620.643 185.779 620.643 Q179.969 620.643 176.89 616.059 Q173.835 611.453 173.835 602.726 Q173.835 593.976 176.89 589.393 Q179.969 584.786 185.779 584.786 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M215.941 588.49 Q212.33 588.49 210.501 592.055 Q208.696 595.596 208.696 602.726 Q208.696 609.832 210.501 613.397 Q212.33 616.939 215.941 616.939 Q219.575 616.939 221.381 613.397 Q223.21 609.832 223.21 602.726 Q223.21 595.596 221.381 592.055 Q219.575 588.49 215.941 588.49 M215.941 584.786 Q221.751 584.786 224.807 589.393 Q227.885 593.976 227.885 602.726 Q227.885 611.453 224.807 616.059 Q221.751 620.643 215.941 620.643 Q210.131 620.643 207.052 616.059 Q203.997 611.453 203.997 602.726 Q203.997 593.976 207.052 589.393 Q210.131 584.786 215.941 584.786 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M146.428 433.524 L154.066 433.524 L154.066 407.159 L145.756 408.825 L145.756 404.566 L154.02 402.899 L158.696 402.899 L158.696 433.524 L166.335 433.524 L166.335 437.459 L146.428 437.459 L146.428 433.524 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M175.826 402.899 L194.182 402.899 L194.182 406.835 L180.108 406.835 L180.108 415.307 Q181.126 414.96 182.145 414.798 Q183.163 414.612 184.182 414.612 Q189.969 414.612 193.349 417.784 Q196.728 420.955 196.728 426.372 Q196.728 431.95 193.256 435.052 Q189.784 438.131 183.464 438.131 Q181.288 438.131 179.02 437.76 Q176.775 437.39 174.367 436.649 L174.367 431.95 Q176.451 433.084 178.673 433.64 Q180.895 434.196 183.372 434.196 Q187.376 434.196 189.714 432.089 Q192.052 429.983 192.052 426.372 Q192.052 422.76 189.714 420.654 Q187.376 418.548 183.372 418.548 Q181.497 418.548 179.622 418.964 Q177.77 419.381 175.826 420.26 L175.826 402.899 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M215.941 405.978 Q212.33 405.978 210.501 409.543 Q208.696 413.085 208.696 420.214 Q208.696 427.321 210.501 430.885 Q212.33 434.427 215.941 434.427 Q219.575 434.427 221.381 430.885 Q223.21 427.321 223.21 420.214 Q223.21 413.085 221.381 409.543 Q219.575 405.978 215.941 405.978 M215.941 402.274 Q221.751 402.274 224.807 406.881 Q227.885 411.464 227.885 420.214 Q227.885 428.941 224.807 433.547 Q221.751 438.131 215.941 438.131 Q210.131 438.131 207.052 433.547 Q203.997 428.941 203.997 420.214 Q203.997 411.464 207.052 406.881 Q210.131 402.274 215.941 402.274 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M149.645 251.012 L165.964 251.012 L165.964 254.948 L144.02 254.948 L144.02 251.012 Q146.682 248.258 151.265 243.628 Q155.872 238.975 157.052 237.633 Q159.298 235.11 160.177 233.374 Q161.08 231.614 161.08 229.925 Q161.08 227.17 159.136 225.434 Q157.215 223.698 154.113 223.698 Q151.914 223.698 149.46 224.462 Q147.029 225.226 144.252 226.776 L144.252 222.054 Q147.076 220.92 149.529 220.341 Q151.983 219.763 154.02 219.763 Q159.39 219.763 162.585 222.448 Q165.779 225.133 165.779 229.624 Q165.779 231.753 164.969 233.675 Q164.182 235.573 162.076 238.165 Q161.497 238.837 158.395 242.054 Q155.293 245.249 149.645 251.012 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M185.779 223.466 Q182.168 223.466 180.339 227.031 Q178.534 230.573 178.534 237.702 Q178.534 244.809 180.339 248.374 Q182.168 251.915 185.779 251.915 Q189.413 251.915 191.219 248.374 Q193.048 244.809 193.048 237.702 Q193.048 230.573 191.219 227.031 Q189.413 223.466 185.779 223.466 M185.779 219.763 Q191.589 219.763 194.645 224.369 Q197.724 228.952 197.724 237.702 Q197.724 246.429 194.645 251.036 Q191.589 255.619 185.779 255.619 Q179.969 255.619 176.89 251.036 Q173.835 246.429 173.835 237.702 Q173.835 228.952 176.89 224.369 Q179.969 219.763 185.779 219.763 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M215.941 223.466 Q212.33 223.466 210.501 227.031 Q208.696 230.573 208.696 237.702 Q208.696 244.809 210.501 248.374 Q212.33 251.915 215.941 251.915 Q219.575 251.915 221.381 248.374 Q223.21 244.809 223.21 237.702 Q223.21 230.573 221.381 227.031 Q219.575 223.466 215.941 223.466 M215.941 219.763 Q221.751 219.763 224.807 224.369 Q227.885 228.952 227.885 237.702 Q227.885 246.429 224.807 251.036 Q221.751 255.619 215.941 255.619 Q210.131 255.619 207.052 251.036 Q203.997 246.429 203.997 237.702 Q203.997 228.952 207.052 224.369 Q210.131 219.763 215.941 219.763 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M67.9044 616.573 L74.6839 616.573 Q71.6602 619.819 70.1643 623.511 Q68.6683 627.172 68.6683 631.309 Q68.6683 639.457 73.6654 643.786 Q78.6307 648.115 88.0519 648.115 Q97.4413 648.115 102.438 643.786 Q107.404 639.457 107.404 631.309 Q107.404 627.172 105.908 623.511 Q104.412 619.819 101.388 616.573 L108.104 616.573 Q110.396 619.946 111.541 623.734 Q112.687 627.49 112.687 631.691 Q112.687 642.481 106.099 648.688 Q99.4783 654.894 88.0519 654.894 Q76.5936 654.894 70.0051 648.688 Q63.3848 642.481 63.3848 631.691 Q63.3848 627.426 64.5306 623.67 Q65.6446 619.883 67.9044 616.573 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M80.2221 593.083 Q80.2221 597.794 83.9142 600.531 Q87.5745 603.268 93.972 603.268 Q100.37 603.268 104.062 600.563 Q107.722 597.826 107.722 593.083 Q107.722 588.404 104.03 585.667 Q100.338 582.93 93.972 582.93 Q87.6381 582.93 83.946 585.667 Q80.2221 588.404 80.2221 593.083 M75.2568 593.083 Q75.2568 585.444 80.2221 581.084 Q85.1873 576.723 93.972 576.723 Q102.725 576.723 107.722 581.084 Q112.687 585.444 112.687 593.083 Q112.687 600.754 107.722 605.114 Q102.725 609.443 93.972 609.443 Q85.1873 609.443 80.2221 605.114 Q75.2568 600.754 75.2568 593.083 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M97.6959 567.62 L76.1162 567.62 L76.1162 561.764 L97.4731 561.764 Q102.534 561.764 105.08 559.791 Q107.595 557.817 107.595 553.87 Q107.595 549.128 104.571 546.391 Q101.547 543.622 96.3273 543.622 L76.1162 543.622 L76.1162 537.765 L111.764 537.765 L111.764 543.622 L106.29 543.622 Q109.536 545.754 111.128 548.587 Q112.687 551.388 112.687 555.112 Q112.687 561.255 108.868 564.438 Q105.048 567.62 97.6959 567.62 M75.2568 552.884 L75.2568 552.884 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M90.2481 496.07 L111.764 496.07 L111.764 501.926 L90.439 501.926 Q85.3783 501.926 82.8638 503.9 Q80.3494 505.873 80.3494 509.82 Q80.3494 514.562 83.3731 517.299 Q86.3968 520.037 91.6167 520.037 L111.764 520.037 L111.764 525.925 L76.1162 525.925 L76.1162 520.037 L81.6544 520.037 Q78.4397 517.936 76.8483 515.103 Q75.2568 512.239 75.2568 508.515 Q75.2568 502.372 79.0763 499.221 Q82.8638 496.07 90.2481 496.07 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M65.9947 478.596 L76.1162 478.596 L76.1162 466.533 L80.6677 466.533 L80.6677 478.596 L100.019 478.596 Q104.38 478.596 105.621 477.418 Q106.863 476.209 106.863 472.549 L106.863 466.533 L111.764 466.533 L111.764 472.549 Q111.764 479.328 109.25 481.906 Q106.703 484.484 100.019 484.484 L80.6677 484.484 L80.6677 488.781 L76.1162 488.781 L76.1162 484.484 L65.9947 484.484 L65.9947 478.596 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M715.784 60.3245 Q714.677 65.6121 713.242 68.6453 Q710.947 73.2771 707.053 73.2771 Q704.102 73.2771 702.011 71.4736 Q699.921 69.67 699.429 66.9647 Q694.06 73.2771 688.485 73.2771 Q683.566 73.2771 680.451 69.5471 Q677.336 65.7761 677.336 59.8326 Q677.336 53.8482 680.287 48.1097 Q683.279 42.3712 687.993 38.8051 Q692.707 35.239 697.544 35.239 Q702.667 35.239 705.291 40.6086 Q705.496 39.133 706.438 38.0263 Q707.381 36.9196 708.816 36.9196 Q709.922 36.9196 710.578 37.5344 Q711.234 38.1083 711.234 39.174 Q711.234 39.7069 710.66 42.0433 L707.627 53.7662 L705.782 61.3492 Q705.086 64.1365 704.881 65.1612 Q704.717 66.186 704.717 67.5386 Q704.717 71.3916 707.217 71.3916 Q708.447 71.3916 709.43 70.6128 Q710.414 69.834 711.193 68.1944 Q711.972 66.5549 712.464 65.0383 Q712.956 63.4807 713.611 61.0213 Q713.816 60.0376 713.98 59.7916 Q714.185 59.5047 714.759 59.5047 Q715.784 59.5047 715.784 60.3245 M704.266 44.4616 Q704.266 44.0927 703.979 43.068 Q703.692 42.0433 703.036 40.6086 Q702.38 39.174 700.946 38.1493 Q699.552 37.0836 697.667 37.0836 Q695.125 37.0836 692.461 39.256 Q689.838 41.4284 687.911 45.1175 Q686.435 48.0687 684.837 54.2581 Q683.279 60.4475 683.279 63.5216 Q683.279 65.4072 683.689 67.0877 Q684.14 68.7273 685.411 70.0799 Q686.681 71.3916 688.649 71.3916 Q693.445 71.3916 698.158 65.4481 Q699.06 64.4234 699.265 64.0135 Q699.511 63.5626 699.839 62.333 L704.02 45.8963 Q704.266 44.6666 704.266 44.4616 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip332)" d="M295.765 964.065 L295.765 967.715 L299.841 967.715 L299.841 964.065 L295.765 964.065 L295.765 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="295.765,964.065 295.765,967.715 299.841,967.715 299.841,964.065 295.765,964.065 "/>
<path clip-path="url(#clip332)" d="M299.841 303.372 L299.841 967.715 L303.918 967.715 L303.918 303.372 L299.841 303.372 L299.841 303.372  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="299.841,303.372 299.841,967.715 303.918,967.715 303.918,303.372 299.841,303.372 "/>
<path clip-path="url(#clip332)" d="M303.918 153.712 L303.918 967.715 L307.994 967.715 L307.994 153.712 L303.918 153.712 L303.918 153.712  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="303.918,153.712 303.918,967.715 307.994,967.715 307.994,153.712 303.918,153.712 "/>
<path clip-path="url(#clip332)" d="M307.994 358.125 L307.994 967.715 L312.07 967.715 L312.07 358.125 L307.994 358.125 L307.994 358.125  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="307.994,358.125 307.994,967.715 312.07,967.715 312.07,358.125 307.994,358.125 "/>
<path clip-path="url(#clip332)" d="M312.07 693.947 L312.07 967.715 L316.147 967.715 L316.147 693.947 L312.07 693.947 L312.07 693.947  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="312.07,693.947 312.07,967.715 316.147,967.715 316.147,693.947 312.07,693.947 "/>
<path clip-path="url(#clip332)" d="M316.147 752.351 L316.147 967.715 L320.223 967.715 L320.223 752.351 L316.147 752.351 L316.147 752.351  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="316.147,752.351 316.147,967.715 320.223,967.715 320.223,752.351 316.147,752.351 "/>
<path clip-path="url(#clip332)" d="M320.223 854.558 L320.223 967.715 L324.299 967.715 L324.299 854.558 L320.223 854.558 L320.223 854.558  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="320.223,854.558 320.223,967.715 324.299,967.715 324.299,854.558 320.223,854.558 "/>
<path clip-path="url(#clip332)" d="M324.299 887.41 L324.299 967.715 L328.376 967.715 L328.376 887.41 L324.299 887.41 L324.299 887.41  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="324.299,887.41 324.299,967.715 328.376,967.715 328.376,887.41 324.299,887.41 "/>
<path clip-path="url(#clip332)" d="M328.376 909.311 L328.376 967.715 L332.452 967.715 L332.452 909.311 L328.376 909.311 L328.376 909.311  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="328.376,909.311 328.376,967.715 332.452,967.715 332.452,909.311 328.376,909.311 "/>
<path clip-path="url(#clip332)" d="M332.452 920.262 L332.452 967.715 L336.528 967.715 L336.528 920.262 L332.452 920.262 L332.452 920.262  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="332.452,920.262 332.452,967.715 336.528,967.715 336.528,920.262 332.452,920.262 "/>
<path clip-path="url(#clip332)" d="M336.528 923.912 L336.528 967.715 L340.604 967.715 L340.604 923.912 L336.528 923.912 L336.528 923.912  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="336.528,923.912 336.528,967.715 340.604,967.715 340.604,923.912 336.528,923.912 "/>
<path clip-path="url(#clip332)" d="M340.604 931.213 L340.604 967.715 L344.681 967.715 L344.681 931.213 L340.604 931.213 L340.604 931.213  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="340.604,931.213 340.604,967.715 344.681,967.715 344.681,931.213 340.604,931.213 "/>
<path clip-path="url(#clip332)" d="M344.681 945.814 L344.681 967.715 L348.757 967.715 L348.757 945.814 L344.681 945.814 L344.681 945.814  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="344.681,945.814 344.681,967.715 348.757,967.715 348.757,945.814 344.681,945.814 "/>
<path clip-path="url(#clip332)" d="M348.757 938.513 L348.757 967.715 L352.833 967.715 L352.833 938.513 L348.757 938.513 L348.757 938.513  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="348.757,938.513 348.757,967.715 352.833,967.715 352.833,938.513 348.757,938.513 "/>
<path clip-path="url(#clip332)" d="M352.833 956.764 L352.833 967.715 L356.91 967.715 L356.91 956.764 L352.833 956.764 L352.833 956.764  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="352.833,956.764 352.833,967.715 356.91,967.715 356.91,956.764 352.833,956.764 "/>
<path clip-path="url(#clip332)" d="M356.91 945.814 L356.91 967.715 L360.986 967.715 L360.986 945.814 L356.91 945.814 L356.91 945.814  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="356.91,945.814 356.91,967.715 360.986,967.715 360.986,945.814 356.91,945.814 "/>
<path clip-path="url(#clip332)" d="M360.986 960.415 L360.986 967.715 L365.062 967.715 L365.062 960.415 L360.986 960.415 L360.986 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="360.986,960.415 360.986,967.715 365.062,967.715 365.062,960.415 360.986,960.415 "/>
<path clip-path="url(#clip332)" d="M365.062 953.114 L365.062 967.715 L369.139 967.715 L369.139 953.114 L365.062 953.114 L365.062 953.114  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="365.062,953.114 365.062,967.715 369.139,967.715 369.139,953.114 365.062,953.114 "/>
<path clip-path="url(#clip332)" d="M369.139 964.065 L369.139 967.715 L373.215 967.715 L373.215 964.065 L369.139 964.065 L369.139 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="369.139,964.065 369.139,967.715 373.215,967.715 373.215,964.065 369.139,964.065 "/>
<path clip-path="url(#clip332)" d="M373.215 956.764 L373.215 967.715 L377.291 967.715 L377.291 956.764 L373.215 956.764 L373.215 956.764  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="373.215,956.764 373.215,967.715 377.291,967.715 377.291,956.764 373.215,956.764 "/>
<path clip-path="url(#clip332)" d="M377.291 956.764 L377.291 967.715 L381.368 967.715 L381.368 956.764 L377.291 956.764 L377.291 956.764  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="377.291,956.764 377.291,967.715 381.368,967.715 381.368,956.764 377.291,956.764 "/>
<path clip-path="url(#clip332)" d="M381.368 960.415 L381.368 967.715 L385.444 967.715 L385.444 960.415 L381.368 960.415 L381.368 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="381.368,960.415 381.368,967.715 385.444,967.715 385.444,960.415 381.368,960.415 "/>
<path clip-path="url(#clip332)" d="M385.444 964.065 L385.444 967.715 L389.52 967.715 L389.52 964.065 L385.444 964.065 L385.444 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="385.444,964.065 385.444,967.715 389.52,967.715 389.52,964.065 385.444,964.065 "/>
<path clip-path="url(#clip332)" d="M389.52 960.415 L389.52 967.715 L393.597 967.715 L393.597 960.415 L389.52 960.415 L389.52 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="389.52,960.415 389.52,967.715 393.597,967.715 393.597,960.415 389.52,960.415 "/>
<path clip-path="url(#clip332)" d="M393.597 960.415 L393.597 967.715 L397.673 967.715 L397.673 960.415 L393.597 960.415 L393.597 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="393.597,960.415 393.597,967.715 397.673,967.715 397.673,960.415 393.597,960.415 "/>
<path clip-path="url(#clip332)" d="M397.673 953.114 L397.673 967.715 L401.749 967.715 L401.749 953.114 L397.673 953.114 L397.673 953.114  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="397.673,953.114 397.673,967.715 401.749,967.715 401.749,953.114 397.673,953.114 "/>
<path clip-path="url(#clip332)" d="M401.749 960.415 L401.749 967.715 L405.826 967.715 L405.826 960.415 L401.749 960.415 L401.749 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="401.749,960.415 401.749,967.715 405.826,967.715 405.826,960.415 401.749,960.415 "/>
<path clip-path="url(#clip332)" d="M405.826 960.415 L405.826 967.715 L409.902 967.715 L409.902 960.415 L405.826 960.415 L405.826 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="405.826,960.415 405.826,967.715 409.902,967.715 409.902,960.415 405.826,960.415 "/>
<path clip-path="url(#clip332)" d="M409.902 964.065 L409.902 967.715 L413.978 967.715 L413.978 964.065 L409.902 964.065 L409.902 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="409.902,964.065 409.902,967.715 413.978,967.715 413.978,964.065 409.902,964.065 "/>
<path clip-path="url(#clip332)" d="M413.978 964.065 L413.978 967.715 L418.055 967.715 L418.055 964.065 L413.978 964.065 L413.978 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="413.978,964.065 413.978,967.715 418.055,967.715 418.055,964.065 413.978,964.065 "/>
<path clip-path="url(#clip332)" d="M418.055 967.715 L418.055 967.715 L422.131 967.715 L422.131 967.715 L418.055 967.715 L418.055 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="418.055,967.715 418.055,967.715 422.131,967.715 418.055,967.715 "/>
<path clip-path="url(#clip332)" d="M422.131 967.715 L422.131 967.715 L426.207 967.715 L426.207 967.715 L422.131 967.715 L422.131 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="422.131,967.715 422.131,967.715 426.207,967.715 422.131,967.715 "/>
<path clip-path="url(#clip332)" d="M426.207 964.065 L426.207 967.715 L430.284 967.715 L430.284 964.065 L426.207 964.065 L426.207 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="426.207,964.065 426.207,967.715 430.284,967.715 430.284,964.065 426.207,964.065 "/>
<path clip-path="url(#clip332)" d="M430.284 964.065 L430.284 967.715 L434.36 967.715 L434.36 964.065 L430.284 964.065 L430.284 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="430.284,964.065 430.284,967.715 434.36,967.715 434.36,964.065 430.284,964.065 "/>
<path clip-path="url(#clip332)" d="M434.36 967.715 L434.36 967.715 L438.436 967.715 L438.436 967.715 L434.36 967.715 L434.36 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="434.36,967.715 434.36,967.715 438.436,967.715 434.36,967.715 "/>
<path clip-path="url(#clip332)" d="M438.436 967.715 L438.436 967.715 L442.512 967.715 L442.512 967.715 L438.436 967.715 L438.436 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="438.436,967.715 438.436,967.715 442.512,967.715 438.436,967.715 "/>
<path clip-path="url(#clip332)" d="M442.512 964.065 L442.512 967.715 L446.589 967.715 L446.589 964.065 L442.512 964.065 L442.512 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="442.512,964.065 442.512,967.715 446.589,967.715 446.589,964.065 442.512,964.065 "/>
<path clip-path="url(#clip332)" d="M446.589 964.065 L446.589 967.715 L450.665 967.715 L450.665 964.065 L446.589 964.065 L446.589 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="446.589,964.065 446.589,967.715 450.665,967.715 450.665,964.065 446.589,964.065 "/>
<path clip-path="url(#clip332)" d="M450.665 964.065 L450.665 967.715 L454.741 967.715 L454.741 964.065 L450.665 964.065 L450.665 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="450.665,964.065 450.665,967.715 454.741,967.715 454.741,964.065 450.665,964.065 "/>
<path clip-path="url(#clip332)" d="M454.741 967.715 L454.741 967.715 L458.818 967.715 L458.818 967.715 L454.741 967.715 L454.741 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="454.741,967.715 454.741,967.715 458.818,967.715 454.741,967.715 "/>
<path clip-path="url(#clip332)" d="M458.818 964.065 L458.818 967.715 L462.894 967.715 L462.894 964.065 L458.818 964.065 L458.818 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="458.818,964.065 458.818,967.715 462.894,967.715 462.894,964.065 458.818,964.065 "/>
<path clip-path="url(#clip332)" d="M462.894 967.715 L462.894 967.715 L466.97 967.715 L466.97 967.715 L462.894 967.715 L462.894 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="462.894,967.715 462.894,967.715 466.97,967.715 462.894,967.715 "/>
<path clip-path="url(#clip332)" d="M466.97 964.065 L466.97 967.715 L471.047 967.715 L471.047 964.065 L466.97 964.065 L466.97 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="466.97,964.065 466.97,967.715 471.047,967.715 471.047,964.065 466.97,964.065 "/>
<path clip-path="url(#clip332)" d="M471.047 964.065 L471.047 967.715 L475.123 967.715 L475.123 964.065 L471.047 964.065 L471.047 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="471.047,964.065 471.047,967.715 475.123,967.715 475.123,964.065 471.047,964.065 "/>
<path clip-path="url(#clip332)" d="M475.123 964.065 L475.123 967.715 L479.199 967.715 L479.199 964.065 L475.123 964.065 L475.123 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="475.123,964.065 475.123,967.715 479.199,967.715 479.199,964.065 475.123,964.065 "/>
<path clip-path="url(#clip332)" d="M479.199 967.715 L479.199 967.715 L483.276 967.715 L483.276 967.715 L479.199 967.715 L479.199 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="479.199,967.715 479.199,967.715 483.276,967.715 479.199,967.715 "/>
<path clip-path="url(#clip332)" d="M483.276 967.715 L483.276 967.715 L487.352 967.715 L487.352 967.715 L483.276 967.715 L483.276 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="483.276,967.715 483.276,967.715 487.352,967.715 483.276,967.715 "/>
<path clip-path="url(#clip332)" d="M487.352 967.715 L487.352 967.715 L491.428 967.715 L491.428 967.715 L487.352 967.715 L487.352 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="487.352,967.715 487.352,967.715 491.428,967.715 487.352,967.715 "/>
<path clip-path="url(#clip332)" d="M491.428 967.715 L491.428 967.715 L495.505 967.715 L495.505 967.715 L491.428 967.715 L491.428 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="491.428,967.715 491.428,967.715 495.505,967.715 491.428,967.715 "/>
<path clip-path="url(#clip332)" d="M495.505 967.715 L495.505 967.715 L499.581 967.715 L499.581 967.715 L495.505 967.715 L495.505 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="495.505,967.715 495.505,967.715 499.581,967.715 495.505,967.715 "/>
<path clip-path="url(#clip332)" d="M499.581 967.715 L499.581 967.715 L503.657 967.715 L503.657 967.715 L499.581 967.715 L499.581 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="499.581,967.715 499.581,967.715 503.657,967.715 499.581,967.715 "/>
<path clip-path="url(#clip332)" d="M503.657 967.715 L503.657 967.715 L507.734 967.715 L507.734 967.715 L503.657 967.715 L503.657 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="503.657,967.715 503.657,967.715 507.734,967.715 503.657,967.715 "/>
<path clip-path="url(#clip332)" d="M507.734 967.715 L507.734 967.715 L511.81 967.715 L511.81 967.715 L507.734 967.715 L507.734 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="507.734,967.715 507.734,967.715 511.81,967.715 507.734,967.715 "/>
<path clip-path="url(#clip332)" d="M511.81 967.715 L511.81 967.715 L515.886 967.715 L515.886 967.715 L511.81 967.715 L511.81 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="511.81,967.715 511.81,967.715 515.886,967.715 511.81,967.715 "/>
<path clip-path="url(#clip332)" d="M515.886 964.065 L515.886 967.715 L519.963 967.715 L519.963 964.065 L515.886 964.065 L515.886 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="515.886,964.065 515.886,967.715 519.963,967.715 519.963,964.065 515.886,964.065 "/>
<path clip-path="url(#clip332)" d="M519.963 960.415 L519.963 967.715 L524.039 967.715 L524.039 960.415 L519.963 960.415 L519.963 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="519.963,960.415 519.963,967.715 524.039,967.715 524.039,960.415 519.963,960.415 "/>
<path clip-path="url(#clip332)" d="M524.039 964.065 L524.039 967.715 L528.115 967.715 L528.115 964.065 L524.039 964.065 L524.039 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="524.039,964.065 524.039,967.715 528.115,967.715 528.115,964.065 524.039,964.065 "/>
<path clip-path="url(#clip332)" d="M528.115 967.715 L528.115 967.715 L532.192 967.715 L532.192 967.715 L528.115 967.715 L528.115 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="528.115,967.715 528.115,967.715 532.192,967.715 528.115,967.715 "/>
<path clip-path="url(#clip332)" d="M532.192 967.715 L532.192 967.715 L536.268 967.715 L536.268 967.715 L532.192 967.715 L532.192 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="532.192,967.715 532.192,967.715 536.268,967.715 532.192,967.715 "/>
<path clip-path="url(#clip332)" d="M536.268 964.065 L536.268 967.715 L540.344 967.715 L540.344 964.065 L536.268 964.065 L536.268 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="536.268,964.065 536.268,967.715 540.344,967.715 540.344,964.065 536.268,964.065 "/>
<path clip-path="url(#clip332)" d="M540.344 967.715 L540.344 967.715 L544.42 967.715 L544.42 967.715 L540.344 967.715 L540.344 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="540.344,967.715 540.344,967.715 544.42,967.715 540.344,967.715 "/>
<path clip-path="url(#clip332)" d="M544.42 967.715 L544.42 967.715 L548.497 967.715 L548.497 967.715 L544.42 967.715 L544.42 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="544.42,967.715 544.42,967.715 548.497,967.715 544.42,967.715 "/>
<path clip-path="url(#clip332)" d="M548.497 967.715 L548.497 967.715 L552.573 967.715 L552.573 967.715 L548.497 967.715 L548.497 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="548.497,967.715 548.497,967.715 552.573,967.715 548.497,967.715 "/>
<path clip-path="url(#clip332)" d="M552.573 967.715 L552.573 967.715 L556.649 967.715 L556.649 967.715 L552.573 967.715 L552.573 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="552.573,967.715 552.573,967.715 556.649,967.715 552.573,967.715 "/>
<path clip-path="url(#clip332)" d="M556.649 967.715 L556.649 967.715 L560.726 967.715 L560.726 967.715 L556.649 967.715 L556.649 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="556.649,967.715 556.649,967.715 560.726,967.715 556.649,967.715 "/>
<path clip-path="url(#clip332)" d="M560.726 967.715 L560.726 967.715 L564.802 967.715 L564.802 967.715 L560.726 967.715 L560.726 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="560.726,967.715 560.726,967.715 564.802,967.715 560.726,967.715 "/>
<path clip-path="url(#clip332)" d="M564.802 967.715 L564.802 967.715 L568.878 967.715 L568.878 967.715 L564.802 967.715 L564.802 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="564.802,967.715 564.802,967.715 568.878,967.715 564.802,967.715 "/>
<path clip-path="url(#clip332)" d="M568.878 967.715 L568.878 967.715 L572.955 967.715 L572.955 967.715 L568.878 967.715 L568.878 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="568.878,967.715 568.878,967.715 572.955,967.715 568.878,967.715 "/>
<path clip-path="url(#clip332)" d="M572.955 967.715 L572.955 967.715 L577.031 967.715 L577.031 967.715 L572.955 967.715 L572.955 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="572.955,967.715 572.955,967.715 577.031,967.715 572.955,967.715 "/>
<path clip-path="url(#clip332)" d="M577.031 967.715 L577.031 967.715 L581.107 967.715 L581.107 967.715 L577.031 967.715 L577.031 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="577.031,967.715 577.031,967.715 581.107,967.715 577.031,967.715 "/>
<path clip-path="url(#clip332)" d="M581.107 964.065 L581.107 967.715 L585.184 967.715 L585.184 964.065 L581.107 964.065 L581.107 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="581.107,964.065 581.107,967.715 585.184,967.715 585.184,964.065 581.107,964.065 "/>
<path clip-path="url(#clip332)" d="M585.184 967.715 L585.184 967.715 L589.26 967.715 L589.26 967.715 L585.184 967.715 L585.184 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="585.184,967.715 585.184,967.715 589.26,967.715 585.184,967.715 "/>
<path clip-path="url(#clip332)" d="M589.26 967.715 L589.26 967.715 L593.336 967.715 L593.336 967.715 L589.26 967.715 L589.26 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="589.26,967.715 589.26,967.715 593.336,967.715 589.26,967.715 "/>
<path clip-path="url(#clip332)" d="M593.336 967.715 L593.336 967.715 L597.413 967.715 L597.413 967.715 L593.336 967.715 L593.336 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="593.336,967.715 593.336,967.715 597.413,967.715 593.336,967.715 "/>
<path clip-path="url(#clip332)" d="M597.413 967.715 L597.413 967.715 L601.489 967.715 L601.489 967.715 L597.413 967.715 L597.413 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="597.413,967.715 597.413,967.715 601.489,967.715 597.413,967.715 "/>
<path clip-path="url(#clip332)" d="M601.489 967.715 L601.489 967.715 L605.565 967.715 L605.565 967.715 L601.489 967.715 L601.489 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="601.489,967.715 601.489,967.715 605.565,967.715 601.489,967.715 "/>
<path clip-path="url(#clip332)" d="M605.565 967.715 L605.565 967.715 L609.642 967.715 L609.642 967.715 L605.565 967.715 L605.565 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="605.565,967.715 605.565,967.715 609.642,967.715 605.565,967.715 "/>
<path clip-path="url(#clip332)" d="M609.642 967.715 L609.642 967.715 L613.718 967.715 L613.718 967.715 L609.642 967.715 L609.642 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="609.642,967.715 609.642,967.715 613.718,967.715 609.642,967.715 "/>
<path clip-path="url(#clip332)" d="M613.718 967.715 L613.718 967.715 L617.794 967.715 L617.794 967.715 L613.718 967.715 L613.718 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="613.718,967.715 613.718,967.715 617.794,967.715 613.718,967.715 "/>
<path clip-path="url(#clip332)" d="M617.794 964.065 L617.794 967.715 L621.871 967.715 L621.871 964.065 L617.794 964.065 L617.794 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="617.794,964.065 617.794,967.715 621.871,967.715 621.871,964.065 617.794,964.065 "/>
<path clip-path="url(#clip332)" d="M621.871 967.715 L621.871 967.715 L625.947 967.715 L625.947 967.715 L621.871 967.715 L621.871 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="621.871,967.715 621.871,967.715 625.947,967.715 621.871,967.715 "/>
<path clip-path="url(#clip332)" d="M625.947 967.715 L625.947 967.715 L630.023 967.715 L630.023 967.715 L625.947 967.715 L625.947 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="625.947,967.715 625.947,967.715 630.023,967.715 625.947,967.715 "/>
<path clip-path="url(#clip332)" d="M630.023 967.715 L630.023 967.715 L634.1 967.715 L634.1 967.715 L630.023 967.715 L630.023 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="630.023,967.715 630.023,967.715 634.1,967.715 630.023,967.715 "/>
<path clip-path="url(#clip332)" d="M634.1 967.715 L634.1 967.715 L638.176 967.715 L638.176 967.715 L634.1 967.715 L634.1 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="634.1,967.715 634.1,967.715 638.176,967.715 634.1,967.715 "/>
<path clip-path="url(#clip332)" d="M638.176 967.715 L638.176 967.715 L642.252 967.715 L642.252 967.715 L638.176 967.715 L638.176 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="638.176,967.715 638.176,967.715 642.252,967.715 638.176,967.715 "/>
<path clip-path="url(#clip332)" d="M642.252 967.715 L642.252 967.715 L646.329 967.715 L646.329 967.715 L642.252 967.715 L642.252 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="642.252,967.715 642.252,967.715 646.329,967.715 642.252,967.715 "/>
<path clip-path="url(#clip332)" d="M646.329 967.715 L646.329 967.715 L650.405 967.715 L650.405 967.715 L646.329 967.715 L646.329 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="646.329,967.715 646.329,967.715 650.405,967.715 646.329,967.715 "/>
<path clip-path="url(#clip332)" d="M650.405 967.715 L650.405 967.715 L654.481 967.715 L654.481 967.715 L650.405 967.715 L650.405 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="650.405,967.715 650.405,967.715 654.481,967.715 650.405,967.715 "/>
<path clip-path="url(#clip332)" d="M654.481 967.715 L654.481 967.715 L658.557 967.715 L658.557 967.715 L654.481 967.715 L654.481 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="654.481,967.715 654.481,967.715 658.557,967.715 654.481,967.715 "/>
<path clip-path="url(#clip332)" d="M658.557 967.715 L658.557 967.715 L662.634 967.715 L662.634 967.715 L658.557 967.715 L658.557 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="658.557,967.715 658.557,967.715 662.634,967.715 658.557,967.715 "/>
<path clip-path="url(#clip332)" d="M662.634 967.715 L662.634 967.715 L666.71 967.715 L666.71 967.715 L662.634 967.715 L662.634 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="662.634,967.715 662.634,967.715 666.71,967.715 662.634,967.715 "/>
<path clip-path="url(#clip332)" d="M666.71 967.715 L666.71 967.715 L670.786 967.715 L670.786 967.715 L666.71 967.715 L666.71 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="666.71,967.715 666.71,967.715 670.786,967.715 666.71,967.715 "/>
<path clip-path="url(#clip332)" d="M670.786 964.065 L670.786 967.715 L674.863 967.715 L674.863 964.065 L670.786 964.065 L670.786 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="670.786,964.065 670.786,967.715 674.863,967.715 674.863,964.065 670.786,964.065 "/>
<path clip-path="url(#clip332)" d="M674.863 964.065 L674.863 967.715 L678.939 967.715 L678.939 964.065 L674.863 964.065 L674.863 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="674.863,964.065 674.863,967.715 678.939,967.715 678.939,964.065 674.863,964.065 "/>
<path clip-path="url(#clip332)" d="M678.939 967.715 L678.939 967.715 L683.015 967.715 L683.015 967.715 L678.939 967.715 L678.939 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="678.939,967.715 678.939,967.715 683.015,967.715 678.939,967.715 "/>
<path clip-path="url(#clip332)" d="M683.015 967.715 L683.015 967.715 L687.092 967.715 L687.092 967.715 L683.015 967.715 L683.015 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="683.015,967.715 683.015,967.715 687.092,967.715 683.015,967.715 "/>
<path clip-path="url(#clip332)" d="M687.092 967.715 L687.092 967.715 L691.168 967.715 L691.168 967.715 L687.092 967.715 L687.092 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="687.092,967.715 687.092,967.715 691.168,967.715 687.092,967.715 "/>
<path clip-path="url(#clip332)" d="M691.168 967.715 L691.168 967.715 L695.244 967.715 L695.244 967.715 L691.168 967.715 L691.168 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="691.168,967.715 691.168,967.715 695.244,967.715 691.168,967.715 "/>
<path clip-path="url(#clip332)" d="M695.244 967.715 L695.244 967.715 L699.321 967.715 L699.321 967.715 L695.244 967.715 L695.244 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="695.244,967.715 695.244,967.715 699.321,967.715 695.244,967.715 "/>
<path clip-path="url(#clip332)" d="M699.321 967.715 L699.321 967.715 L703.397 967.715 L703.397 967.715 L699.321 967.715 L699.321 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="699.321,967.715 699.321,967.715 703.397,967.715 699.321,967.715 "/>
<path clip-path="url(#clip332)" d="M703.397 964.065 L703.397 967.715 L707.473 967.715 L707.473 964.065 L703.397 964.065 L703.397 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="703.397,964.065 703.397,967.715 707.473,967.715 707.473,964.065 703.397,964.065 "/>
<path clip-path="url(#clip332)" d="M707.473 967.715 L707.473 967.715 L711.55 967.715 L711.55 967.715 L707.473 967.715 L707.473 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="707.473,967.715 707.473,967.715 711.55,967.715 707.473,967.715 "/>
<path clip-path="url(#clip332)" d="M711.55 967.715 L711.55 967.715 L715.626 967.715 L715.626 967.715 L711.55 967.715 L711.55 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="711.55,967.715 711.55,967.715 715.626,967.715 711.55,967.715 "/>
<path clip-path="url(#clip332)" d="M715.626 967.715 L715.626 967.715 L719.702 967.715 L719.702 967.715 L715.626 967.715 L715.626 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="715.626,967.715 715.626,967.715 719.702,967.715 715.626,967.715 "/>
<path clip-path="url(#clip332)" d="M719.702 964.065 L719.702 967.715 L723.779 967.715 L723.779 964.065 L719.702 964.065 L719.702 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="719.702,964.065 719.702,967.715 723.779,967.715 723.779,964.065 719.702,964.065 "/>
<path clip-path="url(#clip332)" d="M723.779 967.715 L723.779 967.715 L727.855 967.715 L727.855 967.715 L723.779 967.715 L723.779 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="723.779,967.715 723.779,967.715 727.855,967.715 723.779,967.715 "/>
<path clip-path="url(#clip332)" d="M727.855 967.715 L727.855 967.715 L731.931 967.715 L731.931 967.715 L727.855 967.715 L727.855 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="727.855,967.715 727.855,967.715 731.931,967.715 727.855,967.715 "/>
<path clip-path="url(#clip332)" d="M731.931 967.715 L731.931 967.715 L736.008 967.715 L736.008 967.715 L731.931 967.715 L731.931 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="731.931,967.715 731.931,967.715 736.008,967.715 731.931,967.715 "/>
<path clip-path="url(#clip332)" d="M736.008 967.715 L736.008 967.715 L740.084 967.715 L740.084 967.715 L736.008 967.715 L736.008 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="736.008,967.715 736.008,967.715 740.084,967.715 736.008,967.715 "/>
<path clip-path="url(#clip332)" d="M740.084 964.065 L740.084 967.715 L744.16 967.715 L744.16 964.065 L740.084 964.065 L740.084 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="740.084,964.065 740.084,967.715 744.16,967.715 744.16,964.065 740.084,964.065 "/>
<path clip-path="url(#clip332)" d="M744.16 967.715 L744.16 967.715 L748.237 967.715 L748.237 967.715 L744.16 967.715 L744.16 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="744.16,967.715 744.16,967.715 748.237,967.715 744.16,967.715 "/>
<path clip-path="url(#clip332)" d="M748.237 967.715 L748.237 967.715 L752.313 967.715 L752.313 967.715 L748.237 967.715 L748.237 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="748.237,967.715 748.237,967.715 752.313,967.715 748.237,967.715 "/>
<path clip-path="url(#clip332)" d="M752.313 967.715 L752.313 967.715 L756.389 967.715 L756.389 967.715 L752.313 967.715 L752.313 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="752.313,967.715 752.313,967.715 756.389,967.715 752.313,967.715 "/>
<path clip-path="url(#clip332)" d="M756.389 967.715 L756.389 967.715 L760.465 967.715 L760.465 967.715 L756.389 967.715 L756.389 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="756.389,967.715 756.389,967.715 760.465,967.715 756.389,967.715 "/>
<path clip-path="url(#clip332)" d="M760.465 967.715 L760.465 967.715 L764.542 967.715 L764.542 967.715 L760.465 967.715 L760.465 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="760.465,967.715 760.465,967.715 764.542,967.715 760.465,967.715 "/>
<path clip-path="url(#clip332)" d="M764.542 967.715 L764.542 967.715 L768.618 967.715 L768.618 967.715 L764.542 967.715 L764.542 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="764.542,967.715 764.542,967.715 768.618,967.715 764.542,967.715 "/>
<path clip-path="url(#clip332)" d="M768.618 967.715 L768.618 967.715 L772.694 967.715 L772.694 967.715 L768.618 967.715 L768.618 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="768.618,967.715 768.618,967.715 772.694,967.715 768.618,967.715 "/>
<path clip-path="url(#clip332)" d="M772.694 967.715 L772.694 967.715 L776.771 967.715 L776.771 967.715 L772.694 967.715 L772.694 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="772.694,967.715 772.694,967.715 776.771,967.715 772.694,967.715 "/>
<path clip-path="url(#clip332)" d="M776.771 967.715 L776.771 967.715 L780.847 967.715 L780.847 967.715 L776.771 967.715 L776.771 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="776.771,967.715 776.771,967.715 780.847,967.715 776.771,967.715 "/>
<path clip-path="url(#clip332)" d="M780.847 967.715 L780.847 967.715 L784.923 967.715 L784.923 967.715 L780.847 967.715 L780.847 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="780.847,967.715 780.847,967.715 784.923,967.715 780.847,967.715 "/>
<path clip-path="url(#clip332)" d="M784.923 967.715 L784.923 967.715 L789 967.715 L789 967.715 L784.923 967.715 L784.923 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="784.923,967.715 784.923,967.715 789,967.715 784.923,967.715 "/>
<path clip-path="url(#clip332)" d="M789 967.715 L789 967.715 L793.076 967.715 L793.076 967.715 L789 967.715 L789 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="789,967.715 789,967.715 793.076,967.715 789,967.715 "/>
<path clip-path="url(#clip332)" d="M793.076 964.065 L793.076 967.715 L797.152 967.715 L797.152 964.065 L793.076 964.065 L793.076 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="793.076,964.065 793.076,967.715 797.152,967.715 797.152,964.065 793.076,964.065 "/>
<path clip-path="url(#clip332)" d="M797.152 967.715 L797.152 967.715 L801.229 967.715 L801.229 967.715 L797.152 967.715 L797.152 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="797.152,967.715 797.152,967.715 801.229,967.715 797.152,967.715 "/>
<path clip-path="url(#clip332)" d="M801.229 967.715 L801.229 967.715 L805.305 967.715 L805.305 967.715 L801.229 967.715 L801.229 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="801.229,967.715 801.229,967.715 805.305,967.715 801.229,967.715 "/>
<path clip-path="url(#clip332)" d="M805.305 967.715 L805.305 967.715 L809.381 967.715 L809.381 967.715 L805.305 967.715 L805.305 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="805.305,967.715 805.305,967.715 809.381,967.715 805.305,967.715 "/>
<path clip-path="url(#clip332)" d="M809.381 967.715 L809.381 967.715 L813.458 967.715 L813.458 967.715 L809.381 967.715 L809.381 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="809.381,967.715 809.381,967.715 813.458,967.715 809.381,967.715 "/>
<path clip-path="url(#clip332)" d="M813.458 964.065 L813.458 967.715 L817.534 967.715 L817.534 964.065 L813.458 964.065 L813.458 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="813.458,964.065 813.458,967.715 817.534,967.715 817.534,964.065 813.458,964.065 "/>
<path clip-path="url(#clip332)" d="M817.534 967.715 L817.534 967.715 L821.61 967.715 L821.61 967.715 L817.534 967.715 L817.534 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="817.534,967.715 817.534,967.715 821.61,967.715 817.534,967.715 "/>
<path clip-path="url(#clip332)" d="M821.61 967.715 L821.61 967.715 L825.687 967.715 L825.687 967.715 L821.61 967.715 L821.61 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="821.61,967.715 821.61,967.715 825.687,967.715 821.61,967.715 "/>
<path clip-path="url(#clip332)" d="M825.687 967.715 L825.687 967.715 L829.763 967.715 L829.763 967.715 L825.687 967.715 L825.687 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="825.687,967.715 825.687,967.715 829.763,967.715 825.687,967.715 "/>
<path clip-path="url(#clip332)" d="M829.763 967.715 L829.763 967.715 L833.839 967.715 L833.839 967.715 L829.763 967.715 L829.763 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="829.763,967.715 829.763,967.715 833.839,967.715 829.763,967.715 "/>
<path clip-path="url(#clip332)" d="M833.839 967.715 L833.839 967.715 L837.916 967.715 L837.916 967.715 L833.839 967.715 L833.839 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="833.839,967.715 833.839,967.715 837.916,967.715 833.839,967.715 "/>
<path clip-path="url(#clip332)" d="M837.916 967.715 L837.916 967.715 L841.992 967.715 L841.992 967.715 L837.916 967.715 L837.916 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="837.916,967.715 837.916,967.715 841.992,967.715 837.916,967.715 "/>
<path clip-path="url(#clip332)" d="M841.992 967.715 L841.992 967.715 L846.068 967.715 L846.068 967.715 L841.992 967.715 L841.992 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="841.992,967.715 841.992,967.715 846.068,967.715 841.992,967.715 "/>
<path clip-path="url(#clip332)" d="M846.068 967.715 L846.068 967.715 L850.145 967.715 L850.145 967.715 L846.068 967.715 L846.068 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="846.068,967.715 846.068,967.715 850.145,967.715 846.068,967.715 "/>
<path clip-path="url(#clip332)" d="M850.145 967.715 L850.145 967.715 L854.221 967.715 L854.221 967.715 L850.145 967.715 L850.145 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="850.145,967.715 850.145,967.715 854.221,967.715 850.145,967.715 "/>
<path clip-path="url(#clip332)" d="M854.221 967.715 L854.221 967.715 L858.297 967.715 L858.297 967.715 L854.221 967.715 L854.221 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="854.221,967.715 854.221,967.715 858.297,967.715 854.221,967.715 "/>
<path clip-path="url(#clip332)" d="M858.297 967.715 L858.297 967.715 L862.374 967.715 L862.374 967.715 L858.297 967.715 L858.297 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="858.297,967.715 858.297,967.715 862.374,967.715 858.297,967.715 "/>
<path clip-path="url(#clip332)" d="M862.374 967.715 L862.374 967.715 L866.45 967.715 L866.45 967.715 L862.374 967.715 L862.374 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="862.374,967.715 862.374,967.715 866.45,967.715 862.374,967.715 "/>
<path clip-path="url(#clip332)" d="M866.45 967.715 L866.45 967.715 L870.526 967.715 L870.526 967.715 L866.45 967.715 L866.45 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="866.45,967.715 866.45,967.715 870.526,967.715 866.45,967.715 "/>
<path clip-path="url(#clip332)" d="M870.526 967.715 L870.526 967.715 L874.602 967.715 L874.602 967.715 L870.526 967.715 L870.526 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="870.526,967.715 870.526,967.715 874.602,967.715 870.526,967.715 "/>
<path clip-path="url(#clip332)" d="M874.602 967.715 L874.602 967.715 L878.679 967.715 L878.679 967.715 L874.602 967.715 L874.602 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="874.602,967.715 874.602,967.715 878.679,967.715 874.602,967.715 "/>
<path clip-path="url(#clip332)" d="M878.679 967.715 L878.679 967.715 L882.755 967.715 L882.755 967.715 L878.679 967.715 L878.679 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="878.679,967.715 878.679,967.715 882.755,967.715 878.679,967.715 "/>
<path clip-path="url(#clip332)" d="M882.755 964.065 L882.755 967.715 L886.831 967.715 L886.831 964.065 L882.755 964.065 L882.755 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="882.755,964.065 882.755,967.715 886.831,967.715 886.831,964.065 882.755,964.065 "/>
<path clip-path="url(#clip332)" d="M886.831 967.715 L886.831 967.715 L890.908 967.715 L890.908 967.715 L886.831 967.715 L886.831 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="886.831,967.715 886.831,967.715 890.908,967.715 886.831,967.715 "/>
<path clip-path="url(#clip332)" d="M890.908 967.715 L890.908 967.715 L894.984 967.715 L894.984 967.715 L890.908 967.715 L890.908 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="890.908,967.715 890.908,967.715 894.984,967.715 890.908,967.715 "/>
<path clip-path="url(#clip332)" d="M894.984 967.715 L894.984 967.715 L899.06 967.715 L899.06 967.715 L894.984 967.715 L894.984 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="894.984,967.715 894.984,967.715 899.06,967.715 894.984,967.715 "/>
<path clip-path="url(#clip332)" d="M899.06 967.715 L899.06 967.715 L903.137 967.715 L903.137 967.715 L899.06 967.715 L899.06 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="899.06,967.715 899.06,967.715 903.137,967.715 899.06,967.715 "/>
<path clip-path="url(#clip332)" d="M903.137 967.715 L903.137 967.715 L907.213 967.715 L907.213 967.715 L903.137 967.715 L903.137 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="903.137,967.715 903.137,967.715 907.213,967.715 903.137,967.715 "/>
<path clip-path="url(#clip332)" d="M907.213 967.715 L907.213 967.715 L911.289 967.715 L911.289 967.715 L907.213 967.715 L907.213 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="907.213,967.715 907.213,967.715 911.289,967.715 907.213,967.715 "/>
<path clip-path="url(#clip332)" d="M911.289 967.715 L911.289 967.715 L915.366 967.715 L915.366 967.715 L911.289 967.715 L911.289 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="911.289,967.715 911.289,967.715 915.366,967.715 911.289,967.715 "/>
<path clip-path="url(#clip332)" d="M915.366 964.065 L915.366 967.715 L919.442 967.715 L919.442 964.065 L915.366 964.065 L915.366 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="915.366,964.065 915.366,967.715 919.442,967.715 919.442,964.065 915.366,964.065 "/>
<path clip-path="url(#clip332)" d="M919.442 967.715 L919.442 967.715 L923.518 967.715 L923.518 967.715 L919.442 967.715 L919.442 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="919.442,967.715 919.442,967.715 923.518,967.715 919.442,967.715 "/>
<path clip-path="url(#clip332)" d="M923.518 967.715 L923.518 967.715 L927.595 967.715 L927.595 967.715 L923.518 967.715 L923.518 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="923.518,967.715 923.518,967.715 927.595,967.715 923.518,967.715 "/>
<path clip-path="url(#clip332)" d="M927.595 964.065 L927.595 967.715 L931.671 967.715 L931.671 964.065 L927.595 964.065 L927.595 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="927.595,964.065 927.595,967.715 931.671,967.715 931.671,964.065 927.595,964.065 "/>
<path clip-path="url(#clip332)" d="M931.671 967.715 L931.671 967.715 L935.747 967.715 L935.747 967.715 L931.671 967.715 L931.671 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="931.671,967.715 931.671,967.715 935.747,967.715 931.671,967.715 "/>
<path clip-path="url(#clip332)" d="M935.747 964.065 L935.747 967.715 L939.824 967.715 L939.824 964.065 L935.747 964.065 L935.747 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="935.747,964.065 935.747,967.715 939.824,967.715 939.824,964.065 935.747,964.065 "/>
<path clip-path="url(#clip332)" d="M939.824 967.715 L939.824 967.715 L943.9 967.715 L943.9 967.715 L939.824 967.715 L939.824 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="939.824,967.715 939.824,967.715 943.9,967.715 939.824,967.715 "/>
<path clip-path="url(#clip332)" d="M943.9 967.715 L943.9 967.715 L947.976 967.715 L947.976 967.715 L943.9 967.715 L943.9 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="943.9,967.715 943.9,967.715 947.976,967.715 943.9,967.715 "/>
<path clip-path="url(#clip332)" d="M947.976 967.715 L947.976 967.715 L952.053 967.715 L952.053 967.715 L947.976 967.715 L947.976 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="947.976,967.715 947.976,967.715 952.053,967.715 947.976,967.715 "/>
<path clip-path="url(#clip332)" d="M952.053 956.764 L952.053 967.715 L956.129 967.715 L956.129 956.764 L952.053 956.764 L952.053 956.764  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="952.053,956.764 952.053,967.715 956.129,967.715 956.129,956.764 952.053,956.764 "/>
<path clip-path="url(#clip332)" d="M956.129 964.065 L956.129 967.715 L960.205 967.715 L960.205 964.065 L956.129 964.065 L956.129 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="956.129,964.065 956.129,967.715 960.205,967.715 960.205,964.065 956.129,964.065 "/>
<path clip-path="url(#clip332)" d="M960.205 967.715 L960.205 967.715 L964.282 967.715 L964.282 967.715 L960.205 967.715 L960.205 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="960.205,967.715 960.205,967.715 964.282,967.715 960.205,967.715 "/>
<path clip-path="url(#clip332)" d="M964.282 964.065 L964.282 967.715 L968.358 967.715 L968.358 964.065 L964.282 964.065 L964.282 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="964.282,964.065 964.282,967.715 968.358,967.715 968.358,964.065 964.282,964.065 "/>
<path clip-path="url(#clip332)" d="M968.358 960.415 L968.358 967.715 L972.434 967.715 L972.434 960.415 L968.358 960.415 L968.358 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="968.358,960.415 968.358,967.715 972.434,967.715 972.434,960.415 968.358,960.415 "/>
<path clip-path="url(#clip332)" d="M972.434 967.715 L972.434 967.715 L976.51 967.715 L976.51 967.715 L972.434 967.715 L972.434 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="972.434,967.715 972.434,967.715 976.51,967.715 972.434,967.715 "/>
<path clip-path="url(#clip332)" d="M976.51 949.464 L976.51 967.715 L980.587 967.715 L980.587 949.464 L976.51 949.464 L976.51 949.464  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="976.51,949.464 976.51,967.715 980.587,967.715 980.587,949.464 976.51,949.464 "/>
<path clip-path="url(#clip332)" d="M980.587 964.065 L980.587 967.715 L984.663 967.715 L984.663 964.065 L980.587 964.065 L980.587 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="980.587,964.065 980.587,967.715 984.663,967.715 984.663,964.065 980.587,964.065 "/>
<path clip-path="url(#clip332)" d="M984.663 964.065 L984.663 967.715 L988.739 967.715 L988.739 964.065 L984.663 964.065 L984.663 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="984.663,964.065 984.663,967.715 988.739,967.715 988.739,964.065 984.663,964.065 "/>
<path clip-path="url(#clip332)" d="M988.739 960.415 L988.739 967.715 L992.816 967.715 L992.816 960.415 L988.739 960.415 L988.739 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="988.739,960.415 988.739,967.715 992.816,967.715 992.816,960.415 988.739,960.415 "/>
<path clip-path="url(#clip332)" d="M992.816 964.065 L992.816 967.715 L996.892 967.715 L996.892 964.065 L992.816 964.065 L992.816 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="992.816,964.065 992.816,967.715 996.892,967.715 996.892,964.065 992.816,964.065 "/>
<path clip-path="url(#clip332)" d="M996.892 960.415 L996.892 967.715 L1000.97 967.715 L1000.97 960.415 L996.892 960.415 L996.892 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="996.892,960.415 996.892,967.715 1000.97,967.715 1000.97,960.415 996.892,960.415 "/>
<path clip-path="url(#clip332)" d="M1000.97 964.065 L1000.97 967.715 L1005.04 967.715 L1005.04 964.065 L1000.97 964.065 L1000.97 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1000.97,964.065 1000.97,967.715 1005.04,967.715 1005.04,964.065 1000.97,964.065 "/>
<path clip-path="url(#clip332)" d="M1005.04 964.065 L1005.04 967.715 L1009.12 967.715 L1009.12 964.065 L1005.04 964.065 L1005.04 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1005.04,964.065 1005.04,967.715 1009.12,967.715 1009.12,964.065 1005.04,964.065 "/>
<path clip-path="url(#clip332)" d="M1009.12 960.415 L1009.12 967.715 L1013.2 967.715 L1013.2 960.415 L1009.12 960.415 L1009.12 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1009.12,960.415 1009.12,967.715 1013.2,967.715 1013.2,960.415 1009.12,960.415 "/>
<path clip-path="url(#clip332)" d="M1013.2 960.415 L1013.2 967.715 L1017.27 967.715 L1017.27 960.415 L1013.2 960.415 L1013.2 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1013.2,960.415 1013.2,967.715 1017.27,967.715 1017.27,960.415 1013.2,960.415 "/>
<path clip-path="url(#clip332)" d="M1017.27 956.764 L1017.27 967.715 L1021.35 967.715 L1021.35 956.764 L1017.27 956.764 L1017.27 956.764  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1017.27,956.764 1017.27,967.715 1021.35,967.715 1021.35,956.764 1017.27,956.764 "/>
<path clip-path="url(#clip332)" d="M1021.35 949.464 L1021.35 967.715 L1025.43 967.715 L1025.43 949.464 L1021.35 949.464 L1021.35 949.464  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1021.35,949.464 1021.35,967.715 1025.43,967.715 1025.43,949.464 1021.35,949.464 "/>
<path clip-path="url(#clip332)" d="M1025.43 953.114 L1025.43 967.715 L1029.5 967.715 L1029.5 953.114 L1025.43 953.114 L1025.43 953.114  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1025.43,953.114 1025.43,967.715 1029.5,967.715 1029.5,953.114 1025.43,953.114 "/>
<path clip-path="url(#clip332)" d="M1029.5 953.114 L1029.5 967.715 L1033.58 967.715 L1033.58 953.114 L1029.5 953.114 L1029.5 953.114  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1029.5,953.114 1029.5,967.715 1033.58,967.715 1033.58,953.114 1029.5,953.114 "/>
<path clip-path="url(#clip332)" d="M1033.58 964.065 L1033.58 967.715 L1037.66 967.715 L1037.66 964.065 L1033.58 964.065 L1033.58 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1033.58,964.065 1033.58,967.715 1037.66,967.715 1037.66,964.065 1033.58,964.065 "/>
<path clip-path="url(#clip332)" d="M1037.66 960.415 L1037.66 967.715 L1041.73 967.715 L1041.73 960.415 L1037.66 960.415 L1037.66 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1037.66,960.415 1037.66,967.715 1041.73,967.715 1041.73,960.415 1037.66,960.415 "/>
<path clip-path="url(#clip332)" d="M1041.73 949.464 L1041.73 967.715 L1045.81 967.715 L1045.81 949.464 L1041.73 949.464 L1041.73 949.464  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1041.73,949.464 1041.73,967.715 1045.81,967.715 1045.81,949.464 1041.73,949.464 "/>
<path clip-path="url(#clip332)" d="M1045.81 949.464 L1045.81 967.715 L1049.88 967.715 L1049.88 949.464 L1045.81 949.464 L1045.81 949.464  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1045.81,949.464 1045.81,967.715 1049.88,967.715 1049.88,949.464 1045.81,949.464 "/>
<path clip-path="url(#clip332)" d="M1049.88 953.114 L1049.88 967.715 L1053.96 967.715 L1053.96 953.114 L1049.88 953.114 L1049.88 953.114  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1049.88,953.114 1049.88,967.715 1053.96,967.715 1053.96,953.114 1049.88,953.114 "/>
<path clip-path="url(#clip332)" d="M1053.96 964.065 L1053.96 967.715 L1058.04 967.715 L1058.04 964.065 L1053.96 964.065 L1053.96 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1053.96,964.065 1053.96,967.715 1058.04,967.715 1058.04,964.065 1053.96,964.065 "/>
<path clip-path="url(#clip332)" d="M1058.04 960.415 L1058.04 967.715 L1062.11 967.715 L1062.11 960.415 L1058.04 960.415 L1058.04 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1058.04,960.415 1058.04,967.715 1062.11,967.715 1062.11,960.415 1058.04,960.415 "/>
<path clip-path="url(#clip332)" d="M1062.11 953.114 L1062.11 967.715 L1066.19 967.715 L1066.19 953.114 L1062.11 953.114 L1062.11 953.114  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1062.11,953.114 1062.11,967.715 1066.19,967.715 1066.19,953.114 1062.11,953.114 "/>
<path clip-path="url(#clip332)" d="M1066.19 956.764 L1066.19 967.715 L1070.27 967.715 L1070.27 956.764 L1066.19 956.764 L1066.19 956.764  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1066.19,956.764 1066.19,967.715 1070.27,967.715 1070.27,956.764 1066.19,956.764 "/>
<path clip-path="url(#clip332)" d="M1070.27 953.114 L1070.27 967.715 L1074.34 967.715 L1074.34 953.114 L1070.27 953.114 L1070.27 953.114  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1070.27,953.114 1070.27,967.715 1074.34,967.715 1074.34,953.114 1070.27,953.114 "/>
<path clip-path="url(#clip332)" d="M1074.34 964.065 L1074.34 967.715 L1078.42 967.715 L1078.42 964.065 L1074.34 964.065 L1074.34 964.065  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1074.34,964.065 1074.34,967.715 1078.42,967.715 1078.42,964.065 1074.34,964.065 "/>
<path clip-path="url(#clip332)" d="M1078.42 956.764 L1078.42 967.715 L1082.49 967.715 L1082.49 956.764 L1078.42 956.764 L1078.42 956.764  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1078.42,956.764 1078.42,967.715 1082.49,967.715 1082.49,956.764 1078.42,956.764 "/>
<path clip-path="url(#clip332)" d="M1082.49 967.715 L1082.49 967.715 L1086.57 967.715 L1086.57 967.715 L1082.49 967.715 L1082.49 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1082.49,967.715 1082.49,967.715 1086.57,967.715 1082.49,967.715 "/>
<path clip-path="url(#clip332)" d="M1086.57 956.764 L1086.57 967.715 L1090.65 967.715 L1090.65 956.764 L1086.57 956.764 L1086.57 956.764  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1086.57,956.764 1086.57,967.715 1090.65,967.715 1090.65,956.764 1086.57,956.764 "/>
<path clip-path="url(#clip332)" d="M1090.65 960.415 L1090.65 967.715 L1094.72 967.715 L1094.72 960.415 L1090.65 960.415 L1090.65 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1090.65,960.415 1090.65,967.715 1094.72,967.715 1094.72,960.415 1090.65,960.415 "/>
<path clip-path="url(#clip332)" d="M1094.72 960.415 L1094.72 967.715 L1098.8 967.715 L1098.8 960.415 L1094.72 960.415 L1094.72 960.415  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1094.72,960.415 1094.72,967.715 1098.8,967.715 1098.8,960.415 1094.72,960.415 "/>
<path clip-path="url(#clip332)" d="M1098.8 872.809 L1098.8 967.715 L1102.88 967.715 L1102.88 872.809 L1098.8 872.809 L1098.8 872.809  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1098.8,872.809 1098.8,967.715 1102.88,967.715 1102.88,872.809 1098.8,872.809 "/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="297.803" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="301.879" cy="303.372" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="305.956" cy="153.712" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="310.032" cy="358.125" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="314.108" cy="693.947" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="318.185" cy="752.351" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="322.261" cy="854.558" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="326.337" cy="887.41" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="330.414" cy="909.311" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="334.49" cy="920.262" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="338.566" cy="923.912" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="342.643" cy="931.213" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="346.719" cy="945.814" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="350.795" cy="938.513" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="354.872" cy="956.764" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="358.948" cy="945.814" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="363.024" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="367.101" cy="953.114" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="371.177" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="375.253" cy="956.764" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="379.33" cy="956.764" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="383.406" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="387.482" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="391.558" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="395.635" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="399.711" cy="953.114" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="403.787" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="407.864" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="411.94" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="416.016" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="420.093" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="424.169" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="428.245" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="432.322" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="436.398" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="440.474" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="444.551" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="448.627" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="452.703" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="456.78" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="460.856" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="464.932" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="469.009" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="473.085" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="477.161" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="481.238" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="485.314" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="489.39" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="493.466" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="497.543" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="501.619" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="505.695" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="509.772" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="513.848" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="517.924" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="522.001" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="526.077" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="530.153" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="534.23" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="538.306" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="542.382" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="546.459" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="550.535" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="554.611" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="558.688" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="562.764" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="566.84" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="570.917" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="574.993" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="579.069" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="583.146" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="587.222" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="591.298" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="595.375" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="599.451" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="603.527" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="607.603" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="611.68" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="615.756" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="619.832" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="623.909" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="627.985" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="632.061" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="636.138" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="640.214" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="644.29" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="648.367" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="652.443" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="656.519" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="660.596" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="664.672" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="668.748" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="672.825" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="676.901" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="680.977" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="685.054" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="689.13" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="693.206" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="697.283" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="701.359" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="705.435" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="709.511" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="713.588" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="717.664" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="721.74" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="725.817" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="729.893" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="733.969" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="738.046" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="742.122" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="746.198" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="750.275" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="754.351" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="758.427" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="762.504" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="766.58" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="770.656" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="774.733" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="778.809" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="782.885" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="786.962" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="791.038" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="795.114" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="799.191" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="803.267" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="807.343" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="811.419" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="815.496" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="819.572" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="823.648" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="827.725" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="831.801" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="835.877" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="839.954" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="844.03" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="848.106" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="852.183" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="856.259" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="860.335" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="864.412" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="868.488" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="872.564" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="876.641" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="880.717" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="884.793" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="888.87" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="892.946" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="897.022" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="901.099" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="905.175" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="909.251" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="913.328" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="917.404" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="921.48" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="925.556" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="929.633" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="933.709" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="937.785" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="941.862" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="945.938" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="950.014" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="954.091" cy="956.764" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="958.167" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="962.243" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="966.32" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="970.396" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="974.472" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="978.549" cy="949.464" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="982.625" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="986.701" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="990.778" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="994.854" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="998.93" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1003.01" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1007.08" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1011.16" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1015.24" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1019.31" cy="956.764" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1023.39" cy="949.464" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1027.46" cy="953.114" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1031.54" cy="953.114" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1035.62" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1039.69" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1043.77" cy="949.464" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1047.85" cy="949.464" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1051.92" cy="953.114" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1056" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1060.08" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1064.15" cy="953.114" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1068.23" cy="956.764" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1072.3" cy="953.114" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1076.38" cy="964.065" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1080.46" cy="956.764" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1084.53" cy="967.715" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1088.61" cy="956.764" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1092.69" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1096.76" cy="960.415" r="2"/>
<circle clip-path="url(#clip332)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1100.84" cy="872.809" r="2"/>
<polyline clip-path="url(#clip332)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="309.908,1781.72 309.908,-660.291 "/>
</svg>
<p>(a) Bootstrap samples for each parameter. The red line is the
MLE.</p>
</div>
</div></td>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="300" height="300" viewBox="0 0 1200 1200">
<defs>
  <clipPath id="clip390">
    <rect x="0" y="0" width="1200" height="1200"/>
  </clipPath>
</defs>
<path clip-path="url(#clip390)" d="M0 1200 L1200 1200 L1200 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip391">
    <rect x="240" y="119" width="841" height="841"/>
  </clipPath>
</defs>
<path clip-path="url(#clip390)" d="M245.885 967.715 L1152.76 967.715 L1152.76 153.712 L245.885 153.712  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip392">
    <rect x="245" y="153" width="908" height="815"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="295.765,967.715 295.765,153.712 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="469.711,967.715 469.711,153.712 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="643.658,967.715 643.658,153.712 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="817.604,967.715 817.604,153.712 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="991.551,967.715 991.551,153.712 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,967.715 1152.76,967.715 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,750.648 1152.76,750.648 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,533.58 1152.76,533.58 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,316.513 1152.76,316.513 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,967.715 1152.76,967.715 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="295.765,967.715 295.765,948.817 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="469.711,967.715 469.711,948.817 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="643.658,967.715 643.658,948.817 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="817.604,967.715 817.604,948.817 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="991.551,967.715 991.551,948.817 "/>
<path clip-path="url(#clip390)" d="M295.765 989.034 Q292.154 989.034 290.325 992.599 Q288.52 996.14 288.52 1003.27 Q288.52 1010.38 290.325 1013.94 Q292.154 1017.48 295.765 1017.48 Q299.399 1017.48 301.205 1013.94 Q303.033 1010.38 303.033 1003.27 Q303.033 996.14 301.205 992.599 Q299.399 989.034 295.765 989.034 M295.765 985.33 Q301.575 985.33 304.631 989.937 Q307.709 994.52 307.709 1003.27 Q307.709 1012 304.631 1016.6 Q301.575 1021.19 295.765 1021.19 Q289.955 1021.19 286.876 1016.6 Q283.821 1012 283.821 1003.27 Q283.821 994.52 286.876 989.937 Q289.955 985.33 295.765 985.33 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M362.528 1001.62 L380.884 1001.62 L380.884 1005.55 L366.81 1005.55 L366.81 1014.02 Q367.829 1013.68 368.847 1013.51 Q369.866 1013.33 370.885 1013.33 Q376.672 1013.33 380.051 1016.5 Q383.431 1019.67 383.431 1025.09 Q383.431 1030.67 379.959 1033.77 Q376.486 1036.85 370.167 1036.85 Q367.991 1036.85 365.722 1036.48 Q363.477 1036.11 361.07 1035.36 L361.07 1030.67 Q363.153 1031.8 365.375 1032.36 Q367.597 1032.91 370.074 1032.91 Q374.079 1032.91 376.417 1030.8 Q378.755 1028.7 378.755 1025.09 Q378.755 1021.48 376.417 1019.37 Q374.079 1017.26 370.074 1017.26 Q368.199 1017.26 366.324 1017.68 Q364.473 1018.1 362.528 1018.98 L362.528 1001.62 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M392.644 1030.3 L397.528 1030.3 L397.528 1036.18 L392.644 1036.18 L392.644 1030.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M417.713 1004.69 Q414.102 1004.69 412.273 1008.26 Q410.468 1011.8 410.468 1018.93 Q410.468 1026.04 412.273 1029.6 Q414.102 1033.14 417.713 1033.14 Q421.347 1033.14 423.153 1029.6 Q424.981 1026.04 424.981 1018.93 Q424.981 1011.8 423.153 1008.26 Q421.347 1004.69 417.713 1004.69 M417.713 1000.99 Q423.523 1000.99 426.579 1005.6 Q429.657 1010.18 429.657 1018.93 Q429.657 1027.66 426.579 1032.26 Q423.523 1036.85 417.713 1036.85 Q411.903 1036.85 408.824 1032.26 Q405.769 1027.66 405.769 1018.93 Q405.769 1010.18 408.824 1005.6 Q411.903 1000.99 417.713 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M447.875 1004.69 Q444.264 1004.69 442.435 1008.26 Q440.63 1011.8 440.63 1018.93 Q440.63 1026.04 442.435 1029.6 Q444.264 1033.14 447.875 1033.14 Q451.509 1033.14 453.315 1029.6 Q455.143 1026.04 455.143 1018.93 Q455.143 1011.8 453.315 1008.26 Q451.509 1004.69 447.875 1004.69 M447.875 1000.99 Q453.685 1000.99 456.741 1005.6 Q459.819 1010.18 459.819 1018.93 Q459.819 1027.66 456.741 1032.26 Q453.685 1036.85 447.875 1036.85 Q442.065 1036.85 438.986 1032.26 Q435.93 1027.66 435.93 1018.93 Q435.93 1010.18 438.986 1005.6 Q442.065 1000.99 447.875 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M496.208 1010.71 L485.629 1021.34 L496.208 1031.92 L493.453 1034.72 L482.828 1024.09 L472.203 1034.72 L469.472 1031.92 L480.027 1021.34 L469.472 1010.71 L472.203 1007.91 L482.828 1018.54 L493.453 1007.91 L496.208 1010.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M508.569 1032.24 L516.208 1032.24 L516.208 1005.87 L507.898 1007.54 L507.898 1003.28 L516.162 1001.62 L520.837 1001.62 L520.837 1032.24 L528.476 1032.24 L528.476 1036.18 L508.569 1036.18 L508.569 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M547.921 1004.69 Q544.31 1004.69 542.481 1008.26 Q540.675 1011.8 540.675 1018.93 Q540.675 1026.04 542.481 1029.6 Q544.31 1033.14 547.921 1033.14 Q551.555 1033.14 553.36 1029.6 Q555.189 1026.04 555.189 1018.93 Q555.189 1011.8 553.36 1008.26 Q551.555 1004.69 547.921 1004.69 M547.921 1000.99 Q553.731 1000.99 556.786 1005.6 Q559.865 1010.18 559.865 1018.93 Q559.865 1027.66 556.786 1032.26 Q553.731 1036.85 547.921 1036.85 Q542.11 1036.85 539.032 1032.26 Q535.976 1027.66 535.976 1018.93 Q535.976 1010.18 539.032 1005.6 Q542.11 1000.99 547.921 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M572.56 993.624 Q575.287 994.207 576.811 996.051 Q578.353 997.894 578.353 1000.6 Q578.353 1004.76 575.494 1007.03 Q572.635 1009.31 567.369 1009.31 Q565.601 1009.31 563.721 1008.95 Q561.859 1008.61 559.865 1007.92 L559.865 1004.25 Q561.445 1005.17 563.326 1005.64 Q565.206 1006.11 567.256 1006.11 Q570.83 1006.11 572.692 1004.7 Q574.573 1003.29 574.573 1000.6 Q574.573 998.119 572.824 996.728 Q571.093 995.317 567.99 995.317 L564.717 995.317 L564.717 992.195 L568.14 992.195 Q570.943 992.195 572.429 991.085 Q573.914 989.957 573.914 987.85 Q573.914 985.688 572.372 984.54 Q570.849 983.374 567.99 983.374 Q566.429 983.374 564.642 983.713 Q562.855 984.051 560.711 984.766 L560.711 981.381 Q562.874 980.779 564.755 980.478 Q566.655 980.177 568.328 980.177 Q572.654 980.177 575.174 982.152 Q577.695 984.108 577.695 987.455 Q577.695 989.788 576.359 991.405 Q575.024 993.004 572.56 993.624 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M535.476 1032.24 L543.114 1032.24 L543.114 1005.87 L534.804 1007.54 L534.804 1003.28 L543.068 1001.62 L547.744 1001.62 L547.744 1032.24 L555.383 1032.24 L555.383 1036.18 L535.476 1036.18 L535.476 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M564.827 1030.3 L569.711 1030.3 L569.711 1036.18 L564.827 1036.18 L564.827 1030.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M589.897 1004.69 Q586.285 1004.69 584.457 1008.26 Q582.651 1011.8 582.651 1018.93 Q582.651 1026.04 584.457 1029.6 Q586.285 1033.14 589.897 1033.14 Q593.531 1033.14 595.336 1029.6 Q597.165 1026.04 597.165 1018.93 Q597.165 1011.8 595.336 1008.26 Q593.531 1004.69 589.897 1004.69 M589.897 1000.99 Q595.707 1000.99 598.762 1005.6 Q601.841 1010.18 601.841 1018.93 Q601.841 1027.66 598.762 1032.26 Q595.707 1036.85 589.897 1036.85 Q584.086 1036.85 581.008 1032.26 Q577.952 1027.66 577.952 1018.93 Q577.952 1010.18 581.008 1005.6 Q584.086 1000.99 589.897 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M620.058 1004.69 Q616.447 1004.69 614.619 1008.26 Q612.813 1011.8 612.813 1018.93 Q612.813 1026.04 614.619 1029.6 Q616.447 1033.14 620.058 1033.14 Q623.693 1033.14 625.498 1029.6 Q627.327 1026.04 627.327 1018.93 Q627.327 1011.8 625.498 1008.26 Q623.693 1004.69 620.058 1004.69 M620.058 1000.99 Q625.869 1000.99 628.924 1005.6 Q632.003 1010.18 632.003 1018.93 Q632.003 1027.66 628.924 1032.26 Q625.869 1036.85 620.058 1036.85 Q614.248 1036.85 611.17 1032.26 Q608.114 1027.66 608.114 1018.93 Q608.114 1010.18 611.17 1005.6 Q614.248 1000.99 620.058 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M668.391 1010.71 L657.813 1021.34 L668.391 1031.92 L665.637 1034.72 L655.012 1024.09 L644.387 1034.72 L641.656 1031.92 L652.211 1021.34 L641.656 1010.71 L644.387 1007.91 L655.012 1018.54 L665.637 1007.91 L668.391 1010.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M680.753 1032.24 L688.391 1032.24 L688.391 1005.87 L680.081 1007.54 L680.081 1003.28 L688.345 1001.62 L693.021 1001.62 L693.021 1032.24 L700.66 1032.24 L700.66 1036.18 L680.753 1036.18 L680.753 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M720.104 1004.69 Q716.493 1004.69 714.664 1008.26 Q712.859 1011.8 712.859 1018.93 Q712.859 1026.04 714.664 1029.6 Q716.493 1033.14 720.104 1033.14 Q723.738 1033.14 725.544 1029.6 Q727.373 1026.04 727.373 1018.93 Q727.373 1011.8 725.544 1008.26 Q723.738 1004.69 720.104 1004.69 M720.104 1000.99 Q725.914 1000.99 728.97 1005.6 Q732.049 1010.18 732.049 1018.93 Q732.049 1027.66 728.97 1032.26 Q725.914 1036.85 720.104 1036.85 Q714.294 1036.85 711.215 1032.26 Q708.16 1027.66 708.16 1018.93 Q708.16 1010.18 711.215 1005.6 Q714.294 1000.99 720.104 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M744.725 983.995 L735.133 998.985 L744.725 998.985 L744.725 983.995 M743.728 980.685 L748.505 980.685 L748.505 998.985 L752.511 998.985 L752.511 1002.14 L748.505 1002.14 L748.505 1008.76 L744.725 1008.76 L744.725 1002.14 L732.049 1002.14 L732.049 998.477 L743.728 980.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M709.422 1032.24 L717.061 1032.24 L717.061 1005.87 L708.751 1007.54 L708.751 1003.28 L717.015 1001.62 L721.69 1001.62 L721.69 1032.24 L729.329 1032.24 L729.329 1036.18 L709.422 1036.18 L709.422 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M738.774 1030.3 L743.658 1030.3 L743.658 1036.18 L738.774 1036.18 L738.774 1030.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M753.889 1001.62 L772.246 1001.62 L772.246 1005.55 L758.172 1005.55 L758.172 1014.02 Q759.19 1013.68 760.209 1013.51 Q761.227 1013.33 762.246 1013.33 Q768.033 1013.33 771.412 1016.5 Q774.792 1019.67 774.792 1025.09 Q774.792 1030.67 771.32 1033.77 Q767.848 1036.85 761.528 1036.85 Q759.352 1036.85 757.084 1036.48 Q754.838 1036.11 752.431 1035.36 L752.431 1030.67 Q754.514 1031.8 756.737 1032.36 Q758.959 1032.91 761.436 1032.91 Q765.44 1032.91 767.778 1030.8 Q770.116 1028.7 770.116 1025.09 Q770.116 1021.48 767.778 1019.37 Q765.44 1017.26 761.436 1017.26 Q759.561 1017.26 757.686 1017.68 Q755.834 1018.1 753.889 1018.98 L753.889 1001.62 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M794.005 1004.69 Q790.394 1004.69 788.565 1008.26 Q786.76 1011.8 786.76 1018.93 Q786.76 1026.04 788.565 1029.6 Q790.394 1033.14 794.005 1033.14 Q797.639 1033.14 799.445 1029.6 Q801.273 1026.04 801.273 1018.93 Q801.273 1011.8 799.445 1008.26 Q797.639 1004.69 794.005 1004.69 M794.005 1000.99 Q799.815 1000.99 802.871 1005.6 Q805.949 1010.18 805.949 1018.93 Q805.949 1027.66 802.871 1032.26 Q799.815 1036.85 794.005 1036.85 Q788.195 1036.85 785.116 1032.26 Q782.06 1027.66 782.06 1018.93 Q782.06 1010.18 785.116 1005.6 Q788.195 1000.99 794.005 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M842.338 1010.71 L831.759 1021.34 L842.338 1031.92 L839.583 1034.72 L828.958 1024.09 L818.333 1034.72 L815.602 1031.92 L826.157 1021.34 L815.602 1010.71 L818.333 1007.91 L828.958 1018.54 L839.583 1007.91 L842.338 1010.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M854.699 1032.24 L862.338 1032.24 L862.338 1005.87 L854.028 1007.54 L854.028 1003.28 L862.292 1001.62 L866.967 1001.62 L866.967 1032.24 L874.606 1032.24 L874.606 1036.18 L854.699 1036.18 L854.699 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M894.051 1004.69 Q890.44 1004.69 888.611 1008.26 Q886.805 1011.8 886.805 1018.93 Q886.805 1026.04 888.611 1029.6 Q890.44 1033.14 894.051 1033.14 Q897.685 1033.14 899.49 1029.6 Q901.319 1026.04 901.319 1018.93 Q901.319 1011.8 899.49 1008.26 Q897.685 1004.69 894.051 1004.69 M894.051 1000.99 Q899.861 1000.99 902.916 1005.6 Q905.995 1010.18 905.995 1018.93 Q905.995 1027.66 902.916 1032.26 Q899.861 1036.85 894.051 1036.85 Q888.24 1036.85 885.162 1032.26 Q882.106 1027.66 882.106 1018.93 Q882.106 1010.18 885.162 1005.6 Q888.24 1000.99 894.051 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M918.671 983.995 L909.079 998.985 L918.671 998.985 L918.671 983.995 M917.675 980.685 L922.452 980.685 L922.452 998.985 L926.458 998.985 L926.458 1002.14 L922.452 1002.14 L922.452 1008.76 L918.671 1008.76 L918.671 1002.14 L905.995 1002.14 L905.995 998.477 L917.675 980.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M887.454 1032.24 L903.773 1032.24 L903.773 1036.18 L881.829 1036.18 L881.829 1032.24 Q884.491 1029.49 889.074 1024.86 Q893.681 1020.2 894.861 1018.86 Q897.107 1016.34 897.986 1014.6 Q898.889 1012.84 898.889 1011.15 Q898.889 1008.4 896.945 1006.66 Q895.023 1004.93 891.922 1004.93 Q889.723 1004.93 887.269 1005.69 Q884.838 1006.45 882.061 1008 L882.061 1003.28 Q884.885 1002.15 887.338 1001.57 Q889.792 1000.99 891.829 1000.99 Q897.199 1000.99 900.394 1003.68 Q903.588 1006.36 903.588 1010.85 Q903.588 1012.98 902.778 1014.9 Q901.991 1016.8 899.885 1019.39 Q899.306 1020.06 896.204 1023.28 Q893.102 1026.48 887.454 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M913.588 1030.3 L918.472 1030.3 L918.472 1036.18 L913.588 1036.18 L913.588 1030.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M938.657 1004.69 Q935.046 1004.69 933.218 1008.26 Q931.412 1011.8 931.412 1018.93 Q931.412 1026.04 933.218 1029.6 Q935.046 1033.14 938.657 1033.14 Q942.292 1033.14 944.097 1029.6 Q945.926 1026.04 945.926 1018.93 Q945.926 1011.8 944.097 1008.26 Q942.292 1004.69 938.657 1004.69 M938.657 1000.99 Q944.468 1000.99 947.523 1005.6 Q950.602 1010.18 950.602 1018.93 Q950.602 1027.66 947.523 1032.26 Q944.468 1036.85 938.657 1036.85 Q932.847 1036.85 929.769 1032.26 Q926.713 1027.66 926.713 1018.93 Q926.713 1010.18 929.769 1005.6 Q932.847 1000.99 938.657 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M968.819 1004.69 Q965.208 1004.69 963.38 1008.26 Q961.574 1011.8 961.574 1018.93 Q961.574 1026.04 963.38 1029.6 Q965.208 1033.14 968.819 1033.14 Q972.454 1033.14 974.259 1029.6 Q976.088 1026.04 976.088 1018.93 Q976.088 1011.8 974.259 1008.26 Q972.454 1004.69 968.819 1004.69 M968.819 1000.99 Q974.629 1000.99 977.685 1005.6 Q980.764 1010.18 980.764 1018.93 Q980.764 1027.66 977.685 1032.26 Q974.629 1036.85 968.819 1036.85 Q963.009 1036.85 959.93 1032.26 Q956.875 1027.66 956.875 1018.93 Q956.875 1010.18 959.93 1005.6 Q963.009 1000.99 968.819 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1017.15 1010.71 L1006.57 1021.34 L1017.15 1031.92 L1014.4 1034.72 L1003.77 1024.09 L993.148 1034.72 L990.416 1031.92 L1000.97 1021.34 L990.416 1010.71 L993.148 1007.91 L1003.77 1018.54 L1014.4 1007.91 L1017.15 1010.71 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1029.51 1032.24 L1037.15 1032.24 L1037.15 1005.87 L1028.84 1007.54 L1028.84 1003.28 L1037.11 1001.62 L1041.78 1001.62 L1041.78 1032.24 L1049.42 1032.24 L1049.42 1036.18 L1029.51 1036.18 L1029.51 1032.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1068.87 1004.69 Q1065.25 1004.69 1063.43 1008.26 Q1061.62 1011.8 1061.62 1018.93 Q1061.62 1026.04 1063.43 1029.6 Q1065.25 1033.14 1068.87 1033.14 Q1072.5 1033.14 1074.3 1029.6 Q1076.13 1026.04 1076.13 1018.93 Q1076.13 1011.8 1074.3 1008.26 Q1072.5 1004.69 1068.87 1004.69 M1068.87 1000.99 Q1074.68 1000.99 1077.73 1005.6 Q1080.81 1010.18 1080.81 1018.93 Q1080.81 1027.66 1077.73 1032.26 Q1074.68 1036.85 1068.87 1036.85 Q1063.05 1036.85 1059.98 1032.26 Q1056.92 1027.66 1056.92 1018.93 Q1056.92 1010.18 1059.98 1005.6 Q1063.05 1000.99 1068.87 1000.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1093.49 983.995 L1083.89 998.985 L1093.49 998.985 L1093.49 983.995 M1092.49 980.685 L1097.27 980.685 L1097.27 998.985 L1101.27 998.985 L1101.27 1002.14 L1097.27 1002.14 L1097.27 1008.76 L1093.49 1008.76 L1093.49 1002.14 L1080.81 1002.14 L1080.81 998.477 L1092.49 980.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M709.6 1088.17 Q709.6 1092.81 707.249 1097.35 Q704.931 1101.86 701.195 1104.7 Q697.491 1107.53 693.691 1107.53 Q689.987 1107.53 687.636 1104.7 Q685.317 1101.83 685.317 1097.16 Q685.317 1095.52 685.51 1094.36 Q685.704 1093.2 686.38 1090.49 L691.791 1068.98 Q692.725 1064.95 692.725 1064.92 Q692.725 1064.53 692.564 1064.34 Q692.435 1064.12 691.662 1063.95 Q690.889 1063.79 689.407 1063.79 Q688.795 1063.79 688.506 1063.76 Q688.248 1063.73 688.023 1063.57 Q687.829 1063.38 687.829 1062.99 Q687.829 1062.44 688.055 1062.15 Q688.312 1061.83 688.506 1061.76 Q688.699 1061.7 689.085 1061.67 Q689.633 1061.64 691.469 1061.48 Q693.304 1061.28 694.882 1061.15 Q696.46 1061.02 697.137 1061.02 Q697.523 1061.02 697.716 1061.22 Q697.942 1061.38 697.942 1061.57 L697.974 1061.73 L693.111 1081.6 Q696.944 1077.64 700.84 1077.64 Q704.737 1077.64 707.153 1080.64 Q709.6 1083.63 709.6 1088.17 M704.834 1085.28 Q704.834 1082.22 703.675 1080.67 Q702.547 1079.09 700.679 1079.09 Q699.101 1079.09 697.04 1080.28 Q694.979 1081.47 692.789 1084.37 Q692.37 1084.92 692.209 1085.24 Q692.08 1085.57 691.791 1086.66 L690.341 1092.75 Q689.279 1096.9 689.279 1099.41 Q689.279 1102.83 690.535 1104.44 Q691.791 1106.05 693.691 1106.05 Q695.688 1106.05 697.781 1104.28 Q699.874 1102.51 701.485 1099.28 Q702.483 1097.16 703.642 1092.59 Q704.834 1087.98 704.834 1085.28 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,967.715 245.885,153.712 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,967.715 264.783,967.715 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,750.648 264.783,750.648 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,533.58 264.783,533.58 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,316.513 264.783,316.513 "/>
<path clip-path="url(#clip390)" d="M215.941 953.514 Q212.33 953.514 210.501 957.079 Q208.696 960.62 208.696 967.75 Q208.696 974.856 210.501 978.421 Q212.33 981.963 215.941 981.963 Q219.575 981.963 221.381 978.421 Q223.21 974.856 223.21 967.75 Q223.21 960.62 221.381 957.079 Q219.575 953.514 215.941 953.514 M215.941 949.81 Q221.751 949.81 224.807 954.417 Q227.885 959 227.885 967.75 Q227.885 976.477 224.807 981.083 Q221.751 985.666 215.941 985.666 Q210.131 985.666 207.052 981.083 Q203.997 976.477 203.997 967.75 Q203.997 959 207.052 954.417 Q210.131 949.81 215.941 949.81 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M146.428 763.992 L154.066 763.992 L154.066 737.627 L145.756 739.293 L145.756 735.034 L154.02 733.368 L158.696 733.368 L158.696 763.992 L166.335 763.992 L166.335 767.928 L146.428 767.928 L146.428 763.992 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M185.779 736.446 Q182.168 736.446 180.339 740.011 Q178.534 743.553 178.534 750.682 Q178.534 757.789 180.339 761.354 Q182.168 764.895 185.779 764.895 Q189.413 764.895 191.219 761.354 Q193.048 757.789 193.048 750.682 Q193.048 743.553 191.219 740.011 Q189.413 736.446 185.779 736.446 M185.779 732.743 Q191.589 732.743 194.645 737.349 Q197.724 741.932 197.724 750.682 Q197.724 759.409 194.645 764.016 Q191.589 768.599 185.779 768.599 Q179.969 768.599 176.89 764.016 Q173.835 759.409 173.835 750.682 Q173.835 741.932 176.89 737.349 Q179.969 732.743 185.779 732.743 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M215.941 736.446 Q212.33 736.446 210.501 740.011 Q208.696 743.553 208.696 750.682 Q208.696 757.789 210.501 761.354 Q212.33 764.895 215.941 764.895 Q219.575 764.895 221.381 761.354 Q223.21 757.789 223.21 750.682 Q223.21 743.553 221.381 740.011 Q219.575 736.446 215.941 736.446 M215.941 732.743 Q221.751 732.743 224.807 737.349 Q227.885 741.932 227.885 750.682 Q227.885 759.409 224.807 764.016 Q221.751 768.599 215.941 768.599 Q210.131 768.599 207.052 764.016 Q203.997 759.409 203.997 750.682 Q203.997 741.932 207.052 737.349 Q210.131 732.743 215.941 732.743 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M149.645 546.925 L165.964 546.925 L165.964 550.86 L144.02 550.86 L144.02 546.925 Q146.682 544.17 151.265 539.541 Q155.872 534.888 157.052 533.545 Q159.298 531.022 160.177 529.286 Q161.08 527.527 161.08 525.837 Q161.08 523.083 159.136 521.346 Q157.215 519.61 154.113 519.61 Q151.914 519.61 149.46 520.374 Q147.029 521.138 144.252 522.689 L144.252 517.967 Q147.076 516.833 149.529 516.254 Q151.983 515.675 154.02 515.675 Q159.39 515.675 162.585 518.36 Q165.779 521.045 165.779 525.536 Q165.779 527.666 164.969 529.587 Q164.182 531.485 162.076 534.078 Q161.497 534.749 158.395 537.967 Q155.293 541.161 149.645 546.925 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M185.779 519.379 Q182.168 519.379 180.339 522.944 Q178.534 526.485 178.534 533.615 Q178.534 540.721 180.339 544.286 Q182.168 547.828 185.779 547.828 Q189.413 547.828 191.219 544.286 Q193.048 540.721 193.048 533.615 Q193.048 526.485 191.219 522.944 Q189.413 519.379 185.779 519.379 M185.779 515.675 Q191.589 515.675 194.645 520.282 Q197.724 524.865 197.724 533.615 Q197.724 542.342 194.645 546.948 Q191.589 551.531 185.779 551.531 Q179.969 551.531 176.89 546.948 Q173.835 542.342 173.835 533.615 Q173.835 524.865 176.89 520.282 Q179.969 515.675 185.779 515.675 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M215.941 519.379 Q212.33 519.379 210.501 522.944 Q208.696 526.485 208.696 533.615 Q208.696 540.721 210.501 544.286 Q212.33 547.828 215.941 547.828 Q219.575 547.828 221.381 544.286 Q223.21 540.721 223.21 533.615 Q223.21 526.485 221.381 522.944 Q219.575 519.379 215.941 519.379 M215.941 515.675 Q221.751 515.675 224.807 520.282 Q227.885 524.865 227.885 533.615 Q227.885 542.342 224.807 546.948 Q221.751 551.531 215.941 551.531 Q210.131 551.531 207.052 546.948 Q203.997 542.342 203.997 533.615 Q203.997 524.865 207.052 520.282 Q210.131 515.675 215.941 515.675 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M159.784 315.159 Q163.14 315.876 165.015 318.145 Q166.914 320.413 166.914 323.746 Q166.914 328.862 163.395 331.663 Q159.877 334.464 153.395 334.464 Q151.219 334.464 148.904 334.024 Q146.613 333.608 144.159 332.751 L144.159 328.237 Q146.103 329.371 148.418 329.95 Q150.733 330.529 153.256 330.529 Q157.654 330.529 159.946 328.793 Q162.261 327.057 162.261 323.746 Q162.261 320.691 160.108 318.978 Q157.978 317.242 154.159 317.242 L150.131 317.242 L150.131 313.399 L154.344 313.399 Q157.793 313.399 159.622 312.034 Q161.451 310.645 161.451 308.052 Q161.451 305.39 159.552 303.978 Q157.677 302.543 154.159 302.543 Q152.238 302.543 150.039 302.96 Q147.84 303.376 145.201 304.256 L145.201 300.089 Q147.863 299.348 150.178 298.978 Q152.515 298.608 154.576 298.608 Q159.9 298.608 163.002 301.038 Q166.103 303.446 166.103 307.566 Q166.103 310.436 164.46 312.427 Q162.816 314.395 159.784 315.159 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M185.779 302.311 Q182.168 302.311 180.339 305.876 Q178.534 309.418 178.534 316.547 Q178.534 323.654 180.339 327.219 Q182.168 330.76 185.779 330.76 Q189.413 330.76 191.219 327.219 Q193.048 323.654 193.048 316.547 Q193.048 309.418 191.219 305.876 Q189.413 302.311 185.779 302.311 M185.779 298.608 Q191.589 298.608 194.645 303.214 Q197.724 307.797 197.724 316.547 Q197.724 325.274 194.645 329.881 Q191.589 334.464 185.779 334.464 Q179.969 334.464 176.89 329.881 Q173.835 325.274 173.835 316.547 Q173.835 307.797 176.89 303.214 Q179.969 298.608 185.779 298.608 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M215.941 302.311 Q212.33 302.311 210.501 305.876 Q208.696 309.418 208.696 316.547 Q208.696 323.654 210.501 327.219 Q212.33 330.76 215.941 330.76 Q219.575 330.76 221.381 327.219 Q223.21 323.654 223.21 316.547 Q223.21 309.418 221.381 305.876 Q219.575 302.311 215.941 302.311 M215.941 298.608 Q221.751 298.608 224.807 303.214 Q227.885 307.797 227.885 316.547 Q227.885 325.274 224.807 329.881 Q221.751 334.464 215.941 334.464 Q210.131 334.464 207.052 329.881 Q203.997 325.274 203.997 316.547 Q203.997 307.797 207.052 303.214 Q210.131 298.608 215.941 298.608 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M67.9044 616.573 L74.6839 616.573 Q71.6602 619.819 70.1643 623.511 Q68.6683 627.172 68.6683 631.309 Q68.6683 639.457 73.6654 643.786 Q78.6307 648.115 88.0519 648.115 Q97.4413 648.115 102.438 643.786 Q107.404 639.457 107.404 631.309 Q107.404 627.172 105.908 623.511 Q104.412 619.819 101.388 616.573 L108.104 616.573 Q110.396 619.946 111.541 623.734 Q112.687 627.49 112.687 631.691 Q112.687 642.481 106.099 648.688 Q99.4783 654.894 88.0519 654.894 Q76.5936 654.894 70.0051 648.688 Q63.3848 642.481 63.3848 631.691 Q63.3848 627.426 64.5306 623.67 Q65.6446 619.883 67.9044 616.573 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M80.2221 593.083 Q80.2221 597.794 83.9142 600.531 Q87.5745 603.268 93.972 603.268 Q100.37 603.268 104.062 600.563 Q107.722 597.826 107.722 593.083 Q107.722 588.404 104.03 585.667 Q100.338 582.93 93.972 582.93 Q87.6381 582.93 83.946 585.667 Q80.2221 588.404 80.2221 593.083 M75.2568 593.083 Q75.2568 585.444 80.2221 581.084 Q85.1873 576.723 93.972 576.723 Q102.725 576.723 107.722 581.084 Q112.687 585.444 112.687 593.083 Q112.687 600.754 107.722 605.114 Q102.725 609.443 93.972 609.443 Q85.1873 609.443 80.2221 605.114 Q75.2568 600.754 75.2568 593.083 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M97.6959 567.62 L76.1162 567.62 L76.1162 561.764 L97.4731 561.764 Q102.534 561.764 105.08 559.791 Q107.595 557.817 107.595 553.87 Q107.595 549.128 104.571 546.391 Q101.547 543.622 96.3273 543.622 L76.1162 543.622 L76.1162 537.765 L111.764 537.765 L111.764 543.622 L106.29 543.622 Q109.536 545.754 111.128 548.587 Q112.687 551.388 112.687 555.112 Q112.687 561.255 108.868 564.438 Q105.048 567.62 97.6959 567.62 M75.2568 552.884 L75.2568 552.884 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M90.2481 496.07 L111.764 496.07 L111.764 501.926 L90.439 501.926 Q85.3783 501.926 82.8638 503.9 Q80.3494 505.873 80.3494 509.82 Q80.3494 514.562 83.3731 517.299 Q86.3968 520.037 91.6167 520.037 L111.764 520.037 L111.764 525.925 L76.1162 525.925 L76.1162 520.037 L81.6544 520.037 Q78.4397 517.936 76.8483 515.103 Q75.2568 512.239 75.2568 508.515 Q75.2568 502.372 79.0763 499.221 Q82.8638 496.07 90.2481 496.07 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M65.9947 478.596 L76.1162 478.596 L76.1162 466.533 L80.6677 466.533 L80.6677 478.596 L100.019 478.596 Q104.38 478.596 105.621 477.418 Q106.863 476.209 106.863 472.549 L106.863 466.533 L111.764 466.533 L111.764 472.549 Q111.764 479.328 109.25 481.906 Q106.703 484.484 100.019 484.484 L80.6677 484.484 L80.6677 488.781 L76.1162 488.781 L76.1162 484.484 L65.9947 484.484 L65.9947 478.596 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M712.403 59.1191 Q712.403 65.0216 709.411 70.801 Q706.46 76.5395 701.705 80.1466 Q696.991 83.7537 692.155 83.7537 Q687.441 83.7537 684.449 80.1466 Q681.497 76.4986 681.497 70.5551 Q681.497 68.4647 681.743 66.989 Q681.989 65.5134 682.85 62.0703 L689.736 34.6895 Q690.925 29.5658 690.925 29.5248 Q690.925 29.033 690.72 28.787 Q690.556 28.5001 689.572 28.2952 Q688.588 28.0902 686.703 28.0902 Q685.924 28.0902 685.555 28.0492 Q685.227 28.0082 684.94 27.8033 Q684.695 27.5574 684.695 27.0655 Q684.695 26.3687 684.981 25.9998 Q685.309 25.5899 685.555 25.5079 Q685.801 25.4259 686.293 25.3849 Q686.99 25.3439 689.326 25.139 Q691.663 24.8931 693.671 24.7291 Q695.68 24.5651 696.54 24.5651 Q697.032 24.5651 697.278 24.8111 Q697.565 25.016 697.565 25.262 L697.606 25.4669 L691.417 50.7573 Q696.294 45.7156 701.254 45.7156 Q706.214 45.7156 709.288 49.5276 Q712.403 53.3396 712.403 59.1191 M706.337 55.4301 Q706.337 51.5361 704.861 49.5686 Q703.427 47.5601 701.049 47.5601 Q699.041 47.5601 696.417 49.0767 Q693.794 50.5933 691.007 54.2824 Q690.474 54.9792 690.269 55.3891 Q690.105 55.799 689.736 57.1926 L687.892 64.9396 Q686.539 70.2272 686.539 73.4244 Q686.539 77.7692 688.138 79.8187 Q689.736 81.8682 692.155 81.8682 Q694.696 81.8682 697.36 79.6137 Q700.025 77.3593 702.074 73.2604 Q703.345 70.5551 704.82 64.7346 Q706.337 58.8732 706.337 55.4301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip392)" d="M295.765 741.965 L295.765 967.715 L302.723 967.715 L302.723 741.965 L295.765 741.965 L295.765 741.965  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="295.765,741.965 295.765,967.715 302.723,967.715 302.723,741.965 295.765,741.965 "/>
<path clip-path="url(#clip392)" d="M302.723 153.712 L302.723 967.715 L309.681 967.715 L309.681 153.712 L302.723 153.712 L302.723 153.712  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="302.723,153.712 302.723,967.715 309.681,967.715 309.681,153.712 302.723,153.712 "/>
<path clip-path="url(#clip392)" d="M309.681 600.871 L309.681 967.715 L316.639 967.715 L316.639 600.871 L309.681 600.871 L309.681 600.871  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="309.681,600.871 309.681,967.715 316.639,967.715 316.639,600.871 309.681,600.871 "/>
<path clip-path="url(#clip392)" d="M316.639 781.037 L316.639 967.715 L323.596 967.715 L323.596 781.037 L316.639 781.037 L316.639 781.037  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="316.639,781.037 316.639,967.715 323.596,967.715 323.596,781.037 316.639,781.037 "/>
<path clip-path="url(#clip392)" d="M323.596 891.741 L323.596 967.715 L330.554 967.715 L330.554 891.741 L323.596 891.741 L323.596 891.741  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="323.596,891.741 323.596,967.715 330.554,967.715 330.554,891.741 323.596,891.741 "/>
<path clip-path="url(#clip392)" d="M330.554 917.79 L330.554 967.715 L337.512 967.715 L337.512 917.79 L330.554 917.79 L330.554 917.79  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="330.554,917.79 330.554,967.715 337.512,967.715 337.512,917.79 330.554,917.79 "/>
<path clip-path="url(#clip392)" d="M337.512 930.814 L337.512 967.715 L344.47 967.715 L344.47 930.814 L337.512 930.814 L337.512 930.814  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="337.512,930.814 337.512,967.715 344.47,967.715 344.47,930.814 337.512,930.814 "/>
<path clip-path="url(#clip392)" d="M344.47 943.838 L344.47 967.715 L351.428 967.715 L351.428 943.838 L344.47 943.838 L344.47 943.838  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="344.47,943.838 344.47,967.715 351.428,967.715 351.428,943.838 344.47,943.838 "/>
<path clip-path="url(#clip392)" d="M351.428 948.179 L351.428 967.715 L358.386 967.715 L358.386 948.179 L351.428 948.179 L351.428 948.179  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="351.428,948.179 351.428,967.715 358.386,967.715 358.386,948.179 351.428,948.179 "/>
<path clip-path="url(#clip392)" d="M358.386 952.52 L358.386 967.715 L365.344 967.715 L365.344 952.52 L358.386 952.52 L358.386 952.52  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="358.386,952.52 358.386,967.715 365.344,967.715 365.344,952.52 358.386,952.52 "/>
<path clip-path="url(#clip392)" d="M365.344 959.032 L365.344 967.715 L372.301 967.715 L372.301 959.032 L365.344 959.032 L365.344 959.032  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="365.344,959.032 365.344,967.715 372.301,967.715 372.301,959.032 365.344,959.032 "/>
<path clip-path="url(#clip392)" d="M372.301 956.862 L372.301 967.715 L379.259 967.715 L379.259 956.862 L372.301 956.862 L372.301 956.862  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="372.301,956.862 372.301,967.715 379.259,967.715 379.259,956.862 372.301,956.862 "/>
<path clip-path="url(#clip392)" d="M379.259 959.032 L379.259 967.715 L386.217 967.715 L386.217 959.032 L379.259 959.032 L379.259 959.032  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="379.259,959.032 379.259,967.715 386.217,967.715 386.217,959.032 379.259,959.032 "/>
<path clip-path="url(#clip392)" d="M386.217 961.203 L386.217 967.715 L393.175 967.715 L393.175 961.203 L386.217 961.203 L386.217 961.203  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="386.217,961.203 386.217,967.715 393.175,967.715 393.175,961.203 386.217,961.203 "/>
<path clip-path="url(#clip392)" d="M393.175 963.374 L393.175 967.715 L400.133 967.715 L400.133 963.374 L393.175 963.374 L393.175 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="393.175,963.374 393.175,967.715 400.133,967.715 400.133,963.374 393.175,963.374 "/>
<path clip-path="url(#clip392)" d="M400.133 965.544 L400.133 967.715 L407.091 967.715 L407.091 965.544 L400.133 965.544 L400.133 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="400.133,965.544 400.133,967.715 407.091,967.715 407.091,965.544 400.133,965.544 "/>
<path clip-path="url(#clip392)" d="M407.091 954.691 L407.091 967.715 L414.049 967.715 L414.049 954.691 L407.091 954.691 L407.091 954.691  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="407.091,954.691 407.091,967.715 414.049,967.715 414.049,954.691 407.091,954.691 "/>
<path clip-path="url(#clip392)" d="M414.049 963.374 L414.049 967.715 L421.006 967.715 L421.006 963.374 L414.049 963.374 L414.049 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="414.049,963.374 414.049,967.715 421.006,967.715 421.006,963.374 414.049,963.374 "/>
<path clip-path="url(#clip392)" d="M421.006 965.544 L421.006 967.715 L427.964 967.715 L427.964 965.544 L421.006 965.544 L421.006 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="421.006,965.544 421.006,967.715 427.964,967.715 427.964,965.544 421.006,965.544 "/>
<path clip-path="url(#clip392)" d="M427.964 965.544 L427.964 967.715 L434.922 967.715 L434.922 965.544 L427.964 965.544 L427.964 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="427.964,965.544 427.964,967.715 434.922,967.715 434.922,965.544 427.964,965.544 "/>
<path clip-path="url(#clip392)" d="M434.922 967.715 L434.922 967.715 L441.88 967.715 L441.88 967.715 L434.922 967.715 L434.922 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="434.922,967.715 434.922,967.715 441.88,967.715 434.922,967.715 "/>
<path clip-path="url(#clip392)" d="M441.88 967.715 L441.88 967.715 L448.838 967.715 L448.838 967.715 L441.88 967.715 L441.88 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="441.88,967.715 441.88,967.715 448.838,967.715 441.88,967.715 "/>
<path clip-path="url(#clip392)" d="M448.838 963.374 L448.838 967.715 L455.796 967.715 L455.796 963.374 L448.838 963.374 L448.838 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="448.838,963.374 448.838,967.715 455.796,967.715 455.796,963.374 448.838,963.374 "/>
<path clip-path="url(#clip392)" d="M455.796 963.374 L455.796 967.715 L462.754 967.715 L462.754 963.374 L455.796 963.374 L455.796 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="455.796,963.374 455.796,967.715 462.754,967.715 462.754,963.374 455.796,963.374 "/>
<path clip-path="url(#clip392)" d="M462.754 967.715 L462.754 967.715 L469.711 967.715 L469.711 967.715 L462.754 967.715 L462.754 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="462.754,967.715 462.754,967.715 469.711,967.715 462.754,967.715 "/>
<path clip-path="url(#clip392)" d="M469.711 965.544 L469.711 967.715 L476.669 967.715 L476.669 965.544 L469.711 965.544 L469.711 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="469.711,965.544 469.711,967.715 476.669,967.715 476.669,965.544 469.711,965.544 "/>
<path clip-path="url(#clip392)" d="M476.669 965.544 L476.669 967.715 L483.627 967.715 L483.627 965.544 L476.669 965.544 L476.669 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="476.669,965.544 476.669,967.715 483.627,967.715 483.627,965.544 476.669,965.544 "/>
<path clip-path="url(#clip392)" d="M483.627 963.374 L483.627 967.715 L490.585 967.715 L490.585 963.374 L483.627 963.374 L483.627 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="483.627,963.374 483.627,967.715 490.585,967.715 490.585,963.374 483.627,963.374 "/>
<path clip-path="url(#clip392)" d="M490.585 967.715 L490.585 967.715 L497.543 967.715 L497.543 967.715 L490.585 967.715 L490.585 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="490.585,967.715 490.585,967.715 497.543,967.715 490.585,967.715 "/>
<path clip-path="url(#clip392)" d="M497.543 965.544 L497.543 967.715 L504.501 967.715 L504.501 965.544 L497.543 965.544 L497.543 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="497.543,965.544 497.543,967.715 504.501,967.715 504.501,965.544 497.543,965.544 "/>
<path clip-path="url(#clip392)" d="M504.501 967.715 L504.501 967.715 L511.459 967.715 L511.459 967.715 L504.501 967.715 L504.501 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="504.501,967.715 504.501,967.715 511.459,967.715 504.501,967.715 "/>
<path clip-path="url(#clip392)" d="M511.459 967.715 L511.459 967.715 L518.416 967.715 L518.416 967.715 L511.459 967.715 L511.459 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="511.459,967.715 511.459,967.715 518.416,967.715 511.459,967.715 "/>
<path clip-path="url(#clip392)" d="M518.416 967.715 L518.416 967.715 L525.374 967.715 L525.374 967.715 L518.416 967.715 L518.416 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="518.416,967.715 518.416,967.715 525.374,967.715 518.416,967.715 "/>
<path clip-path="url(#clip392)" d="M525.374 967.715 L525.374 967.715 L532.332 967.715 L532.332 967.715 L525.374 967.715 L525.374 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="525.374,967.715 525.374,967.715 532.332,967.715 525.374,967.715 "/>
<path clip-path="url(#clip392)" d="M532.332 965.544 L532.332 967.715 L539.29 967.715 L539.29 965.544 L532.332 965.544 L532.332 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="532.332,965.544 532.332,967.715 539.29,967.715 539.29,965.544 532.332,965.544 "/>
<path clip-path="url(#clip392)" d="M539.29 963.374 L539.29 967.715 L546.248 967.715 L546.248 963.374 L539.29 963.374 L539.29 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="539.29,963.374 539.29,967.715 546.248,967.715 546.248,963.374 539.29,963.374 "/>
<path clip-path="url(#clip392)" d="M546.248 967.715 L546.248 967.715 L553.206 967.715 L553.206 967.715 L546.248 967.715 L546.248 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="546.248,967.715 546.248,967.715 553.206,967.715 546.248,967.715 "/>
<path clip-path="url(#clip392)" d="M553.206 965.544 L553.206 967.715 L560.164 967.715 L560.164 965.544 L553.206 965.544 L553.206 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="553.206,965.544 553.206,967.715 560.164,967.715 560.164,965.544 553.206,965.544 "/>
<path clip-path="url(#clip392)" d="M560.164 967.715 L560.164 967.715 L567.121 967.715 L567.121 967.715 L560.164 967.715 L560.164 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="560.164,967.715 560.164,967.715 567.121,967.715 560.164,967.715 "/>
<path clip-path="url(#clip392)" d="M567.121 967.715 L567.121 967.715 L574.079 967.715 L574.079 967.715 L567.121 967.715 L567.121 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="567.121,967.715 567.121,967.715 574.079,967.715 567.121,967.715 "/>
<path clip-path="url(#clip392)" d="M574.079 967.715 L574.079 967.715 L581.037 967.715 L581.037 967.715 L574.079 967.715 L574.079 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="574.079,967.715 574.079,967.715 581.037,967.715 574.079,967.715 "/>
<path clip-path="url(#clip392)" d="M581.037 967.715 L581.037 967.715 L587.995 967.715 L587.995 967.715 L581.037 967.715 L581.037 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="581.037,967.715 581.037,967.715 587.995,967.715 581.037,967.715 "/>
<path clip-path="url(#clip392)" d="M587.995 967.715 L587.995 967.715 L594.953 967.715 L594.953 967.715 L587.995 967.715 L587.995 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="587.995,967.715 587.995,967.715 594.953,967.715 587.995,967.715 "/>
<path clip-path="url(#clip392)" d="M594.953 967.715 L594.953 967.715 L601.911 967.715 L601.911 967.715 L594.953 967.715 L594.953 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="594.953,967.715 594.953,967.715 601.911,967.715 594.953,967.715 "/>
<path clip-path="url(#clip392)" d="M601.911 963.374 L601.911 967.715 L608.869 967.715 L608.869 963.374 L601.911 963.374 L601.911 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="601.911,963.374 601.911,967.715 608.869,967.715 608.869,963.374 601.911,963.374 "/>
<path clip-path="url(#clip392)" d="M608.869 967.715 L608.869 967.715 L615.826 967.715 L615.826 967.715 L608.869 967.715 L608.869 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="608.869,967.715 608.869,967.715 615.826,967.715 608.869,967.715 "/>
<path clip-path="url(#clip392)" d="M615.826 967.715 L615.826 967.715 L622.784 967.715 L622.784 967.715 L615.826 967.715 L615.826 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="615.826,967.715 615.826,967.715 622.784,967.715 615.826,967.715 "/>
<path clip-path="url(#clip392)" d="M622.784 967.715 L622.784 967.715 L629.742 967.715 L629.742 967.715 L622.784 967.715 L622.784 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="622.784,967.715 622.784,967.715 629.742,967.715 622.784,967.715 "/>
<path clip-path="url(#clip392)" d="M629.742 967.715 L629.742 967.715 L636.7 967.715 L636.7 967.715 L629.742 967.715 L629.742 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="629.742,967.715 629.742,967.715 636.7,967.715 629.742,967.715 "/>
<path clip-path="url(#clip392)" d="M636.7 967.715 L636.7 967.715 L643.658 967.715 L643.658 967.715 L636.7 967.715 L636.7 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="636.7,967.715 636.7,967.715 643.658,967.715 636.7,967.715 "/>
<path clip-path="url(#clip392)" d="M643.658 967.715 L643.658 967.715 L650.616 967.715 L650.616 967.715 L643.658 967.715 L643.658 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="643.658,967.715 643.658,967.715 650.616,967.715 643.658,967.715 "/>
<path clip-path="url(#clip392)" d="M650.616 967.715 L650.616 967.715 L657.574 967.715 L657.574 967.715 L650.616 967.715 L650.616 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="650.616,967.715 650.616,967.715 657.574,967.715 650.616,967.715 "/>
<path clip-path="url(#clip392)" d="M657.574 965.544 L657.574 967.715 L664.531 967.715 L664.531 965.544 L657.574 965.544 L657.574 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="657.574,965.544 657.574,967.715 664.531,967.715 664.531,965.544 657.574,965.544 "/>
<path clip-path="url(#clip392)" d="M664.531 967.715 L664.531 967.715 L671.489 967.715 L671.489 967.715 L664.531 967.715 L664.531 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="664.531,967.715 664.531,967.715 671.489,967.715 664.531,967.715 "/>
<path clip-path="url(#clip392)" d="M671.489 967.715 L671.489 967.715 L678.447 967.715 L678.447 967.715 L671.489 967.715 L671.489 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="671.489,967.715 671.489,967.715 678.447,967.715 671.489,967.715 "/>
<path clip-path="url(#clip392)" d="M678.447 967.715 L678.447 967.715 L685.405 967.715 L685.405 967.715 L678.447 967.715 L678.447 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="678.447,967.715 678.447,967.715 685.405,967.715 678.447,967.715 "/>
<path clip-path="url(#clip392)" d="M685.405 967.715 L685.405 967.715 L692.363 967.715 L692.363 967.715 L685.405 967.715 L685.405 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="685.405,967.715 685.405,967.715 692.363,967.715 685.405,967.715 "/>
<path clip-path="url(#clip392)" d="M692.363 967.715 L692.363 967.715 L699.321 967.715 L699.321 967.715 L692.363 967.715 L692.363 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="692.363,967.715 692.363,967.715 699.321,967.715 692.363,967.715 "/>
<path clip-path="url(#clip392)" d="M699.321 965.544 L699.321 967.715 L706.279 967.715 L706.279 965.544 L699.321 965.544 L699.321 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="699.321,965.544 699.321,967.715 706.279,967.715 706.279,965.544 699.321,965.544 "/>
<path clip-path="url(#clip392)" d="M706.279 967.715 L706.279 967.715 L713.236 967.715 L713.236 967.715 L706.279 967.715 L706.279 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="706.279,967.715 706.279,967.715 713.236,967.715 706.279,967.715 "/>
<path clip-path="url(#clip392)" d="M713.236 967.715 L713.236 967.715 L720.194 967.715 L720.194 967.715 L713.236 967.715 L713.236 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="713.236,967.715 713.236,967.715 720.194,967.715 713.236,967.715 "/>
<path clip-path="url(#clip392)" d="M720.194 967.715 L720.194 967.715 L727.152 967.715 L727.152 967.715 L720.194 967.715 L720.194 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="720.194,967.715 720.194,967.715 727.152,967.715 720.194,967.715 "/>
<path clip-path="url(#clip392)" d="M727.152 967.715 L727.152 967.715 L734.11 967.715 L734.11 967.715 L727.152 967.715 L727.152 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="727.152,967.715 727.152,967.715 734.11,967.715 727.152,967.715 "/>
<path clip-path="url(#clip392)" d="M734.11 967.715 L734.11 967.715 L741.068 967.715 L741.068 967.715 L734.11 967.715 L734.11 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="734.11,967.715 734.11,967.715 741.068,967.715 734.11,967.715 "/>
<path clip-path="url(#clip392)" d="M741.068 965.544 L741.068 967.715 L748.026 967.715 L748.026 965.544 L741.068 965.544 L741.068 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="741.068,965.544 741.068,967.715 748.026,967.715 748.026,965.544 741.068,965.544 "/>
<path clip-path="url(#clip392)" d="M748.026 967.715 L748.026 967.715 L754.984 967.715 L754.984 967.715 L748.026 967.715 L748.026 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="748.026,967.715 748.026,967.715 754.984,967.715 748.026,967.715 "/>
<path clip-path="url(#clip392)" d="M754.984 965.544 L754.984 967.715 L761.941 967.715 L761.941 965.544 L754.984 965.544 L754.984 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="754.984,965.544 754.984,967.715 761.941,967.715 761.941,965.544 754.984,965.544 "/>
<path clip-path="url(#clip392)" d="M761.941 967.715 L761.941 967.715 L768.899 967.715 L768.899 967.715 L761.941 967.715 L761.941 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="761.941,967.715 761.941,967.715 768.899,967.715 761.941,967.715 "/>
<path clip-path="url(#clip392)" d="M768.899 967.715 L768.899 967.715 L775.857 967.715 L775.857 967.715 L768.899 967.715 L768.899 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="768.899,967.715 768.899,967.715 775.857,967.715 768.899,967.715 "/>
<path clip-path="url(#clip392)" d="M775.857 965.544 L775.857 967.715 L782.815 967.715 L782.815 965.544 L775.857 965.544 L775.857 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="775.857,965.544 775.857,967.715 782.815,967.715 782.815,965.544 775.857,965.544 "/>
<path clip-path="url(#clip392)" d="M782.815 967.715 L782.815 967.715 L789.773 967.715 L789.773 967.715 L782.815 967.715 L782.815 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="782.815,967.715 782.815,967.715 789.773,967.715 782.815,967.715 "/>
<path clip-path="url(#clip392)" d="M789.773 965.544 L789.773 967.715 L796.731 967.715 L796.731 965.544 L789.773 965.544 L789.773 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="789.773,965.544 789.773,967.715 796.731,967.715 796.731,965.544 789.773,965.544 "/>
<path clip-path="url(#clip392)" d="M796.731 967.715 L796.731 967.715 L803.689 967.715 L803.689 967.715 L796.731 967.715 L796.731 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="796.731,967.715 796.731,967.715 803.689,967.715 796.731,967.715 "/>
<path clip-path="url(#clip392)" d="M803.689 967.715 L803.689 967.715 L810.646 967.715 L810.646 967.715 L803.689 967.715 L803.689 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="803.689,967.715 803.689,967.715 810.646,967.715 803.689,967.715 "/>
<path clip-path="url(#clip392)" d="M810.646 967.715 L810.646 967.715 L817.604 967.715 L817.604 967.715 L810.646 967.715 L810.646 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="810.646,967.715 810.646,967.715 817.604,967.715 810.646,967.715 "/>
<path clip-path="url(#clip392)" d="M817.604 967.715 L817.604 967.715 L824.562 967.715 L824.562 967.715 L817.604 967.715 L817.604 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="817.604,967.715 817.604,967.715 824.562,967.715 817.604,967.715 "/>
<path clip-path="url(#clip392)" d="M824.562 967.715 L824.562 967.715 L831.52 967.715 L831.52 967.715 L824.562 967.715 L824.562 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="824.562,967.715 824.562,967.715 831.52,967.715 824.562,967.715 "/>
<path clip-path="url(#clip392)" d="M831.52 965.544 L831.52 967.715 L838.478 967.715 L838.478 965.544 L831.52 965.544 L831.52 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="831.52,965.544 831.52,967.715 838.478,967.715 838.478,965.544 831.52,965.544 "/>
<path clip-path="url(#clip392)" d="M838.478 967.715 L838.478 967.715 L845.436 967.715 L845.436 967.715 L838.478 967.715 L838.478 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="838.478,967.715 838.478,967.715 845.436,967.715 838.478,967.715 "/>
<path clip-path="url(#clip392)" d="M845.436 967.715 L845.436 967.715 L852.394 967.715 L852.394 967.715 L845.436 967.715 L845.436 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="845.436,967.715 845.436,967.715 852.394,967.715 845.436,967.715 "/>
<path clip-path="url(#clip392)" d="M852.394 967.715 L852.394 967.715 L859.351 967.715 L859.351 967.715 L852.394 967.715 L852.394 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="852.394,967.715 852.394,967.715 859.351,967.715 852.394,967.715 "/>
<path clip-path="url(#clip392)" d="M859.351 967.715 L859.351 967.715 L866.309 967.715 L866.309 967.715 L859.351 967.715 L859.351 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="859.351,967.715 859.351,967.715 866.309,967.715 859.351,967.715 "/>
<path clip-path="url(#clip392)" d="M866.309 967.715 L866.309 967.715 L873.267 967.715 L873.267 967.715 L866.309 967.715 L866.309 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="866.309,967.715 866.309,967.715 873.267,967.715 866.309,967.715 "/>
<path clip-path="url(#clip392)" d="M873.267 967.715 L873.267 967.715 L880.225 967.715 L880.225 967.715 L873.267 967.715 L873.267 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="873.267,967.715 873.267,967.715 880.225,967.715 873.267,967.715 "/>
<path clip-path="url(#clip392)" d="M880.225 967.715 L880.225 967.715 L887.183 967.715 L887.183 967.715 L880.225 967.715 L880.225 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="880.225,967.715 880.225,967.715 887.183,967.715 880.225,967.715 "/>
<path clip-path="url(#clip392)" d="M887.183 967.715 L887.183 967.715 L894.141 967.715 L894.141 967.715 L887.183 967.715 L887.183 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="887.183,967.715 887.183,967.715 894.141,967.715 887.183,967.715 "/>
<path clip-path="url(#clip392)" d="M894.141 967.715 L894.141 967.715 L901.099 967.715 L901.099 967.715 L894.141 967.715 L894.141 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="894.141,967.715 894.141,967.715 901.099,967.715 894.141,967.715 "/>
<path clip-path="url(#clip392)" d="M901.099 967.715 L901.099 967.715 L908.056 967.715 L908.056 967.715 L901.099 967.715 L901.099 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="901.099,967.715 901.099,967.715 908.056,967.715 901.099,967.715 "/>
<path clip-path="url(#clip392)" d="M908.056 967.715 L908.056 967.715 L915.014 967.715 L915.014 967.715 L908.056 967.715 L908.056 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="908.056,967.715 908.056,967.715 915.014,967.715 908.056,967.715 "/>
<path clip-path="url(#clip392)" d="M915.014 967.715 L915.014 967.715 L921.972 967.715 L921.972 967.715 L915.014 967.715 L915.014 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="915.014,967.715 915.014,967.715 921.972,967.715 915.014,967.715 "/>
<path clip-path="url(#clip392)" d="M921.972 967.715 L921.972 967.715 L928.93 967.715 L928.93 967.715 L921.972 967.715 L921.972 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="921.972,967.715 921.972,967.715 928.93,967.715 921.972,967.715 "/>
<path clip-path="url(#clip392)" d="M928.93 967.715 L928.93 967.715 L935.888 967.715 L935.888 967.715 L928.93 967.715 L928.93 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="928.93,967.715 928.93,967.715 935.888,967.715 928.93,967.715 "/>
<path clip-path="url(#clip392)" d="M935.888 967.715 L935.888 967.715 L942.846 967.715 L942.846 967.715 L935.888 967.715 L935.888 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="935.888,967.715 935.888,967.715 942.846,967.715 935.888,967.715 "/>
<path clip-path="url(#clip392)" d="M942.846 967.715 L942.846 967.715 L949.804 967.715 L949.804 967.715 L942.846 967.715 L942.846 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="942.846,967.715 942.846,967.715 949.804,967.715 942.846,967.715 "/>
<path clip-path="url(#clip392)" d="M949.804 967.715 L949.804 967.715 L956.761 967.715 L956.761 967.715 L949.804 967.715 L949.804 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="949.804,967.715 949.804,967.715 956.761,967.715 949.804,967.715 "/>
<path clip-path="url(#clip392)" d="M956.761 967.715 L956.761 967.715 L963.719 967.715 L963.719 967.715 L956.761 967.715 L956.761 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="956.761,967.715 956.761,967.715 963.719,967.715 956.761,967.715 "/>
<path clip-path="url(#clip392)" d="M963.719 967.715 L963.719 967.715 L970.677 967.715 L970.677 967.715 L963.719 967.715 L963.719 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="963.719,967.715 963.719,967.715 970.677,967.715 963.719,967.715 "/>
<path clip-path="url(#clip392)" d="M970.677 967.715 L970.677 967.715 L977.635 967.715 L977.635 967.715 L970.677 967.715 L970.677 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="970.677,967.715 970.677,967.715 977.635,967.715 970.677,967.715 "/>
<path clip-path="url(#clip392)" d="M977.635 965.544 L977.635 967.715 L984.593 967.715 L984.593 965.544 L977.635 965.544 L977.635 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="977.635,965.544 977.635,967.715 984.593,967.715 984.593,965.544 977.635,965.544 "/>
<path clip-path="url(#clip392)" d="M984.593 967.715 L984.593 967.715 L991.551 967.715 L991.551 967.715 L984.593 967.715 L984.593 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="984.593,967.715 984.593,967.715 991.551,967.715 984.593,967.715 "/>
<path clip-path="url(#clip392)" d="M991.551 967.715 L991.551 967.715 L998.509 967.715 L998.509 967.715 L991.551 967.715 L991.551 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="991.551,967.715 991.551,967.715 998.509,967.715 991.551,967.715 "/>
<path clip-path="url(#clip392)" d="M998.509 967.715 L998.509 967.715 L1005.47 967.715 L1005.47 967.715 L998.509 967.715 L998.509 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="998.509,967.715 998.509,967.715 1005.47,967.715 998.509,967.715 "/>
<path clip-path="url(#clip392)" d="M1005.47 967.715 L1005.47 967.715 L1012.42 967.715 L1012.42 967.715 L1005.47 967.715 L1005.47 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1005.47,967.715 1005.47,967.715 1012.42,967.715 1005.47,967.715 "/>
<path clip-path="url(#clip392)" d="M1012.42 967.715 L1012.42 967.715 L1019.38 967.715 L1019.38 967.715 L1012.42 967.715 L1012.42 967.715  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1012.42,967.715 1012.42,967.715 1019.38,967.715 1012.42,967.715 "/>
<path clip-path="url(#clip392)" d="M1019.38 963.374 L1019.38 967.715 L1026.34 967.715 L1026.34 963.374 L1019.38 963.374 L1019.38 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1019.38,963.374 1019.38,967.715 1026.34,967.715 1026.34,963.374 1019.38,963.374 "/>
<path clip-path="url(#clip392)" d="M1026.34 965.544 L1026.34 967.715 L1033.3 967.715 L1033.3 965.544 L1026.34 965.544 L1026.34 965.544  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1026.34,965.544 1026.34,967.715 1033.3,967.715 1033.3,965.544 1026.34,965.544 "/>
<path clip-path="url(#clip392)" d="M1033.3 963.374 L1033.3 967.715 L1040.26 967.715 L1040.26 963.374 L1033.3 963.374 L1033.3 963.374  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1033.3,963.374 1033.3,967.715 1040.26,967.715 1040.26,963.374 1033.3,963.374 "/>
<path clip-path="url(#clip392)" d="M1040.26 950.35 L1040.26 967.715 L1047.21 967.715 L1047.21 950.35 L1040.26 950.35 L1040.26 950.35  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1040.26,950.35 1040.26,967.715 1047.21,967.715 1047.21,950.35 1040.26,950.35 "/>
<path clip-path="url(#clip392)" d="M1047.21 956.862 L1047.21 967.715 L1054.17 967.715 L1054.17 956.862 L1047.21 956.862 L1047.21 956.862  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1047.21,956.862 1047.21,967.715 1054.17,967.715 1054.17,956.862 1047.21,956.862 "/>
<path clip-path="url(#clip392)" d="M1054.17 937.326 L1054.17 967.715 L1061.13 967.715 L1061.13 937.326 L1054.17 937.326 L1054.17 937.326  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1054.17,937.326 1054.17,967.715 1061.13,967.715 1061.13,937.326 1054.17,937.326 "/>
<path clip-path="url(#clip392)" d="M1061.13 954.691 L1061.13 967.715 L1068.09 967.715 L1068.09 954.691 L1061.13 954.691 L1061.13 954.691  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1061.13,954.691 1061.13,967.715 1068.09,967.715 1068.09,954.691 1061.13,954.691 "/>
<path clip-path="url(#clip392)" d="M1068.09 928.643 L1068.09 967.715 L1075.04 967.715 L1075.04 928.643 L1068.09 928.643 L1068.09 928.643  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1068.09,928.643 1068.09,967.715 1075.04,967.715 1075.04,928.643 1068.09,928.643 "/>
<path clip-path="url(#clip392)" d="M1075.04 915.619 L1075.04 967.715 L1082 967.715 L1082 915.619 L1075.04 915.619 L1075.04 915.619  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1075.04,915.619 1075.04,967.715 1082,967.715 1082,915.619 1075.04,915.619 "/>
<path clip-path="url(#clip392)" d="M1082 930.814 L1082 967.715 L1088.96 967.715 L1088.96 930.814 L1082 930.814 L1082 930.814  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1082,930.814 1082,967.715 1088.96,967.715 1088.96,930.814 1082,930.814 "/>
<path clip-path="url(#clip392)" d="M1088.96 950.35 L1088.96 967.715 L1095.92 967.715 L1095.92 950.35 L1088.96 950.35 L1088.96 950.35  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1088.96,950.35 1088.96,967.715 1095.92,967.715 1095.92,950.35 1088.96,950.35 "/>
<path clip-path="url(#clip392)" d="M1095.92 952.52 L1095.92 967.715 L1102.88 967.715 L1102.88 952.52 L1095.92 952.52 L1095.92 952.52  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1095.92,952.52 1095.92,967.715 1102.88,967.715 1102.88,952.52 1095.92,952.52 "/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="299.244" cy="741.965" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="306.202" cy="153.712" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="313.16" cy="600.871" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="320.117" cy="781.037" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="327.075" cy="891.741" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="334.033" cy="917.79" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="340.991" cy="930.814" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="347.949" cy="943.838" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="354.907" cy="948.179" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="361.865" cy="952.52" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="368.822" cy="959.032" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="375.78" cy="956.862" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="382.738" cy="959.032" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="389.696" cy="961.203" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="396.654" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="403.612" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="410.57" cy="954.691" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="417.527" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="424.485" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="431.443" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="438.401" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="445.359" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="452.317" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="459.275" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="466.232" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="473.19" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="480.148" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="487.106" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="494.064" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="501.022" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="507.98" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="514.937" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="521.895" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="528.853" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="535.811" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="542.769" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="549.727" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="556.685" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="563.642" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="570.6" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="577.558" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="584.516" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="591.474" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="598.432" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="605.39" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="612.347" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="619.305" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="626.263" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="633.221" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="640.179" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="647.137" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="654.095" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="661.052" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="668.01" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="674.968" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="681.926" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="688.884" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="695.842" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="702.8" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="709.757" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="716.715" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="723.673" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="730.631" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="737.589" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="744.547" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="751.505" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="758.462" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="765.42" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="772.378" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="779.336" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="786.294" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="793.252" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="800.21" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="807.167" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="814.125" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="821.083" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="828.041" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="834.999" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="841.957" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="848.915" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="855.872" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="862.83" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="869.788" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="876.746" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="883.704" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="890.662" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="897.62" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="904.577" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="911.535" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="918.493" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="925.451" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="932.409" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="939.367" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="946.325" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="953.282" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="960.24" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="967.198" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="974.156" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="981.114" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="988.072" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="995.03" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1001.99" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1008.95" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1015.9" cy="967.715" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1022.86" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1029.82" cy="965.544" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1036.78" cy="963.374" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1043.73" cy="950.35" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1050.69" cy="956.862" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1057.65" cy="937.326" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1064.61" cy="954.691" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1071.57" cy="928.643" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1078.52" cy="915.619" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1085.48" cy="930.814" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1092.44" cy="950.35" r="2"/>
<circle clip-path="url(#clip392)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1099.4" cy="952.52" r="2"/>
<polyline clip-path="url(#clip392)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="310.042,1781.72 310.042,-660.291 "/>
</svg>
<p>(b)</p>
</div>
</div></td>
</tr>
</tbody>
</table>

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<tbody>
<tr>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="300" height="300" viewBox="0 0 1200 1200">
<defs>
  <clipPath id="clip450">
    <rect x="0" y="0" width="1200" height="1200"/>
  </clipPath>
</defs>
<path clip-path="url(#clip450)" d="M0 1200 L1200 1200 L1200 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip451">
    <rect x="240" y="119" width="841" height="841"/>
  </clipPath>
</defs>
<path clip-path="url(#clip450)" d="M245.885 986.14 L1152.76 986.14 L1152.76 153.712 L245.885 153.712  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip452">
    <rect x="245" y="153" width="908" height="833"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="291.709,986.14 291.709,153.712 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="494.501,986.14 494.501,153.712 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="697.293,986.14 697.293,153.712 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="900.085,986.14 900.085,153.712 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1102.88,986.14 1102.88,153.712 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,986.14 1152.76,986.14 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,796.952 1152.76,796.952 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,607.764 1152.76,607.764 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,418.575 1152.76,418.575 "/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="245.885,229.387 1152.76,229.387 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,986.14 1152.76,986.14 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="291.709,986.14 291.709,967.242 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="494.501,986.14 494.501,967.242 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="697.293,986.14 697.293,967.242 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="900.085,986.14 900.085,967.242 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1102.88,986.14 1102.88,967.242 "/>
<path clip-path="url(#clip450)" d="M291.709 1007.46 Q288.098 1007.46 286.269 1011.02 Q284.464 1014.56 284.464 1021.69 Q284.464 1028.8 286.269 1032.37 Q288.098 1035.91 291.709 1035.91 Q295.343 1035.91 297.149 1032.37 Q298.978 1028.8 298.978 1021.69 Q298.978 1014.56 297.149 1011.02 Q295.343 1007.46 291.709 1007.46 M291.709 1003.75 Q297.519 1003.75 300.575 1008.36 Q303.653 1012.94 303.653 1021.69 Q303.653 1030.42 300.575 1035.03 Q297.519 1039.61 291.709 1039.61 Q285.899 1039.61 282.82 1035.03 Q279.765 1030.42 279.765 1021.69 Q279.765 1012.94 282.82 1008.36 Q285.899 1003.75 291.709 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M443.112 1035 L459.432 1035 L459.432 1038.94 L437.487 1038.94 L437.487 1035 Q440.149 1032.25 444.733 1027.62 Q449.339 1022.97 450.52 1021.63 Q452.765 1019.1 453.645 1017.37 Q454.547 1015.61 454.547 1013.92 Q454.547 1011.16 452.603 1009.43 Q450.682 1007.69 447.58 1007.69 Q445.381 1007.69 442.927 1008.45 Q440.497 1009.22 437.719 1010.77 L437.719 1006.05 Q440.543 1004.91 442.997 1004.33 Q445.45 1003.75 447.487 1003.75 Q452.858 1003.75 456.052 1006.44 Q459.246 1009.13 459.246 1013.62 Q459.246 1015.75 458.436 1017.67 Q457.649 1019.56 455.543 1022.16 Q454.964 1022.83 451.862 1026.05 Q448.76 1029.24 443.112 1035 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M469.293 1004.38 L487.649 1004.38 L487.649 1008.32 L473.575 1008.32 L473.575 1016.79 Q474.594 1016.44 475.612 1016.28 Q476.631 1016.09 477.649 1016.09 Q483.436 1016.09 486.816 1019.26 Q490.195 1022.44 490.195 1027.85 Q490.195 1033.43 486.723 1036.53 Q483.251 1039.61 476.932 1039.61 Q474.756 1039.61 472.487 1039.24 Q470.242 1038.87 467.834 1038.13 L467.834 1033.43 Q469.918 1034.56 472.14 1035.12 Q474.362 1035.68 476.839 1035.68 Q480.844 1035.68 483.182 1033.57 Q485.519 1031.46 485.519 1027.85 Q485.519 1024.24 483.182 1022.13 Q480.844 1020.03 476.839 1020.03 Q474.964 1020.03 473.089 1020.44 Q471.237 1020.86 469.293 1021.74 L469.293 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M509.408 1007.46 Q505.797 1007.46 503.968 1011.02 Q502.163 1014.56 502.163 1021.69 Q502.163 1028.8 503.968 1032.37 Q505.797 1035.91 509.408 1035.91 Q513.042 1035.91 514.848 1032.37 Q516.677 1028.8 516.677 1021.69 Q516.677 1014.56 514.848 1011.02 Q513.042 1007.46 509.408 1007.46 M509.408 1003.75 Q515.218 1003.75 518.274 1008.36 Q521.353 1012.94 521.353 1021.69 Q521.353 1030.42 518.274 1035.03 Q515.218 1039.61 509.408 1039.61 Q503.598 1039.61 500.519 1035.03 Q497.464 1030.42 497.464 1021.69 Q497.464 1012.94 500.519 1008.36 Q503.598 1003.75 509.408 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M539.57 1007.46 Q535.959 1007.46 534.13 1011.02 Q532.325 1014.56 532.325 1021.69 Q532.325 1028.8 534.13 1032.37 Q535.959 1035.91 539.57 1035.91 Q543.204 1035.91 545.01 1032.37 Q546.839 1028.8 546.839 1021.69 Q546.839 1014.56 545.01 1011.02 Q543.204 1007.46 539.57 1007.46 M539.57 1003.75 Q545.38 1003.75 548.436 1008.36 Q551.515 1012.94 551.515 1021.69 Q551.515 1030.42 548.436 1035.03 Q545.38 1039.61 539.57 1039.61 Q533.76 1039.61 530.681 1035.03 Q527.626 1030.42 527.626 1021.69 Q527.626 1012.94 530.681 1008.36 Q533.76 1003.75 539.57 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M641.83 1004.38 L660.186 1004.38 L660.186 1008.32 L646.112 1008.32 L646.112 1016.79 Q647.131 1016.44 648.15 1016.28 Q649.168 1016.09 650.187 1016.09 Q655.974 1016.09 659.353 1019.26 Q662.733 1022.44 662.733 1027.85 Q662.733 1033.43 659.261 1036.53 Q655.788 1039.61 649.469 1039.61 Q647.293 1039.61 645.025 1039.24 Q642.779 1038.87 640.372 1038.13 L640.372 1033.43 Q642.455 1034.56 644.677 1035.12 Q646.9 1035.68 649.376 1035.68 Q653.381 1035.68 655.719 1033.57 Q658.057 1031.46 658.057 1027.85 Q658.057 1024.24 655.719 1022.13 Q653.381 1020.03 649.376 1020.03 Q647.501 1020.03 645.626 1020.44 Q643.775 1020.86 641.83 1021.74 L641.83 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M681.946 1007.46 Q678.335 1007.46 676.506 1011.02 Q674.7 1014.56 674.7 1021.69 Q674.7 1028.8 676.506 1032.37 Q678.335 1035.91 681.946 1035.91 Q685.58 1035.91 687.385 1032.37 Q689.214 1028.8 689.214 1021.69 Q689.214 1014.56 687.385 1011.02 Q685.58 1007.46 681.946 1007.46 M681.946 1003.75 Q687.756 1003.75 690.811 1008.36 Q693.89 1012.94 693.89 1021.69 Q693.89 1030.42 690.811 1035.03 Q687.756 1039.61 681.946 1039.61 Q676.135 1039.61 673.057 1035.03 Q670.001 1030.42 670.001 1021.69 Q670.001 1012.94 673.057 1008.36 Q676.135 1003.75 681.946 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M712.107 1007.46 Q708.496 1007.46 706.668 1011.02 Q704.862 1014.56 704.862 1021.69 Q704.862 1028.8 706.668 1032.37 Q708.496 1035.91 712.107 1035.91 Q715.742 1035.91 717.547 1032.37 Q719.376 1028.8 719.376 1021.69 Q719.376 1014.56 717.547 1011.02 Q715.742 1007.46 712.107 1007.46 M712.107 1003.75 Q717.918 1003.75 720.973 1008.36 Q724.052 1012.94 724.052 1021.69 Q724.052 1030.42 720.973 1035.03 Q717.918 1039.61 712.107 1039.61 Q706.297 1039.61 703.219 1035.03 Q700.163 1030.42 700.163 1021.69 Q700.163 1012.94 703.219 1008.36 Q706.297 1003.75 712.107 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M742.269 1007.46 Q738.658 1007.46 736.83 1011.02 Q735.024 1014.56 735.024 1021.69 Q735.024 1028.8 736.83 1032.37 Q738.658 1035.91 742.269 1035.91 Q745.904 1035.91 747.709 1032.37 Q749.538 1028.8 749.538 1021.69 Q749.538 1014.56 747.709 1011.02 Q745.904 1007.46 742.269 1007.46 M742.269 1003.75 Q748.08 1003.75 751.135 1008.36 Q754.214 1012.94 754.214 1021.69 Q754.214 1030.42 751.135 1035.03 Q748.08 1039.61 742.269 1039.61 Q736.459 1039.61 733.381 1035.03 Q730.325 1030.42 730.325 1021.69 Q730.325 1012.94 733.381 1008.36 Q736.459 1003.75 742.269 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M843.279 1004.38 L865.501 1004.38 L865.501 1006.37 L852.955 1038.94 L848.071 1038.94 L859.876 1008.32 L843.279 1008.32 L843.279 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M874.668 1004.38 L893.024 1004.38 L893.024 1008.32 L878.95 1008.32 L878.95 1016.79 Q879.969 1016.44 880.987 1016.28 Q882.006 1016.09 883.024 1016.09 Q888.811 1016.09 892.191 1019.26 Q895.571 1022.44 895.571 1027.85 Q895.571 1033.43 892.099 1036.53 Q888.626 1039.61 882.307 1039.61 Q880.131 1039.61 877.862 1039.24 Q875.617 1038.87 873.21 1038.13 L873.21 1033.43 Q875.293 1034.56 877.515 1035.12 Q879.737 1035.68 882.214 1035.68 Q886.219 1035.68 888.557 1033.57 Q890.895 1031.46 890.895 1027.85 Q890.895 1024.24 888.557 1022.13 Q886.219 1020.03 882.214 1020.03 Q880.339 1020.03 878.464 1020.44 Q876.612 1020.86 874.668 1021.74 L874.668 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M914.784 1007.46 Q911.172 1007.46 909.344 1011.02 Q907.538 1014.56 907.538 1021.69 Q907.538 1028.8 909.344 1032.37 Q911.172 1035.91 914.784 1035.91 Q918.418 1035.91 920.223 1032.37 Q922.052 1028.8 922.052 1021.69 Q922.052 1014.56 920.223 1011.02 Q918.418 1007.46 914.784 1007.46 M914.784 1003.75 Q920.594 1003.75 923.649 1008.36 Q926.728 1012.94 926.728 1021.69 Q926.728 1030.42 923.649 1035.03 Q920.594 1039.61 914.784 1039.61 Q908.973 1039.61 905.895 1035.03 Q902.839 1030.42 902.839 1021.69 Q902.839 1012.94 905.895 1008.36 Q908.973 1003.75 914.784 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M944.945 1007.46 Q941.334 1007.46 939.506 1011.02 Q937.7 1014.56 937.7 1021.69 Q937.7 1028.8 939.506 1032.37 Q941.334 1035.91 944.945 1035.91 Q948.58 1035.91 950.385 1032.37 Q952.214 1028.8 952.214 1021.69 Q952.214 1014.56 950.385 1011.02 Q948.58 1007.46 944.945 1007.46 M944.945 1003.75 Q950.756 1003.75 953.811 1008.36 Q956.89 1012.94 956.89 1021.69 Q956.89 1030.42 953.811 1035.03 Q950.756 1039.61 944.945 1039.61 Q939.135 1039.61 936.057 1035.03 Q933.001 1030.42 933.001 1021.69 Q933.001 1012.94 936.057 1008.36 Q939.135 1003.75 944.945 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M1032.32 1035 L1039.96 1035 L1039.96 1008.64 L1031.65 1010.31 L1031.65 1006.05 L1039.91 1004.38 L1044.59 1004.38 L1044.59 1035 L1052.23 1035 L1052.23 1038.94 L1032.32 1038.94 L1032.32 1035 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M1071.67 1007.46 Q1068.06 1007.46 1066.23 1011.02 Q1064.43 1014.56 1064.43 1021.69 Q1064.43 1028.8 1066.23 1032.37 Q1068.06 1035.91 1071.67 1035.91 Q1075.31 1035.91 1077.11 1032.37 Q1078.94 1028.8 1078.94 1021.69 Q1078.94 1014.56 1077.11 1011.02 Q1075.31 1007.46 1071.67 1007.46 M1071.67 1003.75 Q1077.48 1003.75 1080.54 1008.36 Q1083.62 1012.94 1083.62 1021.69 Q1083.62 1030.42 1080.54 1035.03 Q1077.48 1039.61 1071.67 1039.61 Q1065.86 1039.61 1062.78 1035.03 Q1059.73 1030.42 1059.73 1021.69 Q1059.73 1012.94 1062.78 1008.36 Q1065.86 1003.75 1071.67 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M1101.83 1007.46 Q1098.22 1007.46 1096.39 1011.02 Q1094.59 1014.56 1094.59 1021.69 Q1094.59 1028.8 1096.39 1032.37 Q1098.22 1035.91 1101.83 1035.91 Q1105.47 1035.91 1107.27 1032.37 Q1109.1 1028.8 1109.1 1021.69 Q1109.1 1014.56 1107.27 1011.02 Q1105.47 1007.46 1101.83 1007.46 M1101.83 1003.75 Q1107.64 1003.75 1110.7 1008.36 Q1113.78 1012.94 1113.78 1021.69 Q1113.78 1030.42 1110.7 1035.03 Q1107.64 1039.61 1101.83 1039.61 Q1096.02 1039.61 1092.95 1035.03 Q1089.89 1030.42 1089.89 1021.69 Q1089.89 1012.94 1092.95 1008.36 Q1096.02 1003.75 1101.83 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M1132 1007.46 Q1128.39 1007.46 1126.56 1011.02 Q1124.75 1014.56 1124.75 1021.69 Q1124.75 1028.8 1126.56 1032.37 Q1128.39 1035.91 1132 1035.91 Q1135.63 1035.91 1137.44 1032.37 Q1139.27 1028.8 1139.27 1021.69 Q1139.27 1014.56 1137.44 1011.02 Q1135.63 1007.46 1132 1007.46 M1132 1003.75 Q1137.81 1003.75 1140.86 1008.36 Q1143.94 1012.94 1143.94 1021.69 Q1143.94 1030.42 1140.86 1035.03 Q1137.81 1039.61 1132 1039.61 Q1126.19 1039.61 1123.11 1035.03 Q1120.05 1030.42 1120.05 1021.69 Q1120.05 1012.94 1123.11 1008.36 Q1126.19 1003.75 1132 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M1162.16 1007.46 Q1158.55 1007.46 1156.72 1011.02 Q1154.91 1014.56 1154.91 1021.69 Q1154.91 1028.8 1156.72 1032.37 Q1158.55 1035.91 1162.16 1035.91 Q1165.79 1035.91 1167.6 1032.37 Q1169.43 1028.8 1169.43 1021.69 Q1169.43 1014.56 1167.6 1011.02 Q1165.79 1007.46 1162.16 1007.46 M1162.16 1003.75 Q1167.97 1003.75 1171.02 1008.36 Q1174.1 1012.94 1174.1 1021.69 Q1174.1 1030.42 1171.02 1035.03 Q1167.97 1039.61 1162.16 1039.61 Q1156.35 1039.61 1153.27 1035.03 Q1150.21 1030.42 1150.21 1021.69 Q1150.21 1012.94 1153.27 1008.36 Q1156.35 1003.75 1162.16 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M717.01 1071.32 Q717.01 1073.57 713.757 1073.57 L702.679 1073.57 L698.266 1096.15 Q698.17 1096.73 698.041 1097.05 Q697.944 1097.37 697.655 1097.92 Q697.397 1098.43 696.849 1098.69 Q696.334 1098.98 695.561 1098.98 Q694.595 1098.98 694.112 1098.4 Q693.661 1097.85 693.661 1097.18 Q693.661 1096.76 694.305 1094.76 L700.779 1073.57 L695.304 1073.57 Q690.151 1073.57 686.93 1078.66 Q686.415 1079.34 686.125 1079.4 Q685.996 1079.43 685.867 1079.43 Q685.513 1079.43 685.287 1079.27 Q685.094 1079.11 685.094 1078.95 L685.062 1078.79 Q685.062 1078.56 685.577 1077.66 Q686.125 1076.73 687.091 1075.34 Q688.089 1073.96 689.088 1072.86 Q690.408 1071.44 691.89 1070.7 Q693.371 1069.93 694.112 1069.83 Q694.885 1069.74 695.883 1069.74 L714.369 1069.74 Q715.207 1069.74 715.658 1069.8 Q716.108 1069.83 716.559 1070.22 Q717.01 1070.58 717.01 1071.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,986.14 245.885,153.712 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,986.14 264.783,986.14 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,796.952 264.783,796.952 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,607.764 264.783,607.764 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,418.575 264.783,418.575 "/>
<polyline clip-path="url(#clip450)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="245.885,229.387 264.783,229.387 "/>
<path clip-path="url(#clip450)" d="M215.941 971.939 Q212.33 971.939 210.501 975.503 Q208.696 979.045 208.696 986.175 Q208.696 993.281 210.501 996.846 Q212.33 1000.39 215.941 1000.39 Q219.575 1000.39 221.381 996.846 Q223.21 993.281 223.21 986.175 Q223.21 979.045 221.381 975.503 Q219.575 971.939 215.941 971.939 M215.941 968.235 Q221.751 968.235 224.807 972.841 Q227.885 977.425 227.885 986.175 Q227.885 994.901 224.807 999.508 Q221.751 1004.09 215.941 1004.09 Q210.131 1004.09 207.052 999.508 Q203.997 994.901 203.997 986.175 Q203.997 977.425 207.052 972.841 Q210.131 968.235 215.941 968.235 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M175.826 779.672 L194.182 779.672 L194.182 783.607 L180.108 783.607 L180.108 792.079 Q181.126 791.732 182.145 791.57 Q183.163 791.385 184.182 791.385 Q189.969 791.385 193.349 794.556 Q196.728 797.727 196.728 803.144 Q196.728 808.722 193.256 811.824 Q189.784 814.903 183.464 814.903 Q181.288 814.903 179.02 814.533 Q176.775 814.162 174.367 813.422 L174.367 808.722 Q176.451 809.857 178.673 810.412 Q180.895 810.968 183.372 810.968 Q187.376 810.968 189.714 808.861 Q192.052 806.755 192.052 803.144 Q192.052 799.533 189.714 797.426 Q187.376 795.32 183.372 795.32 Q181.497 795.32 179.622 795.736 Q177.77 796.153 175.826 797.033 L175.826 779.672 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M215.941 782.75 Q212.33 782.75 210.501 786.315 Q208.696 789.857 208.696 796.986 Q208.696 804.093 210.501 807.658 Q212.33 811.199 215.941 811.199 Q219.575 811.199 221.381 807.658 Q223.21 804.093 223.21 796.986 Q223.21 789.857 221.381 786.315 Q219.575 782.75 215.941 782.75 M215.941 779.047 Q221.751 779.047 224.807 783.653 Q227.885 788.236 227.885 796.986 Q227.885 805.713 224.807 810.32 Q221.751 814.903 215.941 814.903 Q210.131 814.903 207.052 810.32 Q203.997 805.713 203.997 796.986 Q203.997 788.236 207.052 783.653 Q210.131 779.047 215.941 779.047 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M146.428 621.108 L154.066 621.108 L154.066 594.743 L145.756 596.41 L145.756 592.15 L154.02 590.484 L158.696 590.484 L158.696 621.108 L166.335 621.108 L166.335 625.044 L146.428 625.044 L146.428 621.108 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M185.779 593.562 Q182.168 593.562 180.339 597.127 Q178.534 600.669 178.534 607.798 Q178.534 614.905 180.339 618.47 Q182.168 622.011 185.779 622.011 Q189.413 622.011 191.219 618.47 Q193.048 614.905 193.048 607.798 Q193.048 600.669 191.219 597.127 Q189.413 593.562 185.779 593.562 M185.779 589.859 Q191.589 589.859 194.645 594.465 Q197.724 599.048 197.724 607.798 Q197.724 616.525 194.645 621.132 Q191.589 625.715 185.779 625.715 Q179.969 625.715 176.89 621.132 Q173.835 616.525 173.835 607.798 Q173.835 599.048 176.89 594.465 Q179.969 589.859 185.779 589.859 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M215.941 593.562 Q212.33 593.562 210.501 597.127 Q208.696 600.669 208.696 607.798 Q208.696 614.905 210.501 618.47 Q212.33 622.011 215.941 622.011 Q219.575 622.011 221.381 618.47 Q223.21 614.905 223.21 607.798 Q223.21 600.669 221.381 597.127 Q219.575 593.562 215.941 593.562 M215.941 589.859 Q221.751 589.859 224.807 594.465 Q227.885 599.048 227.885 607.798 Q227.885 616.525 224.807 621.132 Q221.751 625.715 215.941 625.715 Q210.131 625.715 207.052 621.132 Q203.997 616.525 203.997 607.798 Q203.997 599.048 207.052 594.465 Q210.131 589.859 215.941 589.859 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M146.428 431.92 L154.066 431.92 L154.066 405.555 L145.756 407.221 L145.756 402.962 L154.02 401.295 L158.696 401.295 L158.696 431.92 L166.335 431.92 L166.335 435.855 L146.428 435.855 L146.428 431.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M175.826 401.295 L194.182 401.295 L194.182 405.231 L180.108 405.231 L180.108 413.703 Q181.126 413.356 182.145 413.194 Q183.163 413.008 184.182 413.008 Q189.969 413.008 193.349 416.18 Q196.728 419.351 196.728 424.768 Q196.728 430.346 193.256 433.448 Q189.784 436.527 183.464 436.527 Q181.288 436.527 179.02 436.156 Q176.775 435.786 174.367 435.045 L174.367 430.346 Q176.451 431.481 178.673 432.036 Q180.895 432.592 183.372 432.592 Q187.376 432.592 189.714 430.485 Q192.052 428.379 192.052 424.768 Q192.052 421.156 189.714 419.05 Q187.376 416.944 183.372 416.944 Q181.497 416.944 179.622 417.36 Q177.77 417.777 175.826 418.657 L175.826 401.295 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M215.941 404.374 Q212.33 404.374 210.501 407.939 Q208.696 411.481 208.696 418.61 Q208.696 425.717 210.501 429.281 Q212.33 432.823 215.941 432.823 Q219.575 432.823 221.381 429.281 Q223.21 425.717 223.21 418.61 Q223.21 411.481 221.381 407.939 Q219.575 404.374 215.941 404.374 M215.941 400.67 Q221.751 400.67 224.807 405.277 Q227.885 409.86 227.885 418.61 Q227.885 427.337 224.807 431.943 Q221.751 436.527 215.941 436.527 Q210.131 436.527 207.052 431.943 Q203.997 427.337 203.997 418.61 Q203.997 409.86 207.052 405.277 Q210.131 400.67 215.941 400.67 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M149.645 242.732 L165.964 242.732 L165.964 246.667 L144.02 246.667 L144.02 242.732 Q146.682 239.978 151.265 235.348 Q155.872 230.695 157.052 229.353 Q159.298 226.83 160.177 225.093 Q161.08 223.334 161.08 221.644 Q161.08 218.89 159.136 217.154 Q157.215 215.418 154.113 215.418 Q151.914 215.418 149.46 216.181 Q147.029 216.945 144.252 218.496 L144.252 213.774 Q147.076 212.64 149.529 212.061 Q151.983 211.482 154.02 211.482 Q159.39 211.482 162.585 214.168 Q165.779 216.853 165.779 221.343 Q165.779 223.473 164.969 225.394 Q164.182 227.292 162.076 229.885 Q161.497 230.556 158.395 233.774 Q155.293 236.968 149.645 242.732 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M185.779 215.186 Q182.168 215.186 180.339 218.751 Q178.534 222.293 178.534 229.422 Q178.534 236.529 180.339 240.093 Q182.168 243.635 185.779 243.635 Q189.413 243.635 191.219 240.093 Q193.048 236.529 193.048 229.422 Q193.048 222.293 191.219 218.751 Q189.413 215.186 185.779 215.186 M185.779 211.482 Q191.589 211.482 194.645 216.089 Q197.724 220.672 197.724 229.422 Q197.724 238.149 194.645 242.755 Q191.589 247.339 185.779 247.339 Q179.969 247.339 176.89 242.755 Q173.835 238.149 173.835 229.422 Q173.835 220.672 176.89 216.089 Q179.969 211.482 185.779 211.482 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M215.941 215.186 Q212.33 215.186 210.501 218.751 Q208.696 222.293 208.696 229.422 Q208.696 236.529 210.501 240.093 Q212.33 243.635 215.941 243.635 Q219.575 243.635 221.381 240.093 Q223.21 236.529 223.21 229.422 Q223.21 222.293 221.381 218.751 Q219.575 215.186 215.941 215.186 M215.941 211.482 Q221.751 211.482 224.807 216.089 Q227.885 220.672 227.885 229.422 Q227.885 238.149 224.807 242.755 Q221.751 247.339 215.941 247.339 Q210.131 247.339 207.052 242.755 Q203.997 238.149 203.997 229.422 Q203.997 220.672 207.052 216.089 Q210.131 211.482 215.941 211.482 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M67.9044 625.785 L74.6839 625.785 Q71.6602 629.032 70.1643 632.724 Q68.6683 636.384 68.6683 640.522 Q68.6683 648.67 73.6654 652.998 Q78.6307 657.327 88.0519 657.327 Q97.4413 657.327 102.438 652.998 Q107.404 648.67 107.404 640.522 Q107.404 636.384 105.908 632.724 Q104.412 629.032 101.388 625.785 L108.104 625.785 Q110.396 629.159 111.541 632.946 Q112.687 636.702 112.687 640.904 Q112.687 651.693 106.099 657.9 Q99.4783 664.107 88.0519 664.107 Q76.5936 664.107 70.0051 657.9 Q63.3848 651.693 63.3848 640.904 Q63.3848 636.639 64.5306 632.883 Q65.6446 629.095 67.9044 625.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M80.2221 602.296 Q80.2221 607.006 83.9142 609.743 Q87.5745 612.481 93.972 612.481 Q100.37 612.481 104.062 609.775 Q107.722 607.038 107.722 602.296 Q107.722 597.617 104.03 594.88 Q100.338 592.142 93.972 592.142 Q87.6381 592.142 83.946 594.88 Q80.2221 597.617 80.2221 602.296 M75.2568 602.296 Q75.2568 594.657 80.2221 590.296 Q85.1873 585.936 93.972 585.936 Q102.725 585.936 107.722 590.296 Q112.687 594.657 112.687 602.296 Q112.687 609.966 107.722 614.327 Q102.725 618.655 93.972 618.655 Q85.1873 618.655 80.2221 614.327 Q75.2568 609.966 75.2568 602.296 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M97.6959 576.833 L76.1162 576.833 L76.1162 570.976 L97.4731 570.976 Q102.534 570.976 105.08 569.003 Q107.595 567.03 107.595 563.083 Q107.595 558.34 104.571 555.603 Q101.547 552.834 96.3273 552.834 L76.1162 552.834 L76.1162 546.978 L111.764 546.978 L111.764 552.834 L106.29 552.834 Q109.536 554.967 111.128 557.799 Q112.687 560.6 112.687 564.324 Q112.687 570.467 108.868 573.65 Q105.048 576.833 97.6959 576.833 M75.2568 562.096 L75.2568 562.096 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M90.2481 505.282 L111.764 505.282 L111.764 511.139 L90.439 511.139 Q85.3783 511.139 82.8638 513.112 Q80.3494 515.085 80.3494 519.032 Q80.3494 523.775 83.3731 526.512 Q86.3968 529.249 91.6167 529.249 L111.764 529.249 L111.764 535.137 L76.1162 535.137 L76.1162 529.249 L81.6544 529.249 Q78.4397 527.148 76.8483 524.316 Q75.2568 521.451 75.2568 517.727 Q75.2568 511.584 79.0763 508.433 Q82.8638 505.282 90.2481 505.282 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M65.9947 487.808 L76.1162 487.808 L76.1162 475.745 L80.6677 475.745 L80.6677 487.808 L100.019 487.808 Q104.38 487.808 105.621 486.631 Q106.863 485.421 106.863 481.761 L106.863 475.745 L111.764 475.745 L111.764 481.761 Q111.764 488.54 109.25 491.119 Q106.703 493.697 100.019 493.697 L80.6677 493.697 L80.6677 497.993 L76.1162 497.993 L76.1162 493.697 L65.9947 493.697 L65.9947 487.808 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip450)" d="M721.834 37.6649 Q721.834 40.5342 717.694 40.5342 L703.594 40.5342 L697.978 69.2676 Q697.855 70.0055 697.691 70.4153 Q697.568 70.8252 697.199 71.5221 Q696.871 72.1779 696.175 72.5058 Q695.519 72.8747 694.535 72.8747 Q693.305 72.8747 692.691 72.1369 Q692.117 71.4401 692.117 70.5793 Q692.117 70.0464 692.936 67.5051 L701.175 40.5342 L694.207 40.5342 Q687.649 40.5342 683.55 47.0105 Q682.894 47.8712 682.525 47.9532 Q682.361 47.9942 682.197 47.9942 Q681.746 47.9942 681.459 47.7893 Q681.214 47.5843 681.214 47.3794 L681.173 47.1744 Q681.173 46.8875 681.828 45.7398 Q682.525 44.5511 683.755 42.7886 Q685.026 41.026 686.296 39.6324 Q687.977 37.8289 689.862 36.8861 Q691.748 35.9024 692.691 35.7794 Q693.674 35.6564 694.945 35.6564 L718.473 35.6564 Q719.539 35.6564 720.112 35.7384 Q720.686 35.7794 721.26 36.2713 Q721.834 36.7222 721.834 37.6649 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip452)" d="M295.765 316.414 L295.765 986.14 L299.821 986.14 L299.821 316.414 L295.765 316.414 L295.765 316.414  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="295.765,316.414 295.765,986.14 299.821,986.14 299.821,316.414 295.765,316.414 "/>
<path clip-path="url(#clip452)" d="M299.821 153.712 L299.821 986.14 L303.877 986.14 L303.877 153.712 L299.821 153.712 L299.821 153.712  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="299.821,153.712 299.821,986.14 303.877,986.14 303.877,153.712 299.821,153.712 "/>
<path clip-path="url(#clip452)" d="M303.877 426.143 L303.877 986.14 L307.932 986.14 L307.932 426.143 L303.877 426.143 L303.877 426.143  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="303.877,426.143 303.877,986.14 307.932,986.14 307.932,426.143 303.877,426.143 "/>
<path clip-path="url(#clip452)" d="M307.932 649.385 L307.932 986.14 L311.988 986.14 L311.988 649.385 L307.932 649.385 L307.932 649.385  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="307.932,649.385 307.932,986.14 311.988,986.14 311.988,649.385 307.932,649.385 "/>
<path clip-path="url(#clip452)" d="M311.988 740.195 L311.988 986.14 L316.044 986.14 L316.044 740.195 L311.988 740.195 L311.988 740.195  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="311.988,740.195 311.988,986.14 316.044,986.14 316.044,740.195 311.988,740.195 "/>
<path clip-path="url(#clip452)" d="M316.044 868.843 L316.044 986.14 L320.1 986.14 L320.1 868.843 L316.044 868.843 L316.044 868.843  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="316.044,868.843 316.044,986.14 320.1,986.14 320.1,868.843 316.044,868.843 "/>
<path clip-path="url(#clip452)" d="M320.1 899.113 L320.1 986.14 L324.156 986.14 L324.156 899.113 L320.1 899.113 L320.1 899.113  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="320.1,899.113 320.1,986.14 324.156,986.14 324.156,899.113 320.1,899.113 "/>
<path clip-path="url(#clip452)" d="M324.156 918.032 L324.156 986.14 L328.212 986.14 L328.212 918.032 L324.156 918.032 L324.156 918.032  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="324.156,918.032 324.156,986.14 328.212,986.14 328.212,918.032 324.156,918.032 "/>
<path clip-path="url(#clip452)" d="M328.212 940.735 L328.212 986.14 L332.267 986.14 L332.267 940.735 L328.212 940.735 L328.212 940.735  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="328.212,940.735 328.212,986.14 332.267,986.14 332.267,940.735 328.212,940.735 "/>
<path clip-path="url(#clip452)" d="M332.267 936.951 L332.267 986.14 L336.323 986.14 L336.323 936.951 L332.267 936.951 L332.267 936.951  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="332.267,936.951 332.267,986.14 336.323,986.14 336.323,936.951 332.267,936.951 "/>
<path clip-path="url(#clip452)" d="M336.323 948.302 L336.323 986.14 L340.379 986.14 L340.379 948.302 L336.323 948.302 L336.323 948.302  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="336.323,948.302 336.323,986.14 340.379,986.14 340.379,948.302 336.323,948.302 "/>
<path clip-path="url(#clip452)" d="M340.379 955.87 L340.379 986.14 L344.435 986.14 L344.435 955.87 L340.379 955.87 L340.379 955.87  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="340.379,955.87 340.379,986.14 344.435,986.14 344.435,955.87 340.379,955.87 "/>
<path clip-path="url(#clip452)" d="M344.435 959.654 L344.435 986.14 L348.491 986.14 L348.491 959.654 L344.435 959.654 L344.435 959.654  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="344.435,959.654 344.435,986.14 348.491,986.14 348.491,959.654 344.435,959.654 "/>
<path clip-path="url(#clip452)" d="M348.491 967.221 L348.491 986.14 L352.547 986.14 L352.547 967.221 L348.491 967.221 L348.491 967.221  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="348.491,967.221 348.491,986.14 352.547,986.14 352.547,967.221 348.491,967.221 "/>
<path clip-path="url(#clip452)" d="M352.547 967.221 L352.547 986.14 L356.602 986.14 L356.602 967.221 L352.547 967.221 L352.547 967.221  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="352.547,967.221 352.547,986.14 356.602,986.14 356.602,967.221 352.547,967.221 "/>
<path clip-path="url(#clip452)" d="M356.602 974.789 L356.602 986.14 L360.658 986.14 L360.658 974.789 L356.602 974.789 L356.602 974.789  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="356.602,974.789 356.602,986.14 360.658,986.14 360.658,974.789 356.602,974.789 "/>
<path clip-path="url(#clip452)" d="M360.658 974.789 L360.658 986.14 L364.714 986.14 L364.714 974.789 L360.658 974.789 L360.658 974.789  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="360.658,974.789 360.658,986.14 364.714,986.14 364.714,974.789 360.658,974.789 "/>
<path clip-path="url(#clip452)" d="M364.714 978.572 L364.714 986.14 L368.77 986.14 L368.77 978.572 L364.714 978.572 L364.714 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="364.714,978.572 364.714,986.14 368.77,986.14 368.77,978.572 364.714,978.572 "/>
<path clip-path="url(#clip452)" d="M368.77 974.789 L368.77 986.14 L372.826 986.14 L372.826 974.789 L368.77 974.789 L368.77 974.789  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="368.77,974.789 368.77,986.14 372.826,986.14 372.826,974.789 368.77,974.789 "/>
<path clip-path="url(#clip452)" d="M372.826 978.572 L372.826 986.14 L376.882 986.14 L376.882 978.572 L372.826 978.572 L372.826 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="372.826,978.572 372.826,986.14 376.882,986.14 376.882,978.572 372.826,978.572 "/>
<path clip-path="url(#clip452)" d="M376.882 978.572 L376.882 986.14 L380.938 986.14 L380.938 978.572 L376.882 978.572 L376.882 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="376.882,978.572 376.882,986.14 380.938,986.14 380.938,978.572 376.882,978.572 "/>
<path clip-path="url(#clip452)" d="M380.938 974.789 L380.938 986.14 L384.993 986.14 L384.993 974.789 L380.938 974.789 L380.938 974.789  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="380.938,974.789 380.938,986.14 384.993,986.14 384.993,974.789 380.938,974.789 "/>
<path clip-path="url(#clip452)" d="M384.993 971.005 L384.993 986.14 L389.049 986.14 L389.049 971.005 L384.993 971.005 L384.993 971.005  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="384.993,971.005 384.993,986.14 389.049,986.14 389.049,971.005 384.993,971.005 "/>
<path clip-path="url(#clip452)" d="M389.049 986.14 L389.049 986.14 L393.105 986.14 L393.105 986.14 L389.049 986.14 L389.049 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="389.049,986.14 389.049,986.14 393.105,986.14 389.049,986.14 "/>
<path clip-path="url(#clip452)" d="M393.105 978.572 L393.105 986.14 L397.161 986.14 L397.161 978.572 L393.105 978.572 L393.105 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="393.105,978.572 393.105,986.14 397.161,986.14 397.161,978.572 393.105,978.572 "/>
<path clip-path="url(#clip452)" d="M397.161 986.14 L397.161 986.14 L401.217 986.14 L401.217 986.14 L397.161 986.14 L397.161 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="397.161,986.14 397.161,986.14 401.217,986.14 397.161,986.14 "/>
<path clip-path="url(#clip452)" d="M401.217 982.356 L401.217 986.14 L405.273 986.14 L405.273 982.356 L401.217 982.356 L401.217 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="401.217,982.356 401.217,986.14 405.273,986.14 405.273,982.356 401.217,982.356 "/>
<path clip-path="url(#clip452)" d="M405.273 978.572 L405.273 986.14 L409.328 986.14 L409.328 978.572 L405.273 978.572 L405.273 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="405.273,978.572 405.273,986.14 409.328,986.14 409.328,978.572 405.273,978.572 "/>
<path clip-path="url(#clip452)" d="M409.328 974.789 L409.328 986.14 L413.384 986.14 L413.384 974.789 L409.328 974.789 L409.328 974.789  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="409.328,974.789 409.328,986.14 413.384,986.14 413.384,974.789 409.328,974.789 "/>
<path clip-path="url(#clip452)" d="M413.384 982.356 L413.384 986.14 L417.44 986.14 L417.44 982.356 L413.384 982.356 L413.384 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="413.384,982.356 413.384,986.14 417.44,986.14 417.44,982.356 413.384,982.356 "/>
<path clip-path="url(#clip452)" d="M417.44 982.356 L417.44 986.14 L421.496 986.14 L421.496 982.356 L417.44 982.356 L417.44 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="417.44,982.356 417.44,986.14 421.496,986.14 421.496,982.356 417.44,982.356 "/>
<path clip-path="url(#clip452)" d="M421.496 982.356 L421.496 986.14 L425.552 986.14 L425.552 982.356 L421.496 982.356 L421.496 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="421.496,982.356 421.496,986.14 425.552,986.14 425.552,982.356 421.496,982.356 "/>
<path clip-path="url(#clip452)" d="M425.552 986.14 L425.552 986.14 L429.608 986.14 L429.608 986.14 L425.552 986.14 L425.552 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="425.552,986.14 425.552,986.14 429.608,986.14 425.552,986.14 "/>
<path clip-path="url(#clip452)" d="M429.608 986.14 L429.608 986.14 L433.663 986.14 L433.663 986.14 L429.608 986.14 L429.608 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="429.608,986.14 429.608,986.14 433.663,986.14 429.608,986.14 "/>
<path clip-path="url(#clip452)" d="M433.663 986.14 L433.663 986.14 L437.719 986.14 L437.719 986.14 L433.663 986.14 L433.663 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="433.663,986.14 433.663,986.14 437.719,986.14 433.663,986.14 "/>
<path clip-path="url(#clip452)" d="M437.719 986.14 L437.719 986.14 L441.775 986.14 L441.775 986.14 L437.719 986.14 L437.719 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="437.719,986.14 437.719,986.14 441.775,986.14 437.719,986.14 "/>
<path clip-path="url(#clip452)" d="M441.775 982.356 L441.775 986.14 L445.831 986.14 L445.831 982.356 L441.775 982.356 L441.775 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="441.775,982.356 441.775,986.14 445.831,986.14 445.831,982.356 441.775,982.356 "/>
<path clip-path="url(#clip452)" d="M445.831 986.14 L445.831 986.14 L449.887 986.14 L449.887 986.14 L445.831 986.14 L445.831 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="445.831,986.14 445.831,986.14 449.887,986.14 445.831,986.14 "/>
<path clip-path="url(#clip452)" d="M449.887 986.14 L449.887 986.14 L453.943 986.14 L453.943 986.14 L449.887 986.14 L449.887 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="449.887,986.14 449.887,986.14 453.943,986.14 449.887,986.14 "/>
<path clip-path="url(#clip452)" d="M453.943 978.572 L453.943 986.14 L457.998 986.14 L457.998 978.572 L453.943 978.572 L453.943 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="453.943,978.572 453.943,986.14 457.998,986.14 457.998,978.572 453.943,978.572 "/>
<path clip-path="url(#clip452)" d="M457.998 986.14 L457.998 986.14 L462.054 986.14 L462.054 986.14 L457.998 986.14 L457.998 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="457.998,986.14 457.998,986.14 462.054,986.14 457.998,986.14 "/>
<path clip-path="url(#clip452)" d="M462.054 978.572 L462.054 986.14 L466.11 986.14 L466.11 978.572 L462.054 978.572 L462.054 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="462.054,978.572 462.054,986.14 466.11,986.14 466.11,978.572 462.054,978.572 "/>
<path clip-path="url(#clip452)" d="M466.11 986.14 L466.11 986.14 L470.166 986.14 L470.166 986.14 L466.11 986.14 L466.11 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="466.11,986.14 466.11,986.14 470.166,986.14 466.11,986.14 "/>
<path clip-path="url(#clip452)" d="M470.166 986.14 L470.166 986.14 L474.222 986.14 L474.222 986.14 L470.166 986.14 L470.166 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="470.166,986.14 470.166,986.14 474.222,986.14 470.166,986.14 "/>
<path clip-path="url(#clip452)" d="M474.222 986.14 L474.222 986.14 L478.278 986.14 L478.278 986.14 L474.222 986.14 L474.222 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="474.222,986.14 474.222,986.14 478.278,986.14 474.222,986.14 "/>
<path clip-path="url(#clip452)" d="M478.278 982.356 L478.278 986.14 L482.333 986.14 L482.333 982.356 L478.278 982.356 L478.278 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="478.278,982.356 478.278,986.14 482.333,986.14 482.333,982.356 478.278,982.356 "/>
<path clip-path="url(#clip452)" d="M482.333 982.356 L482.333 986.14 L486.389 986.14 L486.389 982.356 L482.333 982.356 L482.333 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="482.333,982.356 482.333,986.14 486.389,986.14 486.389,982.356 482.333,982.356 "/>
<path clip-path="url(#clip452)" d="M486.389 986.14 L486.389 986.14 L490.445 986.14 L490.445 986.14 L486.389 986.14 L486.389 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="486.389,986.14 486.389,986.14 490.445,986.14 486.389,986.14 "/>
<path clip-path="url(#clip452)" d="M490.445 982.356 L490.445 986.14 L494.501 986.14 L494.501 982.356 L490.445 982.356 L490.445 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="490.445,982.356 490.445,986.14 494.501,986.14 494.501,982.356 490.445,982.356 "/>
<path clip-path="url(#clip452)" d="M494.501 978.572 L494.501 986.14 L498.557 986.14 L498.557 978.572 L494.501 978.572 L494.501 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="494.501,978.572 494.501,986.14 498.557,986.14 498.557,978.572 494.501,978.572 "/>
<path clip-path="url(#clip452)" d="M498.557 986.14 L498.557 986.14 L502.613 986.14 L502.613 986.14 L498.557 986.14 L498.557 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="498.557,986.14 498.557,986.14 502.613,986.14 498.557,986.14 "/>
<path clip-path="url(#clip452)" d="M502.613 986.14 L502.613 986.14 L506.668 986.14 L506.668 986.14 L502.613 986.14 L502.613 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="502.613,986.14 502.613,986.14 506.668,986.14 502.613,986.14 "/>
<path clip-path="url(#clip452)" d="M506.668 986.14 L506.668 986.14 L510.724 986.14 L510.724 986.14 L506.668 986.14 L506.668 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="506.668,986.14 506.668,986.14 510.724,986.14 506.668,986.14 "/>
<path clip-path="url(#clip452)" d="M510.724 986.14 L510.724 986.14 L514.78 986.14 L514.78 986.14 L510.724 986.14 L510.724 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="510.724,986.14 510.724,986.14 514.78,986.14 510.724,986.14 "/>
<path clip-path="url(#clip452)" d="M514.78 986.14 L514.78 986.14 L518.836 986.14 L518.836 986.14 L514.78 986.14 L514.78 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="514.78,986.14 514.78,986.14 518.836,986.14 514.78,986.14 "/>
<path clip-path="url(#clip452)" d="M518.836 986.14 L518.836 986.14 L522.892 986.14 L522.892 986.14 L518.836 986.14 L518.836 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="518.836,986.14 518.836,986.14 522.892,986.14 518.836,986.14 "/>
<path clip-path="url(#clip452)" d="M522.892 986.14 L522.892 986.14 L526.948 986.14 L526.948 986.14 L522.892 986.14 L522.892 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="522.892,986.14 522.892,986.14 526.948,986.14 522.892,986.14 "/>
<path clip-path="url(#clip452)" d="M526.948 986.14 L526.948 986.14 L531.003 986.14 L531.003 986.14 L526.948 986.14 L526.948 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="526.948,986.14 526.948,986.14 531.003,986.14 526.948,986.14 "/>
<path clip-path="url(#clip452)" d="M531.003 986.14 L531.003 986.14 L535.059 986.14 L535.059 986.14 L531.003 986.14 L531.003 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="531.003,986.14 531.003,986.14 535.059,986.14 531.003,986.14 "/>
<path clip-path="url(#clip452)" d="M535.059 986.14 L535.059 986.14 L539.115 986.14 L539.115 986.14 L535.059 986.14 L535.059 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="535.059,986.14 535.059,986.14 539.115,986.14 535.059,986.14 "/>
<path clip-path="url(#clip452)" d="M539.115 978.572 L539.115 986.14 L543.171 986.14 L543.171 978.572 L539.115 978.572 L539.115 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="539.115,978.572 539.115,986.14 543.171,986.14 543.171,978.572 539.115,978.572 "/>
<path clip-path="url(#clip452)" d="M543.171 986.14 L543.171 986.14 L547.227 986.14 L547.227 986.14 L543.171 986.14 L543.171 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="543.171,986.14 543.171,986.14 547.227,986.14 543.171,986.14 "/>
<path clip-path="url(#clip452)" d="M547.227 986.14 L547.227 986.14 L551.283 986.14 L551.283 986.14 L547.227 986.14 L547.227 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="547.227,986.14 547.227,986.14 551.283,986.14 547.227,986.14 "/>
<path clip-path="url(#clip452)" d="M551.283 982.356 L551.283 986.14 L555.338 986.14 L555.338 982.356 L551.283 982.356 L551.283 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="551.283,982.356 551.283,986.14 555.338,986.14 555.338,982.356 551.283,982.356 "/>
<path clip-path="url(#clip452)" d="M555.338 986.14 L555.338 986.14 L559.394 986.14 L559.394 986.14 L555.338 986.14 L555.338 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="555.338,986.14 555.338,986.14 559.394,986.14 555.338,986.14 "/>
<path clip-path="url(#clip452)" d="M559.394 986.14 L559.394 986.14 L563.45 986.14 L563.45 986.14 L559.394 986.14 L559.394 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="559.394,986.14 559.394,986.14 563.45,986.14 559.394,986.14 "/>
<path clip-path="url(#clip452)" d="M563.45 982.356 L563.45 986.14 L567.506 986.14 L567.506 982.356 L563.45 982.356 L563.45 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="563.45,982.356 563.45,986.14 567.506,986.14 567.506,982.356 563.45,982.356 "/>
<path clip-path="url(#clip452)" d="M567.506 986.14 L567.506 986.14 L571.562 986.14 L571.562 986.14 L567.506 986.14 L567.506 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="567.506,986.14 567.506,986.14 571.562,986.14 567.506,986.14 "/>
<path clip-path="url(#clip452)" d="M571.562 986.14 L571.562 986.14 L575.618 986.14 L575.618 986.14 L571.562 986.14 L571.562 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="571.562,986.14 571.562,986.14 575.618,986.14 571.562,986.14 "/>
<path clip-path="url(#clip452)" d="M575.618 986.14 L575.618 986.14 L579.673 986.14 L579.673 986.14 L575.618 986.14 L575.618 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="575.618,986.14 575.618,986.14 579.673,986.14 575.618,986.14 "/>
<path clip-path="url(#clip452)" d="M579.673 986.14 L579.673 986.14 L583.729 986.14 L583.729 986.14 L579.673 986.14 L579.673 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="579.673,986.14 579.673,986.14 583.729,986.14 579.673,986.14 "/>
<path clip-path="url(#clip452)" d="M583.729 986.14 L583.729 986.14 L587.785 986.14 L587.785 986.14 L583.729 986.14 L583.729 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="583.729,986.14 583.729,986.14 587.785,986.14 583.729,986.14 "/>
<path clip-path="url(#clip452)" d="M587.785 986.14 L587.785 986.14 L591.841 986.14 L591.841 986.14 L587.785 986.14 L587.785 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="587.785,986.14 587.785,986.14 591.841,986.14 587.785,986.14 "/>
<path clip-path="url(#clip452)" d="M591.841 986.14 L591.841 986.14 L595.897 986.14 L595.897 986.14 L591.841 986.14 L591.841 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="591.841,986.14 591.841,986.14 595.897,986.14 591.841,986.14 "/>
<path clip-path="url(#clip452)" d="M595.897 986.14 L595.897 986.14 L599.953 986.14 L599.953 986.14 L595.897 986.14 L595.897 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="595.897,986.14 595.897,986.14 599.953,986.14 595.897,986.14 "/>
<path clip-path="url(#clip452)" d="M599.953 986.14 L599.953 986.14 L604.009 986.14 L604.009 986.14 L599.953 986.14 L599.953 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="599.953,986.14 599.953,986.14 604.009,986.14 599.953,986.14 "/>
<path clip-path="url(#clip452)" d="M604.009 982.356 L604.009 986.14 L608.064 986.14 L608.064 982.356 L604.009 982.356 L604.009 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="604.009,982.356 604.009,986.14 608.064,986.14 608.064,982.356 604.009,982.356 "/>
<path clip-path="url(#clip452)" d="M608.064 986.14 L608.064 986.14 L612.12 986.14 L612.12 986.14 L608.064 986.14 L608.064 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="608.064,986.14 608.064,986.14 612.12,986.14 608.064,986.14 "/>
<path clip-path="url(#clip452)" d="M612.12 982.356 L612.12 986.14 L616.176 986.14 L616.176 982.356 L612.12 982.356 L612.12 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="612.12,982.356 612.12,986.14 616.176,986.14 616.176,982.356 612.12,982.356 "/>
<path clip-path="url(#clip452)" d="M616.176 986.14 L616.176 986.14 L620.232 986.14 L620.232 986.14 L616.176 986.14 L616.176 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="616.176,986.14 616.176,986.14 620.232,986.14 616.176,986.14 "/>
<path clip-path="url(#clip452)" d="M620.232 986.14 L620.232 986.14 L624.288 986.14 L624.288 986.14 L620.232 986.14 L620.232 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="620.232,986.14 620.232,986.14 624.288,986.14 620.232,986.14 "/>
<path clip-path="url(#clip452)" d="M624.288 986.14 L624.288 986.14 L628.344 986.14 L628.344 986.14 L624.288 986.14 L624.288 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="624.288,986.14 624.288,986.14 628.344,986.14 624.288,986.14 "/>
<path clip-path="url(#clip452)" d="M628.344 986.14 L628.344 986.14 L632.399 986.14 L632.399 986.14 L628.344 986.14 L628.344 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="628.344,986.14 628.344,986.14 632.399,986.14 628.344,986.14 "/>
<path clip-path="url(#clip452)" d="M632.399 986.14 L632.399 986.14 L636.455 986.14 L636.455 986.14 L632.399 986.14 L632.399 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="632.399,986.14 632.399,986.14 636.455,986.14 632.399,986.14 "/>
<path clip-path="url(#clip452)" d="M636.455 986.14 L636.455 986.14 L640.511 986.14 L640.511 986.14 L636.455 986.14 L636.455 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="636.455,986.14 636.455,986.14 640.511,986.14 636.455,986.14 "/>
<path clip-path="url(#clip452)" d="M640.511 986.14 L640.511 986.14 L644.567 986.14 L644.567 986.14 L640.511 986.14 L640.511 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="640.511,986.14 640.511,986.14 644.567,986.14 640.511,986.14 "/>
<path clip-path="url(#clip452)" d="M644.567 986.14 L644.567 986.14 L648.623 986.14 L648.623 986.14 L644.567 986.14 L644.567 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="644.567,986.14 644.567,986.14 648.623,986.14 644.567,986.14 "/>
<path clip-path="url(#clip452)" d="M648.623 982.356 L648.623 986.14 L652.679 986.14 L652.679 982.356 L648.623 982.356 L648.623 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="648.623,982.356 648.623,986.14 652.679,986.14 652.679,982.356 648.623,982.356 "/>
<path clip-path="url(#clip452)" d="M652.679 986.14 L652.679 986.14 L656.734 986.14 L656.734 986.14 L652.679 986.14 L652.679 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="652.679,986.14 652.679,986.14 656.734,986.14 652.679,986.14 "/>
<path clip-path="url(#clip452)" d="M656.734 986.14 L656.734 986.14 L660.79 986.14 L660.79 986.14 L656.734 986.14 L656.734 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="656.734,986.14 656.734,986.14 660.79,986.14 656.734,986.14 "/>
<path clip-path="url(#clip452)" d="M660.79 986.14 L660.79 986.14 L664.846 986.14 L664.846 986.14 L660.79 986.14 L660.79 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="660.79,986.14 660.79,986.14 664.846,986.14 660.79,986.14 "/>
<path clip-path="url(#clip452)" d="M664.846 986.14 L664.846 986.14 L668.902 986.14 L668.902 986.14 L664.846 986.14 L664.846 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="664.846,986.14 664.846,986.14 668.902,986.14 664.846,986.14 "/>
<path clip-path="url(#clip452)" d="M668.902 986.14 L668.902 986.14 L672.958 986.14 L672.958 986.14 L668.902 986.14 L668.902 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="668.902,986.14 668.902,986.14 672.958,986.14 668.902,986.14 "/>
<path clip-path="url(#clip452)" d="M672.958 986.14 L672.958 986.14 L677.014 986.14 L677.014 986.14 L672.958 986.14 L672.958 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="672.958,986.14 672.958,986.14 677.014,986.14 672.958,986.14 "/>
<path clip-path="url(#clip452)" d="M677.014 986.14 L677.014 986.14 L681.069 986.14 L681.069 986.14 L677.014 986.14 L677.014 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="677.014,986.14 677.014,986.14 681.069,986.14 677.014,986.14 "/>
<path clip-path="url(#clip452)" d="M681.069 986.14 L681.069 986.14 L685.125 986.14 L685.125 986.14 L681.069 986.14 L681.069 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="681.069,986.14 681.069,986.14 685.125,986.14 681.069,986.14 "/>
<path clip-path="url(#clip452)" d="M685.125 986.14 L685.125 986.14 L689.181 986.14 L689.181 986.14 L685.125 986.14 L685.125 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="685.125,986.14 685.125,986.14 689.181,986.14 685.125,986.14 "/>
<path clip-path="url(#clip452)" d="M689.181 986.14 L689.181 986.14 L693.237 986.14 L693.237 986.14 L689.181 986.14 L689.181 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="689.181,986.14 689.181,986.14 693.237,986.14 689.181,986.14 "/>
<path clip-path="url(#clip452)" d="M693.237 986.14 L693.237 986.14 L697.293 986.14 L697.293 986.14 L693.237 986.14 L693.237 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="693.237,986.14 693.237,986.14 697.293,986.14 693.237,986.14 "/>
<path clip-path="url(#clip452)" d="M697.293 986.14 L697.293 986.14 L701.349 986.14 L701.349 986.14 L697.293 986.14 L697.293 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="697.293,986.14 697.293,986.14 701.349,986.14 697.293,986.14 "/>
<path clip-path="url(#clip452)" d="M701.349 986.14 L701.349 986.14 L705.404 986.14 L705.404 986.14 L701.349 986.14 L701.349 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="701.349,986.14 701.349,986.14 705.404,986.14 701.349,986.14 "/>
<path clip-path="url(#clip452)" d="M705.404 982.356 L705.404 986.14 L709.46 986.14 L709.46 982.356 L705.404 982.356 L705.404 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="705.404,982.356 705.404,986.14 709.46,986.14 709.46,982.356 705.404,982.356 "/>
<path clip-path="url(#clip452)" d="M709.46 986.14 L709.46 986.14 L713.516 986.14 L713.516 986.14 L709.46 986.14 L709.46 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="709.46,986.14 709.46,986.14 713.516,986.14 709.46,986.14 "/>
<path clip-path="url(#clip452)" d="M713.516 986.14 L713.516 986.14 L717.572 986.14 L717.572 986.14 L713.516 986.14 L713.516 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="713.516,986.14 713.516,986.14 717.572,986.14 713.516,986.14 "/>
<path clip-path="url(#clip452)" d="M717.572 986.14 L717.572 986.14 L721.628 986.14 L721.628 986.14 L717.572 986.14 L717.572 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="717.572,986.14 717.572,986.14 721.628,986.14 717.572,986.14 "/>
<path clip-path="url(#clip452)" d="M721.628 986.14 L721.628 986.14 L725.684 986.14 L725.684 986.14 L721.628 986.14 L721.628 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="721.628,986.14 721.628,986.14 725.684,986.14 721.628,986.14 "/>
<path clip-path="url(#clip452)" d="M725.684 986.14 L725.684 986.14 L729.739 986.14 L729.739 986.14 L725.684 986.14 L725.684 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="725.684,986.14 725.684,986.14 729.739,986.14 725.684,986.14 "/>
<path clip-path="url(#clip452)" d="M729.739 986.14 L729.739 986.14 L733.795 986.14 L733.795 986.14 L729.739 986.14 L729.739 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="729.739,986.14 729.739,986.14 733.795,986.14 729.739,986.14 "/>
<path clip-path="url(#clip452)" d="M733.795 986.14 L733.795 986.14 L737.851 986.14 L737.851 986.14 L733.795 986.14 L733.795 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="733.795,986.14 733.795,986.14 737.851,986.14 733.795,986.14 "/>
<path clip-path="url(#clip452)" d="M737.851 986.14 L737.851 986.14 L741.907 986.14 L741.907 986.14 L737.851 986.14 L737.851 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="737.851,986.14 737.851,986.14 741.907,986.14 737.851,986.14 "/>
<path clip-path="url(#clip452)" d="M741.907 986.14 L741.907 986.14 L745.963 986.14 L745.963 986.14 L741.907 986.14 L741.907 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="741.907,986.14 741.907,986.14 745.963,986.14 741.907,986.14 "/>
<path clip-path="url(#clip452)" d="M745.963 986.14 L745.963 986.14 L750.019 986.14 L750.019 986.14 L745.963 986.14 L745.963 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="745.963,986.14 745.963,986.14 750.019,986.14 745.963,986.14 "/>
<path clip-path="url(#clip452)" d="M750.019 986.14 L750.019 986.14 L754.074 986.14 L754.074 986.14 L750.019 986.14 L750.019 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="750.019,986.14 750.019,986.14 754.074,986.14 750.019,986.14 "/>
<path clip-path="url(#clip452)" d="M754.074 986.14 L754.074 986.14 L758.13 986.14 L758.13 986.14 L754.074 986.14 L754.074 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="754.074,986.14 754.074,986.14 758.13,986.14 754.074,986.14 "/>
<path clip-path="url(#clip452)" d="M758.13 986.14 L758.13 986.14 L762.186 986.14 L762.186 986.14 L758.13 986.14 L758.13 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="758.13,986.14 758.13,986.14 762.186,986.14 758.13,986.14 "/>
<path clip-path="url(#clip452)" d="M762.186 982.356 L762.186 986.14 L766.242 986.14 L766.242 982.356 L762.186 982.356 L762.186 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="762.186,982.356 762.186,986.14 766.242,986.14 766.242,982.356 762.186,982.356 "/>
<path clip-path="url(#clip452)" d="M766.242 986.14 L766.242 986.14 L770.298 986.14 L770.298 986.14 L766.242 986.14 L766.242 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="766.242,986.14 766.242,986.14 770.298,986.14 766.242,986.14 "/>
<path clip-path="url(#clip452)" d="M770.298 982.356 L770.298 986.14 L774.354 986.14 L774.354 982.356 L770.298 982.356 L770.298 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="770.298,982.356 770.298,986.14 774.354,986.14 774.354,982.356 770.298,982.356 "/>
<path clip-path="url(#clip452)" d="M774.354 986.14 L774.354 986.14 L778.409 986.14 L778.409 986.14 L774.354 986.14 L774.354 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="774.354,986.14 774.354,986.14 778.409,986.14 774.354,986.14 "/>
<path clip-path="url(#clip452)" d="M778.409 982.356 L778.409 986.14 L782.465 986.14 L782.465 982.356 L778.409 982.356 L778.409 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="778.409,982.356 778.409,986.14 782.465,986.14 782.465,982.356 778.409,982.356 "/>
<path clip-path="url(#clip452)" d="M782.465 986.14 L782.465 986.14 L786.521 986.14 L786.521 986.14 L782.465 986.14 L782.465 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="782.465,986.14 782.465,986.14 786.521,986.14 782.465,986.14 "/>
<path clip-path="url(#clip452)" d="M786.521 986.14 L786.521 986.14 L790.577 986.14 L790.577 986.14 L786.521 986.14 L786.521 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="786.521,986.14 786.521,986.14 790.577,986.14 786.521,986.14 "/>
<path clip-path="url(#clip452)" d="M790.577 986.14 L790.577 986.14 L794.633 986.14 L794.633 986.14 L790.577 986.14 L790.577 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="790.577,986.14 790.577,986.14 794.633,986.14 790.577,986.14 "/>
<path clip-path="url(#clip452)" d="M794.633 986.14 L794.633 986.14 L798.689 986.14 L798.689 986.14 L794.633 986.14 L794.633 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="794.633,986.14 794.633,986.14 798.689,986.14 794.633,986.14 "/>
<path clip-path="url(#clip452)" d="M798.689 986.14 L798.689 986.14 L802.745 986.14 L802.745 986.14 L798.689 986.14 L798.689 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="798.689,986.14 798.689,986.14 802.745,986.14 798.689,986.14 "/>
<path clip-path="url(#clip452)" d="M802.745 986.14 L802.745 986.14 L806.8 986.14 L806.8 986.14 L802.745 986.14 L802.745 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="802.745,986.14 802.745,986.14 806.8,986.14 802.745,986.14 "/>
<path clip-path="url(#clip452)" d="M806.8 986.14 L806.8 986.14 L810.856 986.14 L810.856 986.14 L806.8 986.14 L806.8 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="806.8,986.14 806.8,986.14 810.856,986.14 806.8,986.14 "/>
<path clip-path="url(#clip452)" d="M810.856 986.14 L810.856 986.14 L814.912 986.14 L814.912 986.14 L810.856 986.14 L810.856 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="810.856,986.14 810.856,986.14 814.912,986.14 810.856,986.14 "/>
<path clip-path="url(#clip452)" d="M814.912 982.356 L814.912 986.14 L818.968 986.14 L818.968 982.356 L814.912 982.356 L814.912 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="814.912,982.356 814.912,986.14 818.968,986.14 818.968,982.356 814.912,982.356 "/>
<path clip-path="url(#clip452)" d="M818.968 986.14 L818.968 986.14 L823.024 986.14 L823.024 986.14 L818.968 986.14 L818.968 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="818.968,986.14 818.968,986.14 823.024,986.14 818.968,986.14 "/>
<path clip-path="url(#clip452)" d="M823.024 986.14 L823.024 986.14 L827.08 986.14 L827.08 986.14 L823.024 986.14 L823.024 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="823.024,986.14 823.024,986.14 827.08,986.14 823.024,986.14 "/>
<path clip-path="url(#clip452)" d="M827.08 986.14 L827.08 986.14 L831.135 986.14 L831.135 986.14 L827.08 986.14 L827.08 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="827.08,986.14 827.08,986.14 831.135,986.14 827.08,986.14 "/>
<path clip-path="url(#clip452)" d="M831.135 986.14 L831.135 986.14 L835.191 986.14 L835.191 986.14 L831.135 986.14 L831.135 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="831.135,986.14 831.135,986.14 835.191,986.14 831.135,986.14 "/>
<path clip-path="url(#clip452)" d="M835.191 986.14 L835.191 986.14 L839.247 986.14 L839.247 986.14 L835.191 986.14 L835.191 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="835.191,986.14 835.191,986.14 839.247,986.14 835.191,986.14 "/>
<path clip-path="url(#clip452)" d="M839.247 982.356 L839.247 986.14 L843.303 986.14 L843.303 982.356 L839.247 982.356 L839.247 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="839.247,982.356 839.247,986.14 843.303,986.14 843.303,982.356 839.247,982.356 "/>
<path clip-path="url(#clip452)" d="M843.303 986.14 L843.303 986.14 L847.359 986.14 L847.359 986.14 L843.303 986.14 L843.303 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="843.303,986.14 843.303,986.14 847.359,986.14 843.303,986.14 "/>
<path clip-path="url(#clip452)" d="M847.359 986.14 L847.359 986.14 L851.415 986.14 L851.415 986.14 L847.359 986.14 L847.359 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="847.359,986.14 847.359,986.14 851.415,986.14 847.359,986.14 "/>
<path clip-path="url(#clip452)" d="M851.415 986.14 L851.415 986.14 L855.47 986.14 L855.47 986.14 L851.415 986.14 L851.415 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="851.415,986.14 851.415,986.14 855.47,986.14 851.415,986.14 "/>
<path clip-path="url(#clip452)" d="M855.47 986.14 L855.47 986.14 L859.526 986.14 L859.526 986.14 L855.47 986.14 L855.47 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="855.47,986.14 855.47,986.14 859.526,986.14 855.47,986.14 "/>
<path clip-path="url(#clip452)" d="M859.526 986.14 L859.526 986.14 L863.582 986.14 L863.582 986.14 L859.526 986.14 L859.526 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="859.526,986.14 859.526,986.14 863.582,986.14 859.526,986.14 "/>
<path clip-path="url(#clip452)" d="M863.582 986.14 L863.582 986.14 L867.638 986.14 L867.638 986.14 L863.582 986.14 L863.582 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="863.582,986.14 863.582,986.14 867.638,986.14 863.582,986.14 "/>
<path clip-path="url(#clip452)" d="M867.638 986.14 L867.638 986.14 L871.694 986.14 L871.694 986.14 L867.638 986.14 L867.638 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="867.638,986.14 867.638,986.14 871.694,986.14 867.638,986.14 "/>
<path clip-path="url(#clip452)" d="M871.694 986.14 L871.694 986.14 L875.75 986.14 L875.75 986.14 L871.694 986.14 L871.694 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="871.694,986.14 871.694,986.14 875.75,986.14 871.694,986.14 "/>
<path clip-path="url(#clip452)" d="M875.75 986.14 L875.75 986.14 L879.805 986.14 L879.805 986.14 L875.75 986.14 L875.75 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="875.75,986.14 875.75,986.14 879.805,986.14 875.75,986.14 "/>
<path clip-path="url(#clip452)" d="M879.805 986.14 L879.805 986.14 L883.861 986.14 L883.861 986.14 L879.805 986.14 L879.805 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="879.805,986.14 879.805,986.14 883.861,986.14 879.805,986.14 "/>
<path clip-path="url(#clip452)" d="M883.861 986.14 L883.861 986.14 L887.917 986.14 L887.917 986.14 L883.861 986.14 L883.861 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="883.861,986.14 883.861,986.14 887.917,986.14 883.861,986.14 "/>
<path clip-path="url(#clip452)" d="M887.917 986.14 L887.917 986.14 L891.973 986.14 L891.973 986.14 L887.917 986.14 L887.917 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="887.917,986.14 887.917,986.14 891.973,986.14 887.917,986.14 "/>
<path clip-path="url(#clip452)" d="M891.973 986.14 L891.973 986.14 L896.029 986.14 L896.029 986.14 L891.973 986.14 L891.973 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="891.973,986.14 891.973,986.14 896.029,986.14 891.973,986.14 "/>
<path clip-path="url(#clip452)" d="M896.029 986.14 L896.029 986.14 L900.085 986.14 L900.085 986.14 L896.029 986.14 L896.029 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="896.029,986.14 896.029,986.14 900.085,986.14 896.029,986.14 "/>
<path clip-path="url(#clip452)" d="M900.085 986.14 L900.085 986.14 L904.14 986.14 L904.14 986.14 L900.085 986.14 L900.085 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="900.085,986.14 900.085,986.14 904.14,986.14 900.085,986.14 "/>
<path clip-path="url(#clip452)" d="M904.14 986.14 L904.14 986.14 L908.196 986.14 L908.196 986.14 L904.14 986.14 L904.14 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="904.14,986.14 904.14,986.14 908.196,986.14 904.14,986.14 "/>
<path clip-path="url(#clip452)" d="M908.196 986.14 L908.196 986.14 L912.252 986.14 L912.252 986.14 L908.196 986.14 L908.196 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="908.196,986.14 908.196,986.14 912.252,986.14 908.196,986.14 "/>
<path clip-path="url(#clip452)" d="M912.252 986.14 L912.252 986.14 L916.308 986.14 L916.308 986.14 L912.252 986.14 L912.252 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="912.252,986.14 912.252,986.14 916.308,986.14 912.252,986.14 "/>
<path clip-path="url(#clip452)" d="M916.308 986.14 L916.308 986.14 L920.364 986.14 L920.364 986.14 L916.308 986.14 L916.308 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="916.308,986.14 916.308,986.14 920.364,986.14 916.308,986.14 "/>
<path clip-path="url(#clip452)" d="M920.364 986.14 L920.364 986.14 L924.42 986.14 L924.42 986.14 L920.364 986.14 L920.364 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="920.364,986.14 920.364,986.14 924.42,986.14 920.364,986.14 "/>
<path clip-path="url(#clip452)" d="M924.42 986.14 L924.42 986.14 L928.475 986.14 L928.475 986.14 L924.42 986.14 L924.42 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="924.42,986.14 924.42,986.14 928.475,986.14 924.42,986.14 "/>
<path clip-path="url(#clip452)" d="M928.475 986.14 L928.475 986.14 L932.531 986.14 L932.531 986.14 L928.475 986.14 L928.475 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="928.475,986.14 928.475,986.14 932.531,986.14 928.475,986.14 "/>
<path clip-path="url(#clip452)" d="M932.531 986.14 L932.531 986.14 L936.587 986.14 L936.587 986.14 L932.531 986.14 L932.531 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="932.531,986.14 932.531,986.14 936.587,986.14 932.531,986.14 "/>
<path clip-path="url(#clip452)" d="M936.587 986.14 L936.587 986.14 L940.643 986.14 L940.643 986.14 L936.587 986.14 L936.587 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="936.587,986.14 936.587,986.14 940.643,986.14 936.587,986.14 "/>
<path clip-path="url(#clip452)" d="M940.643 982.356 L940.643 986.14 L944.699 986.14 L944.699 982.356 L940.643 982.356 L940.643 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="940.643,982.356 940.643,986.14 944.699,986.14 944.699,982.356 940.643,982.356 "/>
<path clip-path="url(#clip452)" d="M944.699 986.14 L944.699 986.14 L948.755 986.14 L948.755 986.14 L944.699 986.14 L944.699 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="944.699,986.14 944.699,986.14 948.755,986.14 944.699,986.14 "/>
<path clip-path="url(#clip452)" d="M948.755 986.14 L948.755 986.14 L952.81 986.14 L952.81 986.14 L948.755 986.14 L948.755 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="948.755,986.14 948.755,986.14 952.81,986.14 948.755,986.14 "/>
<path clip-path="url(#clip452)" d="M952.81 986.14 L952.81 986.14 L956.866 986.14 L956.866 986.14 L952.81 986.14 L952.81 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="952.81,986.14 952.81,986.14 956.866,986.14 952.81,986.14 "/>
<path clip-path="url(#clip452)" d="M956.866 986.14 L956.866 986.14 L960.922 986.14 L960.922 986.14 L956.866 986.14 L956.866 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="956.866,986.14 956.866,986.14 960.922,986.14 956.866,986.14 "/>
<path clip-path="url(#clip452)" d="M960.922 986.14 L960.922 986.14 L964.978 986.14 L964.978 986.14 L960.922 986.14 L960.922 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="960.922,986.14 960.922,986.14 964.978,986.14 960.922,986.14 "/>
<path clip-path="url(#clip452)" d="M964.978 986.14 L964.978 986.14 L969.034 986.14 L969.034 986.14 L964.978 986.14 L964.978 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="964.978,986.14 964.978,986.14 969.034,986.14 964.978,986.14 "/>
<path clip-path="url(#clip452)" d="M969.034 986.14 L969.034 986.14 L973.09 986.14 L973.09 986.14 L969.034 986.14 L969.034 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="969.034,986.14 969.034,986.14 973.09,986.14 969.034,986.14 "/>
<path clip-path="url(#clip452)" d="M973.09 982.356 L973.09 986.14 L977.145 986.14 L977.145 982.356 L973.09 982.356 L973.09 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="973.09,982.356 973.09,986.14 977.145,986.14 977.145,982.356 973.09,982.356 "/>
<path clip-path="url(#clip452)" d="M977.145 986.14 L977.145 986.14 L981.201 986.14 L981.201 986.14 L977.145 986.14 L977.145 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="977.145,986.14 977.145,986.14 981.201,986.14 977.145,986.14 "/>
<path clip-path="url(#clip452)" d="M981.201 986.14 L981.201 986.14 L985.257 986.14 L985.257 986.14 L981.201 986.14 L981.201 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="981.201,986.14 981.201,986.14 985.257,986.14 981.201,986.14 "/>
<path clip-path="url(#clip452)" d="M985.257 986.14 L985.257 986.14 L989.313 986.14 L989.313 986.14 L985.257 986.14 L985.257 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="985.257,986.14 985.257,986.14 989.313,986.14 985.257,986.14 "/>
<path clip-path="url(#clip452)" d="M989.313 986.14 L989.313 986.14 L993.369 986.14 L993.369 986.14 L989.313 986.14 L989.313 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="989.313,986.14 989.313,986.14 993.369,986.14 989.313,986.14 "/>
<path clip-path="url(#clip452)" d="M993.369 986.14 L993.369 986.14 L997.425 986.14 L997.425 986.14 L993.369 986.14 L993.369 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="993.369,986.14 993.369,986.14 997.425,986.14 993.369,986.14 "/>
<path clip-path="url(#clip452)" d="M997.425 982.356 L997.425 986.14 L1001.48 986.14 L1001.48 982.356 L997.425 982.356 L997.425 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="997.425,982.356 997.425,986.14 1001.48,986.14 1001.48,982.356 997.425,982.356 "/>
<path clip-path="url(#clip452)" d="M1001.48 986.14 L1001.48 986.14 L1005.54 986.14 L1005.54 986.14 L1001.48 986.14 L1001.48 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1001.48,986.14 1001.48,986.14 1005.54,986.14 1001.48,986.14 "/>
<path clip-path="url(#clip452)" d="M1005.54 986.14 L1005.54 986.14 L1009.59 986.14 L1009.59 986.14 L1005.54 986.14 L1005.54 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1005.54,986.14 1005.54,986.14 1009.59,986.14 1005.54,986.14 "/>
<path clip-path="url(#clip452)" d="M1009.59 986.14 L1009.59 986.14 L1013.65 986.14 L1013.65 986.14 L1009.59 986.14 L1009.59 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1009.59,986.14 1009.59,986.14 1013.65,986.14 1009.59,986.14 "/>
<path clip-path="url(#clip452)" d="M1013.65 982.356 L1013.65 986.14 L1017.7 986.14 L1017.7 982.356 L1013.65 982.356 L1013.65 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1013.65,982.356 1013.65,986.14 1017.7,986.14 1017.7,982.356 1013.65,982.356 "/>
<path clip-path="url(#clip452)" d="M1017.7 986.14 L1017.7 986.14 L1021.76 986.14 L1021.76 986.14 L1017.7 986.14 L1017.7 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1017.7,986.14 1017.7,986.14 1021.76,986.14 1017.7,986.14 "/>
<path clip-path="url(#clip452)" d="M1021.76 986.14 L1021.76 986.14 L1025.82 986.14 L1025.82 986.14 L1021.76 986.14 L1021.76 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1021.76,986.14 1021.76,986.14 1025.82,986.14 1021.76,986.14 "/>
<path clip-path="url(#clip452)" d="M1025.82 971.005 L1025.82 986.14 L1029.87 986.14 L1029.87 971.005 L1025.82 971.005 L1025.82 971.005  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1025.82,971.005 1025.82,986.14 1029.87,986.14 1029.87,971.005 1025.82,971.005 "/>
<path clip-path="url(#clip452)" d="M1029.87 986.14 L1029.87 986.14 L1033.93 986.14 L1033.93 986.14 L1029.87 986.14 L1029.87 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1029.87,986.14 1029.87,986.14 1033.93,986.14 1029.87,986.14 "/>
<path clip-path="url(#clip452)" d="M1033.93 986.14 L1033.93 986.14 L1037.98 986.14 L1037.98 986.14 L1033.93 986.14 L1033.93 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1033.93,986.14 1033.93,986.14 1037.98,986.14 1033.93,986.14 "/>
<path clip-path="url(#clip452)" d="M1037.98 982.356 L1037.98 986.14 L1042.04 986.14 L1042.04 982.356 L1037.98 982.356 L1037.98 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1037.98,982.356 1037.98,986.14 1042.04,986.14 1042.04,982.356 1037.98,982.356 "/>
<path clip-path="url(#clip452)" d="M1042.04 978.572 L1042.04 986.14 L1046.09 986.14 L1046.09 978.572 L1042.04 978.572 L1042.04 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1042.04,978.572 1042.04,986.14 1046.09,986.14 1046.09,978.572 1042.04,978.572 "/>
<path clip-path="url(#clip452)" d="M1046.09 986.14 L1046.09 986.14 L1050.15 986.14 L1050.15 986.14 L1046.09 986.14 L1046.09 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1046.09,986.14 1046.09,986.14 1050.15,986.14 1046.09,986.14 "/>
<path clip-path="url(#clip452)" d="M1050.15 978.572 L1050.15 986.14 L1054.21 986.14 L1054.21 978.572 L1050.15 978.572 L1050.15 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1050.15,978.572 1050.15,986.14 1054.21,986.14 1054.21,978.572 1050.15,978.572 "/>
<path clip-path="url(#clip452)" d="M1054.21 986.14 L1054.21 986.14 L1058.26 986.14 L1058.26 986.14 L1054.21 986.14 L1054.21 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1054.21,986.14 1054.21,986.14 1058.26,986.14 1054.21,986.14 "/>
<path clip-path="url(#clip452)" d="M1058.26 978.572 L1058.26 986.14 L1062.32 986.14 L1062.32 978.572 L1058.26 978.572 L1058.26 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1058.26,978.572 1058.26,986.14 1062.32,986.14 1062.32,978.572 1058.26,978.572 "/>
<path clip-path="url(#clip452)" d="M1062.32 982.356 L1062.32 986.14 L1066.37 986.14 L1066.37 982.356 L1062.32 982.356 L1062.32 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1062.32,982.356 1062.32,986.14 1066.37,986.14 1066.37,982.356 1062.32,982.356 "/>
<path clip-path="url(#clip452)" d="M1066.37 986.14 L1066.37 986.14 L1070.43 986.14 L1070.43 986.14 L1066.37 986.14 L1066.37 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1066.37,986.14 1066.37,986.14 1070.43,986.14 1066.37,986.14 "/>
<path clip-path="url(#clip452)" d="M1070.43 986.14 L1070.43 986.14 L1074.49 986.14 L1074.49 986.14 L1070.43 986.14 L1070.43 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1070.43,986.14 1070.43,986.14 1074.49,986.14 1070.43,986.14 "/>
<path clip-path="url(#clip452)" d="M1074.49 986.14 L1074.49 986.14 L1078.54 986.14 L1078.54 986.14 L1074.49 986.14 L1074.49 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1074.49,986.14 1074.49,986.14 1078.54,986.14 1074.49,986.14 "/>
<path clip-path="url(#clip452)" d="M1078.54 982.356 L1078.54 986.14 L1082.6 986.14 L1082.6 982.356 L1078.54 982.356 L1078.54 982.356  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1078.54,982.356 1078.54,986.14 1082.6,986.14 1082.6,982.356 1078.54,982.356 "/>
<path clip-path="url(#clip452)" d="M1082.6 978.572 L1082.6 986.14 L1086.65 986.14 L1086.65 978.572 L1082.6 978.572 L1082.6 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1082.6,978.572 1082.6,986.14 1086.65,986.14 1086.65,978.572 1082.6,978.572 "/>
<path clip-path="url(#clip452)" d="M1086.65 986.14 L1086.65 986.14 L1090.71 986.14 L1090.71 986.14 L1086.65 986.14 L1086.65 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1086.65,986.14 1086.65,986.14 1090.71,986.14 1086.65,986.14 "/>
<path clip-path="url(#clip452)" d="M1090.71 978.572 L1090.71 986.14 L1094.76 986.14 L1094.76 978.572 L1090.71 978.572 L1090.71 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1090.71,978.572 1090.71,986.14 1094.76,986.14 1094.76,978.572 1090.71,978.572 "/>
<path clip-path="url(#clip452)" d="M1094.76 978.572 L1094.76 986.14 L1098.82 986.14 L1098.82 978.572 L1094.76 978.572 L1094.76 978.572  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1094.76,978.572 1094.76,986.14 1098.82,986.14 1098.82,978.572 1094.76,978.572 "/>
<path clip-path="url(#clip452)" d="M1098.82 645.601 L1098.82 986.14 L1102.88 986.14 L1102.88 645.601 L1098.82 645.601 L1098.82 645.601  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip452)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1098.82,645.601 1098.82,986.14 1102.88,986.14 1102.88,645.601 1098.82,645.601 "/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="297.793" cy="316.414" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="301.849" cy="153.712" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="305.905" cy="426.143" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="309.96" cy="649.385" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="314.016" cy="740.195" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="318.072" cy="868.843" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="322.128" cy="899.113" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="326.184" cy="918.032" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="330.24" cy="940.735" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="334.295" cy="936.951" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="338.351" cy="948.302" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="342.407" cy="955.87" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="346.463" cy="959.654" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="350.519" cy="967.221" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="354.575" cy="967.221" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="358.63" cy="974.789" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="362.686" cy="974.789" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="366.742" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="370.798" cy="974.789" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="374.854" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="378.91" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="382.965" cy="974.789" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="387.021" cy="971.005" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="391.077" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="395.133" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="399.189" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="403.245" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="407.3" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="411.356" cy="974.789" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="415.412" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="419.468" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="423.524" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="427.58" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="431.635" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="435.691" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="439.747" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="443.803" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="447.859" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="451.915" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="455.97" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="460.026" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="464.082" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="468.138" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="472.194" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="476.25" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="480.306" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="484.361" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="488.417" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="492.473" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="496.529" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="500.585" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="504.641" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="508.696" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="512.752" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="516.808" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="520.864" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="524.92" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="528.976" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="533.031" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="537.087" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="541.143" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="545.199" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="549.255" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="553.311" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="557.366" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="561.422" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="565.478" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="569.534" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="573.59" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="577.646" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="581.701" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="585.757" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="589.813" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="593.869" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="597.925" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="601.981" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="606.036" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="610.092" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="614.148" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="618.204" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="622.26" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="626.316" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="630.371" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="634.427" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="638.483" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="642.539" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="646.595" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="650.651" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="654.706" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="658.762" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="662.818" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="666.874" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="670.93" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="674.986" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="679.041" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="683.097" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="687.153" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="691.209" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="695.265" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="699.321" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="703.377" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="707.432" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="711.488" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="715.544" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="719.6" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="723.656" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="727.712" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="731.767" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="735.823" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="739.879" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="743.935" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="747.991" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="752.047" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="756.102" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="760.158" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="764.214" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="768.27" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="772.326" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="776.382" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="780.437" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="784.493" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="788.549" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="792.605" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="796.661" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="800.717" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="804.772" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="808.828" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="812.884" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="816.94" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="820.996" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="825.052" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="829.107" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="833.163" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="837.219" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="841.275" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="845.331" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="849.387" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="853.442" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="857.498" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="861.554" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="865.61" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="869.666" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="873.722" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="877.777" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="881.833" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="885.889" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="889.945" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="894.001" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="898.057" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="902.113" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="906.168" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="910.224" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="914.28" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="918.336" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="922.392" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="926.448" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="930.503" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="934.559" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="938.615" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="942.671" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="946.727" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="950.783" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="954.838" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="958.894" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="962.95" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="967.006" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="971.062" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="975.118" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="979.173" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="983.229" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="987.285" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="991.341" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="995.397" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="999.453" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1003.51" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1007.56" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1011.62" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1015.68" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1019.73" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1023.79" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1027.84" cy="971.005" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1031.9" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1035.96" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1040.01" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1044.07" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1048.12" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1052.18" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1056.23" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1060.29" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1064.35" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1068.4" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1072.46" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1076.51" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1080.57" cy="982.356" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1084.63" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1088.68" cy="986.14" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1092.74" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1096.79" cy="978.572" r="2"/>
<circle clip-path="url(#clip452)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1100.85" cy="645.601" r="2"/>
<polyline clip-path="url(#clip452)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="306.285,1818.57 306.285,-678.716 "/>
</svg>
<p>(c)</p>
</div>
</div></td>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="300" height="300" viewBox="0 0 1200 1200">
<defs>
  <clipPath id="clip510">
    <rect x="0" y="0" width="1200" height="1200"/>
  </clipPath>
</defs>
<path clip-path="url(#clip510)" d="M0 1200 L1200 1200 L1200 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip511">
    <rect x="240" y="119" width="841" height="841"/>
  </clipPath>
</defs>
<path clip-path="url(#clip510)" d="M244.149 986.14 L1152.76 986.14 L1152.76 153.712 L244.149 153.712  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip512">
    <rect x="244" y="153" width="910" height="833"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="415.423,986.14 415.423,153.712 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="617.587,986.14 617.587,153.712 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="819.751,986.14 819.751,153.712 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1021.92,986.14 1021.92,153.712 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,986.14 1152.76,986.14 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,843.601 1152.76,843.601 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,701.062 1152.76,701.062 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,558.523 1152.76,558.523 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,415.984 1152.76,415.984 "/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,273.445 1152.76,273.445 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 1152.76,986.14 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="415.423,986.14 415.423,967.242 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="617.587,986.14 617.587,967.242 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="819.751,986.14 819.751,967.242 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1021.92,986.14 1021.92,967.242 "/>
<path clip-path="url(#clip510)" d="M354.902 1022.11 L384.578 1022.11 L384.578 1026.05 L354.902 1026.05 L354.902 1022.11 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M395.481 1035 L403.12 1035 L403.12 1008.64 L394.809 1010.31 L394.809 1006.05 L403.073 1004.38 L407.749 1004.38 L407.749 1035 L415.388 1035 L415.388 1038.94 L395.481 1038.94 L395.481 1035 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M435.411 1019.8 Q432.263 1019.8 430.411 1021.95 Q428.582 1024.1 428.582 1027.85 Q428.582 1031.58 430.411 1033.75 Q432.263 1035.91 435.411 1035.91 Q438.559 1035.91 440.388 1033.75 Q442.24 1031.58 442.24 1027.85 Q442.24 1024.1 440.388 1021.95 Q438.559 1019.8 435.411 1019.8 M444.693 1005.14 L444.693 1009.4 Q442.934 1008.57 441.129 1008.13 Q439.346 1007.69 437.587 1007.69 Q432.957 1007.69 430.504 1010.81 Q428.073 1013.94 427.726 1020.26 Q429.092 1018.25 431.152 1017.18 Q433.212 1016.09 435.689 1016.09 Q440.897 1016.09 443.906 1019.26 Q446.939 1022.41 446.939 1027.85 Q446.939 1033.18 443.791 1036.39 Q440.643 1039.61 435.411 1039.61 Q429.416 1039.61 426.245 1035.03 Q423.073 1030.42 423.073 1021.69 Q423.073 1013.5 426.962 1008.64 Q430.851 1003.75 437.402 1003.75 Q439.161 1003.75 440.944 1004.1 Q442.749 1004.45 444.693 1005.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M455.041 1004.38 L473.397 1004.38 L473.397 1008.32 L459.323 1008.32 L459.323 1016.79 Q460.342 1016.44 461.36 1016.28 Q462.379 1016.09 463.397 1016.09 Q469.184 1016.09 472.564 1019.26 Q475.943 1022.44 475.943 1027.85 Q475.943 1033.43 472.471 1036.53 Q468.999 1039.61 462.68 1039.61 Q460.504 1039.61 458.235 1039.24 Q455.99 1038.87 453.582 1038.13 L453.582 1033.43 Q455.666 1034.56 457.888 1035.12 Q460.11 1035.68 462.587 1035.68 Q466.592 1035.68 468.929 1033.57 Q471.267 1031.46 471.267 1027.85 Q471.267 1024.24 468.929 1022.13 Q466.592 1020.03 462.587 1020.03 Q460.712 1020.03 458.837 1020.44 Q456.985 1020.86 455.041 1021.74 L455.041 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M556.569 1022.11 L586.245 1022.11 L586.245 1026.05 L556.569 1026.05 L556.569 1022.11 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M597.147 1035 L604.786 1035 L604.786 1008.64 L596.476 1010.31 L596.476 1006.05 L604.74 1004.38 L609.416 1004.38 L609.416 1035 L617.055 1035 L617.055 1038.94 L597.147 1038.94 L597.147 1035 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M637.078 1019.8 Q633.929 1019.8 632.078 1021.95 Q630.249 1024.1 630.249 1027.85 Q630.249 1031.58 632.078 1033.75 Q633.929 1035.91 637.078 1035.91 Q640.226 1035.91 642.054 1033.75 Q643.906 1031.58 643.906 1027.85 Q643.906 1024.1 642.054 1021.95 Q640.226 1019.8 637.078 1019.8 M646.36 1005.14 L646.36 1009.4 Q644.601 1008.57 642.795 1008.13 Q641.013 1007.69 639.254 1007.69 Q634.624 1007.69 632.17 1010.81 Q629.74 1013.94 629.392 1020.26 Q630.758 1018.25 632.818 1017.18 Q634.879 1016.09 637.355 1016.09 Q642.564 1016.09 645.573 1019.26 Q648.605 1022.41 648.605 1027.85 Q648.605 1033.18 645.457 1036.39 Q642.309 1039.61 637.078 1039.61 Q631.082 1039.61 627.911 1035.03 Q624.74 1030.42 624.74 1021.69 Q624.74 1013.5 628.629 1008.64 Q632.517 1003.75 639.068 1003.75 Q640.828 1003.75 642.61 1004.1 Q644.416 1004.45 646.36 1005.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M666.661 1007.46 Q663.05 1007.46 661.221 1011.02 Q659.415 1014.56 659.415 1021.69 Q659.415 1028.8 661.221 1032.37 Q663.05 1035.91 666.661 1035.91 Q670.295 1035.91 672.101 1032.37 Q673.929 1028.8 673.929 1021.69 Q673.929 1014.56 672.101 1011.02 Q670.295 1007.46 666.661 1007.46 M666.661 1003.75 Q672.471 1003.75 675.526 1008.36 Q678.605 1012.94 678.605 1021.69 Q678.605 1030.42 675.526 1035.03 Q672.471 1039.61 666.661 1039.61 Q660.851 1039.61 657.772 1035.03 Q654.716 1030.42 654.716 1021.69 Q654.716 1012.94 657.772 1008.36 Q660.851 1003.75 666.661 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M759.231 1022.11 L788.906 1022.11 L788.906 1026.05 L759.231 1026.05 L759.231 1022.11 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M799.809 1035 L807.448 1035 L807.448 1008.64 L799.138 1010.31 L799.138 1006.05 L807.402 1004.38 L812.078 1004.38 L812.078 1035 L819.716 1035 L819.716 1038.94 L799.809 1038.94 L799.809 1035 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M829.207 1004.38 L847.563 1004.38 L847.563 1008.32 L833.489 1008.32 L833.489 1016.79 Q834.508 1016.44 835.526 1016.28 Q836.545 1016.09 837.564 1016.09 Q843.351 1016.09 846.73 1019.26 Q850.11 1022.44 850.11 1027.85 Q850.11 1033.43 846.638 1036.53 Q843.165 1039.61 836.846 1039.61 Q834.67 1039.61 832.402 1039.24 Q830.156 1038.87 827.749 1038.13 L827.749 1033.43 Q829.832 1034.56 832.054 1035.12 Q834.277 1035.68 836.753 1035.68 Q840.758 1035.68 843.096 1033.57 Q845.434 1031.46 845.434 1027.85 Q845.434 1024.24 843.096 1022.13 Q840.758 1020.03 836.753 1020.03 Q834.878 1020.03 833.003 1020.44 Q831.152 1020.86 829.207 1021.74 L829.207 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M859.369 1004.38 L877.725 1004.38 L877.725 1008.32 L863.651 1008.32 L863.651 1016.79 Q864.67 1016.44 865.688 1016.28 Q866.707 1016.09 867.725 1016.09 Q873.512 1016.09 876.892 1019.26 Q880.272 1022.44 880.272 1027.85 Q880.272 1033.43 876.799 1036.53 Q873.327 1039.61 867.008 1039.61 Q864.832 1039.61 862.563 1039.24 Q860.318 1038.87 857.911 1038.13 L857.911 1033.43 Q859.994 1034.56 862.216 1035.12 Q864.438 1035.68 866.915 1035.68 Q870.92 1035.68 873.258 1033.57 Q875.596 1031.46 875.596 1027.85 Q875.596 1024.24 873.258 1022.13 Q870.92 1020.03 866.915 1020.03 Q865.04 1020.03 863.165 1020.44 Q861.313 1020.86 859.369 1021.74 L859.369 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M960.897 1022.11 L990.573 1022.11 L990.573 1026.05 L960.897 1026.05 L960.897 1022.11 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M1001.48 1035 L1009.11 1035 L1009.11 1008.64 L1000.8 1010.31 L1000.8 1006.05 L1009.07 1004.38 L1013.74 1004.38 L1013.74 1035 L1021.38 1035 L1021.38 1038.94 L1001.48 1038.94 L1001.48 1035 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M1030.87 1004.38 L1049.23 1004.38 L1049.23 1008.32 L1035.16 1008.32 L1035.16 1016.79 Q1036.17 1016.44 1037.19 1016.28 Q1038.21 1016.09 1039.23 1016.09 Q1045.02 1016.09 1048.4 1019.26 Q1051.78 1022.44 1051.78 1027.85 Q1051.78 1033.43 1048.3 1036.53 Q1044.83 1039.61 1038.51 1039.61 Q1036.34 1039.61 1034.07 1039.24 Q1031.82 1038.87 1029.42 1038.13 L1029.42 1033.43 Q1031.5 1034.56 1033.72 1035.12 Q1035.94 1035.68 1038.42 1035.68 Q1042.42 1035.68 1044.76 1033.57 Q1047.1 1031.46 1047.1 1027.85 Q1047.1 1024.24 1044.76 1022.13 Q1042.42 1020.03 1038.42 1020.03 Q1036.54 1020.03 1034.67 1020.44 Q1032.82 1020.86 1030.87 1021.74 L1030.87 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M1070.99 1007.46 Q1067.38 1007.46 1065.55 1011.02 Q1063.74 1014.56 1063.74 1021.69 Q1063.74 1028.8 1065.55 1032.37 Q1067.38 1035.91 1070.99 1035.91 Q1074.62 1035.91 1076.43 1032.37 Q1078.26 1028.8 1078.26 1021.69 Q1078.26 1014.56 1076.43 1011.02 Q1074.62 1007.46 1070.99 1007.46 M1070.99 1003.75 Q1076.8 1003.75 1079.85 1008.36 Q1082.93 1012.94 1082.93 1021.69 Q1082.93 1030.42 1079.85 1035.03 Q1076.8 1039.61 1070.99 1039.61 Q1065.18 1039.61 1062.1 1035.03 Q1059.04 1030.42 1059.04 1021.69 Q1059.04 1012.94 1062.1 1008.36 Q1065.18 1003.75 1070.99 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M701.545 1051.34 L697.971 1065.76 Q697.777 1066.63 697.616 1066.86 Q697.455 1067.08 697.004 1067.08 Q696.65 1067.08 696.425 1066.92 Q696.232 1066.76 696.232 1066.6 L696.199 1066.44 Q696.199 1066.34 696.264 1065.86 Q696.328 1065.35 696.393 1064.51 Q696.457 1063.67 696.457 1062.74 Q696.457 1057.45 693.88 1055.01 Q691.336 1052.53 686.956 1052.53 Q683.929 1052.53 681.224 1054.14 Q678.518 1055.75 676.972 1058.29 Q675.427 1060.8 675.427 1063.45 Q675.427 1065.86 676.65 1067.41 Q677.874 1068.95 679.452 1069.47 L683.864 1070.66 Q688.502 1071.85 689.823 1072.37 Q691.143 1072.85 692.238 1073.94 Q695.491 1076.97 695.491 1081.67 Q695.491 1085.73 693.204 1089.66 Q690.918 1093.56 686.988 1096.1 Q683.059 1098.61 678.712 1098.61 Q671.079 1098.61 667.568 1093.88 L664.348 1097.62 Q663.736 1098.32 663.543 1098.45 Q663.382 1098.61 663.156 1098.61 Q662.866 1098.61 662.641 1098.42 Q662.415 1098.26 662.415 1097.97 Q662.415 1097.74 664.316 1090.18 Q666.216 1082.61 666.377 1082.45 Q666.634 1082.19 667.053 1082.19 Q667.826 1082.19 667.826 1082.93 Q667.762 1083.12 667.697 1083.38 Q667.311 1085.7 667.311 1086.96 Q667.311 1089.31 668.116 1091.11 Q668.921 1092.88 670.113 1093.91 Q671.336 1094.91 672.979 1095.55 Q674.621 1096.17 676.006 1096.39 Q677.391 1096.58 678.84 1096.58 Q681.964 1096.58 684.734 1094.78 Q687.536 1092.98 689.114 1090.18 Q690.724 1087.37 690.724 1084.44 Q690.724 1082.9 690.306 1081.71 Q689.887 1080.48 689.307 1079.81 Q688.76 1079.13 687.826 1078.61 Q686.892 1078.1 686.248 1077.91 Q685.636 1077.68 684.67 1077.45 L677.617 1075.59 Q674.782 1074.75 672.753 1072.33 Q670.757 1069.92 670.757 1066.21 Q670.757 1062.32 673.043 1058.71 Q675.362 1055.07 679.13 1052.88 Q682.93 1050.66 687.021 1050.66 Q690.241 1050.66 692.721 1051.92 Q695.201 1053.14 696.457 1055.43 L699.645 1051.66 Q700.289 1050.92 700.418 1050.79 Q700.579 1050.66 700.805 1050.66 Q701.191 1050.66 701.352 1050.88 Q701.545 1051.11 701.545 1051.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M708.205 1102.11 Q708.205 1095.32 709.919 1091.63 Q712.308 1086.13 717.944 1086.13 Q719.139 1086.13 720.379 1086.46 Q721.642 1086.78 723.22 1088.02 Q724.82 1089.26 725.79 1091.29 Q727.638 1095.21 727.638 1102.11 Q727.638 1108.85 725.925 1112.52 Q723.423 1117.89 717.899 1117.89 Q715.825 1117.89 713.706 1116.83 Q711.61 1115.77 710.279 1113.22 Q708.205 1109.41 708.205 1102.11 M712.038 1101.55 Q712.038 1108.51 712.534 1111.28 Q713.097 1114.28 714.63 1115.59 Q716.186 1116.88 717.899 1116.88 Q719.748 1116.88 721.281 1115.5 Q722.836 1114.1 723.31 1111.1 Q723.828 1108.15 723.806 1101.55 Q723.806 1095.12 723.355 1092.55 Q722.746 1089.55 721.123 1088.36 Q719.523 1087.14 717.899 1087.14 Q717.291 1087.14 716.637 1087.32 Q716.006 1087.5 715.081 1088.02 Q714.157 1088.54 713.436 1089.82 Q712.737 1091.11 712.399 1093.05 Q712.038 1095.55 712.038 1101.55 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 244.149,153.712 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 263.047,986.14 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,843.601 263.047,843.601 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,701.062 263.047,701.062 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,558.523 263.047,558.523 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,415.984 263.047,415.984 "/>
<polyline clip-path="url(#clip510)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,273.445 263.047,273.445 "/>
<path clip-path="url(#clip510)" d="M214.205 971.939 Q210.594 971.939 208.765 975.503 Q206.96 979.045 206.96 986.175 Q206.96 993.281 208.765 996.846 Q210.594 1000.39 214.205 1000.39 Q217.839 1000.39 219.645 996.846 Q221.473 993.281 221.473 986.175 Q221.473 979.045 219.645 975.503 Q217.839 971.939 214.205 971.939 M214.205 968.235 Q220.015 968.235 223.071 972.841 Q226.149 977.425 226.149 986.175 Q226.149 994.901 223.071 999.508 Q220.015 1004.09 214.205 1004.09 Q208.395 1004.09 205.316 999.508 Q202.261 994.901 202.261 986.175 Q202.261 977.425 205.316 972.841 Q208.395 968.235 214.205 968.235 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M179.066 856.946 L195.386 856.946 L195.386 860.881 L173.441 860.881 L173.441 856.946 Q176.103 854.191 180.687 849.561 Q185.293 844.909 186.474 843.566 Q188.719 841.043 189.599 839.307 Q190.501 837.548 190.501 835.858 Q190.501 833.103 188.557 831.367 Q186.636 829.631 183.534 829.631 Q181.335 829.631 178.881 830.395 Q176.451 831.159 173.673 832.71 L173.673 827.988 Q176.497 826.853 178.951 826.275 Q181.404 825.696 183.441 825.696 Q188.812 825.696 192.006 828.381 Q195.2 831.066 195.2 835.557 Q195.2 837.687 194.39 839.608 Q193.603 841.506 191.497 844.099 Q190.918 844.77 187.816 847.987 Q184.714 851.182 179.066 856.946 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M205.247 826.321 L223.603 826.321 L223.603 830.256 L209.529 830.256 L209.529 838.728 Q210.548 838.381 211.566 838.219 Q212.585 838.034 213.603 838.034 Q219.39 838.034 222.77 841.205 Q226.149 844.376 226.149 849.793 Q226.149 855.372 222.677 858.473 Q219.205 861.552 212.886 861.552 Q210.71 861.552 208.441 861.182 Q206.196 860.811 203.788 860.071 L203.788 855.372 Q205.872 856.506 208.094 857.061 Q210.316 857.617 212.793 857.617 Q216.798 857.617 219.135 855.511 Q221.473 853.404 221.473 849.793 Q221.473 846.182 219.135 844.075 Q216.798 841.969 212.793 841.969 Q210.918 841.969 209.043 842.386 Q207.191 842.802 205.247 843.682 L205.247 826.321 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M174.089 683.782 L192.446 683.782 L192.446 687.717 L178.372 687.717 L178.372 696.189 Q179.39 695.842 180.409 695.68 Q181.427 695.495 182.446 695.495 Q188.233 695.495 191.612 698.666 Q194.992 701.837 194.992 707.254 Q194.992 712.833 191.52 715.934 Q188.048 719.013 181.728 719.013 Q179.552 719.013 177.284 718.643 Q175.039 718.272 172.631 717.532 L172.631 712.833 Q174.714 713.967 176.937 714.522 Q179.159 715.078 181.636 715.078 Q185.64 715.078 187.978 712.972 Q190.316 710.865 190.316 707.254 Q190.316 703.643 187.978 701.536 Q185.64 699.43 181.636 699.43 Q179.761 699.43 177.886 699.847 Q176.034 700.263 174.089 701.143 L174.089 683.782 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M214.205 686.861 Q210.594 686.861 208.765 690.425 Q206.96 693.967 206.96 701.097 Q206.96 708.203 208.765 711.768 Q210.594 715.309 214.205 715.309 Q217.839 715.309 219.645 711.768 Q221.473 708.203 221.473 701.097 Q221.473 693.967 219.645 690.425 Q217.839 686.861 214.205 686.861 M214.205 683.157 Q220.015 683.157 223.071 687.763 Q226.149 692.347 226.149 701.097 Q226.149 709.823 223.071 714.43 Q220.015 719.013 214.205 719.013 Q208.395 719.013 205.316 714.43 Q202.261 709.823 202.261 701.097 Q202.261 692.347 205.316 687.763 Q208.395 683.157 214.205 683.157 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M173.858 541.243 L196.08 541.243 L196.08 543.234 L183.534 575.803 L178.65 575.803 L190.455 545.178 L173.858 545.178 L173.858 541.243 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M205.247 541.243 L223.603 541.243 L223.603 545.178 L209.529 545.178 L209.529 553.65 Q210.548 553.303 211.566 553.141 Q212.585 552.956 213.603 552.956 Q219.39 552.956 222.77 556.127 Q226.149 559.298 226.149 564.715 Q226.149 570.294 222.677 573.395 Q219.205 576.474 212.886 576.474 Q210.71 576.474 208.441 576.104 Q206.196 575.733 203.788 574.993 L203.788 570.294 Q205.872 571.428 208.094 571.983 Q210.316 572.539 212.793 572.539 Q216.798 572.539 219.135 570.433 Q221.473 568.326 221.473 564.715 Q221.473 561.104 219.135 558.997 Q216.798 556.891 212.793 556.891 Q210.918 556.891 209.043 557.308 Q207.191 557.724 205.247 558.604 L205.247 541.243 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M144.691 429.329 L152.33 429.329 L152.33 402.963 L144.02 404.63 L144.02 400.371 L152.284 398.704 L156.96 398.704 L156.96 429.329 L164.599 429.329 L164.599 433.264 L144.691 433.264 L144.691 429.329 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M184.043 401.783 Q180.432 401.783 178.603 405.347 Q176.798 408.889 176.798 416.019 Q176.798 423.125 178.603 426.69 Q180.432 430.231 184.043 430.231 Q187.677 430.231 189.483 426.69 Q191.312 423.125 191.312 416.019 Q191.312 408.889 189.483 405.347 Q187.677 401.783 184.043 401.783 M184.043 398.079 Q189.853 398.079 192.909 402.685 Q195.987 407.269 195.987 416.019 Q195.987 424.745 192.909 429.352 Q189.853 433.935 184.043 433.935 Q178.233 433.935 175.154 429.352 Q172.099 424.745 172.099 416.019 Q172.099 407.269 175.154 402.685 Q178.233 398.079 184.043 398.079 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M214.205 401.783 Q210.594 401.783 208.765 405.347 Q206.96 408.889 206.96 416.019 Q206.96 423.125 208.765 426.69 Q210.594 430.231 214.205 430.231 Q217.839 430.231 219.645 426.69 Q221.473 423.125 221.473 416.019 Q221.473 408.889 219.645 405.347 Q217.839 401.783 214.205 401.783 M214.205 398.079 Q220.015 398.079 223.071 402.685 Q226.149 407.269 226.149 416.019 Q226.149 424.745 223.071 429.352 Q220.015 433.935 214.205 433.935 Q208.395 433.935 205.316 429.352 Q202.261 424.745 202.261 416.019 Q202.261 407.269 205.316 402.685 Q208.395 398.079 214.205 398.079 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M145.687 286.79 L153.326 286.79 L153.326 260.424 L145.016 262.091 L145.016 257.832 L153.279 256.165 L157.955 256.165 L157.955 286.79 L165.594 286.79 L165.594 290.725 L145.687 290.725 L145.687 286.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M179.066 286.79 L195.386 286.79 L195.386 290.725 L173.441 290.725 L173.441 286.79 Q176.103 284.035 180.687 279.405 Q185.293 274.753 186.474 273.41 Q188.719 270.887 189.599 269.151 Q190.501 267.392 190.501 265.702 Q190.501 262.947 188.557 261.211 Q186.636 259.475 183.534 259.475 Q181.335 259.475 178.881 260.239 Q176.451 261.003 173.673 262.554 L173.673 257.832 Q176.497 256.697 178.951 256.119 Q181.404 255.54 183.441 255.54 Q188.812 255.54 192.006 258.225 Q195.2 260.91 195.2 265.401 Q195.2 267.531 194.39 269.452 Q193.603 271.35 191.497 273.943 Q190.918 274.614 187.816 277.831 Q184.714 281.026 179.066 286.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M205.247 256.165 L223.603 256.165 L223.603 260.1 L209.529 260.1 L209.529 268.572 Q210.548 268.225 211.566 268.063 Q212.585 267.878 213.603 267.878 Q219.39 267.878 222.77 271.049 Q226.149 274.22 226.149 279.637 Q226.149 285.216 222.677 288.317 Q219.205 291.396 212.886 291.396 Q210.71 291.396 208.441 291.026 Q206.196 290.655 203.788 289.915 L203.788 285.216 Q205.872 286.35 208.094 286.905 Q210.316 287.461 212.793 287.461 Q216.798 287.461 219.135 285.355 Q221.473 283.248 221.473 279.637 Q221.473 276.026 219.135 273.919 Q216.798 271.813 212.793 271.813 Q210.918 271.813 209.043 272.23 Q207.191 272.646 205.247 273.526 L205.247 256.165 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M67.9044 625.785 L74.6839 625.785 Q71.6602 629.032 70.1643 632.724 Q68.6683 636.384 68.6683 640.522 Q68.6683 648.67 73.6654 652.998 Q78.6307 657.327 88.0519 657.327 Q97.4413 657.327 102.438 652.998 Q107.404 648.67 107.404 640.522 Q107.404 636.384 105.908 632.724 Q104.412 629.032 101.388 625.785 L108.104 625.785 Q110.396 629.159 111.541 632.946 Q112.687 636.702 112.687 640.904 Q112.687 651.693 106.099 657.9 Q99.4783 664.107 88.0519 664.107 Q76.5936 664.107 70.0051 657.9 Q63.3848 651.693 63.3848 640.904 Q63.3848 636.639 64.5306 632.883 Q65.6446 629.095 67.9044 625.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M80.2221 602.296 Q80.2221 607.006 83.9142 609.743 Q87.5745 612.481 93.972 612.481 Q100.37 612.481 104.062 609.775 Q107.722 607.038 107.722 602.296 Q107.722 597.617 104.03 594.88 Q100.338 592.142 93.972 592.142 Q87.6381 592.142 83.946 594.88 Q80.2221 597.617 80.2221 602.296 M75.2568 602.296 Q75.2568 594.657 80.2221 590.296 Q85.1873 585.936 93.972 585.936 Q102.725 585.936 107.722 590.296 Q112.687 594.657 112.687 602.296 Q112.687 609.966 107.722 614.327 Q102.725 618.655 93.972 618.655 Q85.1873 618.655 80.2221 614.327 Q75.2568 609.966 75.2568 602.296 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M97.6959 576.833 L76.1162 576.833 L76.1162 570.976 L97.4731 570.976 Q102.534 570.976 105.08 569.003 Q107.595 567.03 107.595 563.083 Q107.595 558.34 104.571 555.603 Q101.547 552.834 96.3273 552.834 L76.1162 552.834 L76.1162 546.978 L111.764 546.978 L111.764 552.834 L106.29 552.834 Q109.536 554.967 111.128 557.799 Q112.687 560.6 112.687 564.324 Q112.687 570.467 108.868 573.65 Q105.048 576.833 97.6959 576.833 M75.2568 562.096 L75.2568 562.096 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M90.2481 505.282 L111.764 505.282 L111.764 511.139 L90.439 511.139 Q85.3783 511.139 82.8638 513.112 Q80.3494 515.085 80.3494 519.032 Q80.3494 523.775 83.3731 526.512 Q86.3968 529.249 91.6167 529.249 L111.764 529.249 L111.764 535.137 L76.1162 535.137 L76.1162 529.249 L81.6544 529.249 Q78.4397 527.148 76.8483 524.316 Q75.2568 521.451 75.2568 517.727 Q75.2568 511.584 79.0763 508.433 Q82.8638 505.282 90.2481 505.282 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M65.9947 487.808 L76.1162 487.808 L76.1162 475.745 L80.6677 475.745 L80.6677 487.808 L100.019 487.808 Q104.38 487.808 105.621 486.631 Q106.863 485.421 106.863 481.761 L106.863 475.745 L111.764 475.745 L111.764 481.761 Q111.764 488.54 109.25 491.119 Q106.703 493.697 100.019 493.697 L80.6677 493.697 L80.6677 497.993 L76.1162 497.993 L76.1162 493.697 L65.9947 493.697 L65.9947 487.808 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M702.383 12.2341 L697.833 30.5973 Q697.588 31.704 697.383 31.9909 Q697.178 32.2779 696.604 32.2779 Q696.153 32.2779 695.866 32.0729 Q695.62 31.868 695.62 31.663 L695.579 31.4581 Q695.579 31.3351 695.661 30.7203 Q695.743 30.0644 695.825 28.9987 Q695.907 27.933 695.907 26.7443 Q695.907 20.0221 692.628 16.9069 Q689.39 13.7507 683.815 13.7507 Q679.962 13.7507 676.519 15.8002 Q673.076 17.8496 671.108 21.0878 Q669.141 24.2849 669.141 27.6461 Q669.141 30.7203 670.699 32.6878 Q672.256 34.6552 674.265 35.3111 L679.88 36.8277 Q685.783 38.3443 687.463 39.0001 Q689.144 39.6149 690.537 41.0086 Q694.677 44.8616 694.677 50.846 Q694.677 56.0107 691.767 61.0113 Q688.857 65.971 683.856 69.2092 Q678.855 72.4064 673.322 72.4064 Q663.607 72.4064 659.14 66.3809 L655.041 71.1357 Q654.262 72.0375 654.016 72.2014 Q653.811 72.4064 653.524 72.4064 Q653.155 72.4064 652.868 72.1604 Q652.581 71.9555 652.581 71.5866 Q652.581 71.2997 655 61.6672 Q657.418 52.0347 657.623 51.8297 Q657.951 51.5018 658.484 51.5018 Q659.467 51.5018 659.467 52.4446 Q659.386 52.6905 659.304 53.0184 Q658.812 55.9697 658.812 57.5682 Q658.812 60.5605 659.836 62.8559 Q660.861 65.1103 662.378 66.4219 Q663.935 67.6926 666.026 68.5124 Q668.116 69.2912 669.879 69.5781 Q671.641 69.824 673.486 69.824 Q677.462 69.824 680.987 67.5286 Q684.553 65.2332 686.561 61.6672 Q688.611 58.1011 688.611 54.3711 Q688.611 52.4036 688.078 50.887 Q687.545 49.3294 686.807 48.4686 Q686.111 47.6078 684.922 46.952 Q683.733 46.2962 682.913 46.0503 Q682.135 45.7633 680.905 45.4764 L671.928 43.099 Q668.321 42.0333 665.739 38.9591 Q663.198 35.8849 663.198 31.1711 Q663.198 26.2114 666.108 21.6206 Q669.059 16.9889 673.855 14.2016 Q678.691 11.3733 683.897 11.3733 Q687.996 11.3733 691.152 12.9719 Q694.308 14.5295 695.907 17.4397 L699.965 12.644 Q700.785 11.7012 700.949 11.5373 Q701.154 11.3733 701.441 11.3733 Q701.932 11.3733 702.137 11.6603 Q702.383 11.9472 702.383 12.2341 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip510)" d="M710.869 76.8539 Q710.869 68.2174 713.05 63.5119 Q716.091 56.5109 723.265 56.5109 Q724.785 56.5109 726.363 56.9413 Q727.97 57.343 729.979 58.9211 Q732.016 60.4992 733.25 63.0815 Q735.602 68.074 735.602 76.8539 Q735.602 85.4329 733.422 90.1098 Q730.237 96.9386 723.207 96.9386 Q720.567 96.9386 717.87 95.5901 Q715.202 94.2415 713.509 90.9993 Q710.869 86.1503 710.869 76.8539 M715.747 76.1366 Q715.747 85.0026 716.378 88.5317 Q717.096 92.3478 719.047 94.012 Q721.027 95.6475 723.207 95.6475 Q725.56 95.6475 727.511 93.8972 Q729.491 92.1183 730.093 88.3022 Q730.753 84.5435 730.725 76.1366 Q730.725 67.9592 730.151 64.6883 Q729.376 60.8722 727.31 59.3515 Q725.273 57.8021 723.207 57.8021 Q722.432 57.8021 721.6 58.0316 Q720.797 58.2611 719.621 58.9211 Q718.444 59.581 717.526 61.2165 Q716.637 62.8519 716.206 65.3195 Q715.747 68.5044 715.747 76.1366 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip512)" d="M294.124 980.438 L294.124 986.14 L334.557 986.14 L334.557 980.438 L294.124 980.438 L294.124 980.438  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.124,980.438 294.124,986.14 334.557,986.14 334.557,980.438 294.124,980.438 "/>
<path clip-path="url(#clip512)" d="M334.557 980.438 L334.557 986.14 L374.99 986.14 L374.99 980.438 L334.557 980.438 L334.557 980.438  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="334.557,980.438 334.557,986.14 374.99,986.14 374.99,980.438 334.557,980.438 "/>
<path clip-path="url(#clip512)" d="M374.99 963.334 L374.99 986.14 L415.423 986.14 L415.423 963.334 L374.99 963.334 L374.99 963.334  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="374.99,963.334 374.99,986.14 415.423,986.14 415.423,963.334 374.99,963.334 "/>
<path clip-path="url(#clip512)" d="M415.423 946.229 L415.423 986.14 L455.856 986.14 L455.856 946.229 L415.423 946.229 L415.423 946.229  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="415.423,946.229 415.423,986.14 455.856,986.14 455.856,946.229 415.423,946.229 "/>
<path clip-path="url(#clip512)" d="M455.856 866.407 L455.856 986.14 L496.288 986.14 L496.288 866.407 L455.856 866.407 L455.856 866.407  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="455.856,866.407 455.856,986.14 496.288,986.14 496.288,866.407 455.856,866.407 "/>
<path clip-path="url(#clip512)" d="M496.288 701.062 L496.288 986.14 L536.721 986.14 L536.721 701.062 L496.288 701.062 L496.288 701.062  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="496.288,701.062 496.288,986.14 536.721,986.14 536.721,701.062 496.288,701.062 "/>
<path clip-path="url(#clip512)" d="M536.721 592.732 L536.721 986.14 L577.154 986.14 L577.154 592.732 L536.721 592.732 L536.721 592.732  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="536.721,592.732 536.721,986.14 577.154,986.14 577.154,592.732 536.721,592.732 "/>
<path clip-path="url(#clip512)" d="M577.154 450.193 L577.154 986.14 L617.587 986.14 L617.587 450.193 L577.154 450.193 L577.154 450.193  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="577.154,450.193 577.154,986.14 617.587,986.14 617.587,450.193 577.154,450.193 "/>
<path clip-path="url(#clip512)" d="M617.587 296.251 L617.587 986.14 L658.02 986.14 L658.02 296.251 L617.587 296.251 L617.587 296.251  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="617.587,296.251 617.587,986.14 658.02,986.14 658.02,296.251 617.587,296.251 "/>
<path clip-path="url(#clip512)" d="M658.02 176.518 L658.02 986.14 L698.453 986.14 L698.453 176.518 L658.02 176.518 L658.02 176.518  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="658.02,176.518 658.02,986.14 698.453,986.14 698.453,176.518 658.02,176.518 "/>
<path clip-path="url(#clip512)" d="M698.453 153.712 L698.453 986.14 L738.885 986.14 L738.885 153.712 L698.453 153.712 L698.453 153.712  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="698.453,153.712 698.453,986.14 738.885,986.14 738.885,153.712 698.453,153.712 "/>
<path clip-path="url(#clip512)" d="M738.885 256.34 L738.885 986.14 L779.318 986.14 L779.318 256.34 L738.885 256.34 L738.885 256.34  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="738.885,256.34 738.885,986.14 779.318,986.14 779.318,256.34 738.885,256.34 "/>
<path clip-path="url(#clip512)" d="M779.318 353.267 L779.318 986.14 L819.751 986.14 L819.751 353.267 L779.318 353.267 L779.318 353.267  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="779.318,353.267 779.318,986.14 819.751,986.14 819.751,353.267 779.318,353.267 "/>
<path clip-path="url(#clip512)" d="M819.751 729.57 L819.751 986.14 L860.184 986.14 L860.184 729.57 L819.751 729.57 L819.751 729.57  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="819.751,729.57 819.751,986.14 860.184,986.14 860.184,729.57 819.751,729.57 "/>
<path clip-path="url(#clip512)" d="M860.184 786.585 L860.184 986.14 L900.617 986.14 L900.617 786.585 L860.184 786.585 L860.184 786.585  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="860.184,786.585 860.184,986.14 900.617,986.14 900.617,786.585 860.184,786.585 "/>
<path clip-path="url(#clip512)" d="M900.617 889.213 L900.617 986.14 L941.05 986.14 L941.05 889.213 L900.617 889.213 L900.617 889.213  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="900.617,889.213 900.617,986.14 941.05,986.14 941.05,889.213 900.617,889.213 "/>
<path clip-path="url(#clip512)" d="M941.05 957.632 L941.05 986.14 L981.482 986.14 L981.482 957.632 L941.05 957.632 L941.05 957.632  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="941.05,957.632 941.05,986.14 981.482,986.14 981.482,957.632 941.05,957.632 "/>
<path clip-path="url(#clip512)" d="M981.482 974.737 L981.482 986.14 L1021.92 986.14 L1021.92 974.737 L981.482 974.737 L981.482 974.737  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="981.482,974.737 981.482,986.14 1021.92,986.14 1021.92,974.737 981.482,974.737 "/>
<path clip-path="url(#clip512)" d="M1021.92 986.14 L1021.92 986.14 L1062.35 986.14 L1062.35 986.14 L1021.92 986.14 L1021.92 986.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1021.92,986.14 1021.92,986.14 1062.35,986.14 1021.92,986.14 "/>
<path clip-path="url(#clip512)" d="M1062.35 980.438 L1062.35 986.14 L1102.78 986.14 L1102.78 980.438 L1062.35 980.438 L1062.35 980.438  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip512)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1062.35,980.438 1062.35,986.14 1102.78,986.14 1102.78,980.438 1062.35,980.438 "/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="314.341" cy="980.438" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="354.774" cy="980.438" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="395.206" cy="963.334" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="435.639" cy="946.229" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="476.072" cy="866.407" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="516.505" cy="701.062" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="556.938" cy="592.732" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="597.371" cy="450.193" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="637.803" cy="296.251" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="678.236" cy="176.518" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="718.669" cy="153.712" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="759.102" cy="256.34" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="799.535" cy="353.267" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="839.968" cy="729.57" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="880.4" cy="786.585" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="920.833" cy="889.213" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="961.266" cy="957.632" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1001.7" cy="974.737" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1042.13" cy="986.14" r="2"/>
<circle clip-path="url(#clip512)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1082.56" cy="980.438" r="2"/>
<polyline clip-path="url(#clip512)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="690.861,1818.57 690.861,-678.716 "/>
</svg>
<p>(d)</p>
</div>
</div></td>
</tr>
</tbody>
</table>

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<tbody>
<tr>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="300" height="300" viewBox="0 0 1200 1200">
<defs>
  <clipPath id="clip570">
    <rect x="0" y="0" width="1200" height="1200"/>
  </clipPath>
</defs>
<path clip-path="url(#clip570)" d="M0 1200 L1200 1200 L1200 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip571">
    <rect x="240" y="119" width="841" height="841"/>
  </clipPath>
</defs>
<path clip-path="url(#clip570)" d="M244.149 986.14 L1152.76 986.14 L1152.76 153.712 L244.149 153.712  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip572">
    <rect x="244" y="153" width="910" height="833"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="294.124,986.14 294.124,153.712 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="462.594,986.14 462.594,153.712 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="631.065,986.14 631.065,153.712 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="799.535,986.14 799.535,153.712 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="968.005,986.14 968.005,153.712 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1136.47,986.14 1136.47,153.712 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,986.14 1152.76,986.14 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,793.448 1152.76,793.448 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,600.757 1152.76,600.757 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,408.065 1152.76,408.065 "/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,215.373 1152.76,215.373 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 1152.76,986.14 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.124,986.14 294.124,967.242 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="462.594,986.14 462.594,967.242 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="631.065,986.14 631.065,967.242 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="799.535,986.14 799.535,967.242 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="968.005,986.14 968.005,967.242 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1136.47,986.14 1136.47,967.242 "/>
<path clip-path="url(#clip570)" d="M272.307 1007.46 Q268.696 1007.46 266.868 1011.02 Q265.062 1014.56 265.062 1021.69 Q265.062 1028.8 266.868 1032.37 Q268.696 1035.91 272.307 1035.91 Q275.942 1035.91 277.747 1032.37 Q279.576 1028.8 279.576 1021.69 Q279.576 1014.56 277.747 1011.02 Q275.942 1007.46 272.307 1007.46 M272.307 1003.75 Q278.117 1003.75 281.173 1008.36 Q284.252 1012.94 284.252 1021.69 Q284.252 1030.42 281.173 1035.03 Q278.117 1039.61 272.307 1039.61 Q266.497 1039.61 263.418 1035.03 Q260.363 1030.42 260.363 1021.69 Q260.363 1012.94 263.418 1008.36 Q266.497 1003.75 272.307 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M292.469 1033.06 L297.353 1033.06 L297.353 1038.94 L292.469 1038.94 L292.469 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M311.566 1035 L327.886 1035 L327.886 1038.94 L305.941 1038.94 L305.941 1035 Q308.603 1032.25 313.187 1027.62 Q317.793 1022.97 318.974 1021.63 Q321.219 1019.1 322.099 1017.37 Q323.001 1015.61 323.001 1013.92 Q323.001 1011.16 321.057 1009.43 Q319.136 1007.69 316.034 1007.69 Q313.835 1007.69 311.381 1008.45 Q308.951 1009.22 306.173 1010.77 L306.173 1006.05 Q308.997 1004.91 311.451 1004.33 Q313.904 1003.75 315.941 1003.75 Q321.312 1003.75 324.506 1006.44 Q327.701 1009.13 327.701 1013.62 Q327.701 1015.75 326.89 1017.67 Q326.103 1019.56 323.997 1022.16 Q323.418 1022.83 320.316 1026.05 Q317.214 1029.24 311.566 1035 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M440.303 1007.46 Q436.692 1007.46 434.863 1011.02 Q433.058 1014.56 433.058 1021.69 Q433.058 1028.8 434.863 1032.37 Q436.692 1035.91 440.303 1035.91 Q443.937 1035.91 445.743 1032.37 Q447.571 1028.8 447.571 1021.69 Q447.571 1014.56 445.743 1011.02 Q443.937 1007.46 440.303 1007.46 M440.303 1003.75 Q446.113 1003.75 449.169 1008.36 Q452.247 1012.94 452.247 1021.69 Q452.247 1030.42 449.169 1035.03 Q446.113 1039.61 440.303 1039.61 Q434.493 1039.61 431.414 1035.03 Q428.359 1030.42 428.359 1021.69 Q428.359 1012.94 431.414 1008.36 Q434.493 1003.75 440.303 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M460.465 1033.06 L465.349 1033.06 L465.349 1038.94 L460.465 1038.94 L460.465 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M489.701 1020.31 Q493.057 1021.02 494.932 1023.29 Q496.83 1025.56 496.83 1028.89 Q496.83 1034.01 493.312 1036.81 Q489.793 1039.61 483.312 1039.61 Q481.136 1039.61 478.821 1039.17 Q476.53 1038.75 474.076 1037.9 L474.076 1033.38 Q476.02 1034.52 478.335 1035.1 Q480.65 1035.68 483.173 1035.68 Q487.571 1035.68 489.863 1033.94 Q492.178 1032.2 492.178 1028.89 Q492.178 1025.84 490.025 1024.13 Q487.895 1022.39 484.076 1022.39 L480.048 1022.39 L480.048 1018.55 L484.261 1018.55 Q487.71 1018.55 489.539 1017.18 Q491.367 1015.79 491.367 1013.2 Q491.367 1010.54 489.469 1009.13 Q487.594 1007.69 484.076 1007.69 Q482.155 1007.69 479.955 1008.11 Q477.756 1008.52 475.118 1009.4 L475.118 1005.24 Q477.78 1004.5 480.094 1004.13 Q482.432 1003.75 484.492 1003.75 Q489.817 1003.75 492.918 1006.19 Q496.02 1008.59 496.02 1012.71 Q496.02 1015.58 494.377 1017.57 Q492.733 1019.54 489.701 1020.31 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M608.206 1007.46 Q604.595 1007.46 602.766 1011.02 Q600.961 1014.56 600.961 1021.69 Q600.961 1028.8 602.766 1032.37 Q604.595 1035.91 608.206 1035.91 Q611.84 1035.91 613.646 1032.37 Q615.474 1028.8 615.474 1021.69 Q615.474 1014.56 613.646 1011.02 Q611.84 1007.46 608.206 1007.46 M608.206 1003.75 Q614.016 1003.75 617.072 1008.36 Q620.15 1012.94 620.15 1021.69 Q620.15 1030.42 617.072 1035.03 Q614.016 1039.61 608.206 1039.61 Q602.396 1039.61 599.317 1035.03 Q596.262 1030.42 596.262 1021.69 Q596.262 1012.94 599.317 1008.36 Q602.396 1003.75 608.206 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M628.368 1033.06 L633.252 1033.06 L633.252 1038.94 L628.368 1038.94 L628.368 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M656.284 1008.45 L644.479 1026.9 L656.284 1026.9 L656.284 1008.45 M655.058 1004.38 L660.937 1004.38 L660.937 1026.9 L665.868 1026.9 L665.868 1030.79 L660.937 1030.79 L660.937 1038.94 L656.284 1038.94 L656.284 1030.79 L640.683 1030.79 L640.683 1026.28 L655.058 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M777.417 1007.46 Q773.806 1007.46 771.977 1011.02 Q770.171 1014.56 770.171 1021.69 Q770.171 1028.8 771.977 1032.37 Q773.806 1035.91 777.417 1035.91 Q781.051 1035.91 782.857 1032.37 Q784.685 1028.8 784.685 1021.69 Q784.685 1014.56 782.857 1011.02 Q781.051 1007.46 777.417 1007.46 M777.417 1003.75 Q783.227 1003.75 786.282 1008.36 Q789.361 1012.94 789.361 1021.69 Q789.361 1030.42 786.282 1035.03 Q783.227 1039.61 777.417 1039.61 Q771.607 1039.61 768.528 1035.03 Q765.472 1030.42 765.472 1021.69 Q765.472 1012.94 768.528 1008.36 Q771.607 1003.75 777.417 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M797.579 1033.06 L802.463 1033.06 L802.463 1038.94 L797.579 1038.94 L797.579 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M812.694 1004.38 L831.051 1004.38 L831.051 1008.32 L816.977 1008.32 L816.977 1016.79 Q817.995 1016.44 819.014 1016.28 Q820.032 1016.09 821.051 1016.09 Q826.838 1016.09 830.217 1019.26 Q833.597 1022.44 833.597 1027.85 Q833.597 1033.43 830.125 1036.53 Q826.653 1039.61 820.333 1039.61 Q818.157 1039.61 815.889 1039.24 Q813.643 1038.87 811.236 1038.13 L811.236 1033.43 Q813.319 1034.56 815.542 1035.12 Q817.764 1035.68 820.241 1035.68 Q824.245 1035.68 826.583 1033.57 Q828.921 1031.46 828.921 1027.85 Q828.921 1024.24 826.583 1022.13 Q824.245 1020.03 820.241 1020.03 Q818.366 1020.03 816.491 1020.44 Q814.639 1020.86 812.694 1021.74 L812.694 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M945.308 1007.46 Q941.697 1007.46 939.868 1011.02 Q938.063 1014.56 938.063 1021.69 Q938.063 1028.8 939.868 1032.37 Q941.697 1035.91 945.308 1035.91 Q948.942 1035.91 950.748 1032.37 Q952.577 1028.8 952.577 1021.69 Q952.577 1014.56 950.748 1011.02 Q948.942 1007.46 945.308 1007.46 M945.308 1003.75 Q951.118 1003.75 954.174 1008.36 Q957.253 1012.94 957.253 1021.69 Q957.253 1030.42 954.174 1035.03 Q951.118 1039.61 945.308 1039.61 Q939.498 1039.61 936.419 1035.03 Q933.364 1030.42 933.364 1021.69 Q933.364 1012.94 936.419 1008.36 Q939.498 1003.75 945.308 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M965.47 1033.06 L970.354 1033.06 L970.354 1038.94 L965.47 1038.94 L965.47 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M991.118 1019.8 Q987.97 1019.8 986.118 1021.95 Q984.289 1024.1 984.289 1027.85 Q984.289 1031.58 986.118 1033.75 Q987.97 1035.91 991.118 1035.91 Q994.266 1035.91 996.095 1033.75 Q997.947 1031.58 997.947 1027.85 Q997.947 1024.1 996.095 1021.95 Q994.266 1019.8 991.118 1019.8 M1000.4 1005.14 L1000.4 1009.4 Q998.641 1008.57 996.836 1008.13 Q995.053 1007.69 993.294 1007.69 Q988.664 1007.69 986.211 1010.81 Q983.78 1013.94 983.433 1020.26 Q984.799 1018.25 986.859 1017.18 Q988.919 1016.09 991.396 1016.09 Q996.604 1016.09 999.613 1019.26 Q1002.65 1022.41 1002.65 1027.85 Q1002.65 1033.18 999.498 1036.39 Q996.35 1039.61 991.118 1039.61 Q985.123 1039.61 981.952 1035.03 Q978.78 1030.42 978.78 1021.69 Q978.78 1013.5 982.669 1008.64 Q986.558 1003.75 993.109 1003.75 Q994.868 1003.75 996.651 1004.1 Q998.456 1004.45 1000.4 1005.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M1114.31 1007.46 Q1110.7 1007.46 1108.87 1011.02 Q1107.07 1014.56 1107.07 1021.69 Q1107.07 1028.8 1108.87 1032.37 Q1110.7 1035.91 1114.31 1035.91 Q1117.94 1035.91 1119.75 1032.37 Q1121.58 1028.8 1121.58 1021.69 Q1121.58 1014.56 1119.75 1011.02 Q1117.94 1007.46 1114.31 1007.46 M1114.31 1003.75 Q1120.12 1003.75 1123.18 1008.36 Q1126.26 1012.94 1126.26 1021.69 Q1126.26 1030.42 1123.18 1035.03 Q1120.12 1039.61 1114.31 1039.61 Q1108.5 1039.61 1105.42 1035.03 Q1102.37 1030.42 1102.37 1021.69 Q1102.37 1012.94 1105.42 1008.36 Q1108.5 1003.75 1114.31 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M1134.47 1033.06 L1139.36 1033.06 L1139.36 1038.94 L1134.47 1038.94 L1134.47 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M1148.36 1004.38 L1170.58 1004.38 L1170.58 1006.37 L1158.04 1038.94 L1153.15 1038.94 L1164.96 1008.32 L1148.36 1008.32 L1148.36 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M712.721 1073.49 Q712.721 1078.25 710.274 1082.73 Q707.858 1087.2 704.058 1089.91 Q700.29 1092.62 696.49 1092.62 Q692.206 1092.62 689.984 1088.17 Q685.958 1104.31 685.733 1104.72 Q684.799 1106.11 683.446 1106.11 Q682.609 1106.11 682.094 1105.59 Q681.578 1105.11 681.578 1104.34 Q681.611 1104.08 681.804 1103.28 L688.245 1077.35 Q689.726 1071.33 694.203 1067.04 Q698.712 1062.73 703.543 1062.73 Q707.375 1062.73 710.048 1065.59 Q712.721 1068.46 712.721 1073.49 M707.987 1070.65 Q707.987 1067.43 706.699 1065.82 Q705.411 1064.18 703.414 1064.18 Q701.578 1064.18 699.356 1065.66 Q697.166 1067.14 695.234 1070.43 Q694.203 1072.33 693.462 1074.68 Q692.722 1077.03 691.272 1082.82 Q690.757 1085.14 690.757 1085.27 Q690.757 1085.53 690.886 1086.11 Q691.047 1086.69 691.433 1087.59 Q691.852 1088.46 692.432 1089.23 Q693.011 1090.01 694.074 1090.59 Q695.137 1091.13 696.425 1091.13 Q698.422 1091.13 700.548 1089.52 Q702.705 1087.88 704.283 1084.89 Q705.604 1082.47 706.796 1077.93 Q707.987 1073.36 707.987 1070.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 244.149,153.712 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 263.047,986.14 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,793.448 263.047,793.448 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,600.757 263.047,600.757 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,408.065 263.047,408.065 "/>
<polyline clip-path="url(#clip570)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,215.373 263.047,215.373 "/>
<path clip-path="url(#clip570)" d="M214.205 971.939 Q210.594 971.939 208.765 975.503 Q206.96 979.045 206.96 986.175 Q206.96 993.281 208.765 996.846 Q210.594 1000.39 214.205 1000.39 Q217.839 1000.39 219.645 996.846 Q221.473 993.281 221.473 986.175 Q221.473 979.045 219.645 975.503 Q217.839 971.939 214.205 971.939 M214.205 968.235 Q220.015 968.235 223.071 972.841 Q226.149 977.425 226.149 986.175 Q226.149 994.901 223.071 999.508 Q220.015 1004.09 214.205 1004.09 Q208.395 1004.09 205.316 999.508 Q202.261 994.901 202.261 986.175 Q202.261 977.425 205.316 972.841 Q208.395 968.235 214.205 968.235 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M179.066 806.793 L195.386 806.793 L195.386 810.728 L173.441 810.728 L173.441 806.793 Q176.103 804.038 180.687 799.409 Q185.293 794.756 186.474 793.414 Q188.719 790.89 189.599 789.154 Q190.501 787.395 190.501 785.705 Q190.501 782.951 188.557 781.215 Q186.636 779.478 183.534 779.478 Q181.335 779.478 178.881 780.242 Q176.451 781.006 173.673 782.557 L173.673 777.835 Q176.497 776.701 178.951 776.122 Q181.404 775.543 183.441 775.543 Q188.812 775.543 192.006 778.228 Q195.2 780.914 195.2 785.404 Q195.2 787.534 194.39 789.455 Q193.603 791.353 191.497 793.946 Q190.918 794.617 187.816 797.835 Q184.714 801.029 179.066 806.793 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M205.247 776.168 L223.603 776.168 L223.603 780.103 L209.529 780.103 L209.529 788.576 Q210.548 788.228 211.566 788.066 Q212.585 787.881 213.603 787.881 Q219.39 787.881 222.77 791.052 Q226.149 794.224 226.149 799.64 Q226.149 805.219 222.677 808.321 Q219.205 811.4 212.886 811.4 Q210.71 811.4 208.441 811.029 Q206.196 810.659 203.788 809.918 L203.788 805.219 Q205.872 806.353 208.094 806.909 Q210.316 807.464 212.793 807.464 Q216.798 807.464 219.135 805.358 Q221.473 803.251 221.473 799.64 Q221.473 796.029 219.135 793.923 Q216.798 791.816 212.793 791.816 Q210.918 791.816 209.043 792.233 Q207.191 792.65 205.247 793.529 L205.247 776.168 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M174.089 583.477 L192.446 583.477 L192.446 587.412 L178.372 587.412 L178.372 595.884 Q179.39 595.537 180.409 595.375 Q181.427 595.19 182.446 595.19 Q188.233 595.19 191.612 598.361 Q194.992 601.532 194.992 606.949 Q194.992 612.527 191.52 615.629 Q188.048 618.708 181.728 618.708 Q179.552 618.708 177.284 618.338 Q175.039 617.967 172.631 617.226 L172.631 612.527 Q174.714 613.662 176.937 614.217 Q179.159 614.773 181.636 614.773 Q185.64 614.773 187.978 612.666 Q190.316 610.56 190.316 606.949 Q190.316 603.338 187.978 601.231 Q185.64 599.125 181.636 599.125 Q179.761 599.125 177.886 599.541 Q176.034 599.958 174.089 600.838 L174.089 583.477 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M214.205 586.555 Q210.594 586.555 208.765 590.12 Q206.96 593.662 206.96 600.791 Q206.96 607.898 208.765 611.463 Q210.594 615.004 214.205 615.004 Q217.839 615.004 219.645 611.463 Q221.473 607.898 221.473 600.791 Q221.473 593.662 219.645 590.12 Q217.839 586.555 214.205 586.555 M214.205 582.852 Q220.015 582.852 223.071 587.458 Q226.149 592.041 226.149 600.791 Q226.149 609.518 223.071 614.125 Q220.015 618.708 214.205 618.708 Q208.395 618.708 205.316 614.125 Q202.261 609.518 202.261 600.791 Q202.261 592.041 205.316 587.458 Q208.395 582.852 214.205 582.852 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M173.858 390.785 L196.08 390.785 L196.08 392.776 L183.534 425.345 L178.65 425.345 L190.455 394.72 L173.858 394.72 L173.858 390.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M205.247 390.785 L223.603 390.785 L223.603 394.72 L209.529 394.72 L209.529 403.192 Q210.548 402.845 211.566 402.683 Q212.585 402.498 213.603 402.498 Q219.39 402.498 222.77 405.669 Q226.149 408.84 226.149 414.257 Q226.149 419.836 222.677 422.938 Q219.205 426.016 212.886 426.016 Q210.71 426.016 208.441 425.646 Q206.196 425.276 203.788 424.535 L203.788 419.836 Q205.872 420.97 208.094 421.526 Q210.316 422.081 212.793 422.081 Q216.798 422.081 219.135 419.975 Q221.473 417.868 221.473 414.257 Q221.473 410.646 219.135 408.54 Q216.798 406.433 212.793 406.433 Q210.918 406.433 209.043 406.85 Q207.191 407.266 205.247 408.146 L205.247 390.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M144.691 228.718 L152.33 228.718 L152.33 202.353 L144.02 204.019 L144.02 199.76 L152.284 198.093 L156.96 198.093 L156.96 228.718 L164.599 228.718 L164.599 232.653 L144.691 232.653 L144.691 228.718 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M184.043 201.172 Q180.432 201.172 178.603 204.737 Q176.798 208.279 176.798 215.408 Q176.798 222.515 178.603 226.079 Q180.432 229.621 184.043 229.621 Q187.677 229.621 189.483 226.079 Q191.312 222.515 191.312 215.408 Q191.312 208.279 189.483 204.737 Q187.677 201.172 184.043 201.172 M184.043 197.468 Q189.853 197.468 192.909 202.075 Q195.987 206.658 195.987 215.408 Q195.987 224.135 192.909 228.741 Q189.853 233.325 184.043 233.325 Q178.233 233.325 175.154 228.741 Q172.099 224.135 172.099 215.408 Q172.099 206.658 175.154 202.075 Q178.233 197.468 184.043 197.468 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M214.205 201.172 Q210.594 201.172 208.765 204.737 Q206.96 208.279 206.96 215.408 Q206.96 222.515 208.765 226.079 Q210.594 229.621 214.205 229.621 Q217.839 229.621 219.645 226.079 Q221.473 222.515 221.473 215.408 Q221.473 208.279 219.645 204.737 Q217.839 201.172 214.205 201.172 M214.205 197.468 Q220.015 197.468 223.071 202.075 Q226.149 206.658 226.149 215.408 Q226.149 224.135 223.071 228.741 Q220.015 233.325 214.205 233.325 Q208.395 233.325 205.316 228.741 Q202.261 224.135 202.261 215.408 Q202.261 206.658 205.316 202.075 Q208.395 197.468 214.205 197.468 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M67.9044 625.785 L74.6839 625.785 Q71.6602 629.032 70.1643 632.724 Q68.6683 636.384 68.6683 640.522 Q68.6683 648.67 73.6654 652.998 Q78.6307 657.327 88.0519 657.327 Q97.4413 657.327 102.438 652.998 Q107.404 648.67 107.404 640.522 Q107.404 636.384 105.908 632.724 Q104.412 629.032 101.388 625.785 L108.104 625.785 Q110.396 629.159 111.541 632.946 Q112.687 636.702 112.687 640.904 Q112.687 651.693 106.099 657.9 Q99.4783 664.107 88.0519 664.107 Q76.5936 664.107 70.0051 657.9 Q63.3848 651.693 63.3848 640.904 Q63.3848 636.639 64.5306 632.883 Q65.6446 629.095 67.9044 625.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M80.2221 602.296 Q80.2221 607.006 83.9142 609.743 Q87.5745 612.481 93.972 612.481 Q100.37 612.481 104.062 609.775 Q107.722 607.038 107.722 602.296 Q107.722 597.617 104.03 594.88 Q100.338 592.142 93.972 592.142 Q87.6381 592.142 83.946 594.88 Q80.2221 597.617 80.2221 602.296 M75.2568 602.296 Q75.2568 594.657 80.2221 590.296 Q85.1873 585.936 93.972 585.936 Q102.725 585.936 107.722 590.296 Q112.687 594.657 112.687 602.296 Q112.687 609.966 107.722 614.327 Q102.725 618.655 93.972 618.655 Q85.1873 618.655 80.2221 614.327 Q75.2568 609.966 75.2568 602.296 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M97.6959 576.833 L76.1162 576.833 L76.1162 570.976 L97.4731 570.976 Q102.534 570.976 105.08 569.003 Q107.595 567.03 107.595 563.083 Q107.595 558.34 104.571 555.603 Q101.547 552.834 96.3273 552.834 L76.1162 552.834 L76.1162 546.978 L111.764 546.978 L111.764 552.834 L106.29 552.834 Q109.536 554.967 111.128 557.799 Q112.687 560.6 112.687 564.324 Q112.687 570.467 108.868 573.65 Q105.048 576.833 97.6959 576.833 M75.2568 562.096 L75.2568 562.096 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M90.2481 505.282 L111.764 505.282 L111.764 511.139 L90.439 511.139 Q85.3783 511.139 82.8638 513.112 Q80.3494 515.085 80.3494 519.032 Q80.3494 523.775 83.3731 526.512 Q86.3968 529.249 91.6167 529.249 L111.764 529.249 L111.764 535.137 L76.1162 535.137 L76.1162 529.249 L81.6544 529.249 Q78.4397 527.148 76.8483 524.316 Q75.2568 521.451 75.2568 517.727 Q75.2568 511.584 79.0763 508.433 Q82.8638 505.282 90.2481 505.282 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M65.9947 487.808 L76.1162 487.808 L76.1162 475.745 L80.6677 475.745 L80.6677 487.808 L100.019 487.808 Q104.38 487.808 105.621 486.631 Q106.863 485.421 106.863 481.761 L106.863 475.745 L111.764 475.745 L111.764 481.761 Q111.764 488.54 109.25 491.119 Q106.703 493.697 100.019 493.697 L80.6677 493.697 L80.6677 497.993 L76.1162 497.993 L76.1162 493.697 L65.9947 493.697 L65.9947 487.808 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip570)" d="M716.612 40.4256 Q716.612 46.492 713.497 52.1895 Q710.423 57.887 705.586 61.3301 Q700.79 64.7732 695.953 64.7732 Q690.502 64.7732 687.674 59.1167 Q682.55 79.6523 682.263 80.1852 Q681.074 81.9477 679.353 81.9477 Q678.287 81.9477 677.631 81.2919 Q676.975 80.677 676.975 79.6933 Q677.016 79.3654 677.262 78.3406 L685.46 45.3443 Q687.346 37.6793 693.043 32.2277 Q698.782 26.7351 704.93 26.7351 Q709.808 26.7351 713.21 30.3832 Q716.612 34.0312 716.612 40.4256 M710.587 36.8185 Q710.587 32.7196 708.947 30.6701 Q707.307 28.5797 704.766 28.5797 Q702.43 28.5797 699.601 30.4652 Q696.814 32.3507 694.355 36.5316 Q693.043 38.9499 692.1 41.9422 Q691.158 44.9344 689.313 52.3125 Q688.657 55.2637 688.657 55.4276 Q688.657 55.7555 688.821 56.4934 Q689.026 57.2312 689.518 58.3789 Q690.051 59.4856 690.789 60.4693 Q691.527 61.4531 692.879 62.1909 Q694.232 62.8877 695.871 62.8877 Q698.413 62.8877 701.118 60.8382 Q703.864 58.7478 705.873 54.9358 Q707.553 51.8616 709.07 46.0821 Q710.587 40.2616 710.587 36.8185 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip572)" d="M294.124 978.432 L294.124 986.14 L327.818 986.14 L327.818 978.432 L294.124 978.432 L294.124 978.432  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.124,978.432 294.124,986.14 327.818,986.14 327.818,978.432 294.124,978.432 "/>
<path clip-path="url(#clip572)" d="M327.818 970.725 L327.818 986.14 L361.512 986.14 L361.512 970.725 L327.818 970.725 L327.818 970.725  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="327.818,970.725 327.818,986.14 361.512,986.14 361.512,970.725 327.818,970.725 "/>
<path clip-path="url(#clip572)" d="M361.512 963.017 L361.512 986.14 L395.206 986.14 L395.206 963.017 L361.512 963.017 L361.512 963.017  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="361.512,963.017 361.512,986.14 395.206,986.14 395.206,963.017 361.512,963.017 "/>
<path clip-path="url(#clip572)" d="M395.206 939.894 L395.206 986.14 L428.9 986.14 L428.9 939.894 L395.206 939.894 L395.206 939.894  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="395.206,939.894 395.206,986.14 428.9,986.14 428.9,939.894 395.206,939.894 "/>
<path clip-path="url(#clip572)" d="M428.9 909.063 L428.9 986.14 L462.594 986.14 L462.594 909.063 L428.9 909.063 L428.9 909.063  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="428.9,909.063 428.9,986.14 462.594,986.14 462.594,909.063 428.9,909.063 "/>
<path clip-path="url(#clip572)" d="M462.594 878.233 L462.594 986.14 L496.288 986.14 L496.288 878.233 L462.594 878.233 L462.594 878.233  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="462.594,878.233 462.594,986.14 496.288,986.14 496.288,878.233 462.594,878.233 "/>
<path clip-path="url(#clip572)" d="M496.288 747.202 L496.288 986.14 L529.983 986.14 L529.983 747.202 L496.288 747.202 L496.288 747.202  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="496.288,747.202 496.288,986.14 529.983,986.14 529.983,747.202 496.288,747.202 "/>
<path clip-path="url(#clip572)" d="M529.983 724.079 L529.983 986.14 L563.677 986.14 L563.677 724.079 L529.983 724.079 L529.983 724.079  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="529.983,724.079 529.983,986.14 563.677,986.14 563.677,724.079 529.983,724.079 "/>
<path clip-path="url(#clip572)" d="M563.677 585.341 L563.677 986.14 L597.371 986.14 L597.371 585.341 L563.677 585.341 L563.677 585.341  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="563.677,585.341 563.677,986.14 597.371,986.14 597.371,585.341 563.677,585.341 "/>
<path clip-path="url(#clip572)" d="M597.371 554.511 L597.371 986.14 L631.065 986.14 L631.065 554.511 L597.371 554.511 L597.371 554.511  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="597.371,554.511 597.371,986.14 631.065,986.14 631.065,554.511 597.371,554.511 "/>
<path clip-path="url(#clip572)" d="M631.065 500.557 L631.065 986.14 L664.759 986.14 L664.759 500.557 L631.065 500.557 L631.065 500.557  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="631.065,500.557 631.065,986.14 664.759,986.14 664.759,500.557 631.065,500.557 "/>
<path clip-path="url(#clip572)" d="M664.759 400.357 L664.759 986.14 L698.453 986.14 L698.453 400.357 L664.759 400.357 L664.759 400.357  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="664.759,400.357 664.759,986.14 698.453,986.14 698.453,400.357 664.759,400.357 "/>
<path clip-path="url(#clip572)" d="M698.453 153.712 L698.453 986.14 L732.147 986.14 L732.147 153.712 L698.453 153.712 L698.453 153.712  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="698.453,153.712 698.453,986.14 732.147,986.14 732.147,153.712 698.453,153.712 "/>
<path clip-path="url(#clip572)" d="M732.147 192.25 L732.147 986.14 L765.841 986.14 L765.841 192.25 L732.147 192.25 L732.147 192.25  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="732.147,192.25 732.147,986.14 765.841,986.14 765.841,192.25 732.147,192.25 "/>
<path clip-path="url(#clip572)" d="M765.841 261.619 L765.841 986.14 L799.535 986.14 L799.535 261.619 L765.841 261.619 L765.841 261.619  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="765.841,261.619 765.841,986.14 799.535,986.14 799.535,261.619 765.841,261.619 "/>
<path clip-path="url(#clip572)" d="M799.535 223.081 L799.535 986.14 L833.229 986.14 L833.229 223.081 L799.535 223.081 L799.535 223.081  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="799.535,223.081 799.535,986.14 833.229,986.14 833.229,223.081 799.535,223.081 "/>
<path clip-path="url(#clip572)" d="M833.229 392.65 L833.229 986.14 L866.923 986.14 L866.923 392.65 L833.229 392.65 L833.229 392.65  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="833.229,392.65 833.229,986.14 866.923,986.14 866.923,392.65 833.229,392.65 "/>
<path clip-path="url(#clip572)" d="M866.923 485.142 L866.923 986.14 L900.617 986.14 L900.617 485.142 L866.923 485.142 L866.923 485.142  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="866.923,485.142 866.923,986.14 900.617,986.14 900.617,485.142 866.923,485.142 "/>
<path clip-path="url(#clip572)" d="M900.617 716.372 L900.617 986.14 L934.311 986.14 L934.311 716.372 L900.617 716.372 L900.617 716.372  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="900.617,716.372 900.617,986.14 934.311,986.14 934.311,716.372 900.617,716.372 "/>
<path clip-path="url(#clip572)" d="M934.311 700.956 L934.311 986.14 L968.005 986.14 L968.005 700.956 L934.311 700.956 L934.311 700.956  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="934.311,700.956 934.311,986.14 968.005,986.14 968.005,700.956 934.311,700.956 "/>
<path clip-path="url(#clip572)" d="M968.005 839.694 L968.005 986.14 L1001.7 986.14 L1001.7 839.694 L968.005 839.694 L968.005 839.694  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="968.005,839.694 968.005,986.14 1001.7,986.14 1001.7,839.694 968.005,839.694 "/>
<path clip-path="url(#clip572)" d="M1001.7 916.771 L1001.7 986.14 L1035.39 986.14 L1035.39 916.771 L1001.7 916.771 L1001.7 916.771  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1001.7,916.771 1001.7,986.14 1035.39,986.14 1035.39,916.771 1001.7,916.771 "/>
<path clip-path="url(#clip572)" d="M1035.39 955.309 L1035.39 986.14 L1069.09 986.14 L1069.09 955.309 L1035.39 955.309 L1035.39 955.309  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1035.39,955.309 1035.39,986.14 1069.09,986.14 1069.09,955.309 1035.39,955.309 "/>
<path clip-path="url(#clip572)" d="M1069.09 970.725 L1069.09 986.14 L1102.78 986.14 L1102.78 970.725 L1069.09 970.725 L1069.09 970.725  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip572)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1069.09,970.725 1069.09,986.14 1102.78,986.14 1102.78,970.725 1069.09,970.725 "/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="310.971" cy="978.432" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="344.665" cy="970.725" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="378.359" cy="963.017" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="412.053" cy="939.894" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="445.747" cy="909.063" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="479.441" cy="878.233" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="513.135" cy="747.202" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="546.83" cy="724.079" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="580.524" cy="585.341" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="614.218" cy="554.511" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="647.912" cy="500.557" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="681.606" cy="400.357" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="715.3" cy="153.712" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="748.994" cy="192.25" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="782.688" cy="261.619" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="816.382" cy="223.081" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="850.076" cy="392.65" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="883.77" cy="485.142" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="917.464" cy="716.372" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="951.158" cy="700.956" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="984.852" cy="839.694" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1018.55" cy="916.771" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1052.24" cy="955.309" r="2"/>
<circle clip-path="url(#clip572)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1085.93" cy="970.725" r="2"/>
<polyline clip-path="url(#clip572)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="835.852,1818.57 835.852,-678.716 "/>
</svg>
<p>(e)</p>
</div>
</div></td>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="300" height="300" viewBox="0 0 1200 1200">
<defs>
  <clipPath id="clip630">
    <rect x="0" y="0" width="1200" height="1200"/>
  </clipPath>
</defs>
<path clip-path="url(#clip630)" d="M0 1200 L1200 1200 L1200 0 L0 0  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip631">
    <rect x="240" y="119" width="841" height="841"/>
  </clipPath>
</defs>
<path clip-path="url(#clip630)" d="M244.149 986.14 L1152.76 986.14 L1152.76 153.712 L244.149 153.712  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip632">
    <rect x="244" y="153" width="910" height="833"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="334.557,986.14 334.557,153.712 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="536.721,986.14 536.721,153.712 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="738.885,986.14 738.885,153.712 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="941.05,986.14 941.05,153.712 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1143.21,986.14 1143.21,153.712 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,986.14 1152.76,986.14 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,866.366 1152.76,866.366 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,746.592 1152.76,746.592 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,626.819 1152.76,626.819 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,507.045 1152.76,507.045 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,387.271 1152.76,387.271 "/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="244.149,267.497 1152.76,267.497 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 1152.76,986.14 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="334.557,986.14 334.557,967.242 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="536.721,986.14 536.721,967.242 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="738.885,986.14 738.885,967.242 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="941.05,986.14 941.05,967.242 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1143.21,986.14 1143.21,967.242 "/>
<path clip-path="url(#clip630)" d="M315.194 1008.45 L303.388 1026.9 L315.194 1026.9 L315.194 1008.45 M313.967 1004.38 L319.847 1004.38 L319.847 1026.9 L324.777 1026.9 L324.777 1030.79 L319.847 1030.79 L319.847 1038.94 L315.194 1038.94 L315.194 1030.79 L299.592 1030.79 L299.592 1026.28 L313.967 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M332.509 1033.06 L337.393 1033.06 L337.393 1038.94 L332.509 1038.94 L332.509 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M357.578 1007.46 Q353.967 1007.46 352.138 1011.02 Q350.333 1014.56 350.333 1021.69 Q350.333 1028.8 352.138 1032.37 Q353.967 1035.91 357.578 1035.91 Q361.212 1035.91 363.018 1032.37 Q364.846 1028.8 364.846 1021.69 Q364.846 1014.56 363.018 1011.02 Q361.212 1007.46 357.578 1007.46 M357.578 1003.75 Q363.388 1003.75 366.444 1008.36 Q369.522 1012.94 369.522 1021.69 Q369.522 1030.42 366.444 1035.03 Q363.388 1039.61 357.578 1039.61 Q351.768 1039.61 348.689 1035.03 Q345.633 1030.42 345.633 1021.69 Q345.633 1012.94 348.689 1008.36 Q351.768 1003.75 357.578 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M517.856 1008.45 L506.05 1026.9 L517.856 1026.9 L517.856 1008.45 M516.629 1004.38 L522.508 1004.38 L522.508 1026.9 L527.439 1026.9 L527.439 1030.79 L522.508 1030.79 L522.508 1038.94 L517.856 1038.94 L517.856 1030.79 L502.254 1030.79 L502.254 1026.28 L516.629 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M535.17 1033.06 L540.055 1033.06 L540.055 1038.94 L535.17 1038.94 L535.17 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M550.286 1004.38 L568.642 1004.38 L568.642 1008.32 L554.568 1008.32 L554.568 1016.79 Q555.587 1016.44 556.605 1016.28 Q557.624 1016.09 558.642 1016.09 Q564.429 1016.09 567.809 1019.26 Q571.189 1022.44 571.189 1027.85 Q571.189 1033.43 567.717 1036.53 Q564.244 1039.61 557.925 1039.61 Q555.749 1039.61 553.48 1039.24 Q551.235 1038.87 548.828 1038.13 L548.828 1033.43 Q550.911 1034.56 553.133 1035.12 Q555.355 1035.68 557.832 1035.68 Q561.837 1035.68 564.175 1033.57 Q566.513 1031.46 566.513 1027.85 Q566.513 1024.24 564.175 1022.13 Q561.837 1020.03 557.832 1020.03 Q555.957 1020.03 554.082 1020.44 Q552.23 1020.86 550.286 1021.74 L550.286 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M706.05 1004.38 L724.406 1004.38 L724.406 1008.32 L710.332 1008.32 L710.332 1016.79 Q711.351 1016.44 712.369 1016.28 Q713.388 1016.09 714.406 1016.09 Q720.193 1016.09 723.573 1019.26 Q726.953 1022.44 726.953 1027.85 Q726.953 1033.43 723.48 1036.53 Q720.008 1039.61 713.689 1039.61 Q711.513 1039.61 709.244 1039.24 Q706.999 1038.87 704.592 1038.13 L704.592 1033.43 Q706.675 1034.56 708.897 1035.12 Q711.119 1035.68 713.596 1035.68 Q717.601 1035.68 719.939 1033.57 Q722.277 1031.46 722.277 1027.85 Q722.277 1024.24 719.939 1022.13 Q717.601 1020.03 713.596 1020.03 Q711.721 1020.03 709.846 1020.44 Q707.994 1020.86 706.05 1021.74 L706.05 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M736.166 1033.06 L741.05 1033.06 L741.05 1038.94 L736.166 1038.94 L736.166 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M761.235 1007.46 Q757.624 1007.46 755.795 1011.02 Q753.99 1014.56 753.99 1021.69 Q753.99 1028.8 755.795 1032.37 Q757.624 1035.91 761.235 1035.91 Q764.869 1035.91 766.675 1032.37 Q768.503 1028.8 768.503 1021.69 Q768.503 1014.56 766.675 1011.02 Q764.869 1007.46 761.235 1007.46 M761.235 1003.75 Q767.045 1003.75 770.101 1008.36 Q773.179 1012.94 773.179 1021.69 Q773.179 1030.42 770.101 1035.03 Q767.045 1039.61 761.235 1039.61 Q755.425 1039.61 752.346 1035.03 Q749.29 1030.42 749.29 1021.69 Q749.29 1012.94 752.346 1008.36 Q755.425 1003.75 761.235 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M908.712 1004.38 L927.068 1004.38 L927.068 1008.32 L912.994 1008.32 L912.994 1016.79 Q914.013 1016.44 915.031 1016.28 Q916.05 1016.09 917.068 1016.09 Q922.855 1016.09 926.235 1019.26 Q929.614 1022.44 929.614 1027.85 Q929.614 1033.43 926.142 1036.53 Q922.67 1039.61 916.351 1039.61 Q914.175 1039.61 911.906 1039.24 Q909.661 1038.87 907.253 1038.13 L907.253 1033.43 Q909.337 1034.56 911.559 1035.12 Q913.781 1035.68 916.258 1035.68 Q920.263 1035.68 922.601 1033.57 Q924.939 1031.46 924.939 1027.85 Q924.939 1024.24 922.601 1022.13 Q920.263 1020.03 916.258 1020.03 Q914.383 1020.03 912.508 1020.44 Q910.656 1020.86 908.712 1021.74 L908.712 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M938.827 1033.06 L943.712 1033.06 L943.712 1038.94 L938.827 1038.94 L938.827 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M953.943 1004.38 L972.299 1004.38 L972.299 1008.32 L958.225 1008.32 L958.225 1016.79 Q959.244 1016.44 960.262 1016.28 Q961.281 1016.09 962.299 1016.09 Q968.086 1016.09 971.466 1019.26 Q974.846 1022.44 974.846 1027.85 Q974.846 1033.43 971.374 1036.53 Q967.901 1039.61 961.582 1039.61 Q959.406 1039.61 957.137 1039.24 Q954.892 1038.87 952.485 1038.13 L952.485 1033.43 Q954.568 1034.56 956.79 1035.12 Q959.012 1035.68 961.489 1035.68 Q965.494 1035.68 967.832 1033.57 Q970.17 1031.46 970.17 1027.85 Q970.17 1024.24 967.832 1022.13 Q965.494 1020.03 961.489 1020.03 Q959.614 1020.03 957.739 1020.44 Q955.887 1020.86 953.943 1021.74 L953.943 1004.38 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M1121.08 1019.8 Q1117.94 1019.8 1116.08 1021.95 Q1114.26 1024.1 1114.26 1027.85 Q1114.26 1031.58 1116.08 1033.75 Q1117.94 1035.91 1121.08 1035.91 Q1124.23 1035.91 1126.06 1033.75 Q1127.91 1031.58 1127.91 1027.85 Q1127.91 1024.1 1126.06 1021.95 Q1124.23 1019.8 1121.08 1019.8 M1130.37 1005.14 L1130.37 1009.4 Q1128.61 1008.57 1126.8 1008.13 Q1125.02 1007.69 1123.26 1007.69 Q1118.63 1007.69 1116.18 1010.81 Q1113.75 1013.94 1113.4 1020.26 Q1114.76 1018.25 1116.83 1017.18 Q1118.89 1016.09 1121.36 1016.09 Q1126.57 1016.09 1129.58 1019.26 Q1132.61 1022.41 1132.61 1027.85 Q1132.61 1033.18 1129.46 1036.39 Q1126.32 1039.61 1121.08 1039.61 Q1115.09 1039.61 1111.92 1035.03 Q1108.75 1030.42 1108.75 1021.69 Q1108.75 1013.5 1112.64 1008.64 Q1116.52 1003.75 1123.07 1003.75 Q1124.83 1003.75 1126.62 1004.1 Q1128.42 1004.45 1130.37 1005.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M1140.67 1033.06 L1145.55 1033.06 L1145.55 1038.94 L1140.67 1038.94 L1140.67 1033.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M1165.74 1007.46 Q1162.13 1007.46 1160.3 1011.02 Q1158.49 1014.56 1158.49 1021.69 Q1158.49 1028.8 1160.3 1032.37 Q1162.13 1035.91 1165.74 1035.91 Q1169.37 1035.91 1171.18 1032.37 Q1173.01 1028.8 1173.01 1021.69 Q1173.01 1014.56 1171.18 1011.02 Q1169.37 1007.46 1165.74 1007.46 M1165.74 1003.75 Q1171.55 1003.75 1174.6 1008.36 Q1177.68 1012.94 1177.68 1021.69 Q1177.68 1030.42 1174.6 1035.03 Q1171.55 1039.61 1165.74 1039.61 Q1159.93 1039.61 1156.85 1035.03 Q1153.79 1030.42 1153.79 1021.69 Q1153.79 1012.94 1156.85 1008.36 Q1159.93 1003.75 1165.74 1003.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M714.687 1071.35 Q714.687 1073.6 711.466 1073.6 L704.22 1073.6 Q706.249 1076.5 706.249 1080.78 Q706.249 1084.45 704.735 1087.9 Q703.254 1091.31 700.871 1093.7 Q698.52 1096.08 695.557 1097.53 Q692.594 1098.95 689.695 1098.95 Q685.283 1098.95 682.545 1095.95 Q679.808 1092.96 679.808 1088.51 Q679.808 1086.16 680.613 1083.55 Q681.45 1080.94 683.029 1078.53 Q684.607 1076.08 686.668 1074.12 Q688.761 1072.15 691.466 1070.96 Q694.204 1069.77 697.07 1069.77 L712.11 1069.77 Q712.916 1069.77 713.334 1069.83 Q713.785 1069.87 714.236 1070.25 Q714.687 1070.61 714.687 1071.35 M702.03 1080.01 Q702.03 1079.24 701.933 1078.56 Q701.837 1077.85 701.482 1076.92 Q701.128 1075.95 700.516 1075.28 Q699.904 1074.57 698.713 1074.08 Q697.553 1073.6 695.975 1073.6 Q693.753 1073.6 691.305 1074.83 Q688.858 1076.05 686.99 1079.08 Q685.444 1081.65 684.671 1085 Q683.898 1088.35 683.898 1090.35 Q683.898 1093.76 685.541 1095.63 Q687.215 1097.47 689.76 1097.47 Q691.853 1097.47 694.236 1096.05 Q696.619 1094.6 698.455 1091.86 Q700.065 1089.48 701.032 1086 Q702.03 1082.52 702.03 1080.01 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 244.149,153.712 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,986.14 263.047,986.14 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,866.366 263.047,866.366 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,746.592 263.047,746.592 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,626.819 263.047,626.819 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,507.045 263.047,507.045 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,387.271 263.047,387.271 "/>
<polyline clip-path="url(#clip630)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="244.149,267.497 263.047,267.497 "/>
<path clip-path="url(#clip630)" d="M214.205 971.939 Q210.594 971.939 208.765 975.503 Q206.96 979.045 206.96 986.175 Q206.96 993.281 208.765 996.846 Q210.594 1000.39 214.205 1000.39 Q217.839 1000.39 219.645 996.846 Q221.473 993.281 221.473 986.175 Q221.473 979.045 219.645 975.503 Q217.839 971.939 214.205 971.939 M214.205 968.235 Q220.015 968.235 223.071 972.841 Q226.149 977.425 226.149 986.175 Q226.149 994.901 223.071 999.508 Q220.015 1004.09 214.205 1004.09 Q208.395 1004.09 205.316 999.508 Q202.261 994.901 202.261 986.175 Q202.261 977.425 205.316 972.841 Q208.395 968.235 214.205 968.235 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M178.071 879.711 L194.39 879.711 L194.39 883.646 L172.446 883.646 L172.446 879.711 Q175.108 876.956 179.691 872.327 Q184.298 867.674 185.478 866.331 Q187.724 863.808 188.603 862.072 Q189.506 860.313 189.506 858.623 Q189.506 855.868 187.562 854.132 Q185.64 852.396 182.538 852.396 Q180.339 852.396 177.886 853.16 Q175.455 853.924 172.677 855.475 L172.677 850.753 Q175.501 849.618 177.955 849.04 Q180.409 848.461 182.446 848.461 Q187.816 848.461 191.011 851.146 Q194.205 853.831 194.205 858.322 Q194.205 860.452 193.395 862.373 Q192.608 864.271 190.501 866.864 Q189.923 867.535 186.821 870.753 Q183.719 873.947 178.071 879.711 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M214.205 852.165 Q210.594 852.165 208.765 855.73 Q206.96 859.271 206.96 866.401 Q206.96 873.507 208.765 877.072 Q210.594 880.614 214.205 880.614 Q217.839 880.614 219.645 877.072 Q221.473 873.507 221.473 866.401 Q221.473 859.271 219.645 855.73 Q217.839 852.165 214.205 852.165 M214.205 848.461 Q220.015 848.461 223.071 853.068 Q226.149 857.651 226.149 866.401 Q226.149 875.128 223.071 879.734 Q220.015 884.317 214.205 884.317 Q208.395 884.317 205.316 879.734 Q202.261 875.128 202.261 866.401 Q202.261 857.651 205.316 853.068 Q208.395 848.461 214.205 848.461 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M186.89 733.386 L175.085 751.835 L186.89 751.835 L186.89 733.386 M185.663 729.312 L191.543 729.312 L191.543 751.835 L196.474 751.835 L196.474 755.724 L191.543 755.724 L191.543 763.872 L186.89 763.872 L186.89 755.724 L171.289 755.724 L171.289 751.21 L185.663 729.312 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M214.205 732.391 Q210.594 732.391 208.765 735.956 Q206.96 739.497 206.96 746.627 Q206.96 753.733 208.765 757.298 Q210.594 760.84 214.205 760.84 Q217.839 760.84 219.645 757.298 Q221.473 753.733 221.473 746.627 Q221.473 739.497 219.645 735.956 Q217.839 732.391 214.205 732.391 M214.205 728.687 Q220.015 728.687 223.071 733.294 Q226.149 737.877 226.149 746.627 Q226.149 755.354 223.071 759.96 Q220.015 764.544 214.205 764.544 Q208.395 764.544 205.316 759.96 Q202.261 755.354 202.261 746.627 Q202.261 737.877 205.316 733.294 Q208.395 728.687 214.205 728.687 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M184.622 624.955 Q181.474 624.955 179.622 627.108 Q177.793 629.261 177.793 633.011 Q177.793 636.737 179.622 638.913 Q181.474 641.066 184.622 641.066 Q187.77 641.066 189.599 638.913 Q191.45 636.737 191.45 633.011 Q191.45 629.261 189.599 627.108 Q187.77 624.955 184.622 624.955 M193.904 610.302 L193.904 614.562 Q192.145 613.728 190.339 613.289 Q188.557 612.849 186.798 612.849 Q182.168 612.849 179.714 615.974 Q177.284 619.099 176.937 625.418 Q178.302 623.404 180.363 622.339 Q182.423 621.251 184.9 621.251 Q190.108 621.251 193.117 624.423 Q196.15 627.571 196.15 633.011 Q196.15 638.335 193.001 641.552 Q189.853 644.77 184.622 644.77 Q178.626 644.77 175.455 640.187 Q172.284 635.58 172.284 626.853 Q172.284 618.659 176.173 613.798 Q180.062 608.914 186.613 608.914 Q188.372 608.914 190.154 609.261 Q191.96 609.608 193.904 610.302 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M214.205 612.617 Q210.594 612.617 208.765 616.182 Q206.96 619.724 206.96 626.853 Q206.96 633.96 208.765 637.524 Q210.594 641.066 214.205 641.066 Q217.839 641.066 219.645 637.524 Q221.473 633.96 221.473 626.853 Q221.473 619.724 219.645 616.182 Q217.839 612.617 214.205 612.617 M214.205 608.914 Q220.015 608.914 223.071 613.52 Q226.149 618.103 226.149 626.853 Q226.149 635.58 223.071 640.187 Q220.015 644.77 214.205 644.77 Q208.395 644.77 205.316 640.187 Q202.261 635.58 202.261 626.853 Q202.261 618.103 205.316 613.52 Q208.395 608.914 214.205 608.914 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M184.043 507.913 Q180.71 507.913 178.788 509.695 Q176.89 511.478 176.89 514.603 Q176.89 517.728 178.788 519.51 Q180.71 521.292 184.043 521.292 Q187.376 521.292 189.298 519.51 Q191.219 517.704 191.219 514.603 Q191.219 511.478 189.298 509.695 Q187.4 507.913 184.043 507.913 M179.367 505.922 Q176.358 505.181 174.668 503.121 Q173.001 501.061 173.001 498.098 Q173.001 493.955 175.941 491.547 Q178.904 489.14 184.043 489.14 Q189.205 489.14 192.145 491.547 Q195.085 493.955 195.085 498.098 Q195.085 501.061 193.395 503.121 Q191.728 505.181 188.742 505.922 Q192.122 506.709 193.997 509.001 Q195.895 511.292 195.895 514.603 Q195.895 519.626 192.816 522.311 Q189.761 524.996 184.043 524.996 Q178.326 524.996 175.247 522.311 Q172.191 519.626 172.191 514.603 Q172.191 511.292 174.089 509.001 Q175.988 506.709 179.367 505.922 M177.654 498.538 Q177.654 501.223 179.321 502.728 Q181.011 504.232 184.043 504.232 Q187.052 504.232 188.742 502.728 Q190.455 501.223 190.455 498.538 Q190.455 495.853 188.742 494.348 Q187.052 492.843 184.043 492.843 Q181.011 492.843 179.321 494.348 Q177.654 495.853 177.654 498.538 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M214.205 492.843 Q210.594 492.843 208.765 496.408 Q206.96 499.95 206.96 507.079 Q206.96 514.186 208.765 517.751 Q210.594 521.292 214.205 521.292 Q217.839 521.292 219.645 517.751 Q221.473 514.186 221.473 507.079 Q221.473 499.95 219.645 496.408 Q217.839 492.843 214.205 492.843 M214.205 489.14 Q220.015 489.14 223.071 493.746 Q226.149 498.33 226.149 507.079 Q226.149 515.806 223.071 520.413 Q220.015 524.996 214.205 524.996 Q208.395 524.996 205.316 520.413 Q202.261 515.806 202.261 507.079 Q202.261 498.33 205.316 493.746 Q208.395 489.14 214.205 489.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M144.691 400.616 L152.33 400.616 L152.33 374.25 L144.02 375.917 L144.02 371.658 L152.284 369.991 L156.96 369.991 L156.96 400.616 L164.599 400.616 L164.599 404.551 L144.691 404.551 L144.691 400.616 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M184.043 373.07 Q180.432 373.07 178.603 376.634 Q176.798 380.176 176.798 387.306 Q176.798 394.412 178.603 397.977 Q180.432 401.519 184.043 401.519 Q187.677 401.519 189.483 397.977 Q191.312 394.412 191.312 387.306 Q191.312 380.176 189.483 376.634 Q187.677 373.07 184.043 373.07 M184.043 369.366 Q189.853 369.366 192.909 373.972 Q195.987 378.556 195.987 387.306 Q195.987 396.033 192.909 400.639 Q189.853 405.222 184.043 405.222 Q178.233 405.222 175.154 400.639 Q172.099 396.033 172.099 387.306 Q172.099 378.556 175.154 373.972 Q178.233 369.366 184.043 369.366 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M214.205 373.07 Q210.594 373.07 208.765 376.634 Q206.96 380.176 206.96 387.306 Q206.96 394.412 208.765 397.977 Q210.594 401.519 214.205 401.519 Q217.839 401.519 219.645 397.977 Q221.473 394.412 221.473 387.306 Q221.473 380.176 219.645 376.634 Q217.839 373.07 214.205 373.07 M214.205 369.366 Q220.015 369.366 223.071 373.972 Q226.149 378.556 226.149 387.306 Q226.149 396.033 223.071 400.639 Q220.015 405.222 214.205 405.222 Q208.395 405.222 205.316 400.639 Q202.261 396.033 202.261 387.306 Q202.261 378.556 205.316 373.972 Q208.395 369.366 214.205 369.366 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M144.691 280.842 L152.33 280.842 L152.33 254.476 L144.02 256.143 L144.02 251.884 L152.284 250.217 L156.96 250.217 L156.96 280.842 L164.599 280.842 L164.599 284.777 L144.691 284.777 L144.691 280.842 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M178.071 280.842 L194.39 280.842 L194.39 284.777 L172.446 284.777 L172.446 280.842 Q175.108 278.087 179.691 273.458 Q184.298 268.805 185.478 267.462 Q187.724 264.939 188.603 263.203 Q189.506 261.444 189.506 259.754 Q189.506 257 187.562 255.263 Q185.64 253.527 182.538 253.527 Q180.339 253.527 177.886 254.291 Q175.455 255.055 172.677 256.606 L172.677 251.884 Q175.501 250.75 177.955 250.171 Q180.409 249.592 182.446 249.592 Q187.816 249.592 191.011 252.277 Q194.205 254.963 194.205 259.453 Q194.205 261.583 193.395 263.504 Q192.608 265.402 190.501 267.995 Q189.923 268.666 186.821 271.884 Q183.719 275.078 178.071 280.842 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M214.205 253.296 Q210.594 253.296 208.765 256.861 Q206.96 260.402 206.96 267.532 Q206.96 274.638 208.765 278.203 Q210.594 281.745 214.205 281.745 Q217.839 281.745 219.645 278.203 Q221.473 274.638 221.473 267.532 Q221.473 260.402 219.645 256.861 Q217.839 253.296 214.205 253.296 M214.205 249.592 Q220.015 249.592 223.071 254.199 Q226.149 258.782 226.149 267.532 Q226.149 276.259 223.071 280.865 Q220.015 285.449 214.205 285.449 Q208.395 285.449 205.316 280.865 Q202.261 276.259 202.261 267.532 Q202.261 258.782 205.316 254.199 Q208.395 249.592 214.205 249.592 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M67.9044 625.785 L74.6839 625.785 Q71.6602 629.032 70.1643 632.724 Q68.6683 636.384 68.6683 640.522 Q68.6683 648.67 73.6654 652.998 Q78.6307 657.327 88.0519 657.327 Q97.4413 657.327 102.438 652.998 Q107.404 648.67 107.404 640.522 Q107.404 636.384 105.908 632.724 Q104.412 629.032 101.388 625.785 L108.104 625.785 Q110.396 629.159 111.541 632.946 Q112.687 636.702 112.687 640.904 Q112.687 651.693 106.099 657.9 Q99.4783 664.107 88.0519 664.107 Q76.5936 664.107 70.0051 657.9 Q63.3848 651.693 63.3848 640.904 Q63.3848 636.639 64.5306 632.883 Q65.6446 629.095 67.9044 625.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M80.2221 602.296 Q80.2221 607.006 83.9142 609.743 Q87.5745 612.481 93.972 612.481 Q100.37 612.481 104.062 609.775 Q107.722 607.038 107.722 602.296 Q107.722 597.617 104.03 594.88 Q100.338 592.142 93.972 592.142 Q87.6381 592.142 83.946 594.88 Q80.2221 597.617 80.2221 602.296 M75.2568 602.296 Q75.2568 594.657 80.2221 590.296 Q85.1873 585.936 93.972 585.936 Q102.725 585.936 107.722 590.296 Q112.687 594.657 112.687 602.296 Q112.687 609.966 107.722 614.327 Q102.725 618.655 93.972 618.655 Q85.1873 618.655 80.2221 614.327 Q75.2568 609.966 75.2568 602.296 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M97.6959 576.833 L76.1162 576.833 L76.1162 570.976 L97.4731 570.976 Q102.534 570.976 105.08 569.003 Q107.595 567.03 107.595 563.083 Q107.595 558.34 104.571 555.603 Q101.547 552.834 96.3273 552.834 L76.1162 552.834 L76.1162 546.978 L111.764 546.978 L111.764 552.834 L106.29 552.834 Q109.536 554.967 111.128 557.799 Q112.687 560.6 112.687 564.324 Q112.687 570.467 108.868 573.65 Q105.048 576.833 97.6959 576.833 M75.2568 562.096 L75.2568 562.096 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M90.2481 505.282 L111.764 505.282 L111.764 511.139 L90.439 511.139 Q85.3783 511.139 82.8638 513.112 Q80.3494 515.085 80.3494 519.032 Q80.3494 523.775 83.3731 526.512 Q86.3968 529.249 91.6167 529.249 L111.764 529.249 L111.764 535.137 L76.1162 535.137 L76.1162 529.249 L81.6544 529.249 Q78.4397 527.148 76.8483 524.316 Q75.2568 521.451 75.2568 517.727 Q75.2568 511.584 79.0763 508.433 Q82.8638 505.282 90.2481 505.282 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M65.9947 487.808 L76.1162 487.808 L76.1162 475.745 L80.6677 475.745 L80.6677 487.808 L100.019 487.808 Q104.38 487.808 105.621 486.631 Q106.863 485.421 106.863 481.761 L106.863 475.745 L111.764 475.745 L111.764 481.761 Q111.764 488.54 109.25 491.119 Q106.703 493.697 100.019 493.697 L80.6677 493.697 L80.6677 497.993 L76.1162 497.993 L76.1162 493.697 L65.9947 493.697 L65.9947 487.808 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip630)" d="M719.113 37.704 Q719.113 40.5732 715.014 40.5732 L705.792 40.5732 Q708.374 44.2623 708.374 49.7138 Q708.374 54.3866 706.448 58.7725 Q704.562 63.1173 701.529 66.1505 Q698.537 69.1837 694.766 71.0283 Q690.995 72.8318 687.306 72.8318 Q681.69 72.8318 678.206 69.0198 Q674.722 65.2078 674.722 59.5513 Q674.722 56.559 675.747 53.2389 Q676.812 49.9188 678.821 46.8446 Q680.829 43.7294 683.453 41.2291 Q686.117 38.7287 689.56 37.2121 Q693.044 35.6955 696.692 35.6955 L715.834 35.6955 Q716.859 35.6955 717.392 35.7775 Q717.966 35.8185 718.54 36.3103 Q719.113 36.7612 719.113 37.704 M703.005 48.7301 Q703.005 47.7463 702.882 46.8856 Q702.759 45.9838 702.308 44.7951 Q701.857 43.5654 701.078 42.7047 Q700.299 41.8029 698.783 41.1881 Q697.307 40.5732 695.299 40.5732 Q692.47 40.5732 689.355 42.1308 Q686.24 43.6884 683.863 47.5414 Q681.895 50.8205 680.911 55.0834 Q679.928 59.3463 679.928 61.8876 Q679.928 66.2325 682.018 68.6099 Q684.15 70.9463 687.388 70.9463 Q690.052 70.9463 693.085 69.1427 Q696.118 67.2982 698.455 63.8141 Q700.504 60.7809 701.734 56.3541 Q703.005 51.9273 703.005 48.7301 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip632)" d="M294.124 974.162 L294.124 986.14 L334.557 986.14 L334.557 974.162 L294.124 974.162 L294.124 974.162  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.124,974.162 294.124,986.14 334.557,986.14 334.557,974.162 294.124,974.162 "/>
<path clip-path="url(#clip632)" d="M334.557 962.185 L334.557 986.14 L374.99 986.14 L374.99 962.185 L334.557 962.185 L334.557 962.185  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="334.557,962.185 334.557,986.14 374.99,986.14 374.99,962.185 334.557,962.185 "/>
<path clip-path="url(#clip632)" d="M374.99 980.151 L374.99 986.14 L415.423 986.14 L415.423 980.151 L374.99 980.151 L374.99 980.151  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="374.99,980.151 374.99,986.14 415.423,986.14 415.423,980.151 374.99,980.151 "/>
<path clip-path="url(#clip632)" d="M415.423 842.411 L415.423 986.14 L455.856 986.14 L455.856 842.411 L415.423 842.411 L415.423 842.411  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="415.423,842.411 415.423,986.14 455.856,986.14 455.856,842.411 415.423,842.411 "/>
<path clip-path="url(#clip632)" d="M455.856 800.49 L455.856 986.14 L496.288 986.14 L496.288 800.49 L455.856 800.49 L455.856 800.49  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="455.856,800.49 455.856,986.14 496.288,986.14 496.288,800.49 455.856,800.49 "/>
<path clip-path="url(#clip632)" d="M496.288 614.841 L496.288 986.14 L536.721 986.14 L536.721 614.841 L496.288 614.841 L496.288 614.841  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="496.288,614.841 496.288,986.14 536.721,986.14 536.721,614.841 496.288,614.841 "/>
<path clip-path="url(#clip632)" d="M536.721 501.056 L536.721 986.14 L577.154 986.14 L577.154 501.056 L536.721 501.056 L536.721 501.056  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="536.721,501.056 536.721,986.14 577.154,986.14 577.154,501.056 536.721,501.056 "/>
<path clip-path="url(#clip632)" d="M577.154 441.169 L577.154 986.14 L617.587 986.14 L617.587 441.169 L577.154 441.169 L577.154 441.169  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="577.154,441.169 577.154,986.14 617.587,986.14 617.587,441.169 577.154,441.169 "/>
<path clip-path="url(#clip632)" d="M617.587 153.712 L617.587 986.14 L658.02 986.14 L658.02 153.712 L617.587 153.712 L617.587 153.712  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="617.587,153.712 617.587,986.14 658.02,986.14 658.02,153.712 617.587,153.712 "/>
<path clip-path="url(#clip632)" d="M658.02 291.452 L658.02 986.14 L698.453 986.14 L698.453 291.452 L658.02 291.452 L658.02 291.452  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="658.02,291.452 658.02,986.14 698.453,986.14 698.453,291.452 658.02,291.452 "/>
<path clip-path="url(#clip632)" d="M698.453 225.576 L698.453 986.14 L738.885 986.14 L738.885 225.576 L698.453 225.576 L698.453 225.576  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="698.453,225.576 698.453,986.14 738.885,986.14 738.885,225.576 698.453,225.576 "/>
<path clip-path="url(#clip632)" d="M738.885 387.271 L738.885 986.14 L779.318 986.14 L779.318 387.271 L738.885 387.271 L738.885 387.271  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="738.885,387.271 738.885,986.14 779.318,986.14 779.318,387.271 738.885,387.271 "/>
<path clip-path="url(#clip632)" d="M779.318 560.943 L779.318 986.14 L819.751 986.14 L819.751 560.943 L779.318 560.943 L779.318 560.943  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="779.318,560.943 779.318,986.14 819.751,986.14 819.751,560.943 779.318,560.943 "/>
<path clip-path="url(#clip632)" d="M819.751 572.92 L819.751 986.14 L860.184 986.14 L860.184 572.92 L819.751 572.92 L819.751 572.92  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="819.751,572.92 819.751,986.14 860.184,986.14 860.184,572.92 819.751,572.92 "/>
<path clip-path="url(#clip632)" d="M860.184 758.57 L860.184 986.14 L900.617 986.14 L900.617 758.57 L860.184 758.57 L860.184 758.57  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="860.184,758.57 860.184,986.14 900.617,986.14 900.617,758.57 860.184,758.57 "/>
<path clip-path="url(#clip632)" d="M900.617 878.343 L900.617 986.14 L941.05 986.14 L941.05 878.343 L900.617 878.343 L900.617 878.343  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="900.617,878.343 900.617,986.14 941.05,986.14 941.05,878.343 900.617,878.343 "/>
<path clip-path="url(#clip632)" d="M941.05 878.343 L941.05 986.14 L981.482 986.14 L981.482 878.343 L941.05 878.343 L941.05 878.343  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="941.05,878.343 941.05,986.14 981.482,986.14 981.482,878.343 941.05,878.343 "/>
<path clip-path="url(#clip632)" d="M981.482 956.196 L981.482 986.14 L1021.92 986.14 L1021.92 956.196 L981.482 956.196 L981.482 956.196  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="981.482,956.196 981.482,986.14 1021.92,986.14 1021.92,956.196 981.482,956.196 "/>
<path clip-path="url(#clip632)" d="M1021.92 980.151 L1021.92 986.14 L1062.35 986.14 L1062.35 980.151 L1021.92 980.151 L1021.92 980.151  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1021.92,980.151 1021.92,986.14 1062.35,986.14 1062.35,980.151 1021.92,980.151 "/>
<path clip-path="url(#clip632)" d="M1062.35 974.162 L1062.35 986.14 L1102.78 986.14 L1102.78 974.162 L1062.35 974.162 L1062.35 974.162  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip632)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1062.35,974.162 1062.35,986.14 1102.78,986.14 1102.78,974.162 1062.35,974.162 "/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="314.341" cy="974.162" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="354.774" cy="962.185" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="395.206" cy="980.151" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="435.639" cy="842.411" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="476.072" cy="800.49" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="516.505" cy="614.841" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="556.938" cy="501.056" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="597.371" cy="441.169" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="637.803" cy="153.712" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="678.236" cy="291.452" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="718.669" cy="225.576" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="759.102" cy="387.271" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="799.535" cy="560.943" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="839.968" cy="572.92" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="880.4" cy="758.57" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="920.833" cy="878.343" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="961.266" cy="878.343" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1001.7" cy="956.196" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1042.13" cy="980.151" r="2"/>
<circle clip-path="url(#clip632)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1082.56" cy="974.162" r="2"/>
<polyline clip-path="url(#clip632)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="723.094,1818.57 723.094,-678.716 "/>
</svg>
<p>(f)</p>
</div>
</div></td>
</tr>
</tbody>
</table>

Figure 3

We can see that there are some outlying replicates which result in very
high bootstrap samples for the model parameters $a$, $b$, and $\tau$.
But now we can calculate the sampling distribution of the sea-level
sensitivity to temperature, which is $a/\tau$. Plotting this
distribution:

In [1]:
sens = a_boot ./ τ_boot

psens = histogram(sens, xlabel="Sea-Level Sensitivity to Temperature (mm/°C)", ylabel="Count", label=false)
vline!(psens, [p_mle[1] / p_mle[3]], color=:red, label="MLE")
plot!(psens, size=(600, 400))

We can see from
<a href="#fig-p2-sensitivity" class="quarto-xref">Figure 4</a> that the
large outlying samples cancel out in terms of the numerical effect,
which is why the model fit still worked well.[1]

The 90% basic bootstrap confidence interval then becomes

[1] This also highlights a key point for model calibration and
statistical inference: individual model parameters may not be directly
interpretable due to these compensating effects, and it’s valuable to
look at physically-relevant statistics rather than specific parameters.
If the individual model parameters are important for interpretation or
extrapolation, they should be appropriately constrained or prior
information should be used through a Bayesian analysis.

In [1]:
sens_mle = p_mle[1] / p_mle[3]
q_boot = quantile(sens, [0.95, 0.05])
ci_boot = 2 * sens_mle .- q_boot
vspan!(psens, ci_boot, linecolor=:grey, fillcolor=:grey, alpha=0.3, fillalpha=0.3, label="90% CI")

The values for the 90% CI are \[1.3, 4.3\], and the MLE is 3.0. We can
see the impact of the skewed distribution in
<a href="#fig-p2-sensitivity-ci" class="quarto-xref">Figure 5</a>.

### Problem 3

Loading the data and separating out the weather-induced variability:

In [1]:
function load_data(fname)
    date_format = "yyyy-mm-dd HH:MM"
    # this uses the DataFramesMeta package -- it's pretty cool
    return @chain fname begin
        CSV.File(; dateformat=date_format)
        DataFrame
        rename(
            "Time (GMT)" => "time", "Predicted (m)" => "harmonic", "Verified (m)" => "gauge"
        )
        @transform :datetime = (Date.(:Date, "yyyy/mm/dd") + Time.(:time))
        select(:datetime, :gauge, :harmonic)
        @transform :weather = :gauge - :harmonic
        @transform :month = (month.(:datetime))
    end
end

dat = load_data("data/norfolk-hourly-surge-2015.csv")

plot(dat.datetime, dat.weather; ylabel="Gauge Weather Variability (m)", label="Detrended Data", linewidth=1, legend=:topleft, xlabel="Date/Time")

Implementing the moving block bootstrap:

In [1]:
function generate_blocks(d, k)
    n_blocks = length(d) - k + 1
    blocks = zeros(Float64, (k, n_blocks))
    for i = 1:n_blocks
        blocks[:, i] = d[i:(k+i-1)]
    end
    return blocks
end

n_boot = 1_000
function moving_bootstrap(d, k, n_boot)
    blocks = generate_blocks(d, k)
    m = Int64(floor(length(d) / k))
    l = rem(length(d), k)
    n_blocks = size(blocks)[2]
    surge_bootstrap = zeros(length(d), n_boot)
    for i = 1:n_boot
        block_sample_idx = sample(1:n_blocks, m; replace=true)
        surge_bootstrap[1:length(d)-l, i] = reduce(vcat, blocks[:, block_sample_idx])
        if l > 0
            surge_bootstrap[length(d)-l+1:end, i] = blocks[:, sample(1:n_blocks, 1)][1:l]
        end
    end
    return surge_bootstrap
end

k = 20
weather_boot = moving_bootstrap(dat.weather, k, 1_000)

8760×1000 Matrix{Float64}:
 0.257   0.504   0.128  0.145   0.472  …  0.069  0.626  0.165  -0.07   -0.172
 0.267   0.5     0.083  0.136   0.467     0.048  0.625  0.159  -0.088  -0.147
 0.294   0.496   0.052  0.119   0.474     0.138  0.641  0.183  -0.094  -0.132
 0.319   0.504   0.052  0.125   0.503     0.171  0.665  0.184  -0.085  -0.091
 0.31    0.511   0.058  0.141   0.498     0.118  0.703  0.183  -0.125  -0.097
 0.285   0.51    0.07   0.151   0.516  …  0.087  0.744  0.186  -0.153  -0.123
 0.246   0.503   0.084  0.152   0.515     0.078  0.796  0.167  -0.139  -0.128
 0.234   0.487   0.071  0.153   0.518     0.095  0.836  0.164  -0.12   -0.146
 0.223   0.461   0.039  0.162   0.505     0.172  0.874  0.179  -0.108  -0.165
 0.209   0.447   0.009  0.176   0.504     0.19   0.885  0.203  -0.117  -0.177
 ⋮                                     ⋱  ⋮                            
 0.188  -0.113  -0.061  0.14    0.038     0.161  0.089  0.181   0.217   0.068
 0.179  -0.11   -0.061  0.138   0.002     0

Now we add these replicates back to the harmonic prediction, then find
the distribution of the medians.

In [1]:
gauge_boot = mapslices(col -> col + dat.harmonic, weather_boot; dims=1)
median_boot = median(gauge_boot, dims=1)'
pmed = histogram(median_boot, xlabel="Median Tide Gauge Reading (m)", ylabel="Count", label=false)
vline!(pmed, [median(dat.gauge)], label="Observation", color=:red)

The bias in the estimator of the median is the difference between the
bootstrap mean and the sample estimate, which is about 5 mm. The 90%
basic bootstrap confidence interval is \[0.49, 0.517\] m.

Now, let’s repeat the analysis with a block length of 50.

In [1]:
k = 50
weather_boot = moving_bootstrap(dat.weather, k, 1_000)
gauge_boot = mapslices(col -> col + dat.harmonic, weather_boot; dims=1)
median_boot = median(gauge_boot, dims=1)'
pmed = histogram(median_boot, xlabel="Median Tide Gauge Reading (m)", ylabel="Count", label=false)
vline!(pmed, [median(dat.gauge)], label="Observation", color=:red)

Now, the bias is about 5 mm and the 90% basic bootstrap confidence
interval is \[0.483, 0.523\] m.

The two different block sizes produced relatively similar results, with
some slight variation for the confidence interval estimates, which are
shifted slightly higher (but just by a few mm) with the larger block
size. Larger blocks mean that there is a greater probability of sampling
the (rare) larger values within any given block, which might be expected
to produce more occurrences of these values even though the number of
re-sampled blocks is smaller. This would have a result of increasing the
median. But in general, the median estimator seems to be relatively
stable across these block sizes, though even larger blocks might change
this.